In [1]:
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
reviews = pd.read_csv('https://github.com/MoinDalvs/Recommendation_System_Amazon_products/blob/main/ratings.csv?raw=true',
                      names = ['user_id','product_id','Ratings','Time_stamp'])
reviews

,user_id,product_id,Ratings,Time_stamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
...,...,...,...,...
7824477,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824478,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824479,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824480,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


### Feature Engineering Process

In [3]:
reviews['product_code'] = 'product_'+reviews['product_id'].str[8:]
reviews

,user_id,product_id,Ratings,Time_stamp,product_code
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200,product_40
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800,product_44
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600,product_41
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200,product_41
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200,product_41
...,...,...,...,...,...
7824477,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600,product_MW
7824478,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400,product_MW
7824479,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800,product_MW
7824480,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200,product_MW


In [117]:
value_counts = pd.DataFrame(reviews['product_id'].value_counts())
value_counts.rename({'product_id':'Number of Times Rated'}, axis=1, inplace=True)
value_counts

,Number of Times Rated
B0074BW614,18244
B00DR0PDNE,16454
B007WTAJTO,14172
B0019EHU8G,12285
B006GWO5WK,12226
...,...
B004WL91KI,1
B004WL9FK4,1
B004WL9Q2Q,1
B004WL9R8O,1


In [118]:
atleast_100_ratings = value_counts[value_counts['Number of Times Rated']>=500]
atleast_100_ratings.reset_index(inplace=True)
atleast_100_ratings.rename({'index':'product_id'},axis=1,inplace=True)
atleast_100_ratings

,product_id,Number of Times Rated
0,B0074BW614,18244
1,B00DR0PDNE,16454
2,B007WTAJTO,14172
3,B0019EHU8G,12285
4,B006GWO5WK,12226
...,...,...
13199,B000HGMMH6,100
13200,B004289Z5W,100
13201,B000E8BGCO,100
13202,B002WM8ZV0,100


In [119]:
final_reviews = pd.merge(reviews, atleast_100_ratings, on='product_id')
final_reviews

,user_id,product_id,Ratings,Time_stamp,product_code,Number of Times Rated
0,A1BKC6B7YHYTVV,0972683275,4.0,1405382400,product_75,1051
1,AWVFSIB42LHKJ,0972683275,4.0,1405209600,product_75,1051
2,A36MQBNADRH8YY,0972683275,5.0,1405641600,product_75,1051
3,A3SRXMPLAEZ6DP,0972683275,4.0,1405987200,product_75,1051
4,A20XXTXWF2TCPY,0972683275,5.0,1405123200,product_75,1051
...,...,...,...,...,...,...
4471774,A2YJYBE8E07C6J,B00KFAGCUM,5.0,1405641600,product_UM,131
4471775,A16H5VQ54ZCR25,B00KFAGCUM,5.0,1403568000,product_UM,131
4471776,A36UTT01QLVU1R,B00KFAGCUM,5.0,1405296000,product_UM,131
4471777,A1SKCXUJO7BARJ,B00KFAGCUM,5.0,1403395200,product_UM,131


In [120]:
final_reviews['product_code'].value_counts()

product_14    27065
product_NE    20969
product_TO    20593
product_WK    19264
product_PK    18881
              ...  
product_67      101
product_9L      100
product_3L      100
product_7N      100
product_R5      100
Name: product_code, Length: 866, dtype: int64

In [121]:
max_product = final_reviews[final_reviews['product_code']=='product_14']
max_product.reset_index(inplace=True, drop=True)
len(max_product.product_id.unique())

22

In [122]:
min_product = final_reviews[final_reviews['product_code']=='product_R5']
min_product.reset_index(inplace=True, drop=True)
len(min_product.product_id.unique())

1

## Web scraping product name, product links and product Image links 

### Beautiful Soup

In [4]:
cookies = {
    'session-id': '259-3113978-6678618',
    'i18n-prefs': 'INR',
    'ubid-acbin': '260-8554202-6973909',
    'lc-acbin': 'en_IN',
    's_cc': 'true',
    's_nr': '1664868681185-Repeat',
    's_vnum': '2088252956746%26vn%3D2',
    's_dslv': '1664868681187',
    's_sq': '%5B%5BB%5D%5D',
    's_ppv': '66',
    'session-token': 'mZmUeIHWObnOI+UJrx4cIknqrZ9EM8Tr0FRKMLeUZuSd8fpwTWOeqyPHTnpayIRMaglWebgQzLMPUvOlpbsxHy0/OvJEWsrXkjfV34QohkDVvmeQJvYIyCe9136EwYNoAeOiPCDZ0FvwPx0BXLBE4McanIuryMNWQeDinHhgcJdI7usJiYUCshv5r8FiFrOjLemkrMk9wJn6eE5bZlnKmdUxOna61M7W',
    'session-id-time': '2082758401l',
    'csm-hit': 'tb:6GW1RVFP5HJ2X930BVEB+b-QEPPBMW48949YPY594HT|1664876731158&t:1664876731159&adb:adblk_no',
}

headers = {
    'authority': 'www.amazon.in',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'session-id=259-3113978-6678618; i18n-prefs=INR; ubid-acbin=260-8554202-6973909; lc-acbin=en_IN; s_cc=true; s_nr=1664868681185-Repeat; s_vnum=2088252956746%26vn%3D2; s_dslv=1664868681187; s_sq=%5B%5BB%5D%5D; s_ppv=66; session-token=mZmUeIHWObnOI+UJrx4cIknqrZ9EM8Tr0FRKMLeUZuSd8fpwTWOeqyPHTnpayIRMaglWebgQzLMPUvOlpbsxHy0/OvJEWsrXkjfV34QohkDVvmeQJvYIyCe9136EwYNoAeOiPCDZ0FvwPx0BXLBE4McanIuryMNWQeDinHhgcJdI7usJiYUCshv5r8FiFrOjLemkrMk9wJn6eE5bZlnKmdUxOna61M7W; session-id-time=2082758401l; csm-hit=tb:6GW1RVFP5HJ2X930BVEB+b-QEPPBMW48949YPY594HT|1664876731158&t:1664876731159&adb:adblk_no',
    'device-memory': '8',
    'downlink': '10',
    'dpr': '0.8',
    'ect': '4g',
    'referer': 'https://www.amazon.in/ref=nav_logo',
    'rtt': '0',
    'sec-ch-device-memory': '8',
    'sec-ch-dpr': '0.8',
    'sec-ch-ua': '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-viewport-width': '2400',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'service-worker-navigation-preload': 'true',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'viewport-width': '2400',
}

In [5]:
pages = [
    "https://www.amazon.in/s?k=aer&page=1"
]

In [6]:
resp = requests.get(pages[0], cookies=cookies, headers=headers)
resp

<Response [200]>

### To scrape for all Amazon Layout

In [7]:
def get_scraped(category):
    scraper=[]
    base_url = f'https://www.amazon.in/s?k={category.replace(" ", "+")}'
    for x in tqdm(range(1,40)):
        url = base_url+f'&page={x}'
        r = requests.get(url, cookies=cookies, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        if soup.find("div", {"class": "s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"}):
            scrape = soup.find_all("div", {"class": "s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"})
            for item in scrape:
                review = {
                    'product_name': item.find('span',{'class':'a-size-medium a-color-base a-text-normal'}).get_text(),
                    'product_img': item.find('img',{'class':'s-image'}).get('src'),
                    'product_link': 'https://www.amazon.in/' + item.find('a',{'class':'a-link-normal s-no-outline'}, href=True).get('href')
                        }
                scraper.append(review)
        elif soup.find("div", {"class": "sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20"}):
            scrape = soup.find_all("div", {"class": "sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20"})
            for item in scrape:
                if item.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'}):
                    p = item.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'})
                    if p is not None:
                        review = {
                            'product_name': item.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'}).get_text(),
                            'product_img': item.find('img',{'class':'s-image'}).get('src'),
                            'product_link': 'https://www.amazon.in/' + item.find('a',{'class':'a-link-normal s-no-outline'}, href=True).get('href')
                                }
                        scraper.append(review)
                elif item.find('span',{'class':'a-size-medium a-color-base a-text-normal'}):
                    p = item.find('span',{'class':'a-size-medium a-color-base a-text-normal'})
                    if p is not None:
                        review = {
                            'product_name': item.find('span',{'class':'a-size-medium a-color-base a-text-normal'}).get_text(),
                            'product_img': item.find('img',{'class':'s-image'}).get('src'),
                            'product_link': 'https://www.amazon.in/' + item.find('a',{'class':'a-link-normal s-no-outline'}, href=True).get('href')
                                }
                        scraper.append(review)
        if soup.find('span', {'class': 's-pagination-item s-pagination-next s-pagination-disabled'}):
            break
    # Save results to a dataframe, then export as CSV
    df = pd.DataFrame(scraper)
    return df

In [25]:
df = get_scraped('aer')
df

 13%|██████████▋                                                                        | 5/39 [00:10<01:10,  2.08s/it]


,product_name,product_img,product_link
0,Godrej Ezee Aer Power Pocket | Air Freshener- ...,https://m.media-amazon.com/images/I/61L-epiEFP...,https://www.amazon.in//Godrej-aer-Bathroom-Fra...
1,Godrej aer Matic Refill - Automatic Air Freshe...,https://m.media-amazon.com/images/I/61kGN75x0A...,https://www.amazon.in//Godrej-Aer-Matic-Refill...
2,Godrej aer Matic Refill - Automatic Air Freshe...,https://m.media-amazon.com/images/I/61kS0AzQ+u...,https://www.amazon.in//Godrej-Aer-Matic-Refill...
3,Godrej aer Matic Refill - Automatic Air Freshe...,https://m.media-amazon.com/images/I/61463RCBok...,https://www.amazon.in//Godrej-Aer-Matic-Refill...
4,Godrej aer Smart Matic Refill - BLUETOOTH ENAB...,https://m.media-amazon.com/images/I/61MKiHNCpy...,https://www.amazon.in//Godrej-aer-Smart-Matic-...
...,...,...,...
259,Wine Aerator,https://m.media-amazon.com/images/I/61pMu8CwdN...,https://www.amazon.in//Dare-2-Aer-COMINHKPR140...
260,100 Box of Pink Razor Blades Disposable Stainl...,https://m.media-amazon.com/images/I/81l4uziPG1...,https://www.amazon.in//Disposable-Stainless-Ho...
261,40 Twin Blade Premium Disposable Razors in Bul...,https://m.media-amazon.com/images/I/81Fnqjpwxd...,https://www.amazon.in//Twin-Blade-Premium-Disp...
262,"Next-Level Clean Deodorant (Ginger Grapefruit,...",https://m.media-amazon.com/images/I/71H1Fsze2+...,https://www.amazon.in//Next-Level-Clean-Deodor...


### Product AA dataframe

In [40]:
df = get_scraped('aaa batteries')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Duracell Chota Power AAA Alkaline Batteries, L...",https://m.media-amazon.com/images/I/61UmAauFle...,https://www.amazon.in//ANAV-Duracell-Alkaline-...
1,"Duracell Ultra Alkaline AAA Battery, 8 Pcs",https://m.media-amazon.com/images/I/71CX+Z-xVF...,https://www.amazon.in//Duracell-Alkaline-Batte...
2,"Duracell Ultra Alkaline AAA Battery, 4 Pcs",https://m.media-amazon.com/images/I/71nFy6mU+t...,https://www.amazon.in//Duracell-Alkaline-Batte...
3,Duracell Ultra Alkaline Battery AAA - 20 Pieces,https://m.media-amazon.com/images/I/719xp5zEMY...,https://www.amazon.in//Duracell-Ultra-Alkaline...
4,Eveready Red 1012 AAA Batteries - Pack of 10,https://m.media-amazon.com/images/I/71TP9KLQ7T...,https://www.amazon.in//Eveready-Alkaline-Batte...
...,...,...,...
301,ERH India (2 Pcs) Battery Holder for 2 AA Batt...,https://m.media-amazon.com/images/I/51KhobqO1h...,https://www.amazon.in//ERH-India-Holder-Cell-E...
302,"Energizer E96BP2 Energizer Alkaline Battery,""A...",https://m.media-amazon.com/images/I/51A+sZ3ML4...,https://www.amazon.in//Energizer-E96BP2-Alkali...
303,MAXELL 723865 - LR034BP Alkaline Batteries (AA...,https://m.media-amazon.com/images/I/71A3lOBCZ5...,https://www.amazon.in//MAXELL-723865-LR034BP-A...
304,Gilhot® 9 Volt Battery Pack of 10pcs.Non Recha...,https://m.media-amazon.com/images/I/71bHO5x0Oi...,https://www.amazon.in//Gilhot%C2%AE-volt-Batte...


In [37]:
AA = df.copy()

In [41]:
AA = AA.append(df).reset_index(drop=True)
AA

,product_name,product_img,product_link
0,"Duracell Ultra Alkaline AA Battery, 8 Pcs",https://m.media-amazon.com/images/I/71G4VJZ+9W...,https://www.amazon.in//Duracell-Alkaline-Batte...
1,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
2,Duracell Ultra Alkaline Battery AA - 20 Pieces,https://m.media-amazon.com/images/I/71ZykzQzVE...,https://www.amazon.in//Duracell-Ultra-Alkaline...
3,Duracell Chhota Power AA Battery Set of 10 Pcs,https://m.media-amazon.com/images/I/617bDCHC5A...,https://www.amazon.in//Duracell-Chhota-Power-B...
4,Duracell CHHOTA Power Alkaline AA Batteries- 4...,https://m.media-amazon.com/images/I/7177W-oGec...,https://www.amazon.in//Duracell-Chota-Power-Al...
...,...,...,...
913,ERH India (2 Pcs) Battery Holder for 2 AA Batt...,https://m.media-amazon.com/images/I/51KhobqO1h...,https://www.amazon.in//ERH-India-Holder-Cell-E...
914,"Energizer E96BP2 Energizer Alkaline Battery,""A...",https://m.media-amazon.com/images/I/51A+sZ3ML4...,https://www.amazon.in//Energizer-E96BP2-Alkali...
915,MAXELL 723865 - LR034BP Alkaline Batteries (AA...,https://m.media-amazon.com/images/I/71A3lOBCZ5...,https://www.amazon.in//MAXELL-723865-LR034BP-A...
916,Gilhot® 9 Volt Battery Pack of 10pcs.Non Recha...,https://m.media-amazon.com/images/I/71bHO5x0Oi...,https://www.amazon.in//Gilhot%C2%AE-volt-Batte...


In [25]:
product_AA = pd.DataFrame(reviews[reviews['product_code']=='product_AA']).reset_index(drop=True)
product_AA

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1NA7W62VGQ5JZ,B00000JLAA,5.0,1173139200,product_AA
1,A14ME4FQBNFYWH,B00001P4AA,4.0,960595200,product_AA
2,AMGJMFJ63DWWH,B00001P4AA,4.0,959299200,product_AA
3,A11LT1IGS1Q5CZ,B00001P4AA,4.0,968976000,product_AA
4,A27JOWVO9PQAI0,B00001P4AA,4.0,1008201600,product_AA
...,...,...,...,...,...
12445,A5CIP6ECF8LHU,B00JJD8BAA,1.0,1401148800,product_AA
12446,A1L6DDH013USP4,B00K4EWFAA,5.0,1403654400,product_AA
12447,APHX9ICTPNILI,B00K4EWFAA,5.0,1403568000,product_AA
12448,A331BQT4N8E6N2,B00KHMN3AA,5.0,1404086400,product_AA


In [34]:
len(product_AA.product_id.unique())

762

In [42]:
product_AA = pd.DataFrame(reviews[reviews['product_code']=='product_AA']).reset_index(drop=True)
product_AA =  pd.merge(product_AA, 
                       AA.iloc[:len(product_AA.product_id.unique())]\
                       .join(pd.Series(product_AA['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AA.to_csv('product_AA.csv',index=False)
pd.read_csv('product_AA.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1NA7W62VGQ5JZ,B00000JLAA,5.0,1173139200,product_AA,"Duracell Ultra Alkaline AA Battery, 8 Pcs",https://m.media-amazon.com/images/I/71G4VJZ+9W...,https://www.amazon.in//Duracell-Alkaline-Batte...
1,A14ME4FQBNFYWH,B00001P4AA,4.0,960595200,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
2,AMGJMFJ63DWWH,B00001P4AA,4.0,959299200,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
3,A11LT1IGS1Q5CZ,B00001P4AA,4.0,968976000,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
4,A27JOWVO9PQAI0,B00001P4AA,4.0,1008201600,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
...,...,...,...,...,...,...,...,...
12445,A5CIP6ECF8LHU,B00JJD8BAA,1.0,1401148800,product_AA,Powercell AAA Batteries Pack of 30,https://m.media-amazon.com/images/I/71FSugSITy...,https://www.amazon.in//PowerCell-size-Batterie...
12446,A1L6DDH013USP4,B00K4EWFAA,5.0,1403654400,product_AA,"Panasonic EVOLTA Alkaline AAA Battery, Pack of 10",https://m.media-amazon.com/images/I/71nsGOkf06...,https://www.amazon.in//Panasonic-EVOLTA-Alkali...
12447,APHX9ICTPNILI,B00K4EWFAA,5.0,1403568000,product_AA,"Panasonic EVOLTA Alkaline AAA Battery, Pack of 10",https://m.media-amazon.com/images/I/71nsGOkf06...,https://www.amazon.in//Panasonic-EVOLTA-Alkali...
12448,A331BQT4N8E6N2,B00KHMN3AA,5.0,1404086400,product_AA,"Energizer MAX AAA Primary Alkaline Batteries, ...",https://m.media-amazon.com/images/I/715k8klpQP...,https://www.amazon.in//Energizer-Primary-Alkal...


### Product AB dataframe

In [11]:
df = get_scraped('abs builder&i=sporting')
AB = df.copy()
AB

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,IRIS Fitness Ab Roller Abdominal Crunch Exerci...,https://m.media-amazon.com/images/I/51v4F-VeF9...,https://www.amazon.in//IRIS-Abdominal-Exercise...
1,GTC Hand Strengtheners Bands Stretcher Exercis...,https://m.media-amazon.com/images/I/51hVY0TakU...,https://www.amazon.in//GTC-Strengtheners-Stret...
2,MCP Ab Roller/Abs Carver with Thick Knee Pad f...,https://m.media-amazon.com/images/I/51TPz25UYz...,https://www.amazon.in//MCP-Abdominal-Workout-E...
3,ALLYSON FITNESS Forearm Blaster-Wrist Roller-H...,https://m.media-amazon.com/images/I/5173q96DSH...,https://www.amazon.in//ALLYSON-Blaster-Wrist-R...
4,VIRTUAL WORLD Professional Fitness Imported Ab...,https://m.media-amazon.com/images/I/61y2yyAVVi...,https://www.amazon.in//VIRTUAL-Professional-Co...
5,Emeret Super Quiet Double Wheel Ab Roller with...,https://m.media-amazon.com/images/I/61SpMPGqVt...,https://www.amazon.in//Emeret-Roller-Abwheel-A...
6,Wazdorf Anti Skid Double Wheel Total Body AB R...,https://m.media-amazon.com/images/I/71UepQz2LB...,https://www.amazon.in//Wazdorf-Exerciser-Abdom...
7,VAISHNAVI FITNESS ABS Tower Push Up Dips Worko...,https://m.media-amazon.com/images/I/71Ji7gUZLW...,https://www.amazon.in//VAISHNAVI-FITNESS-Worko...
8,virtual World No Gym Full Body Workout Exercis...,https://m.media-amazon.com/images/I/61254GHlY3...,https://www.amazon.in//World-Exerciser-Revofle...
9,Archiz Thermoplastic Elastomer TPE Eco Friendl...,https://m.media-amazon.com/images/I/712xpWkpyK...,https://www.amazon.in//Archiz-Friendly-Resista...


In [43]:
product_AB = pd.DataFrame(reviews[reviews['product_code']=='product_AB']).reset_index(drop=True)
product_AB

,user_id,product_id,Ratings,Time_stamp,product_code
0,A16FYVON6VFP83,B00001P4AB,4.0,983318400,product_AB
1,A1DZ1SBRXHANHX,B00001P4AB,3.0,969753600,product_AB
2,A33QUVOLIPH1A3,B00001P4AB,2.0,1014681600,product_AB
3,A19DAH3XY8U4J3,B00001P4AB,5.0,964656000,product_AB
4,AFHLW2L0G7BOV,B00001QEAB,5.0,949536000,product_AB
...,...,...,...,...,...
347,AQCJWPOKTJ29C,B0000A2QAB,2.0,1104710400,product_AB
348,A2BUVULIN0Q2AQ,B0000A2QAB,1.0,1129075200,product_AB
349,A1HZVIIO3TINAJ,B0000A2QAB,4.0,1065225600,product_AB
350,A24XXI3Y2CV971,B0000A2QAB,1.0,1321833600,product_AB


In [44]:
len(product_AB.product_id.unique())

17

In [49]:
product_AB = pd.DataFrame(reviews[reviews['product_code']=='product_AB']).reset_index(drop=True)
product_AB =  pd.merge(product_AB, 
                       AB.iloc[:len(product_AB.product_id.unique())]\
                       .join(pd.Series(product_AB['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AB.to_csv('product_AB.csv',index=False)
pd.read_csv('product_AB.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A16FYVON6VFP83,B00001P4AB,4.0,983318400,product_AB,PRO365 Ab Roller Combo Pack With Tummy Trimmer...,https://m.media-amazon.com/images/I/619Os004jb...,https://www.amazon.in//PRO365-Exercise-Equipme...
1,A1DZ1SBRXHANHX,B00001P4AB,3.0,969753600,product_AB,PRO365 Ab Roller Combo Pack With Tummy Trimmer...,https://m.media-amazon.com/images/I/619Os004jb...,https://www.amazon.in//PRO365-Exercise-Equipme...
2,A33QUVOLIPH1A3,B00001P4AB,2.0,1014681600,product_AB,PRO365 Ab Roller Combo Pack With Tummy Trimmer...,https://m.media-amazon.com/images/I/619Os004jb...,https://www.amazon.in//PRO365-Exercise-Equipme...
3,A19DAH3XY8U4J3,B00001P4AB,5.0,964656000,product_AB,PRO365 Ab Roller Combo Pack With Tummy Trimmer...,https://m.media-amazon.com/images/I/619Os004jb...,https://www.amazon.in//PRO365-Exercise-Equipme...
4,AFHLW2L0G7BOV,B00001QEAB,5.0,949536000,product_AB,FitBox Sports Adjustable Hand Grip Strengthene...,https://m.media-amazon.com/images/I/71hrS7zaA7...,https://www.amazon.in//FitBox-Sports-Adjustabl...
...,...,...,...,...,...,...,...,...
347,AQCJWPOKTJ29C,B0000A2QAB,2.0,1104710400,product_AB,FITGURU EXERCISE ABS DUAL WHEEL ROLLER SIZE: 2...,https://m.media-amazon.com/images/I/61dWyeereE...,https://www.amazon.in//Exercise-Abdominal-Stab...
348,A2BUVULIN0Q2AQ,B0000A2QAB,1.0,1129075200,product_AB,FITGURU EXERCISE ABS DUAL WHEEL ROLLER SIZE: 2...,https://m.media-amazon.com/images/I/61dWyeereE...,https://www.amazon.in//Exercise-Abdominal-Stab...
349,A1HZVIIO3TINAJ,B0000A2QAB,4.0,1065225600,product_AB,FITGURU EXERCISE ABS DUAL WHEEL ROLLER SIZE: 2...,https://m.media-amazon.com/images/I/61dWyeereE...,https://www.amazon.in//Exercise-Abdominal-Stab...
350,A24XXI3Y2CV971,B0000A2QAB,1.0,1321833600,product_AB,FITGURU EXERCISE ABS DUAL WHEEL ROLLER SIZE: 2...,https://m.media-amazon.com/images/I/61dWyeereE...,https://www.amazon.in//Exercise-Abdominal-Stab...


### Product AC dataframe

In [89]:
df = get_scraped('tower air conditioner')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Voltas Venture Slimline Tower AC (2 Ton White),https://m.media-amazon.com/images/I/31jhpl4GmP...,https://www.amazon.in//Voltas-Venture-Slimline...
1,Symphony Sumo 115 XL Desert Air Cooler For Hom...,https://m.media-amazon.com/images/I/61mBx96iGZ...,https://www.amazon.in//Symphony-Sumo-115-XL-Co...
2,Symphony Diet 3D 30i Portable Tower Air Cooler...,https://m.media-amazon.com/images/I/51Po1SCgSG...,https://www.amazon.in//Symphony-Diet-30i-Touch...
3,Voltas Venture Slimline Tower AC (3 Ton White),https://m.media-amazon.com/images/I/31jhpl4GmP...,https://www.amazon.in//Voltas-Venture-Slimline...
4,Symphony Diet 12T Personal Tower Air Cooler fo...,https://m.media-amazon.com/images/I/51c9CzgVok...,https://www.amazon.in//Symphony-Diet-12-Litre-...
...,...,...,...
301,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...
302,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...
303,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...
304,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...


In [84]:
AC = df.copy()
AC

,product_name,product_img,product_link
0,Panasonic 1.5 Ton 3 Star Wi-Fi Twin-Cool Inver...,https://m.media-amazon.com/images/I/51CFPrvNH7...,https://www.amazon.in//Panasonic-Conditioner-C...
1,Panasonic 1.5 Ton 5 Star Wi-Fi Twin-Cool Inver...,https://m.media-amazon.com/images/I/51CFPrvNH7...,https://www.amazon.in//Panasonic-Conditioner-C...
2,"Whirlpool 1.5 Ton 3 Star, Inverter Split AC (1...",https://m.media-amazon.com/images/I/41uMPN7SoI...,https://www.amazon.in//Whirlpool-Inverter-Spli...
3,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,https://m.media-amazon.com/images/I/51hbo8yQ1E...,https://www.amazon.in//LG-Convertible-Anti-Vir...
4,Lloyd 1.0 Ton 3 Star Inverter Split AC with An...,https://m.media-amazon.com/images/I/41Q1m3KiJ5...,https://www.amazon.in//Lloyd-Anti-Viral-GLS12I...
...,...,...,...
301,Carrier 1.5 Ton 5 Star Inverter Split AC (Copp...,https://m.media-amazon.com/images/I/71Zx6AB3JH...,https://www.amazon.in//Carrier-CAI18EK5R30F0-E...
302,LG 1.5 Ton 3 Star with UV Nano Inverter Split ...,https://m.media-amazon.com/images/I/51lyOjfKhM...,https://www.amazon.in//Inverter-Copper-Convert...
303,"LG 1.5 Ton 3 Star Inverter Split AC (Copper, L...",https://m.media-amazon.com/images/I/61oBNxWvth...,https://www.amazon.in//LG-Inverter-LS-Q18JNXA-...
304,Godrej 1.5 Ton 3 Star Split AC (Copper GSC 18 ...,https://m.media-amazon.com/images/I/41KBiugyQE...,https://www.amazon.in//Godrej-GSC-18-RGN-CWQR/...


In [90]:
AC = AC.append(df).reset_index(drop=True)
AC

,product_name,product_img,product_link
0,Panasonic 1.5 Ton 3 Star Wi-Fi Twin-Cool Inver...,https://m.media-amazon.com/images/I/51CFPrvNH7...,https://www.amazon.in//Panasonic-Conditioner-C...
1,Panasonic 1.5 Ton 5 Star Wi-Fi Twin-Cool Inver...,https://m.media-amazon.com/images/I/51CFPrvNH7...,https://www.amazon.in//Panasonic-Conditioner-C...
2,"Whirlpool 1.5 Ton 3 Star, Inverter Split AC (1...",https://m.media-amazon.com/images/I/41uMPN7SoI...,https://www.amazon.in//Whirlpool-Inverter-Spli...
3,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,https://m.media-amazon.com/images/I/51hbo8yQ1E...,https://www.amazon.in//LG-Convertible-Anti-Vir...
4,Lloyd 1.0 Ton 3 Star Inverter Split AC with An...,https://m.media-amazon.com/images/I/41Q1m3KiJ5...,https://www.amazon.in//Lloyd-Anti-Viral-GLS12I...
...,...,...,...
782,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...
783,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...
784,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...
785,Crompton ® Air Cooler- 10 Liter ISI Certified ...,https://m.media-amazon.com/images/I/614PzsS+n2...,https://www.amazon.in//Crompton-Air-10-Certifi...


In [85]:
product_AC = pd.DataFrame(reviews[reviews['product_code']=='product_AC']).reset_index(drop=True)
product_AC

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1PUPBU225AVMT,B00003CWAC,1.0,1258934400,product_AC
1,A238ORQTQ49E4T,B00004ZCAC,3.0,1126051200,product_AC
2,A1U4JWHUO6QDVP,B0000513AC,4.0,1301616000,product_AC
3,A3UT41TWD7N0D5,B00005AYAC,4.0,1063411200,product_AC
4,AT2J7H5TRZM8Z,B00005B9AC,4.0,1008201600,product_AC
...,...,...,...,...,...
11964,A3N69H23SF6HW8,B00JBGTIAC,5.0,1402358400,product_AC
11965,A1U9ST8KMGZSK,B00JQ04JAC,5.0,1401926400,product_AC
11966,A2OBTRA6Y8LJR9,B00JQ04JAC,5.0,1400371200,product_AC
11967,A2SSILUOK78SX7,B00JQ04JAC,5.0,1398470400,product_AC


In [86]:
len(product_AC.product_id.unique())

756

In [91]:
product_AC = pd.DataFrame(reviews[reviews['product_code']=='product_AC']).reset_index(drop=True)
product_AC =  pd.merge(product_AC, 
                       AC.iloc[:len(product_AC.product_id.unique())]\
                       .join(pd.Series(product_AC['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AC.to_csv('product_AC.csv',index=False)
pd.read_csv('product_AC.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1PUPBU225AVMT,B00003CWAC,1.0,1258934400,product_AC,Panasonic 1.5 Ton 3 Star Wi-Fi Twin-Cool Inver...,https://m.media-amazon.com/images/I/51CFPrvNH7...,https://www.amazon.in//Panasonic-Conditioner-C...
1,A238ORQTQ49E4T,B00004ZCAC,3.0,1126051200,product_AC,Panasonic 1.5 Ton 5 Star Wi-Fi Twin-Cool Inver...,https://m.media-amazon.com/images/I/51CFPrvNH7...,https://www.amazon.in//Panasonic-Conditioner-C...
2,A1U4JWHUO6QDVP,B0000513AC,4.0,1301616000,product_AC,"Whirlpool 1.5 Ton 3 Star, Inverter Split AC (1...",https://m.media-amazon.com/images/I/41uMPN7SoI...,https://www.amazon.in//Whirlpool-Inverter-Spli...
3,A3UT41TWD7N0D5,B00005AYAC,4.0,1063411200,product_AC,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,https://m.media-amazon.com/images/I/51hbo8yQ1E...,https://www.amazon.in//LG-Convertible-Anti-Vir...
4,AT2J7H5TRZM8Z,B00005B9AC,4.0,1008201600,product_AC,Lloyd 1.0 Ton 3 Star Inverter Split AC with An...,https://m.media-amazon.com/images/I/41Q1m3KiJ5...,https://www.amazon.in//Lloyd-Anti-Viral-GLS12I...
...,...,...,...,...,...,...,...,...
11964,A3N69H23SF6HW8,B00JBGTIAC,5.0,1402358400,product_AC,Orient ® Air Cooler- 20 Liter ISI Certified (M...,https://m.media-amazon.com/images/I/511aRDLGWI...,https://www.amazon.in//Orient-20-Smart-Cool-Ki...
11965,A1U9ST8KMGZSK,B00JQ04JAC,5.0,1401926400,product_AC,Orient ® Air Cooler- 20 Liter ISI Certified (M...,https://m.media-amazon.com/images/I/511aRDLGWI...,https://www.amazon.in//Orient-20-Smart-Cool-Ki...
11966,A2OBTRA6Y8LJR9,B00JQ04JAC,5.0,1400371200,product_AC,Orient ® Air Cooler- 20 Liter ISI Certified (M...,https://m.media-amazon.com/images/I/511aRDLGWI...,https://www.amazon.in//Orient-20-Smart-Cool-Ki...
11967,A2SSILUOK78SX7,B00JQ04JAC,5.0,1398470400,product_AC,Orient ® Air Cooler- 20 Liter ISI Certified (M...,https://m.media-amazon.com/images/I/511aRDLGWI...,https://www.amazon.in//Orient-20-Smart-Cool-Ki...


### Product AD dataframe

In [100]:
df = get_scraped('atta dough maker')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Amazon Brand - Solimo Plastic Atta/Dough Maker,https://m.media-amazon.com/images/I/711bfQFRYK...,https://www.amazon.in//Amazon-Brand-Solimo-Pla...
1,Clearline Automatic Electric Dough Kneader Wit...,https://m.media-amazon.com/images/I/31pBfOngn4...,https://www.amazon.in//Clearline-Automatic-Ele...
2,Home Plus Magic 400 Watts Atta Kneader | Food ...,https://m.media-amazon.com/images/I/71FmdweTl+...,https://www.amazon.in//Homeplus-Magic-Maker-ho...
3,"Preethi MGA-524 Atta Kneader, White",https://m.media-amazon.com/images/I/51qcM2EF8a...,https://www.amazon.in//Preethi-Master-MGA-524-...
4,Primelife Plastic 3 in 1 Chop & Churn Manual A...,https://m.media-amazon.com/images/I/61vWfHTjal...,https://www.amazon.in//Primelife-Multicolor-Pl...
...,...,...,...
301,"BIGGTOY Dough Maker, Atta Maker, Atta Machine,...",https://m.media-amazon.com/images/I/41ozYtdlQ-...,https://www.amazon.in//BIGGTOY-Dough-Machine-C...
302,WisTec Stand Mixer 1300 Watts |6.5 Litre SS Bo...,https://m.media-amazon.com/images/I/51FzaOEXEZ...,https://www.amazon.in//WisTec-Copper-Planetary...
303,"Philips HL7703/00 Mixer Grinder 1000 Watt , 4 ...",https://m.media-amazon.com/images/I/61Hh2Z3qw1...,https://www.amazon.in//Philips-HL7703-00-Stain...
304,Ronest ABS Atta Dough Kneader Maker Kitchen Se...,https://m.media-amazon.com/images/I/61IkocZHuW...,https://www.amazon.in//Ronest-Dough-Kneader-Ki...


In [101]:
AD = df.copy()
AD

,product_name,product_img,product_link
0,Amazon Brand - Solimo Plastic Atta/Dough Maker,https://m.media-amazon.com/images/I/711bfQFRYK...,https://www.amazon.in//Amazon-Brand-Solimo-Pla...
1,Clearline Automatic Electric Dough Kneader Wit...,https://m.media-amazon.com/images/I/31pBfOngn4...,https://www.amazon.in//Clearline-Automatic-Ele...
2,Home Plus Magic 400 Watts Atta Kneader | Food ...,https://m.media-amazon.com/images/I/71FmdweTl+...,https://www.amazon.in//Homeplus-Magic-Maker-ho...
3,"Preethi MGA-524 Atta Kneader, White",https://m.media-amazon.com/images/I/51qcM2EF8a...,https://www.amazon.in//Preethi-Master-MGA-524-...
4,Primelife Plastic 3 in 1 Chop & Churn Manual A...,https://m.media-amazon.com/images/I/61vWfHTjal...,https://www.amazon.in//Primelife-Multicolor-Pl...
...,...,...,...
301,"BIGGTOY Dough Maker, Atta Maker, Atta Machine,...",https://m.media-amazon.com/images/I/41ozYtdlQ-...,https://www.amazon.in//BIGGTOY-Dough-Machine-C...
302,WisTec Stand Mixer 1300 Watts |6.5 Litre SS Bo...,https://m.media-amazon.com/images/I/51FzaOEXEZ...,https://www.amazon.in//WisTec-Copper-Planetary...
303,"Philips HL7703/00 Mixer Grinder 1000 Watt , 4 ...",https://m.media-amazon.com/images/I/61Hh2Z3qw1...,https://www.amazon.in//Philips-HL7703-00-Stain...
304,Ronest ABS Atta Dough Kneader Maker Kitchen Se...,https://m.media-amazon.com/images/I/61IkocZHuW...,https://www.amazon.in//Ronest-Dough-Kneader-Ki...


In [96]:
product_AD = pd.DataFrame(reviews[reviews['product_code']=='product_AD']).reset_index(drop=True)
product_AD

,user_id,product_id,Ratings,Time_stamp,product_code
0,A29E2K5C0AOBMJ,B00001P4AD,4.0,965779200,product_AD
1,AF92TH0LYPFWM,B00001P4AD,5.0,972950400,product_AD
2,AVHJAVPDXQEHW,B00003CWAD,1.0,1245715200,product_AD
3,AADDHBS4TTVEM,B00003CWAD,4.0,1391904000,product_AD
4,A2ENSC4GVI8T9Z,B00003CWAD,1.0,1181606400,product_AD
...,...,...,...,...,...
90,A3944IG6I7QHR,B0000A2QAD,5.0,1127260800,product_AD
91,A1D5NPVI86V4JK,B0000A2QAD,1.0,1212624000,product_AD
92,A1CGQLDO0ANVWD,B0000BZLAD,5.0,1405209600,product_AD
93,A11B9T6J0MST74,B0000D8HAD,1.0,1138838400,product_AD


In [97]:
len(product_AD.product_id.unique())

17

In [102]:
product_AD = pd.DataFrame(reviews[reviews['product_code']=='product_AD']).reset_index(drop=True)
product_AD =  pd.merge(product_AD, 
                       AD.iloc[:len(product_AD.product_id.unique())]\
                       .join(pd.Series(product_AD['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AD.to_csv('product_AD.csv',index=False)
pd.read_csv('product_AD.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A29E2K5C0AOBMJ,B00001P4AD,4.0,965779200,product_AD,Amazon Brand - Solimo Plastic Atta/Dough Maker,https://m.media-amazon.com/images/I/711bfQFRYK...,https://www.amazon.in//Amazon-Brand-Solimo-Pla...
1,AF92TH0LYPFWM,B00001P4AD,5.0,972950400,product_AD,Amazon Brand - Solimo Plastic Atta/Dough Maker,https://m.media-amazon.com/images/I/711bfQFRYK...,https://www.amazon.in//Amazon-Brand-Solimo-Pla...
2,AVHJAVPDXQEHW,B00003CWAD,1.0,1245715200,product_AD,Clearline Automatic Electric Dough Kneader Wit...,https://m.media-amazon.com/images/I/31pBfOngn4...,https://www.amazon.in//Clearline-Automatic-Ele...
3,AADDHBS4TTVEM,B00003CWAD,4.0,1391904000,product_AD,Clearline Automatic Electric Dough Kneader Wit...,https://m.media-amazon.com/images/I/31pBfOngn4...,https://www.amazon.in//Clearline-Automatic-Ele...
4,A2ENSC4GVI8T9Z,B00003CWAD,1.0,1181606400,product_AD,Clearline Automatic Electric Dough Kneader Wit...,https://m.media-amazon.com/images/I/31pBfOngn4...,https://www.amazon.in//Clearline-Automatic-Ele...
...,...,...,...,...,...,...,...,...
90,A3944IG6I7QHR,B0000A2QAD,5.0,1127260800,product_AD,OVEERA 2 in 1 Ata Maker Automatic Non-Electric...,https://m.media-amazon.com/images/I/41-gK3iHU0...,https://www.amazon.in//OVEERA-Ata-Maker-Dough-...
91,A1D5NPVI86V4JK,B0000A2QAD,1.0,1212624000,product_AD,OVEERA 2 in 1 Ata Maker Automatic Non-Electric...,https://m.media-amazon.com/images/I/41-gK3iHU0...,https://www.amazon.in//OVEERA-Ata-Maker-Dough-...
92,A1CGQLDO0ANVWD,B0000BZLAD,5.0,1405209600,product_AD,"Kent Atta and Bread Maker for Home, Fully Auto...",https://m.media-amazon.com/images/I/71RveHsDGi...,https://www.amazon.in//KENT-Bread-Maker-550-Wa...
93,A11B9T6J0MST74,B0000D8HAD,1.0,1138838400,product_AD,Lifelong LLHM02 300 W Regalia Plus Hand Mixer ...,https://m.media-amazon.com/images/I/71-+CA9j9S...,https://www.amazon.in//Lifelong-LLHM02-Regalia...


### Product AE dataframe

In [19]:
df = get_scraped('aeroplane clock')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,ZAHEPA Aeroplane Show Pieces Table Clock with ...,https://m.media-amazon.com/images/I/61szqULgq7...,https://www.amazon.in//ZAHEPA-Aeroplane-Design...
1,Meenakshi Handicraft Emporium MHE Decorative B...,https://m.media-amazon.com/images/I/715cPt34xF...,https://www.amazon.in//MHE-Decorative-Brass-Ae...
2,KIDOZ Wooden Clock Aeroplane,https://m.media-amazon.com/images/I/41dsxgosVU...,https://www.amazon.in//KIDOZ-Wooden-Clock-Aero...
3,Store2508® Antique Finish Cast Metal Desk Tabl...,https://m.media-amazon.com/images/I/71Nk-s63RY...,https://www.amazon.in//Store2508%C2%AE-Aeropla...
4,Lasaani Aeroplane Designer ACP Waterproof Wall...,https://m.media-amazon.com/images/I/41A2Uv2g4U...,https://www.amazon.in//Lasaani-Aeroplane-Desig...
...,...,...,...
137,Meri Shopp Weather Vane Yard Wind Direction In...,https://m.media-amazon.com/images/I/61DQpmYQCF...,https://www.amazon.in//Weather-Direction-Indic...
138,Meri Shopp Weather Vane Farm Weather Vane Wind...,https://m.media-amazon.com/images/I/71x0S8vCI9...,https://www.amazon.in//Shopp-Weather-Direction...
139,CALANDIS Weather Vane Farm Bracket Weathervane...,https://m.media-amazon.com/images/I/51J8tXz9vW...,https://www.amazon.in//CALANDIS-Weather-Bracke...
140,Black Stainless Steel Flying Witch Weathervane...,https://m.media-amazon.com/images/I/71leu3XDEz...,https://www.amazon.in//Black-Stainless-Flying-...


In [14]:
AE = df.copy()
AE

,product_name,product_img,product_link
0,SUPER TOY Battery Operated Aeroplane Toy for K...,https://m.media-amazon.com/images/I/615SgNTD0P...,https://www.amazon.in//SUPER-TOY-Battery-Opera...
1,NHR Exceed Helicopter Remote Control & Recharg...,https://m.media-amazon.com/images/I/61SqTcdLUg...,https://www.amazon.in//NHR-Helicopter-Recharge...
2,LEGO 42117 Technic Race Plane Toy to Jet Aerop...,https://m.media-amazon.com/images/I/81X4QrwM6z...,https://www.amazon.in//LEGO-Plane-Building-Blo...
3,Miniature Mart Small Size Indian Fighter Jet A...,https://m.media-amazon.com/images/I/81DB-Q4Sfq...,https://www.amazon.in//Miniature-Mart-Fighter-...
4,Brand Conquer Airplane Launcher Toy Catapult A...,https://m.media-amazon.com/images/I/7168fs-rP-...,https://www.amazon.in//Brand-Conquer-Airplane-...
...,...,...,...
205,pludy innovator Flying Airplane Launcher Gun T...,https://m.media-amazon.com/images/I/71NwnveSKT...,https://www.amazon.in//pludy-innovator-Childre...
206,Cute Plastic Pull Back Plane Cartoon Toys for ...,https://m.media-amazon.com/images/I/718Ch71CnY...,https://www.amazon.in//Toys-Kindergarten-Airpl...
207,BEGED Airplane Launcher Toy Catapult Plane Gun...,https://m.media-amazon.com/images/I/41MkJLg8d4...,https://www.amazon.in//dp/B0BH8W97C5/ref=sr_1_...
208,INAYAT Flying Airplane Launcher Gun Toy with F...,https://m.media-amazon.com/images/I/71j0MYAoLL...,https://www.amazon.in//INAYAT-Airplane-Launche...


In [20]:
AE = AE.append(df).reset_index(drop=True)
AE

,product_name,product_img,product_link
0,SUPER TOY Battery Operated Aeroplane Toy for K...,https://m.media-amazon.com/images/I/615SgNTD0P...,https://www.amazon.in//SUPER-TOY-Battery-Opera...
1,NHR Exceed Helicopter Remote Control & Recharg...,https://m.media-amazon.com/images/I/61SqTcdLUg...,https://www.amazon.in//NHR-Helicopter-Recharge...
2,LEGO 42117 Technic Race Plane Toy to Jet Aerop...,https://m.media-amazon.com/images/I/81X4QrwM6z...,https://www.amazon.in//LEGO-Plane-Building-Blo...
3,Miniature Mart Small Size Indian Fighter Jet A...,https://m.media-amazon.com/images/I/81DB-Q4Sfq...,https://www.amazon.in//Miniature-Mart-Fighter-...
4,Brand Conquer Airplane Launcher Toy Catapult A...,https://m.media-amazon.com/images/I/7168fs-rP-...,https://www.amazon.in//Brand-Conquer-Airplane-...
...,...,...,...
815,Meri Shopp Weather Vane Yard Wind Direction In...,https://m.media-amazon.com/images/I/61DQpmYQCF...,https://www.amazon.in//Weather-Direction-Indic...
816,Meri Shopp Weather Vane Farm Weather Vane Wind...,https://m.media-amazon.com/images/I/71x0S8vCI9...,https://www.amazon.in//Shopp-Weather-Direction...
817,CALANDIS Weather Vane Farm Bracket Weathervane...,https://m.media-amazon.com/images/I/51J8tXz9vW...,https://www.amazon.in//CALANDIS-Weather-Bracke...
818,Black Stainless Steel Flying Witch Weathervane...,https://m.media-amazon.com/images/I/71leu3XDEz...,https://www.amazon.in//Black-Stainless-Flying-...


In [104]:
product_AE = pd.DataFrame(reviews[reviews['product_code']=='product_AE']).reset_index(drop=True)
product_AE

,user_id,product_id,Ratings,Time_stamp,product_code
0,A39ZGVZRR82W7X,B00000JIAE,5.0,957139200,product_AE
1,A354AZBXGBO1LN,B00000JIAE,5.0,944524800,product_AE
2,A1BFAX6KXKS9TU,B00000JIAE,1.0,947721600,product_AE
3,A2ADBSV32ZMKLZ,B00000K4AE,4.0,971222400,product_AE
4,AN3E25IO3EA8D,B00001P4AE,5.0,958867200,product_AE
...,...,...,...,...,...
11091,A33CYQGWRHZSU6,B00K9SYPAE,5.0,1403049600,product_AE
11092,A2P7XNA1CTIUNG,B00KA7NGAE,1.0,1404345600,product_AE
11093,A2U6UHKH3FA8WG,B00KIQITAE,5.0,1405987200,product_AE
11094,A3KHMGYZEUARB8,B00KIQITAE,5.0,1404086400,product_AE


In [105]:
len(product_AE.product_id.unique())

722

In [21]:
product_AE = pd.DataFrame(reviews[reviews['product_code']=='product_AE']).reset_index(drop=True)
product_AE =  pd.merge(product_AE, 
                       AE.iloc[:len(product_AE.product_id.unique())]\
                       .join(pd.Series(product_AE['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AE.to_csv('product_AE.csv',index=False)
pd.read_csv('product_AE.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A39ZGVZRR82W7X,B00000JIAE,5.0,957139200,product_AE,SUPER TOY Battery Operated Aeroplane Toy for K...,https://m.media-amazon.com/images/I/615SgNTD0P...,https://www.amazon.in//SUPER-TOY-Battery-Opera...
1,A354AZBXGBO1LN,B00000JIAE,5.0,944524800,product_AE,SUPER TOY Battery Operated Aeroplane Toy for K...,https://m.media-amazon.com/images/I/615SgNTD0P...,https://www.amazon.in//SUPER-TOY-Battery-Opera...
2,A1BFAX6KXKS9TU,B00000JIAE,1.0,947721600,product_AE,SUPER TOY Battery Operated Aeroplane Toy for K...,https://m.media-amazon.com/images/I/615SgNTD0P...,https://www.amazon.in//SUPER-TOY-Battery-Opera...
3,A2ADBSV32ZMKLZ,B00000K4AE,4.0,971222400,product_AE,NHR Exceed Helicopter Remote Control & Recharg...,https://m.media-amazon.com/images/I/61SqTcdLUg...,https://www.amazon.in//NHR-Helicopter-Recharge...
4,AN3E25IO3EA8D,B00001P4AE,5.0,958867200,product_AE,LEGO 42117 Technic Race Plane Toy to Jet Aerop...,https://m.media-amazon.com/images/I/81X4QrwM6z...,https://www.amazon.in//LEGO-Plane-Building-Blo...
...,...,...,...,...,...,...,...,...
11091,A33CYQGWRHZSU6,B00K9SYPAE,5.0,1403049600,product_AE,NEELEWALA Wooden Wall Clock for Home Stylish L...,https://m.media-amazon.com/images/I/616oa1G7TF...,https://www.amazon.in//NEELEWALA-Stylish-Decor...
11092,A2P7XNA1CTIUNG,B00KA7NGAE,1.0,1404345600,product_AE,EXPORA Retro Stainless Steel Cock Weathervane ...,https://m.media-amazon.com/images/I/51I9INZCD6...,https://www.amazon.in//EXPORA-Retro-Stainless-...
11093,A2U6UHKH3FA8WG,B00KIQITAE,5.0,1405987200,product_AE,XENOTY Twin Bell Edition Vintage Look Metal Ta...,https://m.media-amazon.com/images/I/41zQFjd78x...,https://www.amazon.in//XENOTY-Vintage-Display-...
11094,A3KHMGYZEUARB8,B00KIQITAE,5.0,1404086400,product_AE,XENOTY Twin Bell Edition Vintage Look Metal Ta...,https://m.media-amazon.com/images/I/41zQFjd78x...,https://www.amazon.in//XENOTY-Vintage-Display-...


### Product AF dataframe

In [27]:
df = get_scraped('air fryer')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"PHILIPS Air Fryer HD9200/90, uses up to 90% le...",https://m.media-amazon.com/images/I/614TSm1Lck...,https://www.amazon.in//PHILIPS-Fryer-HD9200-90...
1,PHILIPS Digital Air Fryer HD9252/90 with Touch...,https://m.media-amazon.com/images/I/611y+GGr+t...,https://www.amazon.in//PHILIPS-Digital-HD9252-...
2,"INALSA Air Fryer Digital Nutri Fry-1400W 4L,Sm...",https://m.media-amazon.com/images/I/61ocJM6XH0...,https://www.amazon.in//Inalsa-Digital-Fryer-Nu...
3,KENT 16096 Classic Hot Air Fryer 4L 1300 W | 8...,https://m.media-amazon.com/images/I/61QfWLiC6N...,https://www.amazon.in//KENT-1300W-Classic-Frye...
4,Inalsa Air Fryer Fry-Light-1400W with 4.2 Lite...,https://m.media-amazon.com/images/I/61G-Ty5q+6...,https://www.amazon.in//Inalsa-Fryer-Light-1400...
...,...,...,...
301,Air Fryer 4.2Qt,https://m.media-amazon.com/images/I/7197vrdO6t...,https://www.amazon.in//Presto-03421-Air-Fryer-...
302,GoWISE USA GW22622 2nd Generation Electric Air...,https://m.media-amazon.com/images/I/71PX0ryaSC...,https://www.amazon.in//GoWISE-USA-GW22622-Gene...
303,Cozyna Air Fryer for Gowise Phillips - Set of 5,https://m.media-amazon.com/images/I/91KaIert0J...,https://www.amazon.in//Fryer-Accessories-Gowis...
304,KATEMN Air Fryer replacement Grill Pan 5QT Air...,https://m.media-amazon.com/images/I/715tXRQPFo...,https://www.amazon.in//KATEMN-replacement-Acce...


In [28]:
AF = df.copy()
AF

,product_name,product_img,product_link
0,"PHILIPS Air Fryer HD9200/90, uses up to 90% le...",https://m.media-amazon.com/images/I/614TSm1Lck...,https://www.amazon.in//PHILIPS-Fryer-HD9200-90...
1,PHILIPS Digital Air Fryer HD9252/90 with Touch...,https://m.media-amazon.com/images/I/611y+GGr+t...,https://www.amazon.in//PHILIPS-Digital-HD9252-...
2,"INALSA Air Fryer Digital Nutri Fry-1400W 4L,Sm...",https://m.media-amazon.com/images/I/61ocJM6XH0...,https://www.amazon.in//Inalsa-Digital-Fryer-Nu...
3,KENT 16096 Classic Hot Air Fryer 4L 1300 W | 8...,https://m.media-amazon.com/images/I/61QfWLiC6N...,https://www.amazon.in//KENT-1300W-Classic-Frye...
4,Inalsa Air Fryer Fry-Light-1400W with 4.2 Lite...,https://m.media-amazon.com/images/I/61G-Ty5q+6...,https://www.amazon.in//Inalsa-Fryer-Light-1400...
...,...,...,...
301,Air Fryer 4.2Qt,https://m.media-amazon.com/images/I/7197vrdO6t...,https://www.amazon.in//Presto-03421-Air-Fryer-...
302,GoWISE USA GW22622 2nd Generation Electric Air...,https://m.media-amazon.com/images/I/71PX0ryaSC...,https://www.amazon.in//GoWISE-USA-GW22622-Gene...
303,Cozyna Air Fryer for Gowise Phillips - Set of 5,https://m.media-amazon.com/images/I/91KaIert0J...,https://www.amazon.in//Fryer-Accessories-Gowis...
304,KATEMN Air Fryer replacement Grill Pan 5QT Air...,https://m.media-amazon.com/images/I/715tXRQPFo...,https://www.amazon.in//KATEMN-replacement-Acce...


In [ ]:
# AF = AF.append(df).reset_index(drop=True)
# AF

In [25]:
product_AF = pd.DataFrame(reviews[reviews['product_code']=='product_AF']).reset_index(drop=True)
product_AF

,user_id,product_id,Ratings,Time_stamp,product_code
0,A9SZARXOAPOZR,B00003CWAF,1.0,1155254400,product_AF
1,A1YXI1QYUODR2F,B00004ZCAF,4.0,1270166400,product_AF
2,A6R6FRCY9C2CT,B00004ZDAF,1.0,1031788800,product_AF
3,AU4UPU4VRD9VQ,B00005QFAF,5.0,1255046400,product_AF
4,A2OWIB1MOZ7IR0,B00005QFAF,5.0,1181347200,product_AF
...,...,...,...,...,...
86,A1TEO38SV1GMMC,B0000A0VAF,2.0,1110499200,product_AF
87,A1L2YPLXTMOQSX,B0000A0VAF,5.0,1108166400,product_AF
88,AUS483J3OB1LV,B0000A0VAF,5.0,1090972800,product_AF
89,A14SBWO7UXP6QB,B0000AQJAF,3.0,1258934400,product_AF


In [26]:
len(product_AF.product_id.unique())

15

In [29]:
product_AF = pd.DataFrame(reviews[reviews['product_code']=='product_AF']).reset_index(drop=True)
product_AF =  pd.merge(product_AF, 
                       AF.iloc[:len(product_AF.product_id.unique())]\
                       .join(pd.Series(product_AF['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AF.to_csv('product_AF.csv',index=False)
pd.read_csv('product_AF.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A9SZARXOAPOZR,B00003CWAF,1.0,1155254400,product_AF,"PHILIPS Air Fryer HD9200/90, uses up to 90% le...",https://m.media-amazon.com/images/I/614TSm1Lck...,https://www.amazon.in//PHILIPS-Fryer-HD9200-90...
1,A1YXI1QYUODR2F,B00004ZCAF,4.0,1270166400,product_AF,PHILIPS Digital Air Fryer HD9252/90 with Touch...,https://m.media-amazon.com/images/I/611y+GGr+t...,https://www.amazon.in//PHILIPS-Digital-HD9252-...
2,A6R6FRCY9C2CT,B00004ZDAF,1.0,1031788800,product_AF,"INALSA Air Fryer Digital Nutri Fry-1400W 4L,Sm...",https://m.media-amazon.com/images/I/61ocJM6XH0...,https://www.amazon.in//Inalsa-Digital-Fryer-Nu...
3,AU4UPU4VRD9VQ,B00005QFAF,5.0,1255046400,product_AF,KENT 16096 Classic Hot Air Fryer 4L 1300 W | 8...,https://m.media-amazon.com/images/I/61QfWLiC6N...,https://www.amazon.in//KENT-1300W-Classic-Frye...
4,A2OWIB1MOZ7IR0,B00005QFAF,5.0,1181347200,product_AF,KENT 16096 Classic Hot Air Fryer 4L 1300 W | 8...,https://m.media-amazon.com/images/I/61QfWLiC6N...,https://www.amazon.in//KENT-1300W-Classic-Frye...
...,...,...,...,...,...,...,...,...
86,A1TEO38SV1GMMC,B0000A0VAF,2.0,1110499200,product_AF,INALSA Mellerware Air Fryer Digital 4L MWAF 01...,https://m.media-amazon.com/images/I/519xO62Uml...,https://www.amazon.in//Mellerware-MWAF-AirCris...
87,A1L2YPLXTMOQSX,B0000A0VAF,5.0,1108166400,product_AF,INALSA Mellerware Air Fryer Digital 4L MWAF 01...,https://m.media-amazon.com/images/I/519xO62Uml...,https://www.amazon.in//Mellerware-MWAF-AirCris...
88,AUS483J3OB1LV,B0000A0VAF,5.0,1090972800,product_AF,INALSA Mellerware Air Fryer Digital 4L MWAF 01...,https://m.media-amazon.com/images/I/519xO62Uml...,https://www.amazon.in//Mellerware-MWAF-AirCris...
89,A14SBWO7UXP6QB,B0000AQJAF,3.0,1258934400,product_AF,Philips Essential Air Fryer HD9252/70 with Rap...,https://m.media-amazon.com/images/I/41OWI-Hq9o...,https://www.amazon.in//Philips-Essential-HD925...


### Product AG dataframe

In [38]:
df = get_scraped('game of thrones t shirts')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Dracarys - Game of Thrones Pure Cotton Unisex-...,https://m.media-amazon.com/images/I/71KJmOsm3K...,https://www.amazon.in//VoiletR-Dracarys-Throne...
1,GOT Game of Thrones Dragon and Wolf T-Shirt - ...,https://m.media-amazon.com/images/I/61sZfOM+Ov...,https://www.amazon.in//Indirage-Game-Thrones-D...
2,GOT (Game of Thrones) Targaryen Black Cotton T...,https://m.media-amazon.com/images/I/51bLq14lB3...,https://www.amazon.in//Quote-Marshals-Thrones-...
3,GOT (Game of Thrones) I Know Things Black Cott...,https://m.media-amazon.com/images/I/51PhncP+-D...,https://www.amazon.in//Quote-Marshals-Thrones-...
4,Men's Regular Fit T-Shirt,https://m.media-amazon.com/images/I/512osGrh9m...,https://www.amazon.in//Quote-Marshals-Thrones-...
...,...,...,...
301,Half Sleeve Premium Printed Cotton Music T-Shi...,https://m.media-amazon.com/images/I/618Hb+lMR3...,https://www.amazon.in//Iron-Maiden-Tshirt-Tshi...
302,"Men's Regular Fit, Round Neck Shoulder Print T...",https://m.media-amazon.com/images/I/71mx34X06Z...,https://www.amazon.in//KRA-Regular-Round-Shoul...
303,Dragon Stone Varsity Crop Top (Burgundy),https://m.media-amazon.com/images/I/71g8ySCfPC...,https://www.amazon.in//Shop-Arena-Dragon-Varsi...
304,Jasan Ladakh Flag T-Shirt for Men | Full Sleev...,https://m.media-amazon.com/images/I/61FT8iwrb8...,https://www.amazon.in//Buddhist-Tibetan-T-Shir...


In [35]:
AG = df.copy()
AG

,product_name,product_img,product_link
0,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
1,A Song of Ice and Fire - A Game of Thrones: Th...,https://m.media-amazon.com/images/I/71nK5FjjYV...,https://www.amazon.in//Song-Ice-Fire-Thrones-C...
2,A Game of Thrones: Book 1 (A Song of Ice and F...,https://m.media-amazon.com/images/I/51FnddC2V+...,https://www.amazon.in//Game-Thrones-Song-Ice-F...
3,George R. R. Martin's A Game of Thrones Leathe...,https://m.media-amazon.com/images/I/A1deQNY7L9...,https://www.amazon.in//George-Martins-Thrones-...
4,Fire and Blood: The inspiration for 2022's hig...,https://m.media-amazon.com/images/I/91HQMcu2BN...,https://www.amazon.in//Fire-Blood-Thrones-Targ...
...,...,...,...
301,The Empire's Ruin (Ashes of the Unhewn Throne ...,https://m.media-amazon.com/images/I/81I20aNg8j...,https://www.amazon.in//Empires-Ruin-Ashes-Unhe...
302,Forge of the High Mage,https://m.media-amazon.com/images/I/81WWdb8c92...,https://www.amazon.in//Forge-High-Mage-Ian-Ess...
303,ஷெர்லக் ஹோம்ஸின் சாகசங்கள் - முதலாம் பாகம் (Ta...,https://m.media-amazon.com/images/I/915lwwE-GL...,https://www.amazon.in//%E0%AE%B7%E0%AF%86%E0%A...
304,Game Of Thrones: The Complete Game Of Thrones ...,https://m.media-amazon.com/images/I/91dO3IG6xS...,https://www.amazon.in//Game-Thrones-Complete-C...


In [39]:
AG = AG.append(df).reset_index(drop=True)
AG

,product_name,product_img,product_link
0,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
1,A Song of Ice and Fire - A Game of Thrones: Th...,https://m.media-amazon.com/images/I/71nK5FjjYV...,https://www.amazon.in//Song-Ice-Fire-Thrones-C...
2,A Game of Thrones: Book 1 (A Song of Ice and F...,https://m.media-amazon.com/images/I/51FnddC2V+...,https://www.amazon.in//Game-Thrones-Song-Ice-F...
3,George R. R. Martin's A Game of Thrones Leathe...,https://m.media-amazon.com/images/I/A1deQNY7L9...,https://www.amazon.in//George-Martins-Thrones-...
4,Fire and Blood: The inspiration for 2022's hig...,https://m.media-amazon.com/images/I/91HQMcu2BN...,https://www.amazon.in//Fire-Blood-Thrones-Targ...
...,...,...,...
913,Half Sleeve Premium Printed Cotton Music T-Shi...,https://m.media-amazon.com/images/I/618Hb+lMR3...,https://www.amazon.in//Iron-Maiden-Tshirt-Tshi...
914,"Men's Regular Fit, Round Neck Shoulder Print T...",https://m.media-amazon.com/images/I/71mx34X06Z...,https://www.amazon.in//KRA-Regular-Round-Shoul...
915,Dragon Stone Varsity Crop Top (Burgundy),https://m.media-amazon.com/images/I/71g8ySCfPC...,https://www.amazon.in//Shop-Arena-Dragon-Varsi...
916,Jasan Ladakh Flag T-Shirt for Men | Full Sleev...,https://m.media-amazon.com/images/I/61FT8iwrb8...,https://www.amazon.in//Buddhist-Tibetan-T-Shir...


In [32]:
product_AG = pd.DataFrame(reviews[reviews['product_code']=='product_AG']).reset_index(drop=True)
product_AG

,user_id,product_id,Ratings,Time_stamp,product_code
0,A23LF95HUEIGUR,B00000J1AG,5.0,964828800,product_AG
1,ADEU4RN9QJA0,B00000J1AG,3.0,1067990400,product_AG
2,A1EBQ7E4PZFQQM,B00000J1AG,3.0,1016496000,product_AG
3,A3L9Z3K1H3GYX9,B00000J1AG,4.0,978307200,product_AG
4,A2RLX57YIJCABR,B00000J1AG,5.0,1002931200,product_AG
...,...,...,...,...,...
8142,A1AN5H3BFUQ0MT,B00KEIGLAG,1.0,1402704000,product_AG
8143,A26TR8IH33SG9B,B00KKKOAAG,5.0,1402444800,product_AG
8144,A30V0DX4SPO1TR,B00KKKOAAG,3.0,1404691200,product_AG
8145,A3G244MOACV9UJ,B00KKKOAAG,3.0,1402617600,product_AG


In [33]:
len(product_AG.product_id.unique())

707

In [40]:
product_AG = pd.DataFrame(reviews[reviews['product_code']=='product_AG']).reset_index(drop=True)
product_AG =  pd.merge(product_AG, 
                       AG.iloc[:len(product_AG.product_id.unique())]\
                       .join(pd.Series(product_AG['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AG.to_csv('product_AG.csv',index=False)
pd.read_csv('product_AG.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A23LF95HUEIGUR,B00000J1AG,5.0,964828800,product_AG,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
1,ADEU4RN9QJA0,B00000J1AG,3.0,1067990400,product_AG,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
2,A1EBQ7E4PZFQQM,B00000J1AG,3.0,1016496000,product_AG,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
3,A3L9Z3K1H3GYX9,B00000J1AG,4.0,978307200,product_AG,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
4,A2RLX57YIJCABR,B00000J1AG,5.0,1002931200,product_AG,A Game of Thrones (Reissue): Book 1 (A Song of...,https://m.media-amazon.com/images/I/81YdYcNyY-...,https://www.amazon.in//Game-Thrones-Reissue-So...
...,...,...,...,...,...,...,...,...
8142,A1AN5H3BFUQ0MT,B00KEIGLAG,1.0,1402704000,product_AG,"Tie N Dye Sky blue T-Shirt (Unisex), 100% Cott...",https://m.media-amazon.com/images/I/61r8ZtjXu7...,https://www.amazon.in//Skream-T-Shirt-HANDMADE...
8143,A26TR8IH33SG9B,B00KKKOAAG,5.0,1402444800,product_AG,Womens Game of Thrones Slim Fit Tshirt,https://m.media-amazon.com/images/I/719Wa6Istw...,https://www.amazon.in//Status-Quo-Womens-Thron...
8144,A30V0DX4SPO1TR,B00KKKOAAG,3.0,1404691200,product_AG,Womens Game of Thrones Slim Fit Tshirt,https://m.media-amazon.com/images/I/719Wa6Istw...,https://www.amazon.in//Status-Quo-Womens-Thron...
8145,A3G244MOACV9UJ,B00KKKOAAG,3.0,1402617600,product_AG,Womens Game of Thrones Slim Fit Tshirt,https://m.media-amazon.com/images/I/719Wa6Istw...,https://www.amazon.in//Status-Quo-Womens-Thron...


### Product AH dataframe

In [44]:
df = get_scraped('air horn for car')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
1,Generic One Stop Auto Mobiles Roots Windtone C...,https://m.media-amazon.com/images/I/31CK+XDeF0...,https://www.amazon.in//Generic-Mobiles-Windton...
2,Oshotto Stebel Nautilus Pressure 3 Pipe Twin A...,https://m.media-amazon.com/images/I/71PqlNeqW6...,https://www.amazon.in//Oshotto-Stebel-Nautilus...
3,Automaze Air Horn 12V 150DB Super Loud Train H...,https://m.media-amazon.com/images/I/61O5Jd8kG2...,https://www.amazon.in//Automaze-Compressor-Com...
4,Hella 013410001 Chrome 12V Air 1-Trumpet Horn ...,https://m.media-amazon.com/images/I/71aTWvMeSr...,https://www.amazon.in//HELLA-013410001-Chrome-...
...,...,...,...
301,"Ironctic Dual Trumpet Air Horn, 12V 150db Supe...",https://m.media-amazon.com/images/I/51mNDE89v3...,https://www.amazon.in//Trumpet-150db-Super-Com...
302,Vagary Car/Bikes Air Horn with 12 Volt for Fia...,https://m.media-amazon.com/images/I/61-8MhS7Z3...,https://www.amazon.in//Vagary-Bikes-Horn-Volt-...
303,Negaor 12V Single Trumpet Horn Marine Boat Air...,https://m.media-amazon.com/images/I/51JMDah1PX...,https://www.amazon.in//Negaor-Single-Trumpet-M...
304,Vocado 17609 Red 3 Pipe Car Air Pressure Horn ...,https://m.media-amazon.com/images/I/51+HCP98zy...,https://www.amazon.in//Vocado-Pressure-Vocado%...


In [45]:
AH = df.copy()
AH

,product_name,product_img,product_link
0,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
1,Generic One Stop Auto Mobiles Roots Windtone C...,https://m.media-amazon.com/images/I/31CK+XDeF0...,https://www.amazon.in//Generic-Mobiles-Windton...
2,Oshotto Stebel Nautilus Pressure 3 Pipe Twin A...,https://m.media-amazon.com/images/I/71PqlNeqW6...,https://www.amazon.in//Oshotto-Stebel-Nautilus...
3,Automaze Air Horn 12V 150DB Super Loud Train H...,https://m.media-amazon.com/images/I/61O5Jd8kG2...,https://www.amazon.in//Automaze-Compressor-Com...
4,Hella 013410001 Chrome 12V Air 1-Trumpet Horn ...,https://m.media-amazon.com/images/I/71aTWvMeSr...,https://www.amazon.in//HELLA-013410001-Chrome-...
...,...,...,...
301,"Ironctic Dual Trumpet Air Horn, 12V 150db Supe...",https://m.media-amazon.com/images/I/51mNDE89v3...,https://www.amazon.in//Trumpet-150db-Super-Com...
302,Vagary Car/Bikes Air Horn with 12 Volt for Fia...,https://m.media-amazon.com/images/I/61-8MhS7Z3...,https://www.amazon.in//Vagary-Bikes-Horn-Volt-...
303,Negaor 12V Single Trumpet Horn Marine Boat Air...,https://m.media-amazon.com/images/I/51JMDah1PX...,https://www.amazon.in//Negaor-Single-Trumpet-M...
304,Vocado 17609 Red 3 Pipe Car Air Pressure Horn ...,https://m.media-amazon.com/images/I/51+HCP98zy...,https://www.amazon.in//Vocado-Pressure-Vocado%...


In [ ]:
# AH = AH.append(df).reset_index(drop=True)
# AH

In [42]:
product_AH = pd.DataFrame(reviews[reviews['product_code']=='product_AH']).reset_index(drop=True)
product_AH

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1UWUT1KX2I16U,B00000JBAH,2.0,960595200,product_AH
1,A38X9XX2ZRR0ON,B00000JBAH,3.0,1162339200,product_AH
2,A386UKXMKYS1I9,B00000JBAH,5.0,952560000,product_AH
3,AS0KHBH235SZ7,B00000JBAH,5.0,950227200,product_AH
4,A1DS1VQA47ACCC,B00000JBAH,5.0,1092873600,product_AH
...,...,...,...,...,...
76,A1414YWW0VUQCJ,B0000AE6AH,5.0,1345507200,product_AH
77,A1QTICTBQKS6I6,B0000AE6AH,5.0,1291766400,product_AH
78,A7QMQBGJ2TCQG,B0000AE6AH,5.0,1213833600,product_AH
79,A3G41J1GBVI8KG,B0000AE6AH,3.0,1363564800,product_AH


In [43]:
len(product_AH.product_id.unique())

14

In [46]:
product_AH = pd.DataFrame(reviews[reviews['product_code']=='product_AH']).reset_index(drop=True)
product_AH =  pd.merge(product_AH, 
                       AH.iloc[:len(product_AH.product_id.unique())]\
                       .join(pd.Series(product_AH['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AH.to_csv('product_AH.csv',index=False)
pd.read_csv('product_AH.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1UWUT1KX2I16U,B00000JBAH,2.0,960595200,product_AH,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
1,A38X9XX2ZRR0ON,B00000JBAH,3.0,1162339200,product_AH,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
2,A386UKXMKYS1I9,B00000JBAH,5.0,952560000,product_AH,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
3,AS0KHBH235SZ7,B00000JBAH,5.0,950227200,product_AH,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
4,A1DS1VQA47ACCC,B00000JBAH,5.0,1092873600,product_AH,"Hella 003001681 Multi-Tone Air Horn (12V,580/7...",https://m.media-amazon.com/images/I/41nf1fXev4...,https://www.amazon.in//Hella-003001681-Multi-T...
...,...,...,...,...,...,...,...,...
76,A1414YWW0VUQCJ,B0000AE6AH,5.0,1345507200,product_AH,"Bosch F002H10028 Symphony Horn -Set of 2 (12V,...",https://m.media-amazon.com/images/I/516ox4lXxm...,https://www.amazon.in//Bosch-F002H10028-Sympho...
77,A1QTICTBQKS6I6,B0000AE6AH,5.0,1291766400,product_AH,"Bosch F002H10028 Symphony Horn -Set of 2 (12V,...",https://m.media-amazon.com/images/I/516ox4lXxm...,https://www.amazon.in//Bosch-F002H10028-Sympho...
78,A7QMQBGJ2TCQG,B0000AE6AH,5.0,1213833600,product_AH,"Bosch F002H10028 Symphony Horn -Set of 2 (12V,...",https://m.media-amazon.com/images/I/516ox4lXxm...,https://www.amazon.in//Bosch-F002H10028-Sympho...
79,A3G41J1GBVI8KG,B0000AE6AH,3.0,1363564800,product_AH,"Bosch F002H10028 Symphony Horn -Set of 2 (12V,...",https://m.media-amazon.com/images/I/516ox4lXxm...,https://www.amazon.in//Bosch-F002H10028-Sympho...


### Product AI dataframe

In [53]:
df = get_scraped('air inflator for balloon')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Party Propz New Two Nozzles High Power Electri...,https://m.media-amazon.com/images/I/41RVls2ZTM...,https://www.amazon.in//Party-Propz-Nozzles-Ele...
1,Balloonistics Latex Double Action Manual Ballo...,https://m.media-amazon.com/images/I/51SYKXcUsC...,https://www.amazon.in//Balloonistics-Double-Ba...
2,PartyToko Manual Party Balloon Inflator Plasti...,https://m.media-amazon.com/images/I/51SYKXcUsC...,https://www.amazon.in//PartyToko-Balloon-Infla...
3,Party Hour High Power Electric Balloon Machine...,https://m.media-amazon.com/images/I/61TJUlERVJ...,https://www.amazon.in//Party-Hour-Electric-Inf...
4,Smartcraft Electric Balloon Pump New Two Nozzl...,https://m.media-amazon.com/images/I/41QNMbR0x8...,https://www.amazon.in//Smartcraft-Nozzles-Elec...
...,...,...,...
301,TOYANDONA 2Pcs Hand Held 2- Way Balloon Pump B...,https://m.media-amazon.com/images/I/518AX-g46J...,https://www.amazon.in//TOYANDONA-Balloon-Infla...
302,WorldCare® Mini Plastic Hand Air Pump inflator...,https://m.media-amazon.com/images/I/31IDVAipT2...,https://www.amazon.in//WorldCare%C2%AE-Plastic...
303,Surprises Planner Manual Hand Pump for Rubber ...,https://m.media-amazon.com/images/I/51dfawm1-5...,https://www.amazon.in//Surprises-Planner-Ballo...
304,Advik Enterprises Birthday Latex Balloon Pump ...,https://m.media-amazon.com/images/I/411ojxUeLZ...,https://www.amazon.in//Enterprises-Birthday-Ba...


In [50]:
AI = df.copy()
AI

,product_name,product_img,product_link
0,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
1,Woscher Rapid Performance Portable Tyre Inflat...,https://m.media-amazon.com/images/I/41VsyQQ0Lp...,https://www.amazon.in//Woscher-801-Performance...
2,ShreNik Bicycle Cycle Air Pump for Car Bike In...,https://m.media-amazon.com/images/I/51iIBcbK6W...,https://www.amazon.in//ShreNik-Inflatable-Prod...
3,GoMechanic Gusto T10 Digital Tyre Inflator - 1...,https://m.media-amazon.com/images/I/71fbTkxyC-...,https://www.amazon.in//GoMechanic-Gusto-Digita...
4,Amardeep cycles Rockbros Road Bike Tire Ball I...,https://m.media-amazon.com/images/I/51r8Z7x6id...,https://www.amazon.in//Amardeep-cycles-Rockbro...
...,...,...,...
205,12V Portable Electric Air Compressor for Car a...,https://m.media-amazon.com/images/I/415UxOab2L...,https://www.amazon.in//Portable-Electric-Compr...
206,12V Portable Electric Air Compressor for Car a...,https://m.media-amazon.com/images/I/415UxOab2L...,https://www.amazon.in//Portable-Electric-Compr...
207,KP Cycle Pump Steel Bicycle Air Pump Tyre Infl...,https://m.media-amazon.com/images/I/31jk+Q7LUN...,https://www.amazon.in//Cycle-Bicycle-Inflator-...
208,TYAGY Heavy Duty Double Cylinder High Pressure...,https://m.media-amazon.com/images/I/611PnHt13V...,https://www.amazon.in//TYAGY-Cylinder-Pressure...


In [54]:
AI = AI.append(df).reset_index(drop=True)
AI

,product_name,product_img,product_link
0,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
1,Woscher Rapid Performance Portable Tyre Inflat...,https://m.media-amazon.com/images/I/41VsyQQ0Lp...,https://www.amazon.in//Woscher-801-Performance...
2,ShreNik Bicycle Cycle Air Pump for Car Bike In...,https://m.media-amazon.com/images/I/51iIBcbK6W...,https://www.amazon.in//ShreNik-Inflatable-Prod...
3,GoMechanic Gusto T10 Digital Tyre Inflator - 1...,https://m.media-amazon.com/images/I/71fbTkxyC-...,https://www.amazon.in//GoMechanic-Gusto-Digita...
4,Amardeep cycles Rockbros Road Bike Tire Ball I...,https://m.media-amazon.com/images/I/51r8Z7x6id...,https://www.amazon.in//Amardeep-cycles-Rockbro...
...,...,...,...
799,TOYANDONA 2Pcs Hand Held 2- Way Balloon Pump B...,https://m.media-amazon.com/images/I/518AX-g46J...,https://www.amazon.in//TOYANDONA-Balloon-Infla...
800,WorldCare® Mini Plastic Hand Air Pump inflator...,https://m.media-amazon.com/images/I/31IDVAipT2...,https://www.amazon.in//WorldCare%C2%AE-Plastic...
801,Surprises Planner Manual Hand Pump for Rubber ...,https://m.media-amazon.com/images/I/51dfawm1-5...,https://www.amazon.in//Surprises-Planner-Ballo...
802,Advik Enterprises Birthday Latex Balloon Pump ...,https://m.media-amazon.com/images/I/411ojxUeLZ...,https://www.amazon.in//Enterprises-Birthday-Ba...


In [47]:
product_AI = pd.DataFrame(reviews[reviews['product_code']=='product_AI']).reset_index(drop=True)
product_AI

,user_id,product_id,Ratings,Time_stamp,product_code
0,A23TJQM0SAINWU,B00001O3AI,4.0,1093132800,product_AI
1,A80B2XUD364YO,B00001O3AI,1.0,952041600,product_AI
2,AG5BMW328X5I4,B00001O3AI,4.0,951782400,product_AI
3,A2M2EUUPGQZOCW,B00001O3AI,5.0,949708800,product_AI
4,A1QZMBCO1MWLQ5,B00001O3AI,5.0,947203200,product_AI
...,...,...,...,...,...
10695,ABKDPXY1XB8HX,B00JLLDXAI,5.0,1405036800,product_AI
10696,A47XG1THZKL3H,B00JO3AVAI,1.0,1400457600,product_AI
10697,A3QV161NCAEGUI,B00L8A94AI,5.0,1405382400,product_AI
10698,A39MVSEKHD400M,B00L8A94AI,5.0,1403827200,product_AI


In [48]:
len(product_AI.product_id.unique())

723

In [55]:
product_AI = pd.DataFrame(reviews[reviews['product_code']=='product_AI']).reset_index(drop=True)
product_AI =  pd.merge(product_AI, 
                       AI.iloc[:len(product_AI.product_id.unique())]\
                       .join(pd.Series(product_AI['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AI.to_csv('product_AI.csv',index=False)
pd.read_csv('product_AI.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A23TJQM0SAINWU,B00001O3AI,4.0,1093132800,product_AI,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
1,A80B2XUD364YO,B00001O3AI,1.0,952041600,product_AI,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
2,AG5BMW328X5I4,B00001O3AI,4.0,951782400,product_AI,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
3,A2M2EUUPGQZOCW,B00001O3AI,5.0,949708800,product_AI,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
4,A1QZMBCO1MWLQ5,B00001O3AI,5.0,947203200,product_AI,Woscher 1610 Portable Mini Tyre Inflator with ...,https://m.media-amazon.com/images/I/41ejIw3r-C...,https://www.amazon.in//Woscher-1610-Portable-I...
...,...,...,...,...,...,...,...,...
10695,ABKDPXY1XB8HX,B00JLLDXAI,5.0,1405036800,product_AI,PIKABOO TOYS Balloon Inflator Pack of 3 | Hand...,https://m.media-amazon.com/images/I/51e85VjTqd...,https://www.amazon.in//PIKABOO-Balloon-Inflato...
10696,A47XG1THZKL3H,B00JO3AVAI,1.0,1400457600,product_AI,One-Stop-Shop Two Nozzles High Power Electric ...,https://m.media-amazon.com/images/I/411yUq5nht...,https://www.amazon.in//One-Stop-Shop-Nozzles-E...
10697,A3QV161NCAEGUI,B00L8A94AI,5.0,1405382400,product_AI,Twiclo High Electric high Balloon Pump 600W 11...,https://m.media-amazon.com/images/I/414bjdMh2O...,https://www.amazon.in//Twiclo-Electric-Balloon...
10698,A39MVSEKHD400M,B00L8A94AI,5.0,1403827200,product_AI,Twiclo High Electric high Balloon Pump 600W 11...,https://m.media-amazon.com/images/I/414bjdMh2O...,https://www.amazon.in//Twiclo-Electric-Balloon...


### Product AJ dataframe

In [59]:
df = get_scraped('arctic jackets')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
1,Male Denim Blue Jacket with Integrated Mask & ...,https://m.media-amazon.com/images/I/81tiwnocH-...,https://www.amazon.in//Arctic-Fox-Jacket-Integ...
2,Boys' Arctic Blast Jacket,https://m.media-amazon.com/images/I/61qOl5wHoP...,https://www.amazon.in//Columbia-Arctic-Jacket-...
3,Men's Arctic Trip II Interchange Jacket,https://m.media-amazon.com/images/I/7189RJmVGy...,https://www.amazon.in//Columbia-Arctic-Interch...
4,Men's Arctic Cloth Quilted Snorkel Bomber Remo...,https://m.media-amazon.com/images/I/91XqGJEP5W...,https://www.amazon.in//Tommy-Hilfiger-Quilted-...
...,...,...,...
119,Women Valentine's Day Gift Long Sleeve Nightwe...,https://m.media-amazon.com/images/I/71oNvH4Ukh...,https://www.amazon.in//Valentines-Nightwear-Ju...
120,"Valentine's Day Theme Pajamas, Women Fashion S...",https://m.media-amazon.com/images/I/61iIaqaZ27...,https://www.amazon.in//Valentines-Nightwear-Ju...
121,Women Warm Pajamas Long Sleeve Button Flap Nig...,https://m.media-amazon.com/images/I/61hQJdPtyK...,https://www.amazon.in//Pajamas-Nightwear-Jumps...
122,Women Sexy Fashion Print Long Sleeve Nightwear...,https://m.media-amazon.com/images/I/71q4GevKvY...,https://www.amazon.in//Fashion-Nightwear-Jumps...


In [60]:
AJ = df.copy()
AJ

,product_name,product_img,product_link
0,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
1,Male Denim Blue Jacket with Integrated Mask & ...,https://m.media-amazon.com/images/I/81tiwnocH-...,https://www.amazon.in//Arctic-Fox-Jacket-Integ...
2,Boys' Arctic Blast Jacket,https://m.media-amazon.com/images/I/61qOl5wHoP...,https://www.amazon.in//Columbia-Arctic-Jacket-...
3,Men's Arctic Trip II Interchange Jacket,https://m.media-amazon.com/images/I/7189RJmVGy...,https://www.amazon.in//Columbia-Arctic-Interch...
4,Men's Arctic Cloth Quilted Snorkel Bomber Remo...,https://m.media-amazon.com/images/I/91XqGJEP5W...,https://www.amazon.in//Tommy-Hilfiger-Quilted-...
...,...,...,...
119,Women Valentine's Day Gift Long Sleeve Nightwe...,https://m.media-amazon.com/images/I/71oNvH4Ukh...,https://www.amazon.in//Valentines-Nightwear-Ju...
120,"Valentine's Day Theme Pajamas, Women Fashion S...",https://m.media-amazon.com/images/I/61iIaqaZ27...,https://www.amazon.in//Valentines-Nightwear-Ju...
121,Women Warm Pajamas Long Sleeve Button Flap Nig...,https://m.media-amazon.com/images/I/61hQJdPtyK...,https://www.amazon.in//Pajamas-Nightwear-Jumps...
122,Women Sexy Fashion Print Long Sleeve Nightwear...,https://m.media-amazon.com/images/I/71q4GevKvY...,https://www.amazon.in//Fashion-Nightwear-Jumps...


In [ ]:
# AJ = AJ.append(df).reset_index(drop=True)
# AJ

In [57]:
product_AJ = pd.DataFrame(reviews[reviews['product_code']=='product_AJ']).reset_index(drop=True)
product_AJ

,user_id,product_id,Ratings,Time_stamp,product_code
0,A11ABKRLVEAG6C,B00001U0AJ,1.0,978393600,product_AJ
1,A2K8J2U579XQC4,B00001U0AJ,4.0,980467200,product_AJ
2,A2ZM9BGE3K3SY2,B00001U0AJ,4.0,1064707200,product_AJ
3,A2EZIGYJ638HXP,B00001U0AJ,3.0,999388800,product_AJ
4,A34L11KUIBQSVR,B00001U0AJ,5.0,981331200,product_AJ
...,...,...,...,...,...
69,A18FYH56MEM2AW,B00006BAAJ,4.0,1234569600,product_AJ
70,AOSQIDEPNXJ5Y,B00006BBAJ,1.0,1169510400,product_AJ
71,AFVHT733T7NE,B00009R8AJ,1.0,1252972800,product_AJ
72,A1W3IQRE4XDJAE,B0000A2UAJ,5.0,1064880000,product_AJ


In [58]:
len(product_AJ.product_id.unique())

10

In [61]:
product_AJ = pd.DataFrame(reviews[reviews['product_code']=='product_AJ']).reset_index(drop=True)
product_AJ =  pd.merge(product_AJ, 
                       AJ.iloc[:len(product_AJ.product_id.unique())]\
                       .join(pd.Series(product_AJ['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AJ.to_csv('product_AJ.csv',index=False)
pd.read_csv('product_AJ.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A11ABKRLVEAG6C,B00001U0AJ,1.0,978393600,product_AJ,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
1,A2K8J2U579XQC4,B00001U0AJ,4.0,980467200,product_AJ,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
2,A2ZM9BGE3K3SY2,B00001U0AJ,4.0,1064707200,product_AJ,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
3,A2EZIGYJ638HXP,B00001U0AJ,3.0,999388800,product_AJ,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
4,A34L11KUIBQSVR,B00001U0AJ,5.0,981331200,product_AJ,Men's Arctic Cloth Full Length Quilted Snorkel...,https://m.media-amazon.com/images/I/81hevJzd9y...,https://www.amazon.in//TOMMY-HILFIGER-Arctic-Q...
...,...,...,...,...,...,...,...,...
69,A18FYH56MEM2AW,B00006BAAJ,4.0,1234569600,product_AJ,Men's Arctic Cloth Sherpa Storm Jacket,https://m.media-amazon.com/images/I/91hawMtIQE...,https://www.amazon.in//Dockers-Arctic-Sherpa-J...
70,AOSQIDEPNXJ5Y,B00006BBAJ,1.0,1169510400,product_AJ,Men&#39;s Quilted Arctic Cloth Snap Front Snor...,https://m.media-amazon.com/images/I/91X-9GCtT+...,https://www.amazon.in//Tommy-Hilfiger-Quilted-...
71,AFVHT733T7NE,B00009R8AJ,1.0,1252972800,product_AJ,Men&#39;s Quilted Arctic Cloth Snap Front Snor...,https://m.media-amazon.com/images/I/81qWBbhrPV...,https://www.amazon.in//Tommy-Hilfiger-Quilted-...
72,A1W3IQRE4XDJAE,B0000A2UAJ,5.0,1064880000,product_AJ,Boys & Girls Winter Waterproof Snow Ski Jacket,https://m.media-amazon.com/images/I/71+orJ4PKa...,https://www.amazon.in//Arctic-Paw-Hooded-Comfo...


### Product AK dataframe

In [93]:
df = get_scraped('anime keychain sharingan')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Naruto Anime Red Uchiha Sharingan Design Revol...,https://m.media-amazon.com/images/I/31XzUGwng1...,https://www.amazon.in//Techpro-Naruto-Sharinga...
1,Naruto Anime Red Uchiha Sharingan Design Revol...,https://m.media-amazon.com/images/I/31vsjDxk3K...,https://www.amazon.in//Techpro-Naruto-Sharinga...
2,Naruto Anime Red Uchiha Sharingan Design Revol...,https://m.media-amazon.com/images/I/310TnD-v8t...,https://www.amazon.in//Techpro-Avenger-Super-N...
3,Naruto Anime Copy Ninja Mangekyou Sharingan Ke...,https://m.media-amazon.com/images/I/417ATuh8OE...,https://www.amazon.in//Naruto-Anime-Mangekyou-...
4,Naruto Anime Red Uchiha Sharingan Design Revol...,https://m.media-amazon.com/images/I/41rNf-DCCM...,https://www.amazon.in//Techpro-Naruto-Sharinga...
...,...,...,...
56,Naruto Anime Village Hidden in Leaves Keychain,https://m.media-amazon.com/images/I/51TcnHaTs1...,https://www.amazon.in//Naruto-Village-Hidden-L...
57,Naruto Uzumaki Face Keychain Metal 3D | Kakash...,https://m.media-amazon.com/images/I/41N6vbs7mO...,https://www.amazon.in//Naruto-Uzumaki-Keychain...
58,Naruto Anime Yellow Flash 3 Weapon Keychain Co...,https://m.media-amazon.com/images/I/5115VWSYAd...,https://www.amazon.in//Naruto-Yellow-Keychain-...
59,Naruto Keychain,https://m.media-amazon.com/images/I/619Hf3cuMW...,https://www.amazon.in//Arkanum-Naruto-Figurine...


In [79]:
AK = df.copy()
AK

,product_name,product_img,product_link
0,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
1,Demon Slayer Anime Zenitsu Kimono Lanyard Keyc...,https://m.media-amazon.com/images/I/7160WR1QrE...,https://www.amazon.in//ComicSense-xyz-Slayer-Z...
2,Blue Aura Demon Slayer Tanjiro Keychain for An...,https://m.media-amazon.com/images/I/41pX5ctdy9...,https://www.amazon.in//Blue-Aura-Tanjiro-Keych...
3,Blue Aura Demon Slayer Zenitsu Keychain for An...,https://m.media-amazon.com/images/I/41soA5gXJP...,https://www.amazon.in//Blue-Aura-Zenitsu-Keych...
4,Blue Aura Demon Slayer Single Sided Metal Keyc...,https://m.media-amazon.com/images/I/31ru6Yail1...,https://www.amazon.in//Blue-Aura-Slayer-Keycha...
...,...,...,...
301,"Anime Phone Cover, Tanjiro Phone Case, Anime P...",https://m.media-amazon.com/images/I/51tXrMZAs4...,https://www.amazon.in//Slayer-iPhone-Tanjirou-...
302,"Anime Phone Cover, Zenitsu Phone Case, Anime P...",https://m.media-amazon.com/images/I/51LRcH-r0j...,https://www.amazon.in//Demon-Slayer-iPhone-Zen...
303,"Anime Girls Phone Case, Nezuko Phone Case, Ani...",https://m.media-amazon.com/images/I/51X9Aw1a3B...,https://www.amazon.in//Demon-Slayer-iPhone-Nez...
304,Compatible with Samsung Galaxy S20 FE 5G Case ...,https://m.media-amazon.com/images/I/61NPsWE8Xq...,https://www.amazon.in//Compatible-Samsung-Gala...


In [87]:
AK = AK.append(df).reset_index(drop=True)
AK

,product_name,product_img,product_link
0,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
1,Demon Slayer Anime Zenitsu Kimono Lanyard Keyc...,https://m.media-amazon.com/images/I/7160WR1QrE...,https://www.amazon.in//ComicSense-xyz-Slayer-Z...
2,Blue Aura Demon Slayer Tanjiro Keychain for An...,https://m.media-amazon.com/images/I/41pX5ctdy9...,https://www.amazon.in//Blue-Aura-Tanjiro-Keych...
3,Blue Aura Demon Slayer Zenitsu Keychain for An...,https://m.media-amazon.com/images/I/41soA5gXJP...,https://www.amazon.in//Blue-Aura-Zenitsu-Keych...
4,Blue Aura Demon Slayer Single Sided Metal Keyc...,https://m.media-amazon.com/images/I/31ru6Yail1...,https://www.amazon.in//Blue-Aura-Slayer-Keycha...
...,...,...,...
788,Naruto Anime Village Hidden in Leaves Keychain,https://m.media-amazon.com/images/I/51TcnHaTs1...,https://www.amazon.in//Naruto-Village-Hidden-L...
789,Naruto Uzumaki Face Keychain Metal 3D | Kakash...,https://m.media-amazon.com/images/I/41N6vbs7mO...,https://www.amazon.in//Naruto-Uzumaki-Keychain...
790,Naruto Anime Yellow Flash 3 Weapon Keychain Co...,https://m.media-amazon.com/images/I/5115VWSYAd...,https://www.amazon.in//Naruto-Yellow-Keychain-...
791,Naruto Keychain,https://m.media-amazon.com/images/I/619Hf3cuMW...,https://www.amazon.in//Arkanum-Naruto-Figurine...


In [101]:
product_AK = pd.DataFrame(reviews[reviews['product_code']=='product_AK']).reset_index(drop=True)
product_AK

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3JRAKUG0TB81C,B00004S9AK,4.0,1006992000,product_AK
1,ATW2I5S6S2PFA,B00004S9AK,5.0,968112000,product_AK
2,AFX6DK3RMHQGO,B00004S9AK,5.0,971136000,product_AK
3,A3KYYFT538CZ0O,B00004S9AK,5.0,987206400,product_AK
4,A1GYWOR31AWMGO,B00004S9AK,5.0,989020800,product_AK
...,...,...,...,...,...
16766,A3RI09Q9NQBU93,B00KB3SDAK,5.0,1403568000,product_AK
16767,AZTVJYM4RWO5I,B00KC3YLAK,5.0,1402963200,product_AK
16768,A91WJWB43VEBU,B00KD0WFAK,5.0,1405209600,product_AK
16769,A3QNAVE06V41HJ,B00KH7ERAK,5.0,1402617600,product_AK


In [102]:
len(product_AK.product_id.unique())

737

In [88]:
product_AK = pd.DataFrame(reviews[reviews['product_code']=='product_AK']).reset_index(drop=True)
product_AK =  pd.merge(product_AK, 
                       AK.iloc[:len(product_AK.product_id.unique())]\
                       .join(pd.Series(product_AK['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AK.to_csv('product_AK.csv',index=False)
pd.read_csv('product_AK.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3JRAKUG0TB81C,B00004S9AK,4.0,1006992000,product_AK,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
1,ATW2I5S6S2PFA,B00004S9AK,5.0,968112000,product_AK,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
2,AFX6DK3RMHQGO,B00004S9AK,5.0,971136000,product_AK,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
3,A3KYYFT538CZ0O,B00004S9AK,5.0,987206400,product_AK,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
4,A1GYWOR31AWMGO,B00004S9AK,5.0,989020800,product_AK,Demon Slayer Wooden Keychain for Anime Fans Gi...,https://m.media-amazon.com/images/I/41DJ5D88DD...,https://www.amazon.in//Blue-Aura-Keychain-Gift...
...,...,...,...,...,...,...,...,...
16766,A3RI09Q9NQBU93,B00KB3SDAK,5.0,1403568000,product_AK,AUGEN One Piece 4 Set Action Figure Keychain L...,https://m.media-amazon.com/images/I/61ZT4zEzex...,https://www.amazon.in//AUGEN-Action-Keychain-L...
16767,AZTVJYM4RWO5I,B00KC3YLAK,5.0,1402963200,product_AK,Naruto Anime Red Uchiha Sharingan Design Revol...,https://m.media-amazon.com/images/I/31XzUGwng1...,https://www.amazon.in//Techpro-Naruto-Sharinga...
16768,A91WJWB43VEBU,B00KD0WFAK,5.0,1405209600,product_AK,Naruto Anime Copy Ninja Mangekyou Sharingan Ke...,https://m.media-amazon.com/images/I/417ATuh8OE...,https://www.amazon.in//Naruto-Anime-Mangekyou-...
16769,A3QNAVE06V41HJ,B00KH7ERAK,5.0,1402617600,product_AK,Naruto Anime Red Uchiha Sharingan Design Revol...,https://m.media-amazon.com/images/I/31rIJuZODW...,https://www.amazon.in//Techpro-Naruto-Sharinga...


### Product AL dataframe

In [80]:
df = get_scraped('alexa echo')
df

 49%|███████████████████████████████████████▉                                          | 19/39 [00:20<00:21,  1.06s/it]


,product_name,product_img,product_link
0,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
1,All-new Echo Dot (4th Gen) | #1 smart speaker ...,https://m.media-amazon.com/images/I/61MbLLagiV...,https://www.amazon.in//Echo-Dot-4th-Gen-Blue/d...
2,"Echo Dot (4th Gen, 2020 release) with clock | ...",https://m.media-amazon.com/images/I/61u0y9ADEl...,https://www.amazon.in//All-new-Echo-Dot-with-c...
3,"Echo (4th Gen, 2020 release) | Premium sound p...",https://m.media-amazon.com/images/I/61dgl2srHD...,https://www.amazon.in//All-new-Echo/dp/B085FY9...
4,"Echo Dot (4th Gen, 2020 release)| Smart speake...",https://m.media-amazon.com/images/I/41FgiLFNhF...,https://www.amazon.in//All-new-Echo-Dot/dp/B08...
...,...,...,...
301,Generic 110V Smart Switch Touch Panel Switch S...,https://m.media-amazon.com/images/I/21+y4Y3Ijj...,https://www.amazon.in//Generic-Smart-Switch-Su...
302,Generic Smart Plug Wall Smart Outlet App Contr...,https://m.media-amazon.com/images/I/21md8jxZwf...,https://www.amazon.in//Generic-Outlet-Control-...
303,TNIU Wireless APP Remote Control Power Outlet ...,https://m.media-amazon.com/images/I/41SXLvrA21...,https://www.amazon.in//TNIU-Thermostat-Humidis...
304,GlobalNiche® Smart Plug Wall Smart Outlet App ...,https://m.media-amazon.com/images/I/51MOnV0DIC...,https://www.amazon.in//GlobalNiche%C2%AE-Outle...


In [81]:
AL = df.copy()
AL

,product_name,product_img,product_link
0,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
1,All-new Echo Dot (4th Gen) | #1 smart speaker ...,https://m.media-amazon.com/images/I/61MbLLagiV...,https://www.amazon.in//Echo-Dot-4th-Gen-Blue/d...
2,"Echo Dot (4th Gen, 2020 release) with clock | ...",https://m.media-amazon.com/images/I/61u0y9ADEl...,https://www.amazon.in//All-new-Echo-Dot-with-c...
3,"Echo (4th Gen, 2020 release) | Premium sound p...",https://m.media-amazon.com/images/I/61dgl2srHD...,https://www.amazon.in//All-new-Echo/dp/B085FY9...
4,"Echo Dot (4th Gen, 2020 release)| Smart speake...",https://m.media-amazon.com/images/I/41FgiLFNhF...,https://www.amazon.in//All-new-Echo-Dot/dp/B08...
...,...,...,...
301,Generic 110V Smart Switch Touch Panel Switch S...,https://m.media-amazon.com/images/I/21+y4Y3Ijj...,https://www.amazon.in//Generic-Smart-Switch-Su...
302,Generic Smart Plug Wall Smart Outlet App Contr...,https://m.media-amazon.com/images/I/21md8jxZwf...,https://www.amazon.in//Generic-Outlet-Control-...
303,TNIU Wireless APP Remote Control Power Outlet ...,https://m.media-amazon.com/images/I/41SXLvrA21...,https://www.amazon.in//TNIU-Thermostat-Humidis...
304,GlobalNiche® Smart Plug Wall Smart Outlet App ...,https://m.media-amazon.com/images/I/51MOnV0DIC...,https://www.amazon.in//GlobalNiche%C2%AE-Outle...


In [ ]:
# AL = AL.append(df).reset_index(drop=True)
# AL

In [78]:
product_AL = pd.DataFrame(reviews[reviews['product_code']=='product_AL']).reset_index(drop=True)
product_AL

,user_id,product_id,Ratings,Time_stamp,product_code
0,ADQRZFAWF528M,B00001X5AL,5.0,971395200,product_AL
1,A32OGIZOO8YA4E,B00001X5AL,2.0,945734400,product_AL
2,A2EBLJ740G7EOL,B00001X5AL,3.0,946771200,product_AL
3,A2X4777UKE1BKB,B00001X5AL,5.0,949968000,product_AL
4,A1AC0ZLC6SVXSD,B00001X5AL,3.0,952387200,product_AL
...,...,...,...,...,...
118,A269X242UD6AYE,B0000AE6AL,5.0,1358121600,product_AL
119,A2WSXCSXMXMD2E,B0000AE6AL,5.0,1339372800,product_AL
120,A34VXSEA3W5N8A,B0000AE6AL,4.0,1342915200,product_AL
121,A16P6IGHHQLNYS,B0000AE6AL,5.0,1356480000,product_AL


In [79]:
len(product_AL.product_id.unique())

11

In [82]:
product_AL = pd.DataFrame(reviews[reviews['product_code']=='product_AL']).reset_index(drop=True)
product_AL =  pd.merge(product_AL, 
                       AL.iloc[:len(product_AL.product_id.unique())]\
                       .join(pd.Series(product_AL['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AL.to_csv('product_AL.csv',index=False)
pd.read_csv('product_AL.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ADQRZFAWF528M,B00001X5AL,5.0,971395200,product_AL,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
1,A32OGIZOO8YA4E,B00001X5AL,2.0,945734400,product_AL,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
2,A2EBLJ740G7EOL,B00001X5AL,3.0,946771200,product_AL,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
3,A2X4777UKE1BKB,B00001X5AL,5.0,949968000,product_AL,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
4,A1AC0ZLC6SVXSD,B00001X5AL,3.0,952387200,product_AL,Echo Dot (3rd Gen) - #1 smart speaker brand in...,https://m.media-amazon.com/images/I/61EXU8BuGZ...,https://www.amazon.in//Echo-Dot-3rd-Gen/dp/B07...
...,...,...,...,...,...,...,...,...
118,A269X242UD6AYE,B0000AE6AL,5.0,1358121600,product_AL,"Echo Dot (4th Gen, 2020 release) with clock | ...",https://m.media-amazon.com/images/I/61AAiJ9LJ+...,https://www.amazon.in//All-new-Echo-Dot-with-c...
119,A2WSXCSXMXMD2E,B0000AE6AL,5.0,1339372800,product_AL,"Echo Dot (4th Gen, 2020 release) with clock | ...",https://m.media-amazon.com/images/I/61AAiJ9LJ+...,https://www.amazon.in//All-new-Echo-Dot-with-c...
120,A34VXSEA3W5N8A,B0000AE6AL,4.0,1342915200,product_AL,"Echo Dot (4th Gen, 2020 release) with clock | ...",https://m.media-amazon.com/images/I/61AAiJ9LJ+...,https://www.amazon.in//All-new-Echo-Dot-with-c...
121,A16P6IGHHQLNYS,B0000AE6AL,5.0,1356480000,product_AL,"Echo Dot (4th Gen, 2020 release) with clock | ...",https://m.media-amazon.com/images/I/61AAiJ9LJ+...,https://www.amazon.in//All-new-Echo-Dot-with-c...


### Product AM dataframe

In [99]:
df = get_scraped('anime manga poster')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Risty Shop High Definition Naruto Combo 1046 P...,https://m.media-amazon.com/images/I/91QqFPRi9H...,https://www.amazon.in//Risty-Shop-Definition-D...
1,Risty Shop High Definition Combo 373 Pack of 4...,https://m.media-amazon.com/images/I/910ujz4r4V...,https://www.amazon.in//Risty-Shop-Definition-D...
2,MCSID RAZZ - One Piece - New 3B. wanted Monkey...,https://m.media-amazon.com/images/I/81da3kBbgI...,https://www.amazon.in//MCSID-RAZZ-wanted-Monke...
3,Aesthetic 54 Pcs Mix Anime Posters For Room Wa...,https://m.media-amazon.com/images/I/71OrDcCXIt...,https://www.amazon.in//Aesthetic-Anime-Posters...
4,Risty Shop High Definition one Piece Roronoa Z...,https://m.media-amazon.com/images/I/912gQZ7KUm...,https://www.amazon.in//Risty-Shop-Definition-D...
...,...,...,...
253,Risty Shop High Definition Naruto Kakashi Hata...,https://m.media-amazon.com/images/I/712jzeMg43...,https://www.amazon.in//Risty-Shop-Definition-D...
254,Risty Shop High Definition one Piece 103 Super...,https://m.media-amazon.com/images/I/81n6Yrez62...,https://www.amazon.in//Risty-Shop-Definition-D...
255,IMPOSTER Jujutsu Kaisen Manga Anime Poster - E...,https://m.media-amazon.com/images/I/61OuXpLdoc...,https://www.amazon.in//IMPOSTER-Jujutsu-Kaisen...
256,Poster Best Spike Spiegel Manga Colors Anime C...,https://m.media-amazon.com/images/I/71Qi4FEuNf...,https://www.amazon.in//Poster-Spiegel-Colors-M...


In [96]:
AM = df.copy()
AM

,product_name,product_img,product_link
0,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
1,How to Draw Anime: ( Includes How to Draw Mang...,https://m.media-amazon.com/images/I/81bx7akiMw...,https://www.amazon.in//How-Draw-Anime-Cartoon-...
2,ANIME Girls - Coloring book for ANIME lovers: ...,https://m.media-amazon.com/images/I/61VLOk4yhc...,https://www.amazon.in//ANIME-Girls-Characters-...
3,"Naruto, Volume 1: Uzumaki Naruto",https://m.media-amazon.com/images/I/61LVJN2aIo...,https://www.amazon.in//Naruto-01-Masashi-Kishi...
4,Anyone Can Draw Anime: Easy Step-by-Step Drawi...,https://m.media-amazon.com/images/I/81SfVAp9dJ...,https://www.amazon.in//Anyone-Draw-Anime-Step-...
...,...,...,...
301,Pokemon Center Mega Tokyo Photo Book Japan ani...,https://m.media-amazon.com/images/I/81WLrQMOdO...,https://www.amazon.in//Pokemon-Center-Tokyo-Ph...
302,The Savior's Book Café Story in Another World ...,https://m.media-amazon.com/images/I/81fFYNnkx0...,https://www.amazon.in//Saviors-Story-Another-W...
303,Bleach 01: Strawberry and the Soul Reapers: Vo...,https://m.media-amazon.com/images/I/51+7RZNE2r...,https://www.amazon.in//Bleach-01-Tite-Kubo/dp/...
304,Cute Chibi Girls Coloring Book: Kawaii Chibi G...,https://m.media-amazon.com/images/I/71ZYMLjeMx...,https://www.amazon.in//Cute-Chibi-Girls-Colori...


In [100]:
AM = AM.append(df).reset_index(drop=True)
AM

,product_name,product_img,product_link
0,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
1,How to Draw Anime: ( Includes How to Draw Mang...,https://m.media-amazon.com/images/I/81bx7akiMw...,https://www.amazon.in//How-Draw-Anime-Cartoon-...
2,ANIME Girls - Coloring book for ANIME lovers: ...,https://m.media-amazon.com/images/I/61VLOk4yhc...,https://www.amazon.in//ANIME-Girls-Characters-...
3,"Naruto, Volume 1: Uzumaki Naruto",https://m.media-amazon.com/images/I/61LVJN2aIo...,https://www.amazon.in//Naruto-01-Masashi-Kishi...
4,Anyone Can Draw Anime: Easy Step-by-Step Drawi...,https://m.media-amazon.com/images/I/81SfVAp9dJ...,https://www.amazon.in//Anyone-Draw-Anime-Step-...
...,...,...,...
865,Risty Shop High Definition Naruto Kakashi Hata...,https://m.media-amazon.com/images/I/712jzeMg43...,https://www.amazon.in//Risty-Shop-Definition-D...
866,Risty Shop High Definition one Piece 103 Super...,https://m.media-amazon.com/images/I/81n6Yrez62...,https://www.amazon.in//Risty-Shop-Definition-D...
867,IMPOSTER Jujutsu Kaisen Manga Anime Poster - E...,https://m.media-amazon.com/images/I/61OuXpLdoc...,https://www.amazon.in//IMPOSTER-Jujutsu-Kaisen...
868,Poster Best Spike Spiegel Manga Colors Anime C...,https://m.media-amazon.com/images/I/71Qi4FEuNf...,https://www.amazon.in//Poster-Spiegel-Colors-M...


In [103]:
product_AM = pd.DataFrame(reviews[reviews['product_code']=='product_AM']).reset_index(drop=True)
product_AM

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2AS69LEX6ZUJ4,B00000JBAM,4.0,957052800,product_AM
1,A1LH6RF4UN9VI6,B00000JBAM,5.0,980208000,product_AM
2,A5E2CATGTND1E,B00000JBAM,5.0,971049600,product_AM
3,A354AZBXGBO1LN,B00000JBAM,5.0,949017600,product_AM
4,A149RNR5RH19YY,B00000JBAM,5.0,1021334400,product_AM
...,...,...,...,...,...
13001,AMFQX8QUHYS3J,B00JMRMNAM,5.0,1400803200,product_AM
13002,A38SIL3UX19KIG,B00JPJ1YAM,5.0,1403913600,product_AM
13003,AQJY9F7YIG225,B00K2L4VAM,5.0,1402704000,product_AM
13004,A1IX5X3CLD571J,B00KSAP0AM,4.0,1405382400,product_AM


In [104]:
len(product_AM.product_id.unique())

759

In [105]:
product_AM = pd.DataFrame(reviews[reviews['product_code']=='product_AM']).reset_index(drop=True)
product_AM =  pd.merge(product_AM, 
                       AM.iloc[:len(product_AM.product_id.unique())]\
                       .join(pd.Series(product_AM['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AM.to_csv('product_AM.csv',index=False)
pd.read_csv('product_AM.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2AS69LEX6ZUJ4,B00000JBAM,4.0,957052800,product_AM,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
1,A1LH6RF4UN9VI6,B00000JBAM,5.0,980208000,product_AM,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
2,A5E2CATGTND1E,B00000JBAM,5.0,971049600,product_AM,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
3,A354AZBXGBO1LN,B00000JBAM,5.0,949017600,product_AM,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
4,A149RNR5RH19YY,B00000JBAM,5.0,1021334400,product_AM,"ANIME Coloring Book For Kids, Adults Or Anyone...",https://m.media-amazon.com/images/I/71-2obLmr4...,https://www.amazon.in//ANIME-Coloring-Adults-A...
...,...,...,...,...,...,...,...,...
13001,AMFQX8QUHYS3J,B00JMRMNAM,5.0,1400803200,product_AM,Risty Shop High Definition 113 Superhero Jujut...,https://m.media-amazon.com/images/I/81RPgnOitQ...,https://www.amazon.in//Risty-Shop-Definition-D...
13002,A38SIL3UX19KIG,B00JPJ1YAM,5.0,1403913600,product_AM,Risty Shop High Definition one Piece Luffy Law...,https://m.media-amazon.com/images/I/81uHGlejgz...,https://www.amazon.in//Risty-Shop-Definition-D...
13003,AQJY9F7YIG225,B00K2L4VAM,5.0,1402704000,product_AM,Poster Best Manga Anime Naruto Shippuuden Uchi...,https://m.media-amazon.com/images/I/61KZuYMuAh...,https://www.amazon.in//Poster-Naruto-Shippuude...
13004,A1IX5X3CLD571J,B00KSAP0AM,4.0,1405382400,product_AM,Poster Envy Manga Anime Naruto Shippuuden Uchi...,https://m.media-amazon.com/images/I/61KZuYMuAh...,https://www.amazon.in//Poster-Naruto-Shippuude...


### Product AN dataframe

In [109]:
df = get_scraped('anime necklace')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Anime Attack On Titan Shingeki no Kyojin Badge...,https://m.media-amazon.com/images/I/51ZlWnNfp4...,https://www.amazon.in//Kawn-Attack-Shingeki-Pe...
1,Stylish Partyware Choker for Women & Girls,https://m.media-amazon.com/images/I/51m7V9ufRF...,https://www.amazon.in//Stylish-Partyware-Choke...
2,Street27 Anime Attack On Titan Shingeki no Kyo...,https://m.media-amazon.com/images/I/41NT-l99Gr...,https://www.amazon.in//Street27-Attack-Shingek...
3,Basement Key Attack On Titan Anime Metal Necklace,https://m.media-amazon.com/images/I/31JK7mqbZJ...,https://www.amazon.in//Basement-Attack-Titan-A...
4,Naruto Inspired Akatsuki Red Cloud Pendant Nec...,https://m.media-amazon.com/images/I/31ic3QzmvY...,https://www.amazon.in//RVM-Jewels-Inspired-Aka...
...,...,...,...
301,Pixelated Arrow Multi-Color Alloy/Leather Cord...,https://m.media-amazon.com/images/I/51Atn4SNpy...,https://www.amazon.in//Sarah-Pixelated-Arrow-P...
302,Purple Stone Owl Pendent with Silver chain | p...,https://m.media-amazon.com/images/I/5166jGExut...,https://www.amazon.in//Ishtyle-adda-Pendent-ch...
303,Baby Yoda Necklace Baby Yoda Fan Gift Star War...,https://m.media-amazon.com/images/I/61b8F+m5gY...,https://www.amazon.in//G-Ahora-Necklace-Inspir...
304,Suck UK Skeleton Hand Ring Holder & Dresser Or...,https://m.media-amazon.com/images/I/71m3itPQ-i...,https://www.amazon.in//UK-Skeleton-Organizer-H...


In [111]:
AN = df.copy()
AN

,product_name,product_img,product_link
0,Anime Attack On Titan Shingeki no Kyojin Badge...,https://m.media-amazon.com/images/I/51ZlWnNfp4...,https://www.amazon.in//Kawn-Attack-Shingeki-Pe...
1,Stylish Partyware Choker for Women & Girls,https://m.media-amazon.com/images/I/51m7V9ufRF...,https://www.amazon.in//Stylish-Partyware-Choke...
2,Street27 Anime Attack On Titan Shingeki no Kyo...,https://m.media-amazon.com/images/I/41NT-l99Gr...,https://www.amazon.in//Street27-Attack-Shingek...
3,Basement Key Attack On Titan Anime Metal Necklace,https://m.media-amazon.com/images/I/31JK7mqbZJ...,https://www.amazon.in//Basement-Attack-Titan-A...
4,Naruto Inspired Akatsuki Red Cloud Pendant Nec...,https://m.media-amazon.com/images/I/31ic3QzmvY...,https://www.amazon.in//RVM-Jewels-Inspired-Aka...
...,...,...,...
301,Pixelated Arrow Multi-Color Alloy/Leather Cord...,https://m.media-amazon.com/images/I/51Atn4SNpy...,https://www.amazon.in//Sarah-Pixelated-Arrow-P...
302,Purple Stone Owl Pendent with Silver chain | p...,https://m.media-amazon.com/images/I/5166jGExut...,https://www.amazon.in//Ishtyle-adda-Pendent-ch...
303,Baby Yoda Necklace Baby Yoda Fan Gift Star War...,https://m.media-amazon.com/images/I/61b8F+m5gY...,https://www.amazon.in//G-Ahora-Necklace-Inspir...
304,Suck UK Skeleton Hand Ring Holder & Dresser Or...,https://m.media-amazon.com/images/I/71m3itPQ-i...,https://www.amazon.in//UK-Skeleton-Organizer-H...


In [ ]:
# AN = AN.append(df).reset_index(drop=True)
# AN

In [107]:
product_AN = pd.DataFrame(reviews[reviews['product_code']=='product_AN']).reset_index(drop=True)
product_AN

,user_id,product_id,Ratings,Time_stamp,product_code
0,A18U2MWT8FJKWD,B00003CWAN,5.0,1210204800,product_AN
1,A3W4WGX7SKSI7F,B0000511AN,5.0,1341273600,product_AN
2,A1XRJZ2WJ4ESFS,B0000511AN,3.0,1321315200,product_AN
3,A37XUY9K80C2H5,B0000511AN,5.0,1321056000,product_AN
4,ADNB6NAUOG6W0,B0000511AN,4.0,1313193600,product_AN
...,...,...,...,...,...
315,A3KEKMJOCO3LSD,B0000ALKAN,5.0,1255046400,product_AN
316,A2WPWLEO3XCDP0,B0000ALKAN,5.0,1145664000,product_AN
317,AI79MD2INRFNY,B0000ALKAN,5.0,1394841600,product_AN
318,A1HB16YJ2G3RVD,B0000D8HAN,5.0,1076112000,product_AN


In [108]:
len(product_AN.product_id.unique())

24

In [112]:
product_AN = pd.DataFrame(reviews[reviews['product_code']=='product_AN']).reset_index(drop=True)
product_AN =  pd.merge(product_AN, 
                       AN.iloc[:len(product_AN.product_id.unique())]\
                       .join(pd.Series(product_AN['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AN.to_csv('product_AN.csv',index=False)
pd.read_csv('product_AN.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A18U2MWT8FJKWD,B00003CWAN,5.0,1210204800,product_AN,Anime Attack On Titan Shingeki no Kyojin Badge...,https://m.media-amazon.com/images/I/51ZlWnNfp4...,https://www.amazon.in//Kawn-Attack-Shingeki-Pe...
1,A3W4WGX7SKSI7F,B0000511AN,5.0,1341273600,product_AN,Stylish Partyware Choker for Women & Girls,https://m.media-amazon.com/images/I/51m7V9ufRF...,https://www.amazon.in//Stylish-Partyware-Choke...
2,A1XRJZ2WJ4ESFS,B0000511AN,3.0,1321315200,product_AN,Stylish Partyware Choker for Women & Girls,https://m.media-amazon.com/images/I/51m7V9ufRF...,https://www.amazon.in//Stylish-Partyware-Choke...
3,A37XUY9K80C2H5,B0000511AN,5.0,1321056000,product_AN,Stylish Partyware Choker for Women & Girls,https://m.media-amazon.com/images/I/51m7V9ufRF...,https://www.amazon.in//Stylish-Partyware-Choke...
4,ADNB6NAUOG6W0,B0000511AN,4.0,1313193600,product_AN,Stylish Partyware Choker for Women & Girls,https://m.media-amazon.com/images/I/51m7V9ufRF...,https://www.amazon.in//Stylish-Partyware-Choke...
...,...,...,...,...,...,...,...,...
315,A3KEKMJOCO3LSD,B0000ALKAN,5.0,1255046400,product_AN,Anime Yagami Single L Pendant Necklace Cosplay...,https://m.media-amazon.com/images/I/41UexkNIEc...,https://www.amazon.in//RVM-Jewels-Necklace-Jew...
316,A2WPWLEO3XCDP0,B0000ALKAN,5.0,1145664000,product_AN,Anime Yagami Single L Pendant Necklace Cosplay...,https://m.media-amazon.com/images/I/41UexkNIEc...,https://www.amazon.in//RVM-Jewels-Necklace-Jew...
317,AI79MD2INRFNY,B0000ALKAN,5.0,1394841600,product_AN,Anime Yagami Single L Pendant Necklace Cosplay...,https://m.media-amazon.com/images/I/41UexkNIEc...,https://www.amazon.in//RVM-Jewels-Necklace-Jew...
318,A1HB16YJ2G3RVD,B0000D8HAN,5.0,1076112000,product_AN,Naruto Leaf Necklace Leaf Village Symbol Logo ...,https://m.media-amazon.com/images/I/41c9EtjeE7...,https://www.amazon.in//Krafty-Kustomz-Necklace...


### Product AO dataframe

In [120]:
df = get_scraped('anime outfit for men')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Men's Loose T-Shirt,https://m.media-amazon.com/images/I/41-c32UUsX...,https://www.amazon.in//EYEBOGLER-Casual-Oversi...
1,Men's Stylish Cotton Printed Full Sleeve T-Shi...,https://m.media-amazon.com/images/I/61vv+4HIu8...,https://www.amazon.in//LEWEL-Stylish-Cotton-Pr...
2,Unisex Akatsuki Clouds Naruto Anime Hooded Nec...,https://m.media-amazon.com/images/I/41VobbdN-v...,https://www.amazon.in//TeesTheDay-Akatsuki-Clo...
3,Men's Slim Fit Track pants,https://m.media-amazon.com/images/I/71r3RWbLWA...,https://www.amazon.in//JUGULAR-Solid-Regular-C...
4,Men's Regular Fit Track pants,https://m.media-amazon.com/images/I/81NqBhgsDp...,https://www.amazon.in//SHAUN-Cotton-Trackpants...
...,...,...,...
301,"Adult Cloak Kids Robe, Unisex Men Cape Anime C...",https://m.media-amazon.com/images/I/51v+EZrgxK...,https://www.amazon.in//Cosplay-Halloween-Costu...
302,| Mens Round Neck Solid Tshirt Combo,https://m.media-amazon.com/images/I/618zaae35p...,https://www.amazon.in//FOOTROO-Tee-Round-Solid...
303,Kakashi Naruto Anime Printed Half Sleeve Round...,https://m.media-amazon.com/images/I/61KKUKWV+N...,https://www.amazon.in//VIKCLIQUE-Kakashi-Narut...
304,Printed T Shirt for Men Combo Pack of 3,https://m.media-amazon.com/images/I/71D4mbj8uL...,https://www.amazon.in//Boodbuck-Printed-Shirt-...


In [117]:
AO = df.copy()
AO

,product_name,product_img,product_link
0,Girls Movie Character Halloween Costumes for K...,https://m.media-amazon.com/images/I/61tFsJrkS0...,https://www.amazon.in//Toddlers-Princess-Hallo...
1,Kids Kimono Cosplay Costume Outfit Cardigan To...,https://m.media-amazon.com/images/I/318rgE4PTn...,https://www.amazon.in//Kimono-Cosplay-Costume-...
2,Anime Tanjirou Shinobu Zenitsu Cosplay Costume...,https://m.media-amazon.com/images/I/61LcArxyhu...,https://www.amazon.in//Cosplay-Costume-Tanjiro...
3,【FENGLONG-YB】Cosplay Costume Kimono Anime Cost...,https://m.media-amazon.com/images/I/61GNI5SujN...,https://www.amazon.in//%E3%80%90FENGLONG-YB%E3...
4,Singcoco Baby Boy Halloween Costume Outfit New...,https://m.media-amazon.com/images/I/61-Za8EXe1...,https://www.amazon.in//Singcoco-Halloween-Cost...
...,...,...,...
126,GREAT&LUCKY Cosplay Fairy Pixie Elf Ears - Sof...,https://m.media-amazon.com/images/I/518D0MTRzc...,https://www.amazon.in//GREAT-LUCKY-Cosplay-Fai...
127,CANASOUR Unisex Halloween Kids Costume Party C...,https://m.media-amazon.com/images/I/61UgR4qZYw...,https://www.amazon.in//CANASOUR-Halloween-Cost...
128,Survey Corps Cloak Blanket Cape Cosplay Pullov...,https://m.media-amazon.com/images/I/41KkXanpeF...,https://www.amazon.in//Regiment-Blanket-Cospla...
129,GK-O Anime Sailor Moon Style T-Shirt Cosplay C...,https://m.media-amazon.com/images/I/515k3Qc8gW...,https://www.amazon.in//GK-Japanese-T-Shirt-Har...


In [121]:
AO = AO.append(df).reset_index(drop=True)
AO

,product_name,product_img,product_link
0,Girls Movie Character Halloween Costumes for K...,https://m.media-amazon.com/images/I/61tFsJrkS0...,https://www.amazon.in//Toddlers-Princess-Hallo...
1,Kids Kimono Cosplay Costume Outfit Cardigan To...,https://m.media-amazon.com/images/I/318rgE4PTn...,https://www.amazon.in//Kimono-Cosplay-Costume-...
2,Anime Tanjirou Shinobu Zenitsu Cosplay Costume...,https://m.media-amazon.com/images/I/61LcArxyhu...,https://www.amazon.in//Cosplay-Costume-Tanjiro...
3,【FENGLONG-YB】Cosplay Costume Kimono Anime Cost...,https://m.media-amazon.com/images/I/61GNI5SujN...,https://www.amazon.in//%E3%80%90FENGLONG-YB%E3...
4,Singcoco Baby Boy Halloween Costume Outfit New...,https://m.media-amazon.com/images/I/61-Za8EXe1...,https://www.amazon.in//Singcoco-Halloween-Cost...
...,...,...,...
738,"Adult Cloak Kids Robe, Unisex Men Cape Anime C...",https://m.media-amazon.com/images/I/51v+EZrgxK...,https://www.amazon.in//Cosplay-Halloween-Costu...
739,| Mens Round Neck Solid Tshirt Combo,https://m.media-amazon.com/images/I/618zaae35p...,https://www.amazon.in//FOOTROO-Tee-Round-Solid...
740,Kakashi Naruto Anime Printed Half Sleeve Round...,https://m.media-amazon.com/images/I/61KKUKWV+N...,https://www.amazon.in//VIKCLIQUE-Kakashi-Narut...
741,Printed T Shirt for Men Combo Pack of 3,https://m.media-amazon.com/images/I/71D4mbj8uL...,https://www.amazon.in//Boodbuck-Printed-Shirt-...


In [114]:
product_AO = pd.DataFrame(reviews[reviews['product_code']=='product_AO']).reset_index(drop=True)
product_AO

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3FYKZ8UIVFH64,B00005MEAO,5.0,1340928000,product_AO
1,A30EV4OCLL7RH0,B00005T3AO,2.0,1096761600,product_AO
2,A3R44CH147RV5C,B00005UKAO,5.0,1008201600,product_AO
3,AFP6JAP9Z929Z,B00005UKAO,5.0,1360800000,product_AO
4,A1LAMPDU0E29X4,B000068EAO,3.0,1054425600,product_AO
...,...,...,...,...,...
7286,ATT806EWAXDXR,B00K6AI7AO,5.0,1404777600,product_AO
7287,ARR11O8R4FUTW,B00K6AI7AO,3.0,1405036800,product_AO
7288,A39EB4BMRYKRW4,B00K6AI7AO,4.0,1404777600,product_AO
7289,A11S54HLQM4NGN,B00KHLF8AO,5.0,1404172800,product_AO


In [115]:
len(product_AO.product_id.unique())

691

In [122]:
product_AO = pd.DataFrame(reviews[reviews['product_code']=='product_AO']).reset_index(drop=True)
product_AO =  pd.merge(product_AO, 
                       AO.iloc[:len(product_AO.product_id.unique())]\
                       .join(pd.Series(product_AO['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AO.to_csv('product_AO.csv',index=False)
pd.read_csv('product_AO.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3FYKZ8UIVFH64,B00005MEAO,5.0,1340928000,product_AO,Girls Movie Character Halloween Costumes for K...,https://m.media-amazon.com/images/I/61tFsJrkS0...,https://www.amazon.in//Toddlers-Princess-Hallo...
1,A30EV4OCLL7RH0,B00005T3AO,2.0,1096761600,product_AO,Kids Kimono Cosplay Costume Outfit Cardigan To...,https://m.media-amazon.com/images/I/318rgE4PTn...,https://www.amazon.in//Kimono-Cosplay-Costume-...
2,A3R44CH147RV5C,B00005UKAO,5.0,1008201600,product_AO,Anime Tanjirou Shinobu Zenitsu Cosplay Costume...,https://m.media-amazon.com/images/I/61LcArxyhu...,https://www.amazon.in//Cosplay-Costume-Tanjiro...
3,AFP6JAP9Z929Z,B00005UKAO,5.0,1360800000,product_AO,Anime Tanjirou Shinobu Zenitsu Cosplay Costume...,https://m.media-amazon.com/images/I/61LcArxyhu...,https://www.amazon.in//Cosplay-Costume-Tanjiro...
4,A1LAMPDU0E29X4,B000068EAO,3.0,1054425600,product_AO,【FENGLONG-YB】Cosplay Costume Kimono Anime Cost...,https://m.media-amazon.com/images/I/61GNI5SujN...,https://www.amazon.in//%E3%80%90FENGLONG-YB%E3...
...,...,...,...,...,...,...,...,...
7286,ATT806EWAXDXR,B00K6AI7AO,5.0,1404777600,product_AO,Men's Naruto Kakashi 2 Navy Blue -Hoodie Anime...,https://m.media-amazon.com/images/I/61300wboBx...,https://www.amazon.in//Vfashion-Kakashi-Navy-B...
7287,ARR11O8R4FUTW,B00K6AI7AO,3.0,1405036800,product_AO,Men's Naruto Kakashi 2 Navy Blue -Hoodie Anime...,https://m.media-amazon.com/images/I/61300wboBx...,https://www.amazon.in//Vfashion-Kakashi-Navy-B...
7288,A39EB4BMRYKRW4,B00K6AI7AO,4.0,1404777600,product_AO,Men's Naruto Kakashi 2 Navy Blue -Hoodie Anime...,https://m.media-amazon.com/images/I/61300wboBx...,https://www.amazon.in//Vfashion-Kakashi-Navy-B...
7289,A11S54HLQM4NGN,B00KHLF8AO,5.0,1404172800,product_AO,Stylish Unisex Minato Anime Design Printed Hoo...,https://m.media-amazon.com/images/I/41PauSkvqF...,https://www.amazon.in//Teewink-Stylish-Printed...


### Product AP dataframe

In [126]:
df = get_scraped('air pillow')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Trajectory Inflatable Air Pillow Travel headre...,https://m.media-amazon.com/images/I/81IHRmEcPq...,https://www.amazon.in//Trajectory-Inflatable-T...
1,"Liznoriz esay life Velvet Travel Pillow, Pack ...",https://m.media-amazon.com/images/I/51ksLur2WK...,https://www.amazon.in//esay-life-Travel-Inflat...
2,Rubberized Cotton Travel Pillow (Grey),https://m.media-amazon.com/images/I/51zgcA0-+s...,https://www.amazon.in//SE-Duckback-Khakhi-Air-...
3,Duckback Rubberized Cotton Travel Air Pillow-K...,https://m.media-amazon.com/images/I/313VaICwYK...,https://www.amazon.in//Duckback-Rubberized-Cot...
4,Inflatable Air Pillow Travel headrest Accessor...,https://m.media-amazon.com/images/I/91QN9XK9ny...,https://www.amazon.in//Trajectory-Inflatable-A...
...,...,...,...
301,LOYAL EMPLE Neck Travelling Pillow with Eye Ma...,https://m.media-amazon.com/images/I/61Rjeoqm1q...,https://www.amazon.in//LOYAL-EMPLE-Travelling-...
302,KriTech Velvet Soft Comfortable Easy to Carry ...,https://m.media-amazon.com/images/I/416K34UQPe...,https://www.amazon.in//KriTech-Comfortable-U-S...
303,Walberrie Velvet Soft Air Inflatable Navy Blue...,https://m.media-amazon.com/images/I/31nv6kifI6...,https://www.amazon.in//Walberrie-Velvet-Inflat...
304,RAV Pack of 1 Cotton Travelling Pillow with ai...,https://m.media-amazon.com/images/I/61Q1Er6Eb5...,https://www.amazon.in//RAV-Cotton-Travelling-P...


In [127]:
AP = df.copy()
AP

,product_name,product_img,product_link
0,Trajectory Inflatable Air Pillow Travel headre...,https://m.media-amazon.com/images/I/81IHRmEcPq...,https://www.amazon.in//Trajectory-Inflatable-T...
1,"Liznoriz esay life Velvet Travel Pillow, Pack ...",https://m.media-amazon.com/images/I/51ksLur2WK...,https://www.amazon.in//esay-life-Travel-Inflat...
2,Rubberized Cotton Travel Pillow (Grey),https://m.media-amazon.com/images/I/51zgcA0-+s...,https://www.amazon.in//SE-Duckback-Khakhi-Air-...
3,Duckback Rubberized Cotton Travel Air Pillow-K...,https://m.media-amazon.com/images/I/313VaICwYK...,https://www.amazon.in//Duckback-Rubberized-Cot...
4,Inflatable Air Pillow Travel headrest Accessor...,https://m.media-amazon.com/images/I/91QN9XK9ny...,https://www.amazon.in//Trajectory-Inflatable-A...
...,...,...,...
301,LOYAL EMPLE Neck Travelling Pillow with Eye Ma...,https://m.media-amazon.com/images/I/61Rjeoqm1q...,https://www.amazon.in//LOYAL-EMPLE-Travelling-...
302,KriTech Velvet Soft Comfortable Easy to Carry ...,https://m.media-amazon.com/images/I/416K34UQPe...,https://www.amazon.in//KriTech-Comfortable-U-S...
303,Walberrie Velvet Soft Air Inflatable Navy Blue...,https://m.media-amazon.com/images/I/31nv6kifI6...,https://www.amazon.in//Walberrie-Velvet-Inflat...
304,RAV Pack of 1 Cotton Travelling Pillow with ai...,https://m.media-amazon.com/images/I/61Q1Er6Eb5...,https://www.amazon.in//RAV-Cotton-Travelling-P...


In [ ]:
# AP = AP.append(df).reset_index(drop=True)
# AP

In [124]:
product_AP = pd.DataFrame(reviews[reviews['product_code']=='product_AP']).reset_index(drop=True)
product_AP

,user_id,product_id,Ratings,Time_stamp,product_code
0,A27RIHXBUGZNQ6,B00004Z6AP,5.0,1375142400,product_AP
1,A2NEQ89J8NI997,B000050YAP,3.0,987552000,product_AP
2,A23GFTVIETX7DS,B000050YAP,4.0,1044403200,product_AP
3,A1QZ2VZP6B9OX7,B000050YAP,4.0,997142400,product_AP
4,A2R2Z3RT7RCB1L,B000050YAP,5.0,1010361600,product_AP
...,...,...,...,...,...
115,A3UFDY7GPV3R8J,B0000AE6AP,5.0,1368144000,product_AP
116,A1NG5JLG4OLJB4,B0000AE6AP,5.0,1403136000,product_AP
117,AP579SJ9EVHLI,B0000AE6AP,5.0,1361491200,product_AP
118,A8G2YP67ZFS3K,B0000AE6AP,5.0,1374364800,product_AP


In [125]:
len(product_AP.product_id.unique())

15

In [128]:
product_AP = pd.DataFrame(reviews[reviews['product_code']=='product_AP']).reset_index(drop=True)
product_AP =  pd.merge(product_AP, 
                       AP.iloc[:len(product_AP.product_id.unique())]\
                       .join(pd.Series(product_AP['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AP.to_csv('product_AP.csv',index=False)
pd.read_csv('product_AP.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A27RIHXBUGZNQ6,B00004Z6AP,5.0,1375142400,product_AP,Trajectory Inflatable Air Pillow Travel headre...,https://m.media-amazon.com/images/I/81IHRmEcPq...,https://www.amazon.in//Trajectory-Inflatable-T...
1,A2NEQ89J8NI997,B000050YAP,3.0,987552000,product_AP,"Liznoriz esay life Velvet Travel Pillow, Pack ...",https://m.media-amazon.com/images/I/51ksLur2WK...,https://www.amazon.in//esay-life-Travel-Inflat...
2,A23GFTVIETX7DS,B000050YAP,4.0,1044403200,product_AP,"Liznoriz esay life Velvet Travel Pillow, Pack ...",https://m.media-amazon.com/images/I/51ksLur2WK...,https://www.amazon.in//esay-life-Travel-Inflat...
3,A1QZ2VZP6B9OX7,B000050YAP,4.0,997142400,product_AP,"Liznoriz esay life Velvet Travel Pillow, Pack ...",https://m.media-amazon.com/images/I/51ksLur2WK...,https://www.amazon.in//esay-life-Travel-Inflat...
4,A2R2Z3RT7RCB1L,B000050YAP,5.0,1010361600,product_AP,"Liznoriz esay life Velvet Travel Pillow, Pack ...",https://m.media-amazon.com/images/I/51ksLur2WK...,https://www.amazon.in//esay-life-Travel-Inflat...
...,...,...,...,...,...,...,...,...
115,A3UFDY7GPV3R8J,B0000AE6AP,5.0,1368144000,product_AP,"CHANCY Air Inflammable Velvet Travel Pillow, N...",https://m.media-amazon.com/images/I/41rxxvcg6v...,https://www.amazon.in//CHANCY-Inflammable-Slee...
116,A1NG5JLG4OLJB4,B0000AE6AP,5.0,1403136000,product_AP,"CHANCY Air Inflammable Velvet Travel Pillow, N...",https://m.media-amazon.com/images/I/41rxxvcg6v...,https://www.amazon.in//CHANCY-Inflammable-Slee...
117,AP579SJ9EVHLI,B0000AE6AP,5.0,1361491200,product_AP,"CHANCY Air Inflammable Velvet Travel Pillow, N...",https://m.media-amazon.com/images/I/41rxxvcg6v...,https://www.amazon.in//CHANCY-Inflammable-Slee...
118,A8G2YP67ZFS3K,B0000AE6AP,5.0,1374364800,product_AP,"CHANCY Air Inflammable Velvet Travel Pillow, N...",https://m.media-amazon.com/images/I/41rxxvcg6v...,https://www.amazon.in//CHANCY-Inflammable-Slee...


### Product AQ dataframe

In [136]:
df = get_scraped('aquarium plants')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Petzlifeworld Fish Tank Decoration Landscape O...,https://m.media-amazon.com/images/I/61q67h2jBJ...,https://www.amazon.in//Petzlifeworld-Decoratio...
1,"Rotala Ramosior ""Sunset"" | Rotala sp. | Rotála...",https://m.media-amazon.com/images/I/519o8I00dY...,https://www.amazon.in//Ramosior-ram%C3%B3sior-...
2,BURAQ Aquarium Plants Fish Tank Decorations Ar...,https://m.media-amazon.com/images/I/81B-ZqRYsx...,https://www.amazon.in//BURAQ-Decorations-Artif...
3,Wowpets Live Aquarium Plants Seeds (Betta Bulb...,https://m.media-amazon.com/images/I/6112EKeQwO...,https://www.amazon.in//Wowpets-Aquarium-Plante...
4,CNZ 10-piece Green Plastic Aquarium Tank Plant...,https://m.media-amazon.com/images/I/61n+ySRNa3...,https://www.amazon.in//CNZ-10-Piece-Plastic-Aq...
...,...,...,...
283,Aquarium/Fishtank Decorative Artificial Plants,https://m.media-amazon.com/images/I/61-uUQKudQ...,https://www.amazon.in//JAINSONS-PET-PRODUCTS-D...
284,JAINSONS PET PRODUCTS® Natural Looking Aquariu...,https://m.media-amazon.com/images/I/61EzEzb5zA...,https://www.amazon.in//JAINSONS-PET-PRODUCTS%C...
285,JAINSONS PET PRODUCTS® Aquarium Resin Artifici...,https://m.media-amazon.com/images/I/71kb8-DpNw...,https://www.amazon.in//Jainsons-Aquarium-Resin...
286,Maalavya Java Moss Box Live Aquatic Water Plan...,https://m.media-amazon.com/images/I/41GHWCCsUM...,https://www.amazon.in//Maalavya-Java-Aquatic-W...


In [133]:
AQ = df.copy()
AQ

,product_name,product_img,product_link
0,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
1,Pet Konnect Sobo FK-1503 Aquarium Filter Pump,https://m.media-amazon.com/images/I/61IFrBd20E...,https://www.amazon.in//Pet-Konnect-FK-1503-Aqu...
2,Venus Aqua® 6005F Internal Power Filter,https://m.media-amazon.com/images/I/81ekCKDMEV...,https://www.amazon.in//Venus-6005F-Internal-Po...
3,Sunsun HW-303A Aquarium Canister Filter Sunsun...,https://m.media-amazon.com/images/I/51bVqTlF9n...,https://www.amazon.in//SunsuSunsun-HW-303A-Aqu...
4,"Sobo WP 5001 Aquarium Internal Filter, 1 Piece",https://m.media-amazon.com/images/I/31HEuKeXDa...,https://www.amazon.in//Sobo-5001-Aquarium-Inte...
...,...,...,...
205,Spiryfi 3 PCS - HM Aqua 5 Micron Water Filter ...,https://m.media-amazon.com/images/I/61xCOwvgPA...,https://www.amazon.in//Spiryfi-PCS-Micron-Suit...
206,Submersible Aquarium Internal Filter Water Pum...,https://m.media-amazon.com/images/I/61bQrfzVNH...,https://www.amazon.in//Submersible-Aquarium-In...
207,SOBO WP-1150F AQUARIUM FILTER,https://m.media-amazon.com/images/I/51s6MckWfq...,https://www.amazon.in//SOBO-WP-1150F-AQUARIUM-...
208,Xilong Aquarium Fish Tank Multi Function 3 in ...,https://m.media-amazon.com/images/I/61jiQQ9FZO...,https://www.amazon.in//Xilong-Aquarium-Functio...


In [137]:
AQ = AQ.append(df).reset_index(drop=True)
AQ

,product_name,product_img,product_link
0,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
1,Pet Konnect Sobo FK-1503 Aquarium Filter Pump,https://m.media-amazon.com/images/I/61IFrBd20E...,https://www.amazon.in//Pet-Konnect-FK-1503-Aqu...
2,Venus Aqua® 6005F Internal Power Filter,https://m.media-amazon.com/images/I/81ekCKDMEV...,https://www.amazon.in//Venus-6005F-Internal-Po...
3,Sunsun HW-303A Aquarium Canister Filter Sunsun...,https://m.media-amazon.com/images/I/51bVqTlF9n...,https://www.amazon.in//SunsuSunsun-HW-303A-Aqu...
4,"Sobo WP 5001 Aquarium Internal Filter, 1 Piece",https://m.media-amazon.com/images/I/31HEuKeXDa...,https://www.amazon.in//Sobo-5001-Aquarium-Inte...
...,...,...,...
751,Aquarium/Fishtank Decorative Artificial Plants,https://m.media-amazon.com/images/I/61-uUQKudQ...,https://www.amazon.in//JAINSONS-PET-PRODUCTS-D...
752,JAINSONS PET PRODUCTS® Natural Looking Aquariu...,https://m.media-amazon.com/images/I/61EzEzb5zA...,https://www.amazon.in//JAINSONS-PET-PRODUCTS%C...
753,JAINSONS PET PRODUCTS® Aquarium Resin Artifici...,https://m.media-amazon.com/images/I/71kb8-DpNw...,https://www.amazon.in//Jainsons-Aquarium-Resin...
754,Maalavya Java Moss Box Live Aquatic Water Plan...,https://m.media-amazon.com/images/I/41GHWCCsUM...,https://www.amazon.in//Maalavya-Java-Aquatic-W...


In [130]:
product_AQ = pd.DataFrame(reviews[reviews['product_code']=='product_AQ']).reset_index(drop=True)
product_AQ

,user_id,product_id,Ratings,Time_stamp,product_code
0,AD9ZL63H0VKCZ,B00004ZCAQ,5.0,1386892800,product_AQ
1,A212M2SLD3OYN6,B00004ZCAQ,1.0,1397606400,product_AQ
2,A3H7I249HK26NI,B00004ZCAQ,5.0,1206057600,product_AQ
3,A1SX9PHJWTDMF9,B00004ZCAQ,5.0,1310860800,product_AQ
4,AMAYUGZZOP39X,B00004ZCAQ,5.0,1331683200,product_AQ
...,...,...,...,...,...
11654,A1UMSB7LAW0RIR,B00KOLNVAQ,4.0,1405900800,product_AQ
11655,A11K70GWSB7A5Z,B00KOLNVAQ,3.0,1403568000,product_AQ
11656,A1WQPJLK01B424,B00KOLNVAQ,1.0,1405296000,product_AQ
11657,A1XJOSJN6FHFO0,B00KOLNVAQ,5.0,1403654400,product_AQ


In [131]:
len(product_AQ.product_id.unique())

742

In [138]:
product_AQ = pd.DataFrame(reviews[reviews['product_code']=='product_AQ']).reset_index(drop=True)
product_AQ =  pd.merge(product_AQ, 
                       AQ.iloc[:len(product_AQ.product_id.unique())]\
                       .join(pd.Series(product_AQ['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AQ.to_csv('product_AQ.csv',index=False)
pd.read_csv('product_AQ.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AD9ZL63H0VKCZ,B00004ZCAQ,5.0,1386892800,product_AQ,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
1,A212M2SLD3OYN6,B00004ZCAQ,1.0,1397606400,product_AQ,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
2,A3H7I249HK26NI,B00004ZCAQ,5.0,1206057600,product_AQ,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
3,A1SX9PHJWTDMF9,B00004ZCAQ,5.0,1310860800,product_AQ,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
4,AMAYUGZZOP39X,B00004ZCAQ,5.0,1331683200,product_AQ,Despacito Filter Sponge for Aquarium Fish Tank...,https://m.media-amazon.com/images/I/71+2XsI3UC...,https://www.amazon.in//DESPACITO%C2%AE-XY-2902...
...,...,...,...,...,...,...,...,...
11654,A1UMSB7LAW0RIR,B00KOLNVAQ,4.0,1405900800,product_AQ,Mistletoe� Product Ring Shape Plastic Green Gr...,https://m.media-amazon.com/images/I/81GQ6Gr69A...,https://www.amazon.in//Shape-Plastic-Green-Gra...
11655,A11K70GWSB7A5Z,B00KOLNVAQ,3.0,1403568000,product_AQ,Mistletoe� Product Ring Shape Plastic Green Gr...,https://m.media-amazon.com/images/I/81GQ6Gr69A...,https://www.amazon.in//Shape-Plastic-Green-Gra...
11656,A1WQPJLK01B424,B00KOLNVAQ,1.0,1405296000,product_AQ,Mistletoe� Product Ring Shape Plastic Green Gr...,https://m.media-amazon.com/images/I/81GQ6Gr69A...,https://www.amazon.in//Shape-Plastic-Green-Gra...
11657,A1XJOSJN6FHFO0,B00KOLNVAQ,5.0,1403654400,product_AQ,Mistletoe� Product Ring Shape Plastic Green Gr...,https://m.media-amazon.com/images/I/81GQ6Gr69A...,https://www.amazon.in//Shape-Plastic-Green-Gra...


### Product AR dataframe

In [39]:
df = get_scraped('artificial rose')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
1,SATYAM KRAFT 1 Pcs Artificial Multiflora Flowe...,https://m.media-amazon.com/images/I/81zfrEGpLf...,https://www.amazon.in//SATYAM-KRAFT-Artificial...
2,Lavanaya Silver Artificial 24 Karat Gold Rose ...,https://m.media-amazon.com/images/I/51mx9XHoYv...,https://www.amazon.in//LAKSHMI-ENTERPRISES-Art...
3,Fourwalls Artificial Polyester and Plastic Ros...,https://m.media-amazon.com/images/I/31PctPdRtt...,https://www.amazon.in//Fourwalls-Artificial-Fa...
4,BS AMOR Multicolour Shade Roses for DIY Weddin...,https://m.media-amazon.com/images/I/81TiqDWMyQ...,https://www.amazon.in//BS-AMOR-Flowers-Arrange...
...,...,...,...
301,Okeyplus SGR Product Artificial red Rose Bonsa...,https://m.media-amazon.com/images/I/41sIccKlvA...,https://www.amazon.in//Okeyplus-Product-Artifi...
302,Petals and Roses Pink Valley Rose. Beautiful A...,https://m.media-amazon.com/images/I/71sRjtPRhK...,https://www.amazon.in//Petals-Valley-Beautiful...
303,Breewell Mini Artificial Yellow Roses Bonsai F...,https://m.media-amazon.com/images/I/61pmH7rcEf...,https://www.amazon.in//BreewellTM-Artificial-O...
304,"Relanta-Beautiful White Rose Bunch Decorative,...",https://m.media-amazon.com/images/I/51f+fNQcC9...,https://www.amazon.in//Relanta-Beautiful-Decor...


In [40]:
AR = df.copy()
AR

,product_name,product_img,product_link
0,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
1,SATYAM KRAFT 1 Pcs Artificial Multiflora Flowe...,https://m.media-amazon.com/images/I/81zfrEGpLf...,https://www.amazon.in//SATYAM-KRAFT-Artificial...
2,Lavanaya Silver Artificial 24 Karat Gold Rose ...,https://m.media-amazon.com/images/I/51mx9XHoYv...,https://www.amazon.in//LAKSHMI-ENTERPRISES-Art...
3,Fourwalls Artificial Polyester and Plastic Ros...,https://m.media-amazon.com/images/I/31PctPdRtt...,https://www.amazon.in//Fourwalls-Artificial-Fa...
4,BS AMOR Multicolour Shade Roses for DIY Weddin...,https://m.media-amazon.com/images/I/81TiqDWMyQ...,https://www.amazon.in//BS-AMOR-Flowers-Arrange...
...,...,...,...
301,Okeyplus SGR Product Artificial red Rose Bonsa...,https://m.media-amazon.com/images/I/41sIccKlvA...,https://www.amazon.in//Okeyplus-Product-Artifi...
302,Petals and Roses Pink Valley Rose. Beautiful A...,https://m.media-amazon.com/images/I/71sRjtPRhK...,https://www.amazon.in//Petals-Valley-Beautiful...
303,Breewell Mini Artificial Yellow Roses Bonsai F...,https://m.media-amazon.com/images/I/61pmH7rcEf...,https://www.amazon.in//BreewellTM-Artificial-O...
304,"Relanta-Beautiful White Rose Bunch Decorative,...",https://m.media-amazon.com/images/I/51f+fNQcC9...,https://www.amazon.in//Relanta-Beautiful-Decor...


In [ ]:
# AR = AR.append(df).reset_index(drop=True)
# AR

In [37]:
product_AR = pd.DataFrame(reviews[reviews['product_code']=='product_AR']).reset_index(drop=True)
product_AR

,user_id,product_id,Ratings,Time_stamp,product_code
0,A37STTLP6UXK6J,B00000J0AR,1.0,1191974400,product_AR
1,A1J5LI9FUHQNRO,B00000J0AR,2.0,1206576000,product_AR
2,A3D296RVX0JVKB,B00000J0AR,3.0,1045612800,product_AR
3,A2G4OJFE2Q4HJO,B00000J0AR,5.0,1372204800,product_AR
4,A1JETHCL85UIBW,B00000J0AR,2.0,1259798400,product_AR
...,...,...,...,...,...
186,A30UB7SES197S3,B00009V3AR,3.0,1379116800,product_AR
187,A2DCIK85U0RKSC,B0000A2UAR,1.0,1064448000,product_AR
188,A3C602P4QQYY2V,B0000A2UAR,1.0,1171843200,product_AR
189,A253QSTOLRMTO0,B0000A2UAR,5.0,1093651200,product_AR


In [38]:
len(product_AR.product_id.unique())

17

In [41]:
product_AR = pd.DataFrame(reviews[reviews['product_code']=='product_AR']).reset_index(drop=True)
product_AR =  pd.merge(product_AR, 
                       AR.iloc[:len(product_AR.product_id.unique())]\
                       .join(pd.Series(product_AR['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AR.to_csv('product_AR.csv',index=False)
pd.read_csv('product_AR.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A37STTLP6UXK6J,B00000J0AR,1.0,1191974400,product_AR,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
1,A1J5LI9FUHQNRO,B00000J0AR,2.0,1206576000,product_AR,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
2,A3D296RVX0JVKB,B00000J0AR,3.0,1045612800,product_AR,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
3,A2G4OJFE2Q4HJO,B00000J0AR,5.0,1372204800,product_AR,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
4,A1JETHCL85UIBW,B00000J0AR,2.0,1259798400,product_AR,"Fourwalls Artificial Rose Bouquet (Red, 1 Pc)",https://m.media-amazon.com/images/I/613G46ipeA...,https://www.amazon.in//Fourwalls-Artificial-Ve...
...,...,...,...,...,...,...,...,...
186,A30UB7SES197S3,B00009V3AR,3.0,1379116800,product_AR,TIED RIBBONS Artificial Rose Flowers Bunches f...,https://m.media-amazon.com/images/I/71k4IPY7wV...,https://www.amazon.in//TIED-RIBBONS-Artificial...
187,A2DCIK85U0RKSC,B0000A2UAR,1.0,1064448000,product_AR,Artificial 24K Golden and Red Rose with Gift B...,https://m.media-amazon.com/images/I/41czDc99yr...,https://www.amazon.in//Token-Love-Valentine-Ar...
188,A3C602P4QQYY2V,B0000A2UAR,1.0,1171843200,product_AR,Artificial 24K Golden and Red Rose with Gift B...,https://m.media-amazon.com/images/I/41czDc99yr...,https://www.amazon.in//Token-Love-Valentine-Ar...
189,A253QSTOLRMTO0,B0000A2UAR,5.0,1093651200,product_AR,Artificial 24K Golden and Red Rose with Gift B...,https://m.media-amazon.com/images/I/41czDc99yr...,https://www.amazon.in//Token-Love-Valentine-Ar...


### Product AS dataframe

In [56]:
df = get_scraped('agarbatti stand steel')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,NATULIX Stainless Steel Agarbatti Stand | Ince...,https://m.media-amazon.com/images/I/51eWFb5sZJ...,https://www.amazon.in//NATULIX%C2%AE-Stainless...
1,RIDDHAH COLLECTIONS Stainless Steel Agarbatti ...,https://m.media-amazon.com/images/I/411uRwU3Xn...,https://www.amazon.in//RIDDHAH-COLLECTIONS-Saf...
2,Mini House Stainless Steel Agarbatti Incense S...,https://m.media-amazon.com/images/I/511RKnJSjp...,https://www.amazon.in//Mini-House-Stainless-Ag...
3,Velofeil® Stainless Steel Agarbatti Incense St...,https://m.media-amazon.com/images/I/51kEHFLczI...,https://www.amazon.in//Velofeil-Stainless-Agar...
4,Kriwin Metal Agarbatti Stand With Dhoop Holder...,https://m.media-amazon.com/images/I/41bVm05y1U...,https://www.amazon.in//KRIWIN%C2%AE-Safety-Aga...
...,...,...,...
301,GODHEAD® Stainless Steel Agarbatti Stand | Inc...,https://m.media-amazon.com/images/I/41t+S4qG28...,https://www.amazon.in//GODHEAD%C2%AE-Stainless...
302,Fabusnap Stainless Steel Agarbatti Stand | Inc...,https://m.media-amazon.com/images/I/31kREgHz9I...,https://www.amazon.in//Fabusnap-Stainless-Agar...
303,Incense Stick Holder Stainless Steel Agarbatti...,https://m.media-amazon.com/images/I/618kMJKFX3...,https://www.amazon.in//Incense-Holder-Stainles...
304,BANSIGOODS Diwali Pooja Celebration Gift Stain...,https://m.media-amazon.com/images/I/51euK-S1hd...,https://www.amazon.in//BANSIGOODS-Celebration-...


In [47]:
AS = df.copy()
AS

,product_name,product_img,product_link
0,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
1,KC KULLICRAFT Marble Soapstone Incense Stick H...,https://m.media-amazon.com/images/I/81JnAmeyl1...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
2,KC KULLICRAFT Marble Soapstone Incense Stick H...,https://m.media-amazon.com/images/I/81n84CLa94...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
3,KC KULLICRAFT Marble Soapstone Bottle Shape Ag...,https://m.media-amazon.com/images/I/81X4tOvYi4...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
4,KC KULLICRAFT Marble Soapstone Incense Stick H...,https://m.media-amazon.com/images/I/81tleGKrBz...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
...,...,...,...
205,Pooja Creation Stone Incense And Candle Holder...,https://m.media-amazon.com/images/I/41InnrTlN+...,https://www.amazon.in//Pooja-Creation-Hand-Car...
206,Natural India Wooden Craft Bamboo Incense Stic...,https://m.media-amazon.com/images/I/41owds1+35...,https://www.amazon.in//Natural-India-Wooden-In...
207,Vaishnavi Art Collection Marble Agarbatti Stan...,https://m.media-amazon.com/images/I/71YAL1OT0K...,https://www.amazon.in//Vaishnavi-Art-Collectio...
208,Desi Decor Marble Stoneware Incense Agarbatti ...,https://m.media-amazon.com/images/I/31Zuu7Z1bM...,https://www.amazon.in//Desi-Decor-Stoneware-In...


In [53]:
AS = AS.append(df).reset_index(drop=True)
AS

,product_name,product_img,product_link
0,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
1,KC KULLICRAFT Marble Soapstone Incense Stick H...,https://m.media-amazon.com/images/I/81JnAmeyl1...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
2,KC KULLICRAFT Marble Soapstone Incense Stick H...,https://m.media-amazon.com/images/I/81n84CLa94...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
3,KC KULLICRAFT Marble Soapstone Bottle Shape Ag...,https://m.media-amazon.com/images/I/81X4tOvYi4...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
4,KC KULLICRAFT Marble Soapstone Incense Stick H...,https://m.media-amazon.com/images/I/81tleGKrBz...,https://www.amazon.in//KC-KULLICRAFT-Soapstone...
...,...,...,...
769,Articia Aluminum Incense Holder & Ash Catcher ...,https://m.media-amazon.com/images/I/81ete3PBlb...,https://www.amazon.in//Articia-Aluminum-Agarba...
770,Aapkidukan Stainless Steel Agarbatti Stand,https://m.media-amazon.com/images/I/41bDn-DLCn...,https://www.amazon.in//Aapkidukan-Stainless-St...
771,faas Incense Holder Agarbatti Stand with Ash C...,https://m.media-amazon.com/images/I/51dOAVlWC8...,https://www.amazon.in//faas-Incense-Holder-Aga...
772,Labdhi Stainless Steel Agarbatti Stand | Incen...,https://m.media-amazon.com/images/I/41VXVnmZ0F...,https://www.amazon.in//Labdhi-Stainless-Agarba...


In [44]:
product_AS = pd.DataFrame(reviews[reviews['product_code']=='product_AS']).reset_index(drop=True)
product_AS

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3T5MAHVS6DL69,B00004Z5AS,1.0,1402358400,product_AS
1,A312ZWTKCLVKDQ,B00004Z5AS,5.0,1389312000,product_AS
2,AGUOJBOF1LQUI,B00004Z5AS,2.0,1217030400,product_AS
3,A3VFT39XQRH9QU,B00004Z5AS,5.0,1405036800,product_AS
4,A2EZB9PA031UYC,B00004Z5AS,4.0,1265846400,product_AS
...,...,...,...,...,...
13430,A2DQ0UDQIWJMWQ,B00JV8P4AS,5.0,1405123200,product_AS
13431,A18DHWNUY0CXNB,B00K1J11AS,5.0,1403481600,product_AS
13432,A3RDFZC3XOHOLR,B00KC6GIAS,3.0,1404086400,product_AS
13433,A15EIOBJMWZNPE,B00KC6GIAS,1.0,1405814400,product_AS


In [45]:
len(product_AS.product_id.unique())

704

In [54]:
product_AS = pd.DataFrame(reviews[reviews['product_code']=='product_AS']).reset_index(drop=True)
product_AS =  pd.merge(product_AS, 
                       AS.iloc[:len(product_AS.product_id.unique())]\
                       .join(pd.Series(product_AS['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AS.to_csv('product_AS.csv',index=False)
pd.read_csv('product_AS.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3T5MAHVS6DL69,B00004Z5AS,1.0,1402358400,product_AS,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
1,A312ZWTKCLVKDQ,B00004Z5AS,5.0,1389312000,product_AS,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
2,AGUOJBOF1LQUI,B00004Z5AS,2.0,1217030400,product_AS,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
3,A3VFT39XQRH9QU,B00004Z5AS,5.0,1405036800,product_AS,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
4,A2EZB9PA031UYC,B00004Z5AS,4.0,1265846400,product_AS,KC KULLICRAFT Handcrafted Soapstone Marble Wor...,https://m.media-amazon.com/images/I/81qH4NAsOY...,https://www.amazon.in//KC-KULLICRAFT-Handcraft...
...,...,...,...,...,...,...,...,...
13430,A2DQ0UDQIWJMWQ,B00JV8P4AS,5.0,1405123200,product_AS,UnequeTrend Agarbatti Stand with Plate | 7 Inc...,https://m.media-amazon.com/images/I/41XUc9UhqG...,https://www.amazon.in//UnequeTrend-Agarbatti-I...
13431,A18DHWNUY0CXNB,B00K1J11AS,5.0,1403481600,product_AS,Ashoka Dry Fruit Stainless Steel Agarbatti Sta...,https://m.media-amazon.com/images/I/41zXptVmS4...,https://www.amazon.in//Ashoka-Dry-Fruit-Stainl...
13432,A3RDFZC3XOHOLR,B00KC6GIAS,3.0,1404086400,product_AS,Phool Batti Stainless Steel Agarbatti/Incense ...,https://m.media-amazon.com/images/I/41P4t27sRU...,https://www.amazon.in//Phool-Batti-Stainless-A...
13433,A15EIOBJMWZNPE,B00KC6GIAS,1.0,1405814400,product_AS,Phool Batti Stainless Steel Agarbatti/Incense ...,https://m.media-amazon.com/images/I/41P4t27sRU...,https://www.amazon.in//Phool-Batti-Stainless-A...


### Product AT dataframe

In [59]:
df = get_scraped('atta')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
1,"Fortune Chakki Fresh Atta, 5kg",https://m.media-amazon.com/images/I/61AOiVfvtB...,https://www.amazon.in//Fortune-Chakki-Fresh-At...
2,"Aashirvaad Organic Atta, 1 kg, 100% Organic Wh...",https://m.media-amazon.com/images/I/91U4oQatpX...,https://www.amazon.in//Aashirvaad-Organic-Atta...
3,"Aashirvaad Superior MP Atta, 5kg Pack",https://m.media-amazon.com/images/I/81+FF+Rsym...,https://www.amazon.in//Aashirvaad-Superior-MP-...
4,"Fortune Chakki Fresh Atta, 10 kg",https://m.media-amazon.com/images/I/618GH1zvSR...,https://www.amazon.in//Fortune-Chakki-Fresh-At...
...,...,...,...
205,Native Organica Cold Pressed Khapli Wheat Atta...,https://m.media-amazon.com/images/I/31uutDMwBu...,https://www.amazon.in//Native-Organica-Pressed...
206,K-Pra K Pra Rajgira Flour | Rajgira Peeth | Am...,https://m.media-amazon.com/images/I/71f3gyMOG6...,https://www.amazon.in//K-Pra-Rajgira-Flour-Att...
207,Conscious Food Refined Wheat Flour | Chakki Gr...,https://m.media-amazon.com/images/I/81G6y0Dt9c...,https://www.amazon.in//Conscious-Food-Organic-...
208,Farm Desi Natural Black Wheat Atta 1 Kg,https://m.media-amazon.com/images/I/41Rmdo5643...,https://www.amazon.in//Farm-Desi-Natural-Black...


In [60]:
AT = df.copy()
AT

,product_name,product_img,product_link
0,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
1,"Fortune Chakki Fresh Atta, 5kg",https://m.media-amazon.com/images/I/61AOiVfvtB...,https://www.amazon.in//Fortune-Chakki-Fresh-At...
2,"Aashirvaad Organic Atta, 1 kg, 100% Organic Wh...",https://m.media-amazon.com/images/I/91U4oQatpX...,https://www.amazon.in//Aashirvaad-Organic-Atta...
3,"Aashirvaad Superior MP Atta, 5kg Pack",https://m.media-amazon.com/images/I/81+FF+Rsym...,https://www.amazon.in//Aashirvaad-Superior-MP-...
4,"Fortune Chakki Fresh Atta, 10 kg",https://m.media-amazon.com/images/I/618GH1zvSR...,https://www.amazon.in//Fortune-Chakki-Fresh-At...
...,...,...,...
205,Native Organica Cold Pressed Khapli Wheat Atta...,https://m.media-amazon.com/images/I/31uutDMwBu...,https://www.amazon.in//Native-Organica-Pressed...
206,K-Pra K Pra Rajgira Flour | Rajgira Peeth | Am...,https://m.media-amazon.com/images/I/71f3gyMOG6...,https://www.amazon.in//K-Pra-Rajgira-Flour-Att...
207,Conscious Food Refined Wheat Flour | Chakki Gr...,https://m.media-amazon.com/images/I/81G6y0Dt9c...,https://www.amazon.in//Conscious-Food-Organic-...
208,Farm Desi Natural Black Wheat Atta 1 Kg,https://m.media-amazon.com/images/I/41Rmdo5643...,https://www.amazon.in//Farm-Desi-Natural-Black...


In [ ]:
# AT = AT.append(df).reset_index(drop=True)
# AT

In [57]:
product_AT = pd.DataFrame(reviews[reviews['product_code']=='product_AT']).reset_index(drop=True)
product_AT

,user_id,product_id,Ratings,Time_stamp,product_code
0,AK3VALQTT1EWO,B00000JBAT,1.0,947116800,product_AT
1,A3T8G23JTU5C26,B00000JBAT,4.0,962928000,product_AT
2,A1R9QOPV6HVEKF,B00000JBAT,3.0,950486400,product_AT
3,A1ABVP0DV1ZN89,B00000JBAT,5.0,929232000,product_AT
4,A1XOX87IOIJ9IJ,B00000JBAT,4.0,945820800,product_AT
...,...,...,...,...,...
264,A1Y0IHS2CUGPLJ,B0000A5AAT,3.0,1210982400,product_AT
265,A2YZCPQGNAKT4R,B0000A5AAT,5.0,1306022400,product_AT
266,ABIB8OLFEJL5Q,B0000AJ5AT,5.0,1067990400,product_AT
267,A3ID4Z6GXFXC8G,B0000AJ5AT,5.0,1069200000,product_AT


In [58]:
len(product_AT.product_id.unique())

17

In [61]:
product_AT = pd.DataFrame(reviews[reviews['product_code']=='product_AT']).reset_index(drop=True)
product_AT =  pd.merge(product_AT, 
                       AT.iloc[:len(product_AT.product_id.unique())]\
                       .join(pd.Series(product_AT['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AT.to_csv('product_AT.csv',index=False)
pd.read_csv('product_AT.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AK3VALQTT1EWO,B00000JBAT,1.0,947116800,product_AT,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
1,A3T8G23JTU5C26,B00000JBAT,4.0,962928000,product_AT,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
2,A1R9QOPV6HVEKF,B00000JBAT,3.0,950486400,product_AT,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
3,A1ABVP0DV1ZN89,B00000JBAT,5.0,929232000,product_AT,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
4,A1XOX87IOIJ9IJ,B00000JBAT,4.0,945820800,product_AT,"Aashirvaad Shudh Chakki Atta, 10kg Pack",https://m.media-amazon.com/images/I/81pDk4-gYQ...,https://www.amazon.in//Aashirvaad-Shudh-Chakki...
...,...,...,...,...,...,...,...,...
264,A1Y0IHS2CUGPLJ,B0000A5AAT,3.0,1210982400,product_AT,"Pillsbury Gold Atta 5kg, Sharbati Atta, Madhya...",https://m.media-amazon.com/images/I/71+ZAUgr8v...,https://www.amazon.in//Pillsbury-Sharbati-Madh...
265,A2YZCPQGNAKT4R,B0000A5AAT,5.0,1306022400,product_AT,"Pillsbury Gold Atta 5kg, Sharbati Atta, Madhya...",https://m.media-amazon.com/images/I/71+ZAUgr8v...,https://www.amazon.in//Pillsbury-Sharbati-Madh...
266,ABIB8OLFEJL5Q,B0000AJ5AT,5.0,1067990400,product_AT,"Aashirvaad Organic Atta, 5 kg, 100% Organic Wh...",https://m.media-amazon.com/images/I/91U4oQatpX...,https://www.amazon.in//Natures-Superfoods-Aash...
267,A3ID4Z6GXFXC8G,B0000AJ5AT,5.0,1069200000,product_AT,"Aashirvaad Organic Atta, 5 kg, 100% Organic Wh...",https://m.media-amazon.com/images/I/91U4oQatpX...,https://www.amazon.in//Natures-Superfoods-Aash...


### Product AU dataframe

In [103]:
df = get_scraped('automation sensor')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,JAIN AUTOMATION M18 PNP No Inductive Proximity...,https://m.media-amazon.com/images/I/51dxuzo7SF...,https://www.amazon.in//JAIN-AUTOMATION-Inducti...
1,HomeOne Technologies- Motion Sensor- Smart Mot...,https://m.media-amazon.com/images/I/513eX-u+E7...,https://www.amazon.in//DEFT-Degree-Pa-PIR-Dete...
2,Electronic spices HC-SR501 Pyroelectric Infrar...,https://m.media-amazon.com/images/I/51-liqYKY4...,https://www.amazon.in//Electronicspices-HC-SR5...
3,Robotbanao HC-SR501 Passive Infrared (PIR) Mot...,https://m.media-amazon.com/images/I/61jnYFfb6q...,https://www.amazon.in//Robotbanao-Hcsr501-Moti...
4,JAIN AUTOMATION M18 NPN NO Inductive Proximity...,https://m.media-amazon.com/images/I/512WPcXbyh...,https://www.amazon.in//JAIN-AUTOMATION-Inducti...
...,...,...,...
301,PIR Motion Sensor Detector Module (Pyroelectri...,https://m.media-amazon.com/images/I/416JTeIewH...,https://www.amazon.in//Motion-Sensor-Detector-...
302,VMS INDIA AM312 Mini Infrared PIR Human Motion...,https://m.media-amazon.com/images/I/51W0AxR582...,https://www.amazon.in//VMS-INDIA-Infrared-Dete...
303,Smarty Pet Pet Droid Sound Module & 3 Motion S...,https://m.media-amazon.com/images/I/41d0WQD3Ds...,https://www.amazon.in//Smarty-Pet-Module-Motio...
304,Protium 32 Steps LED Stair Lights Controller W...,https://m.media-amazon.com/images/I/51bFaq9Zb+...,https://www.amazon.in//Protium-Lights-Controll...


In [100]:
AU = df.copy()
AU

,product_name,product_img,product_link
0,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81Qkb3J+Op...,https://www.amazon.in//ThinkerPlace-Educationa...
1,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81ZxcVK0A2...,https://www.amazon.in//ThinkerPlace-Educationa...
2,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
3,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81PKfCtrWV...,https://www.amazon.in//ThinkerPlace-Educationa...
4,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
...,...,...,...
283,"Speed Reduction Motor, Transmission Parts 300R...",https://m.media-amazon.com/images/I/51BNErbX-+...,https://www.amazon.in//Reduction-Transmission-...
284,Qoretronix ESP8266 Development Board Small Bat...,https://m.media-amazon.com/images/I/41ecYbX4XO...,https://www.amazon.in//Qoretronix-ESP8266-Deve...
285,HomeMate WiFi 1 Node Smart Switch | 16A | No H...,https://m.media-amazon.com/images/I/61l5I+xeg0...,https://www.amazon.in//HomeMate-WiFi-Multi-Sma...
286,2022 Guide to UX/UI Design In 45 Minutes for B...,https://m.media-amazon.com/images/I/61GgULMDkR...,https://www.amazon.in//2022-Guide-Design-Minut...


In [104]:
AU = AU.append(df).reset_index(drop=True)
AU

,product_name,product_img,product_link
0,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81Qkb3J+Op...,https://www.amazon.in//ThinkerPlace-Educationa...
1,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81ZxcVK0A2...,https://www.amazon.in//ThinkerPlace-Educationa...
2,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
3,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81PKfCtrWV...,https://www.amazon.in//ThinkerPlace-Educationa...
4,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
...,...,...,...
781,PIR Motion Sensor Detector Module (Pyroelectri...,https://m.media-amazon.com/images/I/416JTeIewH...,https://www.amazon.in//Motion-Sensor-Detector-...
782,VMS INDIA AM312 Mini Infrared PIR Human Motion...,https://m.media-amazon.com/images/I/51W0AxR582...,https://www.amazon.in//VMS-INDIA-Infrared-Dete...
783,Smarty Pet Pet Droid Sound Module & 3 Motion S...,https://m.media-amazon.com/images/I/41d0WQD3Ds...,https://www.amazon.in//Smarty-Pet-Module-Motio...
784,Protium 32 Steps LED Stair Lights Controller W...,https://m.media-amazon.com/images/I/51bFaq9Zb+...,https://www.amazon.in//Protium-Lights-Controll...


In [63]:
product_AU = pd.DataFrame(reviews[reviews['product_code']=='product_AU']).reset_index(drop=True)
product_AU

,user_id,product_id,Ratings,Time_stamp,product_code
0,A112XTXZQDVE91,B00000J4AU,5.0,1387152000,product_AU
1,A3MQ87ZMVV33BZ,B00001QEAU,5.0,1355529600,product_AU
2,A1AC0ZLC6SVXSD,B0000513AU,3.0,1389225600,product_AU
3,A2QMSUZRFBOQGW,B0000513AU,5.0,1388275200,product_AU
4,A1YW0WALRA96OJ,B0000513AU,5.0,1284768000,product_AU
...,...,...,...,...,...
9962,A3FFG4RP12MVSG,B00KH7NNAU,3.0,1402876800,product_AU
9963,AXD6XMUQYZIQJ,B00KH7NNAU,5.0,1401926400,product_AU
9964,A2T8QYO9DOSMOF,B00KH7NNAU,5.0,1403481600,product_AU
9965,A6TIH1VVEK33F,B00KQBUXAU,5.0,1403308800,product_AU


In [64]:
len(product_AU.product_id.unique())

723

In [105]:
product_AU = pd.DataFrame(reviews[reviews['product_code']=='product_AU']).reset_index(drop=True)
product_AU =  pd.merge(product_AU, 
                       AU.iloc[:len(product_AU.product_id.unique())]\
                       .join(pd.Series(product_AU['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AU.to_csv('product_AU.csv',index=False)
pd.read_csv('product_AU.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A112XTXZQDVE91,B00000J4AU,5.0,1387152000,product_AU,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81Qkb3J+Op...,https://www.amazon.in//ThinkerPlace-Educationa...
1,A3MQ87ZMVV33BZ,B00001QEAU,5.0,1355529600,product_AU,ThinkerPlace STEM Educational Home Automation ...,https://m.media-amazon.com/images/I/81ZxcVK0A2...,https://www.amazon.in//ThinkerPlace-Educationa...
2,A1AC0ZLC6SVXSD,B0000513AU,3.0,1389225600,product_AU,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
3,A2QMSUZRFBOQGW,B0000513AU,5.0,1388275200,product_AU,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
4,A1YW0WALRA96OJ,B0000513AU,5.0,1284768000,product_AU,MY CRADLLE Automatic Kit for Cradles for Baby ...,https://m.media-amazon.com/images/I/51wwyWDmql...,https://www.amazon.in//MY-CRADLLE-Automatic-Cr...
...,...,...,...,...,...,...,...,...
9962,A3FFG4RP12MVSG,B00KH7NNAU,3.0,1402876800,product_AU,Robotbanao HC-SR501 Pyroelectric Infrared PIR ...,https://m.media-amazon.com/images/I/61624qpt04...,https://www.amazon.in//Robotbanao-HC-SR501-Pyr...
9963,AXD6XMUQYZIQJ,B00KH7NNAU,5.0,1401926400,product_AU,Robotbanao HC-SR501 Pyroelectric Infrared PIR ...,https://m.media-amazon.com/images/I/61624qpt04...,https://www.amazon.in//Robotbanao-HC-SR501-Pyr...
9964,A2T8QYO9DOSMOF,B00KH7NNAU,5.0,1403481600,product_AU,Robotbanao HC-SR501 Pyroelectric Infrared PIR ...,https://m.media-amazon.com/images/I/61624qpt04...,https://www.amazon.in//Robotbanao-HC-SR501-Pyr...
9965,A6TIH1VVEK33F,B00KQBUXAU,5.0,1403308800,product_AU,PIR Motion Detector Sensor Module,https://m.media-amazon.com/images/I/61ZgVCMxk1...,https://www.amazon.in//PIR-Motion-Detector-Sen...


### Product AV dataframe

In [77]:
df = get_scraped('avengers toys')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Avenger Toy Set for Kids Super Hero Action Fig...,https://m.media-amazon.com/images/I/51xwAOKn--...,https://www.amazon.in//Avenger-Action-Children...
1,"Teddylover Elephant Soft Toy, Baby Toy, Kids T...",https://m.media-amazon.com/images/I/51R2ZQRQvp...,https://www.amazon.in//Teddylover-Elephant-Sof...
2,RKZ Avenger Super Hero Action Figure Toy Set (...,https://m.media-amazon.com/images/I/51rt2g7vbW...,https://www.amazon.in//RKZ-Superheroes-Genuine...
3,SaleOn Set of 4 Cars Toys Avengers Character F...,https://m.media-amazon.com/images/I/81lXRs1LnL...,https://www.amazon.in//SaleOn-Avengers-Charact...
4,SaleOn Set of 6 Cars Toys Avengers Character F...,https://m.media-amazon.com/images/I/71uwcrU2sl...,https://www.amazon.in//SaleOn-Avengers-Charact...
...,...,...,...
283,MARVEL Avengers Titan Hero Series Blast Gear D...,https://m.media-amazon.com/images/I/71XboxdV4R...,https://www.amazon.in//Marvel-Avengers-Deluxe-...
284,OKO Heros Action Figure Toys Set for Kids Aven...,https://m.media-amazon.com/images/I/61XRR5Y+0H...,https://www.amazon.in//OKO-Action-Figure-Aveng...
285,BEAU STUTI Super Heroes 6” Inch Action Figures...,https://m.media-amazon.com/images/I/41fhucHd75...,https://www.amazon.in//BEAU-STUTI-Heroes-Figur...
286,HALO NATION® Dance Superhero Swinging Hero Lig...,https://m.media-amazon.com/images/I/41kGubQ9Zx...,https://www.amazon.in//HALO-Superhero-Swinging...


In [78]:
AV = df.copy()
AV

,product_name,product_img,product_link
0,Avenger Toy Set for Kids Super Hero Action Fig...,https://m.media-amazon.com/images/I/51xwAOKn--...,https://www.amazon.in//Avenger-Action-Children...
1,"Teddylover Elephant Soft Toy, Baby Toy, Kids T...",https://m.media-amazon.com/images/I/51R2ZQRQvp...,https://www.amazon.in//Teddylover-Elephant-Sof...
2,RKZ Avenger Super Hero Action Figure Toy Set (...,https://m.media-amazon.com/images/I/51rt2g7vbW...,https://www.amazon.in//RKZ-Superheroes-Genuine...
3,SaleOn Set of 4 Cars Toys Avengers Character F...,https://m.media-amazon.com/images/I/81lXRs1LnL...,https://www.amazon.in//SaleOn-Avengers-Charact...
4,SaleOn Set of 6 Cars Toys Avengers Character F...,https://m.media-amazon.com/images/I/71uwcrU2sl...,https://www.amazon.in//SaleOn-Avengers-Charact...
...,...,...,...
283,MARVEL Avengers Titan Hero Series Blast Gear D...,https://m.media-amazon.com/images/I/71XboxdV4R...,https://www.amazon.in//Marvel-Avengers-Deluxe-...
284,OKO Heros Action Figure Toys Set for Kids Aven...,https://m.media-amazon.com/images/I/61XRR5Y+0H...,https://www.amazon.in//OKO-Action-Figure-Aveng...
285,BEAU STUTI Super Heroes 6” Inch Action Figures...,https://m.media-amazon.com/images/I/41fhucHd75...,https://www.amazon.in//BEAU-STUTI-Heroes-Figur...
286,HALO NATION® Dance Superhero Swinging Hero Lig...,https://m.media-amazon.com/images/I/41kGubQ9Zx...,https://www.amazon.in//HALO-Superhero-Swinging...


In [ ]:
# AV = AV.append(df).reset_index(drop=True)
# AV

In [75]:
product_AV = pd.DataFrame(reviews[reviews['product_code']=='product_AV']).reset_index(drop=True)
product_AV

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1OZHOW375BUCS,B00003CWAV,1.0,1296518400,product_AV
1,A2O85WY92QDWQ6,B00005A9AV,1.0,1071446400,product_AV
2,A1YE7P5S285TQ5,B00005A9AV,5.0,1162512000,product_AV
3,A1YQBLD4QM4LSE,B00005A9AV,5.0,1122422400,product_AV
4,A3D2LWR3NFCUAZ,B00005A9AV,4.0,1137110400,product_AV
...,...,...,...,...,...
182,A1MRU4PM2VDOZE,B0000ALKAV,1.0,1382400000,product_AV
183,A3175NVEI4RRHH,B0000ALKAV,5.0,1395792000,product_AV
184,A29XNECA1WWSK1,B0000ALKAV,5.0,1371427200,product_AV
185,A37KAUN3Q2765G,B0000ALKAV,5.0,1303084800,product_AV


In [76]:
len(product_AV.product_id.unique())

14

In [79]:
product_AV = pd.DataFrame(reviews[reviews['product_code']=='product_AV']).reset_index(drop=True)
product_AV =  pd.merge(product_AV, 
                       AV.iloc[:len(product_AV.product_id.unique())]\
                       .join(pd.Series(product_AV['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AV.to_csv('product_AV.csv',index=False)
pd.read_csv('product_AV.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1OZHOW375BUCS,B00003CWAV,1.0,1296518400,product_AV,Avenger Toy Set for Kids Super Hero Action Fig...,https://m.media-amazon.com/images/I/51xwAOKn--...,https://www.amazon.in//Avenger-Action-Children...
1,A2O85WY92QDWQ6,B00005A9AV,1.0,1071446400,product_AV,"Teddylover Elephant Soft Toy, Baby Toy, Kids T...",https://m.media-amazon.com/images/I/51R2ZQRQvp...,https://www.amazon.in//Teddylover-Elephant-Sof...
2,A1YE7P5S285TQ5,B00005A9AV,5.0,1162512000,product_AV,"Teddylover Elephant Soft Toy, Baby Toy, Kids T...",https://m.media-amazon.com/images/I/51R2ZQRQvp...,https://www.amazon.in//Teddylover-Elephant-Sof...
3,A1YQBLD4QM4LSE,B00005A9AV,5.0,1122422400,product_AV,"Teddylover Elephant Soft Toy, Baby Toy, Kids T...",https://m.media-amazon.com/images/I/51R2ZQRQvp...,https://www.amazon.in//Teddylover-Elephant-Sof...
4,A3D2LWR3NFCUAZ,B00005A9AV,4.0,1137110400,product_AV,"Teddylover Elephant Soft Toy, Baby Toy, Kids T...",https://m.media-amazon.com/images/I/51R2ZQRQvp...,https://www.amazon.in//Teddylover-Elephant-Sof...
...,...,...,...,...,...,...,...,...
182,A1MRU4PM2VDOZE,B0000ALKAV,1.0,1382400000,product_AV,Super Hero Toy Set of 5 Super Hero Character w...,https://m.media-amazon.com/images/I/51r0xldxt4...,https://www.amazon.in//Character-Twisting-Join...
183,A3175NVEI4RRHH,B0000ALKAV,5.0,1395792000,product_AV,Super Hero Toy Set of 5 Super Hero Character w...,https://m.media-amazon.com/images/I/51r0xldxt4...,https://www.amazon.in//Character-Twisting-Join...
184,A29XNECA1WWSK1,B0000ALKAV,5.0,1371427200,product_AV,Super Hero Toy Set of 5 Super Hero Character w...,https://m.media-amazon.com/images/I/51r0xldxt4...,https://www.amazon.in//Character-Twisting-Join...
185,A37KAUN3Q2765G,B0000ALKAV,5.0,1303084800,product_AV,Super Hero Toy Set of 5 Super Hero Character w...,https://m.media-amazon.com/images/I/51r0xldxt4...,https://www.amazon.in//Character-Twisting-Join...


### Product AW dataframe

In [110]:
df = get_scraped('automatic washing machine cover')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,KANUSHI Industries® Washable & Dustproof Top L...,https://m.media-amazon.com/images/I/81z1fy0Fzd...,https://www.amazon.in//KANUSHI-Industries%C2%A...
1,Classic® LG Top Load Washing Machine Cover Sui...,https://m.media-amazon.com/images/I/81n2tlvz52...,https://www.amazon.in//Classic-Washing-Machine...
2,Stylista Washing Machine Cover Compatible for ...,https://m.media-amazon.com/images/I/41U-7XFknd...,https://www.amazon.in//Stylista-Whirlpool-Full...
3,Stylista Washing Machine Cover Compatible for ...,https://m.media-amazon.com/images/I/61HSssPiAD...,https://www.amazon.in//Stylista-Compatible-Inv...
4,Glassiano Fully Automatic Top Load Waterproof ...,https://m.media-amazon.com/images/I/71rAUlrCiF...,https://www.amazon.in//Glassiano-Waterproof-Du...
...,...,...,...
301,Classic® LG Top Load Washing Machine Cover Sui...,https://m.media-amazon.com/images/I/81qi3XwDBQ...,https://www.amazon.in//Classic-Washing-Machine...
302,KVAR Printed design with water dust Protection...,https://m.media-amazon.com/images/I/61SfEzXF8V...,https://www.amazon.in//Printed-Protection-Auto...
303,Wings Star Washing Machine Cover for Fully Aut...,https://m.media-amazon.com/images/I/71YP1JRdFl...,https://www.amazon.in//Wings-Star-Automatic-36...
304,SANAVYA Samsung Back Panel Top Load Washing Ma...,https://m.media-amazon.com/images/I/41fVH6KEDT...,https://www.amazon.in//Generic-Samsung-Washing...


In [107]:
AW = df.copy()
AW

,product_name,product_img,product_link
0,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
1,"Samsung 8 Kg 5 Star AI Control & Wi-Fi, Fully-...",https://m.media-amazon.com/images/I/61sIXFuim6...,https://www.amazon.in//Samsung-Control-Fully-A...
2,Samsung 6.0 Kg Inverter 5 Star Fully-Automatic...,https://m.media-amazon.com/images/I/71i8f3-Dsu...,https://www.amazon.in//Samsung-Fully-Automatic...
3,IFB 6.5 Kg Fully-Automatic Front Loading Washi...,https://m.media-amazon.com/images/I/71hrC9lDoV...,https://www.amazon.in//IFB-Fully-Automatic-Loa...
4,IFB 6 Kg 5 Star Front Load Washing Machine 2X ...,https://m.media-amazon.com/images/I/61Gbp0b1lm...,https://www.amazon.in//IFB-DIVA-AQUA-BXS-6008/...
...,...,...,...
301,Samsung 6.5 Kg Inverter 5 starFully-Automatic ...,https://m.media-amazon.com/images/I/71+o3GHal8...,https://www.amazon.in//Samsung-starFully-Autom...
302,VOLTAS BEKO 7 kg Fully Automatic Top Loading W...,https://m.media-amazon.com/images/I/317tU65kzf...,https://www.amazon.in//VOLTAS-BEKO-Automatic-L...
303,Bosch 10Kg/6Kg Inverter Washer Dryer (WNA254U0...,https://m.media-amazon.com/images/I/61j441YamT...,https://www.amazon.in//Bosch-Inverter-Washer-W...
304,LG 7.0 Kg Inverter Fully-Automatic Top Loading...,https://m.media-amazon.com/images/I/81KOzY3A68...,https://www.amazon.in//LG-Inverter-Fully-Autom...


In [111]:
AW = AW.append(df).reset_index(drop=True)
AW

,product_name,product_img,product_link
0,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
1,"Samsung 8 Kg 5 Star AI Control & Wi-Fi, Fully-...",https://m.media-amazon.com/images/I/61sIXFuim6...,https://www.amazon.in//Samsung-Control-Fully-A...
2,Samsung 6.0 Kg Inverter 5 Star Fully-Automatic...,https://m.media-amazon.com/images/I/71i8f3-Dsu...,https://www.amazon.in//Samsung-Fully-Automatic...
3,IFB 6.5 Kg Fully-Automatic Front Loading Washi...,https://m.media-amazon.com/images/I/71hrC9lDoV...,https://www.amazon.in//IFB-Fully-Automatic-Loa...
4,IFB 6 Kg 5 Star Front Load Washing Machine 2X ...,https://m.media-amazon.com/images/I/61Gbp0b1lm...,https://www.amazon.in//IFB-DIVA-AQUA-BXS-6008/...
...,...,...,...
897,Classic® LG Top Load Washing Machine Cover Sui...,https://m.media-amazon.com/images/I/81qi3XwDBQ...,https://www.amazon.in//Classic-Washing-Machine...
898,KVAR Printed design with water dust Protection...,https://m.media-amazon.com/images/I/61SfEzXF8V...,https://www.amazon.in//Printed-Protection-Auto...
899,Wings Star Washing Machine Cover for Fully Aut...,https://m.media-amazon.com/images/I/71YP1JRdFl...,https://www.amazon.in//Wings-Star-Automatic-36...
900,SANAVYA Samsung Back Panel Top Load Washing Ma...,https://m.media-amazon.com/images/I/41fVH6KEDT...,https://www.amazon.in//Generic-Samsung-Washing...


In [81]:
product_AW = pd.DataFrame(reviews[reviews['product_code']=='product_AW']).reset_index(drop=True)
product_AW

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2H7WU1PW1AE3P,B00004U3AW,5.0,990230400,product_AW
1,A2S07LNFU0W0U3,B00004U3AW,4.0,968371200,product_AW
2,ATOFTVML8LX4D,B00004U3AW,4.0,975715200,product_AW
3,A1C7E59Y13WKBL,B00004U3AW,4.0,965088000,product_AW
4,A2EZANNJFL0MUE,B00004U3AW,5.0,980985600,product_AW
...,...,...,...,...,...
9238,A23G1BMY68VXZ1,B00K3046AW,3.0,1404432000,product_AW
9239,A86628I79X0JL,B00K74FCAW,5.0,1404259200,product_AW
9240,AZPATS2IPIKWW,B00K85X2AW,5.0,1404432000,product_AW
9241,A3O29DVIRDS6F0,B00K85X2AW,5.0,1405814400,product_AW


In [82]:
len(product_AW.product_id.unique())

739

In [112]:
product_AW = pd.DataFrame(reviews[reviews['product_code']=='product_AW']).reset_index(drop=True)
product_AW =  pd.merge(product_AW, 
                       AW.iloc[:len(product_AW.product_id.unique())]\
                       .join(pd.Series(product_AW['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AW.to_csv('product_AW.csv',index=False)
pd.read_csv('product_AW.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2H7WU1PW1AE3P,B00004U3AW,5.0,990230400,product_AW,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
1,A2S07LNFU0W0U3,B00004U3AW,4.0,968371200,product_AW,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
2,ATOFTVML8LX4D,B00004U3AW,4.0,975715200,product_AW,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
3,A1C7E59Y13WKBL,B00004U3AW,4.0,965088000,product_AW,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
4,A2EZANNJFL0MUE,B00004U3AW,5.0,980985600,product_AW,Samsung 6.0 Kg Inverter 5 star Fully-Automatic...,https://m.media-amazon.com/images/I/71m+C6830p...,https://www.amazon.in//Samsung-Inverter-Fully-...
...,...,...,...,...,...,...,...,...
9238,A23G1BMY68VXZ1,B00K3046AW,3.0,1404432000,product_AW,Classic® Whirlpool Front Panel Top Load Washin...,https://m.media-amazon.com/images/I/71x84PvEtf...,https://www.amazon.in//Classic-Whirlpool-Washi...
9239,A86628I79X0JL,B00K74FCAW,5.0,1404259200,product_AW,AMAZOR Spin Cap Suitable for Samsung Washing M...,https://m.media-amazon.com/images/I/61I5-JZjJB...,https://www.amazon.in//Suitable-Samsung-Washin...
9240,AZPATS2IPIKWW,B00K85X2AW,5.0,1404432000,product_AW,Stylista Semi Automatic Washing Machine Cover ...,https://m.media-amazon.com/images/I/716pEalC7e...,https://www.amazon.in//Stylista-Automatic-Wash...
9241,A3O29DVIRDS6F0,B00K85X2AW,5.0,1405814400,product_AW,Stylista Semi Automatic Washing Machine Cover ...,https://m.media-amazon.com/images/I/716pEalC7e...,https://www.amazon.in//Stylista-Automatic-Wash...


### Product AX dataframe

In [116]:
df = get_scraped('axe')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,AXE Antiperspirant Deodorant For Men 48H Sweat...,https://m.media-amazon.com/images/I/619k8vn6xB...,https://www.amazon.in//AXE-Antiperspirant-Deod...
1,AXE Denim Lather Shaving Cream (78g) - Pack of 2,https://m.media-amazon.com/images/I/31cSNlso84...,https://www.amazon.in//AXE-Denim-Lather-Shavin...
2,CLOUD MART AXE Fruit JUICER with HIGH Quality ...,https://m.media-amazon.com/images/I/81au6w-ER1...,https://www.amazon.in//CLOUD-MART-Quality-Plas...
3,AXE Dark Temptation Deodorant Bodyspray for Me...,https://m.media-amazon.com/images/I/51SCJ+b2bJ...,https://www.amazon.in//AXE-Dark-Temptation-Deo...
4,Axe After Shave Lotion - 50ml (Dark Temptation),https://m.media-amazon.com/images/I/51x6KmBByg...,https://www.amazon.in//Axe-After-Shave-Lotion-...
...,...,...,...
91,SHREETA Funny & Cool Axe Pizza Cutter - Plasti...,https://m.media-amazon.com/images/I/31UcYUB4xy...,https://www.amazon.in//SHREETA-Funny-Cool-Pizz...
92,"Brut Original Deodorant For Men, 200ml",https://m.media-amazon.com/images/I/31zlrHhKLk...,https://www.amazon.in//Brut-Original-Deodorant...
93,Mortein Insta5 Mosquito Repellent Liquid Refil...,https://m.media-amazon.com/images/I/61LTT8lFah...,https://www.amazon.in//Mortein-Liquid-Vaporize...
94,"Nivea Fresh Active Original Deodorant for Men,...",https://m.media-amazon.com/images/I/41LhTwHvEv...,https://www.amazon.in//Nivea-Fresh-Active-Orig...


In [117]:
AX = df.copy()
AX

,product_name,product_img,product_link
0,AXE Antiperspirant Deodorant For Men 48H Sweat...,https://m.media-amazon.com/images/I/619k8vn6xB...,https://www.amazon.in//AXE-Antiperspirant-Deod...
1,AXE Denim Lather Shaving Cream (78g) - Pack of 2,https://m.media-amazon.com/images/I/31cSNlso84...,https://www.amazon.in//AXE-Denim-Lather-Shavin...
2,CLOUD MART AXE Fruit JUICER with HIGH Quality ...,https://m.media-amazon.com/images/I/81au6w-ER1...,https://www.amazon.in//CLOUD-MART-Quality-Plas...
3,AXE Dark Temptation Deodorant Bodyspray for Me...,https://m.media-amazon.com/images/I/51SCJ+b2bJ...,https://www.amazon.in//AXE-Dark-Temptation-Deo...
4,Axe After Shave Lotion - 50ml (Dark Temptation),https://m.media-amazon.com/images/I/51x6KmBByg...,https://www.amazon.in//Axe-After-Shave-Lotion-...
...,...,...,...
91,SHREETA Funny & Cool Axe Pizza Cutter - Plasti...,https://m.media-amazon.com/images/I/31UcYUB4xy...,https://www.amazon.in//SHREETA-Funny-Cool-Pizz...
92,"Brut Original Deodorant For Men, 200ml",https://m.media-amazon.com/images/I/31zlrHhKLk...,https://www.amazon.in//Brut-Original-Deodorant...
93,Mortein Insta5 Mosquito Repellent Liquid Refil...,https://m.media-amazon.com/images/I/61LTT8lFah...,https://www.amazon.in//Mortein-Liquid-Vaporize...
94,"Nivea Fresh Active Original Deodorant for Men,...",https://m.media-amazon.com/images/I/41LhTwHvEv...,https://www.amazon.in//Nivea-Fresh-Active-Orig...


In [ ]:
# AX = AX.append(df).reset_index(drop=True)
# AX

In [114]:
product_AX = pd.DataFrame(reviews[reviews['product_code']=='product_AX']).reset_index(drop=True)
product_AX

,user_id,product_id,Ratings,Time_stamp,product_code
0,ARF0S69N5C96X,B00003CWAX,4.0,962064000,product_AX
1,A348WXK3XZMH55,B00004SGAX,5.0,1155945600,product_AX
2,A1EQ1AIWFNZOO7,B00004SGAX,5.0,961286400,product_AX
3,AU8DH6AOA1YOJ,B00004SGAX,2.0,1006646400,product_AX
4,A1X2GX05KINAMP,B00004SGAX,4.0,1008115200,product_AX
...,...,...,...,...,...
99,A2MWYW5L3J6YKV,B0000CFWAX,1.0,1117411200,product_AX
100,A1RTSQCCNLHSWV,B0000CFWAX,5.0,1084406400,product_AX
101,A2GRYXLWZ44OZZ,B0000CFWAX,4.0,1067472000,product_AX
102,A3TLRMLVP7WM5E,B0000DB4AX,5.0,1323129600,product_AX


In [115]:
len(product_AX.product_id.unique())

13

In [118]:
product_AX = pd.DataFrame(reviews[reviews['product_code']=='product_AX']).reset_index(drop=True)
product_AX =  pd.merge(product_AX, 
                       AX.iloc[:len(product_AX.product_id.unique())]\
                       .join(pd.Series(product_AX['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AX.to_csv('product_AX.csv',index=False)
pd.read_csv('product_AX.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ARF0S69N5C96X,B00003CWAX,4.0,962064000,product_AX,AXE Antiperspirant Deodorant For Men 48H Sweat...,https://m.media-amazon.com/images/I/619k8vn6xB...,https://www.amazon.in//AXE-Antiperspirant-Deod...
1,A348WXK3XZMH55,B00004SGAX,5.0,1155945600,product_AX,AXE Denim Lather Shaving Cream (78g) - Pack of 2,https://m.media-amazon.com/images/I/31cSNlso84...,https://www.amazon.in//AXE-Denim-Lather-Shavin...
2,A1EQ1AIWFNZOO7,B00004SGAX,5.0,961286400,product_AX,AXE Denim Lather Shaving Cream (78g) - Pack of 2,https://m.media-amazon.com/images/I/31cSNlso84...,https://www.amazon.in//AXE-Denim-Lather-Shavin...
3,AU8DH6AOA1YOJ,B00004SGAX,2.0,1006646400,product_AX,AXE Denim Lather Shaving Cream (78g) - Pack of 2,https://m.media-amazon.com/images/I/31cSNlso84...,https://www.amazon.in//AXE-Denim-Lather-Shavin...
4,A1X2GX05KINAMP,B00004SGAX,4.0,1008115200,product_AX,AXE Denim Lather Shaving Cream (78g) - Pack of 2,https://m.media-amazon.com/images/I/31cSNlso84...,https://www.amazon.in//AXE-Denim-Lather-Shavin...
...,...,...,...,...,...,...,...,...
99,A2MWYW5L3J6YKV,B0000CFWAX,1.0,1117411200,product_AX,DIONE UK AXE Brand Universal Oil for Cold and ...,https://m.media-amazon.com/images/I/61fEwEIcEk...,https://www.amazon.in//DIONE-UK-AXE-Universal-...
100,A1RTSQCCNLHSWV,B0000CFWAX,5.0,1084406400,product_AX,DIONE UK AXE Brand Universal Oil for Cold and ...,https://m.media-amazon.com/images/I/61fEwEIcEk...,https://www.amazon.in//DIONE-UK-AXE-Universal-...
101,A2GRYXLWZ44OZZ,B0000CFWAX,4.0,1067472000,product_AX,DIONE UK AXE Brand Universal Oil for Cold and ...,https://m.media-amazon.com/images/I/61fEwEIcEk...,https://www.amazon.in//DIONE-UK-AXE-Universal-...
102,A3TLRMLVP7WM5E,B0000DB4AX,5.0,1323129600,product_AX,"Axe Signature Body Perfume - Suave, 122ml Bottle",https://m.media-amazon.com/images/I/61pjdpz4nk...,https://www.amazon.in//Axe-Signature-Body-Perf...


### Product AY dataframe

In [125]:
df = get_scraped('ayurvedic shampoo')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"DABUR Vatika Ayurvedic Shampoo, 640ml : Power ...",https://m.media-amazon.com/images/I/71H2aQfpoo...,https://www.amazon.in//Dabur-Almond-Shampoo-64...
1,Neem Tulsi & Aloevera Shampoo - For Damaged Ha...,https://m.media-amazon.com/images/I/51aErZcsHF...,https://www.amazon.in//Neem-Tulsi-Aloevera-Sha...
2,"Indulekha Bringha Ayurvedic Shampoo 200 ml, fo...",https://m.media-amazon.com/images/I/51ODfZLXmV...,https://www.amazon.in//Indulekha-Bringha-Anti-...
3,Satvayush Khadi Amla Reetha Shikakai Bhringraj...,https://m.media-amazon.com/images/I/71zzdedDOw...,https://www.amazon.in//Satvayush-Shikakai-Bhri...
4,Sesa Ayurvedic Medicinal Shampoo - Hair Fall C...,https://m.media-amazon.com/images/I/61zcpbzxCe...,https://www.amazon.in//Sesa-Ayurvedic-Medicina...
...,...,...,...
253,MEDIMIX Total Care Shampoo | 200ml | Minimizes...,https://m.media-amazon.com/images/I/71-geUFM8o...,https://www.amazon.in//MEDIMIX-Shampoo-Minimiz...
254,Kama Ayurveda Himalayan Deodar Hair Cleanser (...,https://m.media-amazon.com/images/I/51CmnFRmHD...,https://www.amazon.in//Kama-Ayurveda-Himalayan...
255,"Biotique Onion Black Seed Shampoo For Fresh, S...",https://m.media-amazon.com/images/I/51cV3fr-Wl...,https://www.amazon.in//Biotique-Onion-Shampoo-...
256,KHADI NATURAL Honey and Vanilla Hair Cleanser ...,https://m.media-amazon.com/images/I/61XBdpA3Bx...,https://www.amazon.in//Khadi-Natural-Herbal-Va...


In [122]:
AY = df.copy()
AY

,product_name,product_img,product_link
0,Just Herbs Skin-Rejuvenating Ayurvedic Face Wa...,https://m.media-amazon.com/images/I/61stHma03P...,https://www.amazon.in//Just-Herbs-Skin-Rejuven...
1,"Medimix Ayurvedic Anti Pimple Face Wash, 100ml...",https://m.media-amazon.com/images/I/81-vBGGdAx...,https://www.amazon.in//Medimix-Ayurvedic-Essen...
2,Blue Nectar Honey Face Wash for Glowing Skin f...,https://m.media-amazon.com/images/I/71dQb2ux06...,https://www.amazon.in//Blue-Nectar-Ayurvedic-H...
3,"KHADI NATURAL Ayurvedic Neem Face Wash, 210ml",https://m.media-amazon.com/images/I/51PnM7ozrb...,https://www.amazon.in//Khadi-Natural-Neem-Face...
4,Aroma Magic Neem & Tea Tree Face Wash 100ml,https://m.media-amazon.com/images/I/61ibACk+GQ...,https://www.amazon.in//Aroma-Magic-Neem-Tree-1...
...,...,...,...
301,Dr.Rashel Gold Foaming Face Wash For Radiance ...,https://m.media-amazon.com/images/I/51a4Y+3H39...,https://www.amazon.in//Dr-Rashel-Radiance-Enha...
302,Fair and Handsome Instant Radiance Face Wash |...,https://m.media-amazon.com/images/I/61Jnn0D30R...,https://www.amazon.in//Fair-Handsome-Pro-Pepti...
303,Luster Cosmetics Anti Acne Face Wash | Made Wi...,https://m.media-amazon.com/images/I/51BLxSOmg-...,https://www.amazon.in//Luster-Cosmetics-Natura...
304,Ayurvedic Fruity Gleem Face Wash,https://m.media-amazon.com/images/I/810l0sJIW+...,https://www.amazon.in//Ayurvedic-Fruity-Gleem-...


In [126]:
AY = AY.append(df).reset_index(drop=True)
AY

,product_name,product_img,product_link
0,Just Herbs Skin-Rejuvenating Ayurvedic Face Wa...,https://m.media-amazon.com/images/I/61stHma03P...,https://www.amazon.in//Just-Herbs-Skin-Rejuven...
1,"Medimix Ayurvedic Anti Pimple Face Wash, 100ml...",https://m.media-amazon.com/images/I/81-vBGGdAx...,https://www.amazon.in//Medimix-Ayurvedic-Essen...
2,Blue Nectar Honey Face Wash for Glowing Skin f...,https://m.media-amazon.com/images/I/71dQb2ux06...,https://www.amazon.in//Blue-Nectar-Ayurvedic-H...
3,"KHADI NATURAL Ayurvedic Neem Face Wash, 210ml",https://m.media-amazon.com/images/I/51PnM7ozrb...,https://www.amazon.in//Khadi-Natural-Neem-Face...
4,Aroma Magic Neem & Tea Tree Face Wash 100ml,https://m.media-amazon.com/images/I/61ibACk+GQ...,https://www.amazon.in//Aroma-Magic-Neem-Tree-1...
...,...,...,...
847,MEDIMIX Total Care Shampoo | 200ml | Minimizes...,https://m.media-amazon.com/images/I/71-geUFM8o...,https://www.amazon.in//MEDIMIX-Shampoo-Minimiz...
848,Kama Ayurveda Himalayan Deodar Hair Cleanser (...,https://m.media-amazon.com/images/I/51CmnFRmHD...,https://www.amazon.in//Kama-Ayurveda-Himalayan...
849,"Biotique Onion Black Seed Shampoo For Fresh, S...",https://m.media-amazon.com/images/I/51cV3fr-Wl...,https://www.amazon.in//Biotique-Onion-Shampoo-...
850,KHADI NATURAL Honey and Vanilla Hair Cleanser ...,https://m.media-amazon.com/images/I/61XBdpA3Bx...,https://www.amazon.in//Khadi-Natural-Herbal-Va...


In [119]:
product_AY = pd.DataFrame(reviews[reviews['product_code']=='product_AY']).reset_index(drop=True)
product_AY

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2ZQ52RGHYYMZK,B00003CWAY,1.0,1006041600,product_AY
1,AVMC17IX9DT1B,B00003CWAY,5.0,961113600,product_AY
2,A4IJAIG9455PT,B00004SGAY,5.0,1005782400,product_AY
3,AVBKUMR73WDTT,B00004SGAY,4.0,1010016000,product_AY
4,AZLMDL2HJVMSM,B00004SGAY,5.0,1381708800,product_AY
...,...,...,...,...,...
11167,A2V5G12RGQRI7Y,B00JRPLEAY,1.0,1401494400,product_AY
11168,A3SFXBB6KO52ZG,B00JRPLEAY,5.0,1400803200,product_AY
11169,A45OAQ9WJ5AI,B00JRPLEAY,5.0,1401667200,product_AY
11170,A13LZTH5CDM20P,B00KOW9VAY,5.0,1403568000,product_AY


In [120]:
len(product_AY.product_id.unique())

684

In [127]:
product_AY = pd.DataFrame(reviews[reviews['product_code']=='product_AY']).reset_index(drop=True)
product_AY =  pd.merge(product_AY, 
                       AY.iloc[:len(product_AY.product_id.unique())]\
                       .join(pd.Series(product_AY['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AY.to_csv('product_AY.csv',index=False)
pd.read_csv('product_AY.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2ZQ52RGHYYMZK,B00003CWAY,1.0,1006041600,product_AY,Just Herbs Skin-Rejuvenating Ayurvedic Face Wa...,https://m.media-amazon.com/images/I/61stHma03P...,https://www.amazon.in//Just-Herbs-Skin-Rejuven...
1,AVMC17IX9DT1B,B00003CWAY,5.0,961113600,product_AY,Just Herbs Skin-Rejuvenating Ayurvedic Face Wa...,https://m.media-amazon.com/images/I/61stHma03P...,https://www.amazon.in//Just-Herbs-Skin-Rejuven...
2,A4IJAIG9455PT,B00004SGAY,5.0,1005782400,product_AY,"Medimix Ayurvedic Anti Pimple Face Wash, 100ml...",https://m.media-amazon.com/images/I/81-vBGGdAx...,https://www.amazon.in//Medimix-Ayurvedic-Essen...
3,AVBKUMR73WDTT,B00004SGAY,4.0,1010016000,product_AY,"Medimix Ayurvedic Anti Pimple Face Wash, 100ml...",https://m.media-amazon.com/images/I/81-vBGGdAx...,https://www.amazon.in//Medimix-Ayurvedic-Essen...
4,AZLMDL2HJVMSM,B00004SGAY,5.0,1381708800,product_AY,"Medimix Ayurvedic Anti Pimple Face Wash, 100ml...",https://m.media-amazon.com/images/I/81-vBGGdAx...,https://www.amazon.in//Medimix-Ayurvedic-Essen...
...,...,...,...,...,...,...,...,...
11167,A2V5G12RGQRI7Y,B00JRPLEAY,1.0,1401494400,product_AY,Rangoli Herbal Karishma Premium Shampoo 100ML ...,https://m.media-amazon.com/images/I/51UIPDRTM-...,https://www.amazon.in//Rangoli-Karishma-Ayurve...
11168,A3SFXBB6KO52ZG,B00JRPLEAY,5.0,1400803200,product_AY,Rangoli Herbal Karishma Premium Shampoo 100ML ...,https://m.media-amazon.com/images/I/51UIPDRTM-...,https://www.amazon.in//Rangoli-Karishma-Ayurve...
11169,A45OAQ9WJ5AI,B00JRPLEAY,5.0,1401667200,product_AY,Rangoli Herbal Karishma Premium Shampoo 100ML ...,https://m.media-amazon.com/images/I/51UIPDRTM-...,https://www.amazon.in//Rangoli-Karishma-Ayurve...
11170,A13LZTH5CDM20P,B00KOW9VAY,5.0,1403568000,product_AY,Natural Vibes | Tea Tree Anti Hair Fall & Thin...,https://m.media-amazon.com/images/I/519eYKveGy...,https://www.amazon.in//Natural-Vibes-Ayurvedic...


### Product AZ dataframe

In [131]:
df = get_scraped('a to z toys for kids')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
1,Zitto Premium Wooden Small Alphabets A to Z Ed...,https://m.media-amazon.com/images/I/81yikGjT3z...,https://www.amazon.in//Zitto-Premium-Wooden-Al...
2,Baybee Wooden English Alphabets and Color Lear...,https://m.media-amazon.com/images/I/71pe5RLx8g...,https://www.amazon.in//CaseSmall-Alphabet-Geom...
3,Premium Wooden Capital Alphabets A to Z Educat...,https://m.media-amazon.com/images/I/71jTgR-YtV...,https://www.amazon.in//Premium-Wooden-Capital-...
4,SYGA Wooden 26 English Upper Case (A to Z) Alp...,https://m.media-amazon.com/images/I/71cnvIJvfp...,https://www.amazon.in//English-Alphabets-Stenc...
...,...,...,...
301,Esmi Intelligent Book | Educational & Learning...,https://m.media-amazon.com/images/I/71il9dWggI...,https://www.amazon.in//Esmi-Intelligent-Educat...
302,WHIZ KIDS Wooden Educational Peg Board for Kid...,https://m.media-amazon.com/images/I/51x9FiRfg7...,https://www.amazon.in//Educational-Kids-Presch...
303,Elegant Lifestyle Super Hero Goku Dragon Ballz...,https://m.media-amazon.com/images/I/41CsSJ+wEc...,https://www.amazon.in//Elegant-Lifestyle-Japan...
304,Ratna's Squeezy Toys Duck 3 pcs Pack for Infan...,https://m.media-amazon.com/images/I/31d8ETzk7C...,https://www.amazon.in//Ratnas-squeezy-infants-...


In [132]:
AZ = df.copy()
AZ

,product_name,product_img,product_link
0,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
1,Zitto Premium Wooden Small Alphabets A to Z Ed...,https://m.media-amazon.com/images/I/81yikGjT3z...,https://www.amazon.in//Zitto-Premium-Wooden-Al...
2,Baybee Wooden English Alphabets and Color Lear...,https://m.media-amazon.com/images/I/71pe5RLx8g...,https://www.amazon.in//CaseSmall-Alphabet-Geom...
3,Premium Wooden Capital Alphabets A to Z Educat...,https://m.media-amazon.com/images/I/71jTgR-YtV...,https://www.amazon.in//Premium-Wooden-Capital-...
4,SYGA Wooden 26 English Upper Case (A to Z) Alp...,https://m.media-amazon.com/images/I/71cnvIJvfp...,https://www.amazon.in//English-Alphabets-Stenc...
...,...,...,...
301,Esmi Intelligent Book | Educational & Learning...,https://m.media-amazon.com/images/I/71il9dWggI...,https://www.amazon.in//Esmi-Intelligent-Educat...
302,WHIZ KIDS Wooden Educational Peg Board for Kid...,https://m.media-amazon.com/images/I/51x9FiRfg7...,https://www.amazon.in//Educational-Kids-Presch...
303,Elegant Lifestyle Super Hero Goku Dragon Ballz...,https://m.media-amazon.com/images/I/41CsSJ+wEc...,https://www.amazon.in//Elegant-Lifestyle-Japan...
304,Ratna's Squeezy Toys Duck 3 pcs Pack for Infan...,https://m.media-amazon.com/images/I/31d8ETzk7C...,https://www.amazon.in//Ratnas-squeezy-infants-...


In [ ]:
# AZ = AZ.append(df).reset_index(drop=True)
# AZ

In [129]:
product_AZ = pd.DataFrame(reviews[reviews['product_code']=='product_AZ']).reset_index(drop=True)
product_AZ

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3HQKIJ6ZFQB87,B00001X5AZ,2.0,945216000,product_AZ
1,A2NR5SNMBOKJF8,B00001X5AZ,3.0,1362528000,product_AZ
2,A7Y6AVS576M03,B00001X5AZ,5.0,1172707200,product_AZ
3,AHMWKMZ7BMGB5,B00001X5AZ,1.0,1320796800,product_AZ
4,A1UK8ZBY28LKKP,B00001X5AZ,3.0,939772800,product_AZ
...,...,...,...,...,...
137,AYDNPZDF06AOD,B0000DC6AZ,3.0,1125964800,product_AZ
138,A3FQZ5TZLC781L,B0000DC6AZ,1.0,1130198400,product_AZ
139,A2GXJZ3JDYSCDK,B0000DC6AZ,5.0,1090195200,product_AZ
140,A21WF2BQ5M9XOZ,B0000DC6AZ,5.0,1140739200,product_AZ


In [130]:
len(product_AZ.product_id.unique())

21

In [133]:
product_AZ = pd.DataFrame(reviews[reviews['product_code']=='product_AZ']).reset_index(drop=True)
product_AZ =  pd.merge(product_AZ, 
                       AZ.iloc[:len(product_AZ.product_id.unique())]\
                       .join(pd.Series(product_AZ['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_AZ.to_csv('product_AZ.csv',index=False)
pd.read_csv('product_AZ.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3HQKIJ6ZFQB87,B00001X5AZ,2.0,945216000,product_AZ,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
1,A2NR5SNMBOKJF8,B00001X5AZ,3.0,1362528000,product_AZ,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
2,A7Y6AVS576M03,B00001X5AZ,5.0,1172707200,product_AZ,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
3,AHMWKMZ7BMGB5,B00001X5AZ,1.0,1320796800,product_AZ,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
4,A1UK8ZBY28LKKP,B00001X5AZ,3.0,939772800,product_AZ,Chocozone Wooden English Alphabets and Color L...,https://m.media-amazon.com/images/I/61qnvpYyG9...,https://www.amazon.in//Chocozone-English-Alpha...
...,...,...,...,...,...,...,...,...
137,AYDNPZDF06AOD,B0000DC6AZ,3.0,1125964800,product_AZ,Lots4kids Ratna's Alpha Magnetic Learning Boar...,https://m.media-amazon.com/images/I/71qt-85r5T...,https://www.amazon.in//LOTS4KIDS-Alpha-Magneti...
138,A3FQZ5TZLC781L,B0000DC6AZ,1.0,1130198400,product_AZ,Lots4kids Ratna's Alpha Magnetic Learning Boar...,https://m.media-amazon.com/images/I/71qt-85r5T...,https://www.amazon.in//LOTS4KIDS-Alpha-Magneti...
139,A2GXJZ3JDYSCDK,B0000DC6AZ,5.0,1090195200,product_AZ,Lots4kids Ratna's Alpha Magnetic Learning Boar...,https://m.media-amazon.com/images/I/71qt-85r5T...,https://www.amazon.in//LOTS4KIDS-Alpha-Magneti...
140,A21WF2BQ5M9XOZ,B0000DC6AZ,5.0,1140739200,product_AZ,Lots4kids Ratna's Alpha Magnetic Learning Boar...,https://m.media-amazon.com/images/I/71qt-85r5T...,https://www.amazon.in//LOTS4KIDS-Alpha-Magneti...


### Product A1 dataframe

In [137]:
df = get_scraped('a1 paper')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,inr | Ivory Drawing Paper Natural White Cartri...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
1,OMGEET A1 WHITE CHART PAPER 20 PIECE,https://m.media-amazon.com/images/I/11CSXxDsk8...,https://www.amazon.in//OMGEET-WHITE-CHART-PAPE...
2,inr | Ivory A1 Drawing Paper Pack of 25 Sheets...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
3,DESIGNERS DEN A1 White Pattern Paper -Pack of ...,https://m.media-amazon.com/images/I/61IYi08mgL...,https://www.amazon.in//designers-den-White-Pat...
4,inr | Ivory A1 Drawing Paper Pack of 12 Sheets...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
...,...,...,...
205,Creative Educational Aids P. Ltd. - CRE0642 Ac...,https://m.media-amazon.com/images/I/71ojTKbdV6...,https://www.amazon.in//Creative-Education-Aids...
206,A1 Decor Marble Granite Look White Marble Glos...,https://m.media-amazon.com/images/I/71Iog7V33w...,https://www.amazon.in//A1-Decor-Decorative-Rem...
207,"DSR Smooth Finish A3 Size, 400 GSM Ivory Drawi...",https://m.media-amazon.com/images/I/41FPn-B4ZC...,https://www.amazon.in//DSR-Smooth-Finish-Drawi...
208,SPARTAN KIDS® Flash Cards for Kids (Set of 20)...,https://m.media-amazon.com/images/I/91f6im6XoA...,https://www.amazon.in//SPARTAN-KIDS%C2%AE-Flas...


In [138]:
A1 = df.copy()
A1

,product_name,product_img,product_link
0,inr | Ivory Drawing Paper Natural White Cartri...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
1,OMGEET A1 WHITE CHART PAPER 20 PIECE,https://m.media-amazon.com/images/I/11CSXxDsk8...,https://www.amazon.in//OMGEET-WHITE-CHART-PAPE...
2,inr | Ivory A1 Drawing Paper Pack of 25 Sheets...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
3,DESIGNERS DEN A1 White Pattern Paper -Pack of ...,https://m.media-amazon.com/images/I/61IYi08mgL...,https://www.amazon.in//designers-den-White-Pat...
4,inr | Ivory A1 Drawing Paper Pack of 12 Sheets...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
...,...,...,...
205,Creative Educational Aids P. Ltd. - CRE0642 Ac...,https://m.media-amazon.com/images/I/71ojTKbdV6...,https://www.amazon.in//Creative-Education-Aids...
206,A1 Decor Marble Granite Look White Marble Glos...,https://m.media-amazon.com/images/I/71Iog7V33w...,https://www.amazon.in//A1-Decor-Decorative-Rem...
207,"DSR Smooth Finish A3 Size, 400 GSM Ivory Drawi...",https://m.media-amazon.com/images/I/41FPn-B4ZC...,https://www.amazon.in//DSR-Smooth-Finish-Drawi...
208,SPARTAN KIDS® Flash Cards for Kids (Set of 20)...,https://m.media-amazon.com/images/I/91f6im6XoA...,https://www.amazon.in//SPARTAN-KIDS%C2%AE-Flas...


In [ ]:
# A1 = A1.append(df).reset_index(drop=True)
# A1

In [135]:
product_A1 = pd.DataFrame(reviews[reviews['product_code']=='product_A1']).reset_index(drop=True)
product_A1

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2JBGKJ5LFEP47,B00001O3A1,5.0,993254400,product_A1
1,A102UJN17M7VT0,B00001O3A1,5.0,1006992000,product_A1
2,A5K3MY209ABZR,B00001O3A1,3.0,950313600,product_A1
3,A39AWL2FKWDFK6,B00004SDA1,4.0,1004313600,product_A1
4,A3OEJX0R2T49VH,B00004SDA1,5.0,982022400,product_A1
...,...,...,...,...,...
395,A3NCIN6TNL0MGA,B0000DCEA1,5.0,1128643200,product_A1
396,A20D2I308Z2671,B0000DCEA1,5.0,1141948800,product_A1
397,AOCLP52IYG6E3,B0000DCEA1,5.0,1142208000,product_A1
398,AJBV1GIE67GOH,B0000DCEA1,5.0,1138752000,product_A1


In [136]:
len(product_A1.product_id.unique())

24

In [139]:
product_A1 = pd.DataFrame(reviews[reviews['product_code']=='product_A1']).reset_index(drop=True)
product_A1 =  pd.merge(product_A1, 
                       A1.iloc[:len(product_A1.product_id.unique())]\
                       .join(pd.Series(product_A1['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A1.to_csv('product_A1.csv',index=False)
pd.read_csv('product_A1.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2JBGKJ5LFEP47,B00001O3A1,5.0,993254400,product_A1,inr | Ivory Drawing Paper Natural White Cartri...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
1,A102UJN17M7VT0,B00001O3A1,5.0,1006992000,product_A1,inr | Ivory Drawing Paper Natural White Cartri...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
2,A5K3MY209ABZR,B00001O3A1,3.0,950313600,product_A1,inr | Ivory Drawing Paper Natural White Cartri...,https://m.media-amazon.com/images/I/41tgtfzIpi...,https://www.amazon.in//Drawing-Natural-Cartrid...
3,A39AWL2FKWDFK6,B00004SDA1,4.0,1004313600,product_A1,OMGEET A1 WHITE CHART PAPER 20 PIECE,https://m.media-amazon.com/images/I/11CSXxDsk8...,https://www.amazon.in//OMGEET-WHITE-CHART-PAPE...
4,A3OEJX0R2T49VH,B00004SDA1,5.0,982022400,product_A1,OMGEET A1 WHITE CHART PAPER 20 PIECE,https://m.media-amazon.com/images/I/11CSXxDsk8...,https://www.amazon.in//OMGEET-WHITE-CHART-PAPE...
...,...,...,...,...,...,...,...,...
395,A3NCIN6TNL0MGA,B0000DCEA1,5.0,1128643200,product_A1,inr | A1 BLACK COLOR CHART PAPER Special Imper...,https://m.media-amazon.com/images/I/31DhmKeNrW...,https://www.amazon.in//Special-Imperial-Sheets...
396,A20D2I308Z2671,B0000DCEA1,5.0,1141948800,product_A1,inr | A1 BLACK COLOR CHART PAPER Special Imper...,https://m.media-amazon.com/images/I/31DhmKeNrW...,https://www.amazon.in//Special-Imperial-Sheets...
397,AOCLP52IYG6E3,B0000DCEA1,5.0,1142208000,product_A1,inr | A1 BLACK COLOR CHART PAPER Special Imper...,https://m.media-amazon.com/images/I/31DhmKeNrW...,https://www.amazon.in//Special-Imperial-Sheets...
398,AJBV1GIE67GOH,B0000DCEA1,5.0,1138752000,product_A1,inr | A1 BLACK COLOR CHART PAPER Special Imper...,https://m.media-amazon.com/images/I/31DhmKeNrW...,https://www.amazon.in//Special-Imperial-Sheets...


### Product A2 dataframe

In [147]:
df = get_scraped('a2 size drawing board')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,VISHA MDF Half Imperial Drawing Board (A2 Size...,https://m.media-amazon.com/images/I/61x9Jt0Eny...,https://www.amazon.in//Vishal-Drawing-Board-Im...
1,Eduway 16”x23” Inches Multipurpose Drawing Boa...,https://m.media-amazon.com/images/I/61WHnXn8hU...,https://www.amazon.in//Eduway-Multipurpose-Eng...
2,XSOURCE Wooden Lightweight Drawing Board 17 x ...,https://m.media-amazon.com/images/I/71V-r0DSUo...,https://www.amazon.in//XSOURCE-Lightweight-Thi...
3,Indian Traders (Indian Natural Pinewood Drawin...,https://m.media-amazon.com/images/I/51FH+p5pTX...,https://www.amazon.in//Traders-Pinewood-Imperi...
4,POPULAR MagicA2M Drawing Board MDF – 16’’x23’’...,https://m.media-amazon.com/images/I/71OADD0DhF...,https://www.amazon.in//POPULAR-MagicA2M-Drawin...
...,...,...,...
187,Roger & Moris Spruce Wood Mini Black Board Dou...,https://m.media-amazon.com/images/I/51jxTZtzdm...,https://www.amazon.in//Roger-Moris-Black-Doubl...
188,Roger & Moris Mini Black Board Double Framed E...,https://m.media-amazon.com/images/I/51jxTZtzdm...,https://www.amazon.in//Roger-Moris-Black-Doubl...
189,Roger & Moris Artist Canvas Panel - Cotton Med...,https://m.media-amazon.com/images/I/51YR9XK0rN...,https://www.amazon.in//Roger-Moris-Artist-Canv...
190,Brunte My Stylish Drawing Board Slate White Bo...,https://m.media-amazon.com/images/I/31zNIWyeKK...,https://www.amazon.in//Brunte-Stylish-Drawing-...


In [144]:
A2 = df.copy()
A2

,product_name,product_img,product_link
0,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
1,ARTIOS Mixed Media Cartridge Paper A2 240 GSM ...,https://m.media-amazon.com/images/I/51uNs6y6TG...,https://www.amazon.in//ARTIOS-Mixed-Media-Cart...
2,ArtRight Artists' Sketching Paper (A2 Pack of ...,https://m.media-amazon.com/images/I/81gq1J7i5w...,https://www.amazon.in//ArtRight-Cartridge-Pape...
3,inr | Ivory A2 Drawing Paper Natural White Car...,https://m.media-amazon.com/images/I/41hyIarmyR...,https://www.amazon.in//Drawing-Natural-Cartrid...
4,inr | Ivory A2 Drawing Paper Natural White Car...,https://m.media-amazon.com/images/I/21vD+K7Yhg...,https://www.amazon.in//A2-Drawing-Cartridge-Pa...
...,...,...,...
253,Sketch & Drawing Paper A2 10sheets,https://m.media-amazon.com/images/I/51+xhA24KL...,https://www.amazon.in//Sketch-Drawing-Paper-A2...
254,inr | Ivory Drawing Paper Natural White Cartri...,https://m.media-amazon.com/images/I/31w94G940q...,https://www.amazon.in//Drawing-Natural-Cartrid...
255,ARTIOS Mixed Media Cartridge Paper A4 240 GSM ...,https://m.media-amazon.com/images/I/51IgWjtb+Y...,https://www.amazon.in//ARTIOS-Mixed-Media-Cart...
256,Drawing Board 16’’x 23’’ – Half Imperial A2 Si...,https://m.media-amazon.com/images/I/61w-NyQmJi...,https://www.amazon.in//Drawing-Board-16x-Archi...


In [148]:
A2 = A2.append(df).reset_index(drop=True)
A2

,product_name,product_img,product_link
0,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
1,ARTIOS Mixed Media Cartridge Paper A2 240 GSM ...,https://m.media-amazon.com/images/I/51uNs6y6TG...,https://www.amazon.in//ARTIOS-Mixed-Media-Cart...
2,ArtRight Artists' Sketching Paper (A2 Pack of ...,https://m.media-amazon.com/images/I/81gq1J7i5w...,https://www.amazon.in//ArtRight-Cartridge-Pape...
3,inr | Ivory A2 Drawing Paper Natural White Car...,https://m.media-amazon.com/images/I/41hyIarmyR...,https://www.amazon.in//Drawing-Natural-Cartrid...
4,inr | Ivory A2 Drawing Paper Natural White Car...,https://m.media-amazon.com/images/I/21vD+K7Yhg...,https://www.amazon.in//A2-Drawing-Cartridge-Pa...
...,...,...,...
733,Roger & Moris Spruce Wood Mini Black Board Dou...,https://m.media-amazon.com/images/I/51jxTZtzdm...,https://www.amazon.in//Roger-Moris-Black-Doubl...
734,Roger & Moris Mini Black Board Double Framed E...,https://m.media-amazon.com/images/I/51jxTZtzdm...,https://www.amazon.in//Roger-Moris-Black-Doubl...
735,Roger & Moris Artist Canvas Panel - Cotton Med...,https://m.media-amazon.com/images/I/51YR9XK0rN...,https://www.amazon.in//Roger-Moris-Artist-Canv...
736,Brunte My Stylish Drawing Board Slate White Bo...,https://m.media-amazon.com/images/I/31zNIWyeKK...,https://www.amazon.in//Brunte-Stylish-Drawing-...


In [141]:
product_A2 = pd.DataFrame(reviews[reviews['product_code']=='product_A2']).reset_index(drop=True)
product_A2

,user_id,product_id,Ratings,Time_stamp,product_code
0,ALUNVOQRXOZIA,B00000J0A2,4.0,1197244800,product_A2
1,A3AKB4GVGGWIVI,B00000J0A2,3.0,973900800,product_A2
2,AHUZ9S2BDOM3H,B00000J0A2,2.0,1042416000,product_A2
3,A355B9HM87SFHE,B00000J0A2,3.0,980208000,product_A2
4,A1YCOL87OT3QTH,B00000J0A2,4.0,1359849600,product_A2
...,...,...,...,...,...
11310,A20AJETVZ4LM5X,B00JUQH1A2,1.0,1404950400,product_A2
11311,ATRRKMHSGU0TH,B00JUQH1A2,5.0,1402876800,product_A2
11312,A2977EM75SD4KO,B00JUQH1A2,5.0,1405296000,product_A2
11313,A2IRTZV9VHALRI,B00K1JBHA2,5.0,1404086400,product_A2


In [142]:
len(product_A2.product_id.unique())

707

In [149]:
product_A2 = pd.DataFrame(reviews[reviews['product_code']=='product_A2']).reset_index(drop=True)
product_A2 =  pd.merge(product_A2, 
                       A2.iloc[:len(product_A2.product_id.unique())]\
                       .join(pd.Series(product_A2['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A2.to_csv('product_A2.csv',index=False)
pd.read_csv('product_A2.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ALUNVOQRXOZIA,B00000J0A2,4.0,1197244800,product_A2,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
1,A3AKB4GVGGWIVI,B00000J0A2,3.0,973900800,product_A2,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
2,AHUZ9S2BDOM3H,B00000J0A2,2.0,1042416000,product_A2,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
3,A355B9HM87SFHE,B00000J0A2,3.0,980208000,product_A2,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
4,A1YCOL87OT3QTH,B00000J0A2,4.0,1359849600,product_A2,"Brustro Artists Drawing Paper 200 GSM A2 Size,...",https://m.media-amazon.com/images/I/81+ulTQiyQ...,https://www.amazon.in//Brustro-Artists-Drawing...
...,...,...,...,...,...,...,...,...
11310,A20AJETVZ4LM5X,B00JUQH1A2,1.0,1404950400,product_A2,Variety Canvas Pantonic 8x10 Inch 400 GSM Canv...,https://m.media-amazon.com/images/I/61k-xNaBhY...,https://www.amazon.in//Variety-Canvas-Pantonic...
11311,ATRRKMHSGU0TH,B00JUQH1A2,5.0,1402876800,product_A2,Variety Canvas Pantonic 8x10 Inch 400 GSM Canv...,https://m.media-amazon.com/images/I/61k-xNaBhY...,https://www.amazon.in//Variety-Canvas-Pantonic...
11312,A2977EM75SD4KO,B00JUQH1A2,5.0,1405296000,product_A2,Variety Canvas Pantonic 8x10 Inch 400 GSM Canv...,https://m.media-amazon.com/images/I/61k-xNaBhY...,https://www.amazon.in//Variety-Canvas-Pantonic...
11313,A2IRTZV9VHALRI,B00K1JBHA2,5.0,1404086400,product_A2,DYKIDRAH Magic Slate for Kids Pen Doodle pad e...,https://m.media-amazon.com/images/I/41mX0KpuJK...,https://www.amazon.in//DYKIDRAH-erasable-Learn...


### Product A3 dataframe

In [153]:
df = get_scraped('a3 sketch book')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
1,Zen Sangam Artists’ - A3 Spiral Doodle Sketchb...,https://m.media-amazon.com/images/I/91VJvmCE6P...,https://www.amazon.in//Zen-Sangam-Sheets-Doodl...
2,Zen Sangam Artists’ - A3 Spiral Sketchpad Draw...,https://m.media-amazon.com/images/I/5177JVGDHd...,https://www.amazon.in//Zen-Sangam-Fineliners-O...
3,Navneet Youva | Soft Cover Drawing Book | A3 S...,https://m.media-amazon.com/images/I/71hsAMHn8p...,https://www.amazon.in//Navneet-Youva-Bound-Dra...
4,Brustro Artists Sketch Book Wiro Bound A3 Size...,https://m.media-amazon.com/images/I/71RWcdq8Z6...,https://www.amazon.in//Brustro-Artists-Sketch-...
...,...,...,...
301,Friends Gallery Best Quality Wire Bound Design...,https://m.media-amazon.com/images/I/71ZF8jVZY3...,https://www.amazon.in//Everyone-Affection-Ital...
302,"Amazon Brand - Solimo Sketching Book, 100 Shee...",https://m.media-amazon.com/images/I/81QX04BDkX...,https://www.amazon.in//Amazon-Brand-Solimo-Ske...
303,Drawing Book Family Store Forest Theme Designe...,https://m.media-amazon.com/images/I/61IhPo+yLV...,https://www.amazon.in//Drawing-Book-Family-Des...
304,Drawing Book Family Store Presents Wire Bound ...,https://m.media-amazon.com/images/I/71ITkJqrue...,https://www.amazon.in//Drawing-Book-Presents-D...


In [154]:
A3 = df.copy()
A3

,product_name,product_img,product_link
0,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
1,Zen Sangam Artists’ - A3 Spiral Doodle Sketchb...,https://m.media-amazon.com/images/I/91VJvmCE6P...,https://www.amazon.in//Zen-Sangam-Sheets-Doodl...
2,Zen Sangam Artists’ - A3 Spiral Sketchpad Draw...,https://m.media-amazon.com/images/I/5177JVGDHd...,https://www.amazon.in//Zen-Sangam-Fineliners-O...
3,Navneet Youva | Soft Cover Drawing Book | A3 S...,https://m.media-amazon.com/images/I/71hsAMHn8p...,https://www.amazon.in//Navneet-Youva-Bound-Dra...
4,Brustro Artists Sketch Book Wiro Bound A3 Size...,https://m.media-amazon.com/images/I/71RWcdq8Z6...,https://www.amazon.in//Brustro-Artists-Sketch-...
...,...,...,...
301,Friends Gallery Best Quality Wire Bound Design...,https://m.media-amazon.com/images/I/71ZF8jVZY3...,https://www.amazon.in//Everyone-Affection-Ital...
302,"Amazon Brand - Solimo Sketching Book, 100 Shee...",https://m.media-amazon.com/images/I/81QX04BDkX...,https://www.amazon.in//Amazon-Brand-Solimo-Ske...
303,Drawing Book Family Store Forest Theme Designe...,https://m.media-amazon.com/images/I/61IhPo+yLV...,https://www.amazon.in//Drawing-Book-Family-Des...
304,Drawing Book Family Store Presents Wire Bound ...,https://m.media-amazon.com/images/I/71ITkJqrue...,https://www.amazon.in//Drawing-Book-Presents-D...


In [ ]:
# A3 = A3.append(df).reset_index(drop=True)
# A3

In [151]:
product_A3 = pd.DataFrame(reviews[reviews['product_code']=='product_A3']).reset_index(drop=True)
product_A3

,user_id,product_id,Ratings,Time_stamp,product_code
0,A10FMNPNXT1FXY,B00000DMA3,5.0,1095465600,product_A3
1,A3UGKTDC32B9O5,B00000DMA3,3.0,1136332800,product_A3
2,A3A19ZBW81L4SL,B00000DMA3,4.0,1013126400,product_A3
3,A27D8EMHCTVPIF,B00000DMA3,2.0,1131408000,product_A3
4,A3UFCX1AE4TKZE,B00000DMA3,4.0,1104796800,product_A3
...,...,...,...,...,...
443,A2DZE219Q98EB2,B00009XVA3,5.0,1369785600,product_A3
444,APV09K5W9D0QQ,B0000C1IA3,5.0,1219708800,product_A3
445,A3EW91IEKZ3F9S,B0000C1IA3,4.0,1251849600,product_A3
446,A1SQMGEQ9WH3JL,B0000C1IA3,5.0,1197417600,product_A3


In [152]:
len(product_A3.product_id.unique())

22

In [155]:
product_A3 = pd.DataFrame(reviews[reviews['product_code']=='product_A3']).reset_index(drop=True)
product_A3 =  pd.merge(product_A3, 
                       A3.iloc[:len(product_A3.product_id.unique())]\
                       .join(pd.Series(product_A3['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A3.to_csv('product_A3.csv',index=False)
pd.read_csv('product_A3.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A10FMNPNXT1FXY,B00000DMA3,5.0,1095465600,product_A3,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
1,A3UGKTDC32B9O5,B00000DMA3,3.0,1136332800,product_A3,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
2,A3A19ZBW81L4SL,B00000DMA3,4.0,1013126400,product_A3,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
3,A27D8EMHCTVPIF,B00000DMA3,2.0,1131408000,product_A3,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
4,A3UFCX1AE4TKZE,B00000DMA3,4.0,1104796800,product_A3,"Brustro Artists Wiro Bound Sketch Book, A3 Siz...",https://m.media-amazon.com/images/I/61cZVACEgv...,https://www.amazon.in//Brustro-Bound-Artists-S...
...,...,...,...,...,...,...,...,...
443,A2DZE219Q98EB2,B00009XVA3,5.0,1369785600,product_A3,"SHARMA BUSINESS A3 Sketch Book Kids Student, A...",https://m.media-amazon.com/images/I/31nCkXhvyT...,https://www.amazon.in//SHARMA-BUSINESS-Student...
444,APV09K5W9D0QQ,B0000C1IA3,5.0,1219708800,product_A3,Paper Plane Design Artist Sketch Book 200 Page...,https://m.media-amazon.com/images/I/71pPcRSAVs...,https://www.amazon.in//Design-Artist-Sketch-Sh...
445,A3EW91IEKZ3F9S,B0000C1IA3,4.0,1251849600,product_A3,Paper Plane Design Artist Sketch Book 200 Page...,https://m.media-amazon.com/images/I/71pPcRSAVs...,https://www.amazon.in//Design-Artist-Sketch-Sh...
446,A1SQMGEQ9WH3JL,B0000C1IA3,5.0,1197417600,product_A3,Paper Plane Design Artist Sketch Book 200 Page...,https://m.media-amazon.com/images/I/71pPcRSAVs...,https://www.amazon.in//Design-Artist-Sketch-Sh...


### Product A4 dataframe

In [165]:
df = get_scraped('a4 paper cutter')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"3A Featuretail Paper Cutter, A4 Paper Trimmer ...",https://m.media-amazon.com/images/I/61WHxs7apl...,https://www.amazon.in//3A-Featuretail-Automati...
1,JD9 Paper Cutter A4 Heavy Duty Professional Pa...,https://m.media-amazon.com/images/I/71iirmK-bk...,https://www.amazon.in//JD9-Professional-Trimme...
2,Firbon A4 Paper Cutter 12 Inch Titanium Straig...,https://m.media-amazon.com/images/I/71y-HDZdyU...,https://www.amazon.in//Firbon-Titanium-Straigh...
3,JD9 Paper Cutter A4 Heavy Duty Professional Pa...,https://m.media-amazon.com/images/I/719jiX94Ga...,https://www.amazon.in//JD9-Professional-Trimme...
4,CraftDev A500MB Portable Paper Trimmer 12 Inch...,https://m.media-amazon.com/images/I/71bAXU-1BC...,https://www.amazon.in//CraftDev-A500MB-Portabl...
...,...,...,...
253,Paper Trimmer A4 Guillotine Paper Cutter Cutti...,https://m.media-amazon.com/images/I/516pbeLgtA...,https://www.amazon.in//Trimmer-Guillotine-Cutt...
254,Kosh PVC Spiral Book Binding Sheet A/4 (Set of...,https://m.media-amazon.com/images/I/417iNsH7dk...,https://www.amazon.in//Kosh-Spiral-Binding-She...
255,VERENA 12 Inch A4 Heavy Duty Professional Pape...,https://m.media-amazon.com/images/I/517zHU3Low...,https://www.amazon.in//VERENA-Professional-Tri...
256,pekdi Multifunctional A4 Paper Trimmer Cutters...,https://m.media-amazon.com/images/I/61b3VPB5X3...,https://www.amazon.in//pekdi-Multifunctional-T...


In [160]:
A4 = df.copy()
A4

,product_name,product_img,product_link
0,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
1,Askprints 50 Sheet A5 Sketchbook Set of 2-5.8 ...,https://m.media-amazon.com/images/I/81gNNXUNWY...,https://www.amazon.in//Askprints-Sketchbook-Sp...
2,Paper Pebbles Pansy Flower Pattern Sketch Book...,https://m.media-amazon.com/images/I/71BpXrpqf8...,https://www.amazon.in//Paper-Pebbles-Flower-Pa...
3,Paper Plane design Spiral Bound Artists Sketch...,https://m.media-amazon.com/images/I/81c32Ok9wf...,https://www.amazon.in//design-Spiral-Artists-S...
4,"KAMAL Drawing and Sketch Pad for Artists, 120L...",https://m.media-amazon.com/images/I/71biNqSZNd...,https://www.amazon.in//KAMAL-Drawing-Artists-A...
...,...,...,...
109,Glory A4 Size My Artist Drawing pad Book for W...,https://m.media-amazon.com/images/I/81RQVa0bVB...,https://www.amazon.in//Artist-Drawing-Colours-...
110,MOREL (A4 Size) Drawing Sketch Book 250 Pages ...,https://m.media-amazon.com/images/I/51mb9rJ5sZ...,https://www.amazon.in//MOREL-Drawing-Colorful-...
111,ESCAPER Ethnic Floral Mandala Theme Sketch Boo...,https://m.media-amazon.com/images/I/A1RbruQDgE...,https://www.amazon.in//ESCAPER-Ethnic-Floral-M...
112,ESCAPER Fine Mandala Theme Sketch Book (A4 Siz...,https://m.media-amazon.com/images/I/91oe0thRei...,https://www.amazon.in//ESCAPER-Fine-Mandala-Th...


In [166]:
A4 = A4.append(df).reset_index(drop=True)
A4

,product_name,product_img,product_link
0,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
1,Askprints 50 Sheet A5 Sketchbook Set of 2-5.8 ...,https://m.media-amazon.com/images/I/81gNNXUNWY...,https://www.amazon.in//Askprints-Sketchbook-Sp...
2,Paper Pebbles Pansy Flower Pattern Sketch Book...,https://m.media-amazon.com/images/I/71BpXrpqf8...,https://www.amazon.in//Paper-Pebbles-Flower-Pa...
3,Paper Plane design Spiral Bound Artists Sketch...,https://m.media-amazon.com/images/I/81c32Ok9wf...,https://www.amazon.in//design-Spiral-Artists-S...
4,"KAMAL Drawing and Sketch Pad for Artists, 120L...",https://m.media-amazon.com/images/I/71biNqSZNd...,https://www.amazon.in//KAMAL-Drawing-Artists-A...
...,...,...,...
817,Paper Trimmer A4 Guillotine Paper Cutter Cutti...,https://m.media-amazon.com/images/I/516pbeLgtA...,https://www.amazon.in//Trimmer-Guillotine-Cutt...
818,Kosh PVC Spiral Book Binding Sheet A/4 (Set of...,https://m.media-amazon.com/images/I/417iNsH7dk...,https://www.amazon.in//Kosh-Spiral-Binding-She...
819,VERENA 12 Inch A4 Heavy Duty Professional Pape...,https://m.media-amazon.com/images/I/517zHU3Low...,https://www.amazon.in//VERENA-Professional-Tri...
820,pekdi Multifunctional A4 Paper Trimmer Cutters...,https://m.media-amazon.com/images/I/61b3VPB5X3...,https://www.amazon.in//pekdi-Multifunctional-T...


In [157]:
product_A4 = pd.DataFrame(reviews[reviews['product_code']=='product_A4']).reset_index(drop=True)
product_A4

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3PROZ0C88T7MZ,B00000DMA4,4.0,959299200,product_A4
1,A3R5UY152HB2TA,B00000DMA4,1.0,1069372800,product_A4
2,A3U6W3M2UDRK1N,B00000DMA4,3.0,961372800,product_A4
3,A2WFZRROXKW6QH,B00000DMA4,1.0,972345600,product_A4
4,A2OUI0T52PTL,B00000DMA4,1.0,1010620800,product_A4
...,...,...,...,...,...
12689,A30HJ8TQ4MN7AO,B00L26YDA4,5.0,1405555200,product_A4
12690,A3NEAETOSXDBOM,B00L26YDA4,3.0,1403827200,product_A4
12691,A3A4ZAIBQWKOZS,B00L26YDA4,5.0,1403827200,product_A4
12692,A3R19YKNL641X3,B00L26YDA4,4.0,1405209600,product_A4


In [158]:
len(product_A4.product_id.unique())

728

In [167]:
product_A4 = pd.DataFrame(reviews[reviews['product_code']=='product_A4']).reset_index(drop=True)
product_A4 =  pd.merge(product_A4, 
                       A4.iloc[:len(product_A4.product_id.unique())]\
                       .join(pd.Series(product_A4['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A4.to_csv('product_A4.csv',index=False)
pd.read_csv('product_A4.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3PROZ0C88T7MZ,B00000DMA4,4.0,959299200,product_A4,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
1,A3R5UY152HB2TA,B00000DMA4,1.0,1069372800,product_A4,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
2,A3U6W3M2UDRK1N,B00000DMA4,3.0,961372800,product_A4,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
3,A2WFZRROXKW6QH,B00000DMA4,1.0,972345600,product_A4,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
4,A2OUI0T52PTL,B00000DMA4,1.0,1010620800,product_A4,Sundaram Sketch Book A4 Spiral- 100 pages (D-1...,https://m.media-amazon.com/images/I/61nxxUUAoZ...,https://www.amazon.in//Sundaram-D-15-Sketch-Bo...
...,...,...,...,...,...,...,...,...
12689,A30HJ8TQ4MN7AO,B00L26YDA4,5.0,1405555200,product_A4,VR SHOPEE A4 Slim Clipboard Folder Conference ...,https://m.media-amazon.com/images/I/711pE6Fx92...,https://www.amazon.in//VR-SHOPEE-Clipboard-Con...
12690,A3NEAETOSXDBOM,B00L26YDA4,3.0,1403827200,product_A4,VR SHOPEE A4 Slim Clipboard Folder Conference ...,https://m.media-amazon.com/images/I/711pE6Fx92...,https://www.amazon.in//VR-SHOPEE-Clipboard-Con...
12691,A3A4ZAIBQWKOZS,B00L26YDA4,5.0,1403827200,product_A4,VR SHOPEE A4 Slim Clipboard Folder Conference ...,https://m.media-amazon.com/images/I/711pE6Fx92...,https://www.amazon.in//VR-SHOPEE-Clipboard-Con...
12692,A3R19YKNL641X3,B00L26YDA4,4.0,1405209600,product_A4,VR SHOPEE A4 Slim Clipboard Folder Conference ...,https://m.media-amazon.com/images/I/711pE6Fx92...,https://www.amazon.in//VR-SHOPEE-Clipboard-Con...


### Product A5 dataframe

In [171]:
df = get_scraped('a5 paper')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
1,IMPRINT A5 Copier Paper(SMALL SIZE-Not A4 Size...,https://m.media-amazon.com/images/I/71+UJE-5Gq...,https://www.amazon.in//IMPRINT-Copier-SIZE-Not...
2,IMPRINT A5 Copier Pre Punched Paper Pack of 50...,https://m.media-amazon.com/images/I/71-cDQP56W...,https://www.amazon.in//IMPRINT-Copier-Punched-...
3,"Green Way JK Copier Paper A5 500 Sheets, 75 GS...",https://m.media-amazon.com/images/I/71kmALRtsk...,https://www.amazon.in//Green-Way-Copier-Paper-...
4,JK Paper Sparkle A5 Copier 70 GSM- Box (10 rea...,https://m.media-amazon.com/images/I/71bFMKjYQZ...,https://www.amazon.in//JK-Paper-Sparkle-Copier...
...,...,...,...
187,A5 Black Diary/Journal with imported Italian B...,https://m.media-amazon.com/images/I/71mDyLY1nf...,https://www.amazon.in//Black-Diary-Journal-Ita...
188,AccuPrints Black Hard Bound A5 or 5.8 * 8.3 in...,https://m.media-amazon.com/images/I/81HltUN+7i...,https://www.amazon.in//AccuPrints-Personal-Not...
189,Ayush paper Mixed Media Sketchbook A5 Portrait...,https://m.media-amazon.com/images/I/511bXLJA6j...,https://www.amazon.in//Ayush-paper-Sketchbook-...
190,Black Panther A5 Size Wiro Bound Spiral Notebo...,https://m.media-amazon.com/images/I/41unH0bBDx...,https://www.amazon.in//Panther-Spiral-Notebook...


In [172]:
A5 = df.copy()
A5

,product_name,product_img,product_link
0,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
1,IMPRINT A5 Copier Paper(SMALL SIZE-Not A4 Size...,https://m.media-amazon.com/images/I/71+UJE-5Gq...,https://www.amazon.in//IMPRINT-Copier-SIZE-Not...
2,IMPRINT A5 Copier Pre Punched Paper Pack of 50...,https://m.media-amazon.com/images/I/71-cDQP56W...,https://www.amazon.in//IMPRINT-Copier-Punched-...
3,"Green Way JK Copier Paper A5 500 Sheets, 75 GS...",https://m.media-amazon.com/images/I/71kmALRtsk...,https://www.amazon.in//Green-Way-Copier-Paper-...
4,JK Paper Sparkle A5 Copier 70 GSM- Box (10 rea...,https://m.media-amazon.com/images/I/71bFMKjYQZ...,https://www.amazon.in//JK-Paper-Sparkle-Copier...
...,...,...,...
187,A5 Black Diary/Journal with imported Italian B...,https://m.media-amazon.com/images/I/71mDyLY1nf...,https://www.amazon.in//Black-Diary-Journal-Ita...
188,AccuPrints Black Hard Bound A5 or 5.8 * 8.3 in...,https://m.media-amazon.com/images/I/81HltUN+7i...,https://www.amazon.in//AccuPrints-Personal-Not...
189,Ayush paper Mixed Media Sketchbook A5 Portrait...,https://m.media-amazon.com/images/I/511bXLJA6j...,https://www.amazon.in//Ayush-paper-Sketchbook-...
190,Black Panther A5 Size Wiro Bound Spiral Notebo...,https://m.media-amazon.com/images/I/41unH0bBDx...,https://www.amazon.in//Panther-Spiral-Notebook...


In [ ]:
# A5 = A5.append(df).reset_index(drop=True)
# A5

In [169]:
product_A5 = pd.DataFrame(reviews[reviews['product_code']=='product_A5']).reset_index(drop=True)
product_A5

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2H1I0AR67NWAC,B00000J4A5,3.0,945388800,product_A5
1,A1LPUD5YTDY7IT,B00000J4A5,5.0,964396800,product_A5
2,AMPKCKXMSHVPI,B00000J4A5,5.0,953510400,product_A5
3,AT2BYFEVLVITT,B00000J4A5,5.0,959817600,product_A5
4,A2VET6XXV90UDQ,B00000J4A5,5.0,986947200,product_A5
...,...,...,...,...,...
307,A2P4WYXZGLDISR,B0000CEPA5,3.0,1086307200,product_A5
308,A3JV3WNI21NBCT,B0000CEPA5,4.0,1084147200,product_A5
309,A387YKJRG26KE5,B0000CEPA5,4.0,1100736000,product_A5
310,A1IT2WY4YTOD9T,B0000CEPA5,1.0,1152316800,product_A5


In [170]:
len(product_A5.product_id.unique())

29

In [173]:
product_A5 = pd.DataFrame(reviews[reviews['product_code']=='product_A5']).reset_index(drop=True)
product_A5 =  pd.merge(product_A5, 
                       A5.iloc[:len(product_A5.product_id.unique())]\
                       .join(pd.Series(product_A5['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A5.to_csv('product_A5.csv',index=False)
pd.read_csv('product_A5.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2H1I0AR67NWAC,B00000J4A5,3.0,945388800,product_A5,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
1,A1LPUD5YTDY7IT,B00000J4A5,5.0,964396800,product_A5,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
2,AMPKCKXMSHVPI,B00000J4A5,5.0,953510400,product_A5,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
3,AT2BYFEVLVITT,B00000J4A5,5.0,959817600,product_A5,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
4,A2VET6XXV90UDQ,B00000J4A5,5.0,986947200,product_A5,SHARMA BUSINESS A5 COPY AND MULTIPURPOSE PAPER...,https://m.media-amazon.com/images/I/61wiMgk3uV...,https://www.amazon.in//SHARMA-BUSINESS-MULTIPU...
...,...,...,...,...,...,...,...,...
307,A2P4WYXZGLDISR,B0000CEPA5,3.0,1086307200,product_A5,Brustro Black Paper - 200 GSM A5 Pack of 2 (40...,https://m.media-amazon.com/images/I/81dwpOWiVA...,https://www.amazon.in//Brustro-Black-Paper-Pac...
308,A3JV3WNI21NBCT,B0000CEPA5,4.0,1084147200,product_A5,Brustro Black Paper - 200 GSM A5 Pack of 2 (40...,https://m.media-amazon.com/images/I/81dwpOWiVA...,https://www.amazon.in//Brustro-Black-Paper-Pac...
309,A387YKJRG26KE5,B0000CEPA5,4.0,1100736000,product_A5,Brustro Black Paper - 200 GSM A5 Pack of 2 (40...,https://m.media-amazon.com/images/I/81dwpOWiVA...,https://www.amazon.in//Brustro-Black-Paper-Pac...
310,A1IT2WY4YTOD9T,B0000CEPA5,1.0,1152316800,product_A5,Brustro Black Paper - 200 GSM A5 Pack of 2 (40...,https://m.media-amazon.com/images/I/81dwpOWiVA...,https://www.amazon.in//Brustro-Black-Paper-Pac...


### Product A6 dataframe

In [13]:
df = get_scraped('a6 binder')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"A6 PVC Binder with Pages,Laser",https://m.media-amazon.com/images/I/41Qku9g19E...,https://www.amazon.in//A6-PVC-Binder-Pages-Las...
1,Binder Index PP Dividers for 8.5 inch/6 inch A...,https://m.media-amazon.com/images/I/61AHuGnkrn...,https://www.amazon.in//A6-Divider-Paper-Cherry...
2,"Budget Binder, A6 Notebook Binder PU Leather C...",https://m.media-amazon.com/images/I/51auSZXSW2...,https://www.amazon.in//Budget-Notebook-Leather...
3,"A6 Notebook Binder, A6 Budget Binder Portable ...",https://m.media-amazon.com/images/I/61VexZVGNz...,https://www.amazon.in//Notebook-Binder-Budget-...
4,"Cash Budget Binder, Soft Tear Resistant A6 Bud...",https://m.media-amazon.com/images/I/51Ykn19+Sq...,https://www.amazon.in//Budget-Resistant-Waterp...
...,...,...,...
301,Layfoxz Adjustable 6-Hole Desktop Punch Punche...,https://m.media-amazon.com/images/I/51wdaYdjks...,https://www.amazon.in//Layfoxz-Adjustable-Desk...
302,"Budget Binder, Cash Budget Binder Minimalist L...",https://m.media-amazon.com/images/I/615FaSW+8j...,https://www.amazon.in//Minimalist-Waterproof-L...
303,"Yuanbigtai Budget Binder,Budget Planner,A6 PU ...",https://m.media-amazon.com/images/I/71VgocDboC...,https://www.amazon.in//Yuanbigtai-Planner-Incl...
304,VMS INDIA Gold Stamping Index Dividers Tabbed ...,https://m.media-amazon.com/images/I/51BdJHlvJQ...,https://www.amazon.in//VMS-INDIA-Stamping-Divi...


In [10]:
A6 = df.copy()
A6

,product_name,product_img,product_link
0,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
1,"INNAXA A6, Travelers' Notebook Thread-Bound Jo...",https://m.media-amazon.com/images/I/615ww23-iC...,https://www.amazon.in//INNAXA-Travelers-Notebo...
2,Kitabby – Linen Series – A6 Size – Hard Bound ...,https://m.media-amazon.com/images/I/71c13a0UXH...,https://www.amazon.in//Kitabby-Notebook-Elasti...
3,A6 Size Ruled Pages Graphic Diary 73,https://m.media-amazon.com/images/I/61YF3DXKcP...,https://www.amazon.in//Size-RULED-Pages-Graphi...
4,A6 Size Ruled Pages Graphic Diary 98,https://m.media-amazon.com/images/I/61V3TSA55n...,https://www.amazon.in//Size-RULED-Pages-Graphi...
...,...,...,...
301,A6 Size Ruled Pages Graphic Diary 38,https://m.media-amazon.com/images/I/61HmQeTxZw...,https://www.amazon.in//Size-RULED-Pages-Graphi...
302,Whats Your Kick (CSK) - Recycled Soft cover De...,https://m.media-amazon.com/images/I/71Fp1AIT4F...,https://www.amazon.in//Whats-Your-Kick-CSK-Aer...
303,Whats Your Kick - Recycled Soft cover Designer...,https://m.media-amazon.com/images/I/81+q0mubLA...,https://www.amazon.in//Whats-Your-Kick-Recycle...
304,"Denpetec Super Thick Notebook Stationery,A4 A5...",https://m.media-amazon.com/images/I/61sNAg-0UL...,https://www.amazon.in//Denpetec-Notebook-Stati...


In [14]:
A6 = A6.append(df).reset_index(drop=True)
A6

,product_name,product_img,product_link
0,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
1,"INNAXA A6, Travelers' Notebook Thread-Bound Jo...",https://m.media-amazon.com/images/I/615ww23-iC...,https://www.amazon.in//INNAXA-Travelers-Notebo...
2,Kitabby – Linen Series – A6 Size – Hard Bound ...,https://m.media-amazon.com/images/I/71c13a0UXH...,https://www.amazon.in//Kitabby-Notebook-Elasti...
3,A6 Size Ruled Pages Graphic Diary 73,https://m.media-amazon.com/images/I/61YF3DXKcP...,https://www.amazon.in//Size-RULED-Pages-Graphi...
4,A6 Size Ruled Pages Graphic Diary 98,https://m.media-amazon.com/images/I/61V3TSA55n...,https://www.amazon.in//Size-RULED-Pages-Graphi...
...,...,...,...
865,Layfoxz Adjustable 6-Hole Desktop Punch Punche...,https://m.media-amazon.com/images/I/51wdaYdjks...,https://www.amazon.in//Layfoxz-Adjustable-Desk...
866,"Budget Binder, Cash Budget Binder Minimalist L...",https://m.media-amazon.com/images/I/615FaSW+8j...,https://www.amazon.in//Minimalist-Waterproof-L...
867,"Yuanbigtai Budget Binder,Budget Planner,A6 PU ...",https://m.media-amazon.com/images/I/71VgocDboC...,https://www.amazon.in//Yuanbigtai-Planner-Incl...
868,VMS INDIA Gold Stamping Index Dividers Tabbed ...,https://m.media-amazon.com/images/I/51BdJHlvJQ...,https://www.amazon.in//VMS-INDIA-Stamping-Divi...


In [7]:
product_A6 = pd.DataFrame(reviews[reviews['product_code']=='product_A6']).reset_index(drop=True)
product_A6

,user_id,product_id,Ratings,Time_stamp,product_code
0,A10SQ620SCF6ZX,B00003CWA6,5.0,1250380800,product_A6
1,A2O71BU2LQ9NML,B00003CWA6,5.0,1006387200,product_A6
2,A3BLY1XKLOIV1W,B00003CWA6,3.0,1078012800,product_A6
3,A3FN1E32ENP19R,B00003CWA6,4.0,1074384000,product_A6
4,AG1O81FZRO9TL,B00003CWA6,4.0,1252281600,product_A6
...,...,...,...,...,...
16842,A3PAAKKKSILNTT,B00K3E0CA6,5.0,1404172800,product_A6
16843,ARNFHZL93K06S,B00K3E0CA6,5.0,1404345600,product_A6
16844,A2SBB5KVPDAOTI,B00KDQYLA6,1.0,1405987200,product_A6
16845,A1IM5PNIP5Y128,B00KDQYLA6,5.0,1405123200,product_A6


In [8]:
len(product_A6.product_id.unique())

693

In [15]:
product_A6 = pd.DataFrame(reviews[reviews['product_code']=='product_A6']).reset_index(drop=True)
product_A6 =  pd.merge(product_A6, 
                       A6.iloc[:len(product_A6.product_id.unique())]\
                       .join(pd.Series(product_A6['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A6.to_csv('product_A6.csv',index=False)
pd.read_csv('product_A6.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A10SQ620SCF6ZX,B00003CWA6,5.0,1250380800,product_A6,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
1,A2O71BU2LQ9NML,B00003CWA6,5.0,1006387200,product_A6,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
2,A3BLY1XKLOIV1W,B00003CWA6,3.0,1078012800,product_A6,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
3,A3FN1E32ENP19R,B00003CWA6,4.0,1074384000,product_A6,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
4,AG1O81FZRO9TL,B00003CWA6,4.0,1252281600,product_A6,Luxor Subject 4 Single Ruled Pocket Notebook -...,https://m.media-amazon.com/images/I/61NIBSakl6...,https://www.amazon.in//Luxor-Subject-Single-Po...
...,...,...,...,...,...,...,...,...
16842,A3PAAKKKSILNTT,B00K3E0CA6,5.0,1404172800,product_A6,Twavang 15 PCS A6 Binder Pockets with PU Leath...,https://m.media-amazon.com/images/I/512FxTycCD...,https://www.amazon.in//Twavang-Notebook-Refill...
16843,ARNFHZL93K06S,B00K3E0CA6,5.0,1404345600,product_A6,Twavang 15 PCS A6 Binder Pockets with PU Leath...,https://m.media-amazon.com/images/I/512FxTycCD...,https://www.amazon.in//Twavang-Notebook-Refill...
16844,A2SBB5KVPDAOTI,B00KDQYLA6,1.0,1405987200,product_A6,"Budget Binder, Portable Clear View A6 Home Bud...",https://m.media-amazon.com/images/I/61esLjzbWn...,https://www.amazon.in//Budget-Binder-Portable-...
16845,A1IM5PNIP5Y128,B00KDQYLA6,5.0,1405123200,product_A6,"Budget Binder, Portable Clear View A6 Home Bud...",https://m.media-amazon.com/images/I/61esLjzbWn...,https://www.amazon.in//Budget-Binder-Portable-...


### Product A7 dataframe

In [45]:
df = get_scraped('a7 cover')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Pikkme Samsung Galaxy A7 2018 Back Cover | Hyb...,https://m.media-amazon.com/images/I/71VvPh5H2s...,https://www.amazon.in//Pikkme-Samsung-Galaxy-P...
1,MOBITUSSION Silicone Shockproof 360 Degree Pro...,https://m.media-amazon.com/images/I/41Qv-N91Wr...,https://www.amazon.in//MobiTussion-Shockproof-...
2,WOW Imagine Shock Proof Flip Cover Back Case C...,https://m.media-amazon.com/images/I/71DbfKmyAi...,https://www.amazon.in//WOW-Imagine-Magnetic-Co...
3,Plus Back Cover for Samsung Galaxy A7 (2018) (...,https://m.media-amazon.com/images/I/31nNTu0LzJ...,https://www.amazon.in//Plus-Protective-Transpa...
4,Pikkme Bumper for Samsung Galaxy A7 2018 (Ther...,https://m.media-amazon.com/images/I/51Gdat5Uqt...,https://www.amazon.in//Pikkme-Bumper-Samsung-T...
...,...,...,...
301,HOBBYTRONICS Rubber Protective and Anti Shock ...,https://m.media-amazon.com/images/I/51TiXN-IcL...,https://www.amazon.in//HOBBYTRONICS-Protective...
302,Backer The Brand Compatible with Samsung Galax...,https://m.media-amazon.com/images/I/31V0RoFS2j...,https://www.amazon.in//Backer-Brand-Samsung-Ga...
303,Oppo A7 Transparent Cover Dustproof and Waterp...,https://m.media-amazon.com/images/I/41jDUzqdc9...,https://www.amazon.in//Transparent-Cover-Dustp...
304,ELRACases® Lovers Divine Couple Lord Radha Kri...,https://m.media-amazon.com/images/I/71YJN0rtvG...,https://www.amazon.in//ELRACases%C2%AE-Back-Co...


In [46]:
A7 = df.copy()
A7

,product_name,product_img,product_link
0,Pikkme Samsung Galaxy A7 2018 Back Cover | Hyb...,https://m.media-amazon.com/images/I/71VvPh5H2s...,https://www.amazon.in//Pikkme-Samsung-Galaxy-P...
1,MOBITUSSION Silicone Shockproof 360 Degree Pro...,https://m.media-amazon.com/images/I/41Qv-N91Wr...,https://www.amazon.in//MobiTussion-Shockproof-...
2,WOW Imagine Shock Proof Flip Cover Back Case C...,https://m.media-amazon.com/images/I/71DbfKmyAi...,https://www.amazon.in//WOW-Imagine-Magnetic-Co...
3,Plus Back Cover for Samsung Galaxy A7 (2018) (...,https://m.media-amazon.com/images/I/31nNTu0LzJ...,https://www.amazon.in//Plus-Protective-Transpa...
4,Pikkme Bumper for Samsung Galaxy A7 2018 (Ther...,https://m.media-amazon.com/images/I/51Gdat5Uqt...,https://www.amazon.in//Pikkme-Bumper-Samsung-T...
...,...,...,...
301,HOBBYTRONICS Rubber Protective and Anti Shock ...,https://m.media-amazon.com/images/I/51TiXN-IcL...,https://www.amazon.in//HOBBYTRONICS-Protective...
302,Backer The Brand Compatible with Samsung Galax...,https://m.media-amazon.com/images/I/31V0RoFS2j...,https://www.amazon.in//Backer-Brand-Samsung-Ga...
303,Oppo A7 Transparent Cover Dustproof and Waterp...,https://m.media-amazon.com/images/I/41jDUzqdc9...,https://www.amazon.in//Transparent-Cover-Dustp...
304,ELRACases® Lovers Divine Couple Lord Radha Kri...,https://m.media-amazon.com/images/I/71YJN0rtvG...,https://www.amazon.in//ELRACases%C2%AE-Back-Co...


In [ ]:
# A7 = A7.append(df).reset_index(drop=True)
# A7

In [19]:
product_A7 = pd.DataFrame(reviews[reviews['product_code']=='product_A7']).reset_index(drop=True)
product_A7

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2CJX4YUXLV4FT,B00004Z7A7,1.0,1115164800,product_A7
1,A351KD10AM8MJN,B00004Z7A7,3.0,978134400,product_A7
2,A39I2YYIVLBE96,B00005AYA7,5.0,1052784000,product_A7
3,A1BDPRQ7BEUNLP,B00005AYA7,3.0,1009497600,product_A7
4,A5W7E366W2F2E,B00005AYA7,4.0,1032220800,product_A7
...,...,...,...,...,...
102,A3BRJW85SMKZF,B0000AE6A7,3.0,1281484800,product_A7
103,A3OEIPLU13BEW2,B0000AE6A7,3.0,1282521600,product_A7
104,A1VOG6AJASHFRA,B0000AE6A7,4.0,1244073600,product_A7
105,A3EXKKVITRQ03S,B0000CC7A7,3.0,1171324800,product_A7


In [20]:
len(product_A7.product_id.unique())

14

In [47]:
product_A7 = pd.DataFrame(reviews[reviews['product_code']=='product_A7']).reset_index(drop=True)
product_A7 =  pd.merge(product_A7, 
                       A7.iloc[:len(product_A7.product_id.unique())]\
                       .join(pd.Series(product_A7['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A7.to_csv('product_A7.csv',index=False)
pd.read_csv('product_A7.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2CJX4YUXLV4FT,B00004Z7A7,1.0,1115164800,product_A7,Pikkme Samsung Galaxy A7 2018 Back Cover | Hyb...,https://m.media-amazon.com/images/I/71VvPh5H2s...,https://www.amazon.in//Pikkme-Samsung-Galaxy-P...
1,A351KD10AM8MJN,B00004Z7A7,3.0,978134400,product_A7,Pikkme Samsung Galaxy A7 2018 Back Cover | Hyb...,https://m.media-amazon.com/images/I/71VvPh5H2s...,https://www.amazon.in//Pikkme-Samsung-Galaxy-P...
2,A39I2YYIVLBE96,B00005AYA7,5.0,1052784000,product_A7,MOBITUSSION Silicone Shockproof 360 Degree Pro...,https://m.media-amazon.com/images/I/41Qv-N91Wr...,https://www.amazon.in//MobiTussion-Shockproof-...
3,A1BDPRQ7BEUNLP,B00005AYA7,3.0,1009497600,product_A7,MOBITUSSION Silicone Shockproof 360 Degree Pro...,https://m.media-amazon.com/images/I/41Qv-N91Wr...,https://www.amazon.in//MobiTussion-Shockproof-...
4,A5W7E366W2F2E,B00005AYA7,4.0,1032220800,product_A7,MOBITUSSION Silicone Shockproof 360 Degree Pro...,https://m.media-amazon.com/images/I/41Qv-N91Wr...,https://www.amazon.in//MobiTussion-Shockproof-...
...,...,...,...,...,...,...,...,...
102,A3BRJW85SMKZF,B0000AE6A7,3.0,1281484800,product_A7,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71QyTXlfuR...,https://www.amazon.in//Rubberised-Samsung-Gala...
103,A3OEIPLU13BEW2,B0000AE6A7,3.0,1282521600,product_A7,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71QyTXlfuR...,https://www.amazon.in//Rubberised-Samsung-Gala...
104,A1VOG6AJASHFRA,B0000AE6A7,4.0,1244073600,product_A7,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71QyTXlfuR...,https://www.amazon.in//Rubberised-Samsung-Gala...
105,A3EXKKVITRQ03S,B0000CC7A7,3.0,1171324800,product_A7,Pikkme Bumper for Samsung Galaxy A7 2018 (Ther...,https://m.media-amazon.com/images/I/51wJrPu+-P...,https://www.amazon.in//Pikkme-Bumper-Samsung-T...


### Product A8 dataframe

In [55]:
df = get_scraped('a8 samsung tablet')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Samsung Galaxy Tab A8 26.69 cm (10.5 inch) Dis...,https://m.media-amazon.com/images/I/91veRYPjpe...,https://www.amazon.in//Samsung-Galaxy-Display-...
1,Samsung Galaxy Tab A8 26.69 cm (10.5 inch) Dis...,https://m.media-amazon.com/images/I/91veRYPjpe...,https://www.amazon.in//Samsung-Galaxy-Display-...
2,Samsung Galaxy Tab A8 26.69 cm (10.5 inch) Dis...,https://m.media-amazon.com/images/I/91veRYPjpe...,https://www.amazon.in//Samsung-Galaxy-Display-...
3,Samsung Galaxy Tab A8 26.69 cm (10.5 inch) Dis...,https://m.media-amazon.com/images/I/91afx8hEj9...,https://www.amazon.in//Samsung-Galaxy-Display-...
4,Samsung Galaxy Tab A8 10.5 inches(26cm) Displa...,https://m.media-amazon.com/images/I/91afx8hEj9...,https://www.amazon.in//Samsung-Galaxy-Display-...
...,...,...,...
87,(Renewed) Samsung Galaxy Tab S7 27.81 cm (11 i...,https://m.media-amazon.com/images/I/81scJqZDEJ...,https://www.amazon.in//Renewed-Samsung-Galaxy-...
88,Samsung Galaxy Tab S7 FE 31.5 cm (12.4 inch) L...,https://m.media-amazon.com/images/I/9135RJhz8x...,https://www.amazon.in//Samsung-Galaxy-Display-...
89,Samsung Galaxy Tab S7 FE 31.5 cm (12.4 inch) L...,https://m.media-amazon.com/images/I/910xLQicbV...,https://www.amazon.in//Samsung-Galaxy-Display-...
90,"Samsung Galaxy Tab Active PRO 10.1"" | 64GB & W...",https://m.media-amazon.com/images/I/71XFm0upzQ...,https://www.amazon.in//Samsung-Galaxy-Active-W...


In [52]:
A8 = df.copy()
A8

,product_name,product_img,product_link
0,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
1,Robustrion Deer Pattern Flip Stand Case Cover ...,https://m.media-amazon.com/images/I/81TcqSe5mO...,https://www.amazon.in//Robustrion-Pattern-Sams...
2,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61OShUky-o...,https://www.amazon.in//Robustrion-Trifold-Sams...
3,Robustrion Trifold Flip Stand Case Cover for S...,https://m.media-amazon.com/images/I/61AZ4KCQjf...,https://www.amazon.in//Robustrion-Trifold-Sams...
4,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/71TriWYOJQ...,https://www.amazon.in//Robustrion-Trifold-Sams...
...,...,...,...
301,"Lustree Samsung Galaxy Tab A7 Lite 8.7"" Flip C...",https://m.media-amazon.com/images/I/51Gpr9ga-m...,https://www.amazon.in//Lustree-Samsung-Galaxy-...
302,SwooK Case Compatible for Samsung Galaxy Tab A...,https://m.media-amazon.com/images/I/81EYHHNe5t...,https://www.amazon.in//SwooK-Compatible-8-7-In...
303,ProElite Smart Deer Flip case Cover for Samsun...,https://m.media-amazon.com/images/I/51P+P3aIWu...,https://www.amazon.in//ProElite-Smart-Samsung-...
304,TGK Texture Leather Case with Viewing Stand Fl...,https://m.media-amazon.com/images/I/911VlbctcG...,https://www.amazon.in//TGK-Texture-Leather-Vie...


In [56]:
A8 = A8.append(df).reset_index(drop=True)
A8

,product_name,product_img,product_link
0,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
1,Robustrion Deer Pattern Flip Stand Case Cover ...,https://m.media-amazon.com/images/I/81TcqSe5mO...,https://www.amazon.in//Robustrion-Pattern-Sams...
2,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61OShUky-o...,https://www.amazon.in//Robustrion-Trifold-Sams...
3,Robustrion Trifold Flip Stand Case Cover for S...,https://m.media-amazon.com/images/I/61AZ4KCQjf...,https://www.amazon.in//Robustrion-Trifold-Sams...
4,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/71TriWYOJQ...,https://www.amazon.in//Robustrion-Trifold-Sams...
...,...,...,...
699,(Renewed) Samsung Galaxy Tab S7 27.81 cm (11 i...,https://m.media-amazon.com/images/I/81scJqZDEJ...,https://www.amazon.in//Renewed-Samsung-Galaxy-...
700,Samsung Galaxy Tab S7 FE 31.5 cm (12.4 inch) L...,https://m.media-amazon.com/images/I/9135RJhz8x...,https://www.amazon.in//Samsung-Galaxy-Display-...
701,Samsung Galaxy Tab S7 FE 31.5 cm (12.4 inch) L...,https://m.media-amazon.com/images/I/910xLQicbV...,https://www.amazon.in//Samsung-Galaxy-Display-...
702,"Samsung Galaxy Tab Active PRO 10.1"" | 64GB & W...",https://m.media-amazon.com/images/I/71XFm0upzQ...,https://www.amazon.in//Samsung-Galaxy-Active-W...


In [49]:
product_A8 = pd.DataFrame(reviews[reviews['product_code']=='product_A8']).reset_index(drop=True)
product_A8

,user_id,product_id,Ratings,Time_stamp,product_code
0,AW5ZBAAH66X31,B00004THA8,5.0,966470400,product_A8
1,A2Y3W2AX1WA3RW,B00004THA8,5.0,975456000,product_A8
2,A1HH53GDXDUBR9,B00004THA8,5.0,1345507200,product_A8
3,A6A43XREFCVMQ,B00004THA8,5.0,1256515200,product_A8
4,A1M45SOC657HLD,B00004THA8,5.0,969840000,product_A8
...,...,...,...,...,...
8721,A3JT30AGDKKPOY,B00K8HWFA8,5.0,1404345600,product_A8
8722,A17DA1GRR385S3,B00K9EC7A8,2.0,1404604800,product_A8
8723,A23PUELNTQBA6L,B00KBLUPA8,4.0,1402704000,product_A8
8724,AZU0JS3AM2783,B00KBLUPA8,5.0,1404604800,product_A8


In [50]:
len(product_A8.product_id.unique())

704

In [57]:
product_A8 = pd.DataFrame(reviews[reviews['product_code']=='product_A8']).reset_index(drop=True)
product_A8 =  pd.merge(product_A8, 
                       A8.iloc[:len(product_A8.product_id.unique())]\
                       .join(pd.Series(product_A8['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A8.to_csv('product_A8.csv',index=False)
pd.read_csv('product_A8.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AW5ZBAAH66X31,B00004THA8,5.0,966470400,product_A8,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
1,A2Y3W2AX1WA3RW,B00004THA8,5.0,975456000,product_A8,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
2,A1HH53GDXDUBR9,B00004THA8,5.0,1345507200,product_A8,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
3,A6A43XREFCVMQ,B00004THA8,5.0,1256515200,product_A8,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
4,A1M45SOC657HLD,B00004THA8,5.0,969840000,product_A8,Robustrion Smart Trifold Hard Back Flip Stand ...,https://m.media-amazon.com/images/I/61ofnjvVOC...,https://www.amazon.in//Robustrion-Trifold-Sams...
...,...,...,...,...,...,...,...,...
8721,A3JT30AGDKKPOY,B00K8HWFA8,5.0,1404345600,product_A8,Samsung Galaxy Tab S7 FE 31.5 cm (12.4 inch) L...,https://m.media-amazon.com/images/I/9135RJhz8x...,https://www.amazon.in//Samsung-Galaxy-Display-...
8722,A17DA1GRR385S3,B00K9EC7A8,2.0,1404604800,product_A8,Samsung Galaxy Tab S7 FE 31.5 cm (12.4 inch) L...,https://m.media-amazon.com/images/I/910xLQicbV...,https://www.amazon.in//Samsung-Galaxy-Display-...
8723,A23PUELNTQBA6L,B00KBLUPA8,4.0,1402704000,product_A8,"Samsung Galaxy Tab Active PRO 10.1"" | 64GB & W...",https://m.media-amazon.com/images/I/71XFm0upzQ...,https://www.amazon.in//Samsung-Galaxy-Active-W...
8724,AZU0JS3AM2783,B00KBLUPA8,5.0,1404604800,product_A8,"Samsung Galaxy Tab Active PRO 10.1"" | 64GB & W...",https://m.media-amazon.com/images/I/71XFm0upzQ...,https://www.amazon.in//Samsung-Galaxy-Active-W...


### Product A9 dataframe

In [61]:
df = get_scraped('a9 samsung back cover')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
1,Amazon Brand - Solimo Mobile (Soft & Flexible ...,https://m.media-amazon.com/images/I/61+dsoGyBj...,https://www.amazon.in//Amazon-Brand-Shockproof...
2,A rtistque Liquid TPU Silicone Matte Shockproo...,https://m.media-amazon.com/images/I/61TgMhMNce...,https://www.amazon.in//rtistque-Silicone-Shock...
3,TheGiftKart Back Cover for Samsung Galaxy A72(...,https://m.media-amazon.com/images/I/61-rhsTJNM...,https://www.amazon.in//TheGiftKart-Samsung-Gal...
4,"KC Back Cover for Samsung Galaxy A72, Real Dri...",https://m.media-amazon.com/images/I/71lCGarVxr...,https://www.amazon.in//Cover-Galaxy-Real-Dried...
...,...,...,...
301,"KC Back Cover for Samsung Galaxy A72, Non Movi...",https://m.media-amazon.com/images/I/71MWQmeNZ9...,https://www.amazon.in//Cover-Galaxy-A72-Pearls...
302,Mobile Genics Back Case Cover for Samsung Gala...,https://m.media-amazon.com/images/I/61egBqN3fl...,https://www.amazon.in//Mobile-Genics-Transpare...
303,A rtistque Liquid TPU Silicone Matte Shockproo...,https://m.media-amazon.com/images/I/51FB50Kv6P...,https://www.amazon.in//rtistque-Silicone-Shock...
304,BEJOSH Liquid Silicone Back Cover Compatible f...,https://m.media-amazon.com/images/I/512jAfkb7o...,https://www.amazon.in//BEJOSH-Liquid-Silicone-...


In [62]:
A9 = df.copy()
A9

,product_name,product_img,product_link
0,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
1,Amazon Brand - Solimo Mobile (Soft & Flexible ...,https://m.media-amazon.com/images/I/61+dsoGyBj...,https://www.amazon.in//Amazon-Brand-Shockproof...
2,A rtistque Liquid TPU Silicone Matte Shockproo...,https://m.media-amazon.com/images/I/61TgMhMNce...,https://www.amazon.in//rtistque-Silicone-Shock...
3,TheGiftKart Back Cover for Samsung Galaxy A72(...,https://m.media-amazon.com/images/I/61-rhsTJNM...,https://www.amazon.in//TheGiftKart-Samsung-Gal...
4,"KC Back Cover for Samsung Galaxy A72, Real Dri...",https://m.media-amazon.com/images/I/71lCGarVxr...,https://www.amazon.in//Cover-Galaxy-Real-Dried...
...,...,...,...
301,"KC Back Cover for Samsung Galaxy A72, Non Movi...",https://m.media-amazon.com/images/I/71MWQmeNZ9...,https://www.amazon.in//Cover-Galaxy-A72-Pearls...
302,Mobile Genics Back Case Cover for Samsung Gala...,https://m.media-amazon.com/images/I/61egBqN3fl...,https://www.amazon.in//Mobile-Genics-Transpare...
303,A rtistque Liquid TPU Silicone Matte Shockproo...,https://m.media-amazon.com/images/I/51FB50Kv6P...,https://www.amazon.in//rtistque-Silicone-Shock...
304,BEJOSH Liquid Silicone Back Cover Compatible f...,https://m.media-amazon.com/images/I/512jAfkb7o...,https://www.amazon.in//BEJOSH-Liquid-Silicone-...


In [ ]:
# A9 = A9.append(df).reset_index(drop=True)
# A9

In [59]:
product_A9 = pd.DataFrame(reviews[reviews['product_code']=='product_A9']).reset_index(drop=True)
product_A9

,user_id,product_id,Ratings,Time_stamp,product_code
0,A24UNXVA4YLXBT,B00001P4A9,5.0,988588800,product_A9
1,A3K6TS5BM12P8V,B00001P4A9,5.0,959904000,product_A9
2,A3BMW6R5R9JC0M,B00001P4A9,1.0,1059696000,product_A9
3,A27JOWVO9PQAI0,B00001P4A9,4.0,1008201600,product_A9
4,A17OH1IUJWWPUE,B00001P4A9,1.0,973814400,product_A9
...,...,...,...,...,...
303,A3UL8CTCZNGKPK,B0000CC7A9,2.0,1105056000,product_A9
304,A3DK6YBH2VNRL6,B0000CC7A9,4.0,1220659200,product_A9
305,A323I7XS4AFGS6,B0000CC7A9,4.0,1217376000,product_A9
306,A3877PC93ZNXAH,B0000CC7A9,5.0,1191196800,product_A9


In [60]:
len(product_A9.product_id.unique())

16

In [63]:
product_A9 = pd.DataFrame(reviews[reviews['product_code']=='product_A9']).reset_index(drop=True)
product_A9 =  pd.merge(product_A9, 
                       A9.iloc[:len(product_A9.product_id.unique())]\
                       .join(pd.Series(product_A9['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A9.to_csv('product_A9.csv',index=False)
pd.read_csv('product_A9.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A24UNXVA4YLXBT,B00001P4A9,5.0,988588800,product_A9,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
1,A3K6TS5BM12P8V,B00001P4A9,5.0,959904000,product_A9,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
2,A3BMW6R5R9JC0M,B00001P4A9,1.0,1059696000,product_A9,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
3,A27JOWVO9PQAI0,B00001P4A9,4.0,1008201600,product_A9,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
4,A17OH1IUJWWPUE,B00001P4A9,1.0,973814400,product_A9,Sadgatih Sapno Ki Udaan Back Cover for Samsung...,https://m.media-amazon.com/images/I/41YHJuxWYt...,https://www.amazon.in//SADGATIH-SAPNO-UDAAN-Ga...
...,...,...,...,...,...,...,...,...
303,A3UL8CTCZNGKPK,B0000CC7A9,2.0,1105056000,product_A9,KAPA Soft Fabric Cloth Hybrid for Samsung Gala...,https://m.media-amazon.com/images/I/91JTZP+r5K...,https://www.amazon.in//Fabric-Hybrid-Samsung-S...
304,A3DK6YBH2VNRL6,B0000CC7A9,4.0,1220659200,product_A9,KAPA Soft Fabric Cloth Hybrid for Samsung Gala...,https://m.media-amazon.com/images/I/91JTZP+r5K...,https://www.amazon.in//Fabric-Hybrid-Samsung-S...
305,A323I7XS4AFGS6,B0000CC7A9,4.0,1217376000,product_A9,KAPA Soft Fabric Cloth Hybrid for Samsung Gala...,https://m.media-amazon.com/images/I/91JTZP+r5K...,https://www.amazon.in//Fabric-Hybrid-Samsung-S...
306,A3877PC93ZNXAH,B0000CC7A9,5.0,1191196800,product_A9,KAPA Soft Fabric Cloth Hybrid for Samsung Gala...,https://m.media-amazon.com/images/I/91JTZP+r5K...,https://www.amazon.in//Fabric-Hybrid-Samsung-S...


### Product A0 dataframe

In [69]:
df = get_scraped('antic clock')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,KridayKraft Prince Home Decor & Gifts Metal Kr...,https://m.media-amazon.com/images/I/81OeAM3C1L...,https://www.amazon.in//Prince-Home-Decor-Gifts...
1,Navdeshwar Shivling Pindi Metal & Stone for Ho...,https://m.media-amazon.com/images/I/61exuN+DRi...,https://www.amazon.in//Navdeshwar-Shivling-Off...
2,Two Moustaches Brass Vintage Motor Car Showpie...,https://m.media-amazon.com/images/I/71ocK+6sOn...,https://www.amazon.in//Two-Moustaches-Showpiec...
3,eCraftIndia Brass Horse Tableware Antique Show...,https://m.media-amazon.com/images/I/81nl+CcA2H...,https://www.amazon.in//eCraftIndia-Brass-Table...
4,JH Gallery Recycled Material Rajasthani Musici...,https://m.media-amazon.com/images/I/91QcDGOwZ3...,https://www.amazon.in//Figurines-Rajasthani-Sh...
...,...,...,...
301,Dinine Craft® Resin Handicraft Hand Painted Sh...,https://m.media-amazon.com/images/I/81fxFNDOW+...,https://www.amazon.in//Dininecraft%C2%AE-Handi...
302,UniqueArt White Stone God Ganesha with Rosewoo...,https://m.media-amazon.com/images/I/51RU365g51...,https://www.amazon.in//UniqueArt-Ganesha-Dashb...
303,Proudly Rural Home Decor Items for Living Room...,https://m.media-amazon.com/images/I/41nCDr9Ega...,https://www.amazon.in//Proudly-Showpiece-Sitti...
304,Sehaz Artworks Wooden Wall Hangings for Home D...,https://m.media-amazon.com/images/I/81o-KlpBYA...,https://www.amazon.in//Sehaz-Artworks-Wooden-H...


In [68]:
A0 = df.copy()
A0

,product_name,product_img,product_link
0,Antique Look Jewellery Silver Plated Tradition...,https://m.media-amazon.com/images/I/91zRc8gsan...,https://www.amazon.in//YouBella-Antique-Jewell...
1,Oxidised Jewellery Choker Necklace Set for Wom...,https://m.media-amazon.com/images/I/81XMDZevzQ...,https://www.amazon.in//Total-Fashion-Oxidised-...
2,Necklace Set for Women,https://m.media-amazon.com/images/I/6114nKyrk2...,https://www.amazon.in//Estele-Sublime-Devotion...
3,Oxidized German Silver Necklace with Earring f...,https://m.media-amazon.com/images/I/81C81BW30k...,https://www.amazon.in//Sasitrends-German-Antiq...
4,Golden Brass Alloy German Silver Antique Gold ...,https://m.media-amazon.com/images/I/81nGy-cNYV...,https://www.amazon.in//Sasitrends-German-Silve...
...,...,...,...
301,Glorious Pearl Gold Plated Wedding Jewellery C...,https://m.media-amazon.com/images/I/71LbL302mN...,https://www.amazon.in//Sukkhi-Jewellery-Women-...
302,Combo Of 4 Antique Adjustable Nose Pin For Wom...,https://m.media-amazon.com/images/I/61Xc4YO6dp...,https://www.amazon.in//ZAVERI-PEARLS-Women-Sil...
303,Antique Oxidized Silver Mirror Choker Set With...,https://m.media-amazon.com/images/I/71K6FVR2yT...,https://www.amazon.in//Antique-Oxidized-Silver...
304,Gold plated Handcrafted Antique gold Ruby Chok...,https://m.media-amazon.com/images/I/71IV9lMzI6...,https://www.amazon.in//AccessHer-plated-Handcr...


In [71]:
A0 = A0.append(df).reset_index(drop=True)
A0

,product_name,product_img,product_link
0,Antique Look Jewellery Silver Plated Tradition...,https://m.media-amazon.com/images/I/91zRc8gsan...,https://www.amazon.in//YouBella-Antique-Jewell...
1,Oxidised Jewellery Choker Necklace Set for Wom...,https://m.media-amazon.com/images/I/81XMDZevzQ...,https://www.amazon.in//Total-Fashion-Oxidised-...
2,Necklace Set for Women,https://m.media-amazon.com/images/I/6114nKyrk2...,https://www.amazon.in//Estele-Sublime-Devotion...
3,Oxidized German Silver Necklace with Earring f...,https://m.media-amazon.com/images/I/81C81BW30k...,https://www.amazon.in//Sasitrends-German-Antiq...
4,Golden Brass Alloy German Silver Antique Gold ...,https://m.media-amazon.com/images/I/81nGy-cNYV...,https://www.amazon.in//Sasitrends-German-Silve...
...,...,...,...
913,Dinine Craft® Resin Handicraft Hand Painted Sh...,https://m.media-amazon.com/images/I/81fxFNDOW+...,https://www.amazon.in//Dininecraft%C2%AE-Handi...
914,UniqueArt White Stone God Ganesha with Rosewoo...,https://m.media-amazon.com/images/I/51RU365g51...,https://www.amazon.in//UniqueArt-Ganesha-Dashb...
915,Proudly Rural Home Decor Items for Living Room...,https://m.media-amazon.com/images/I/41nCDr9Ega...,https://www.amazon.in//Proudly-Showpiece-Sitti...
916,Sehaz Artworks Wooden Wall Hangings for Home D...,https://m.media-amazon.com/images/I/81o-KlpBYA...,https://www.amazon.in//Sehaz-Artworks-Wooden-H...


In [73]:
product_A0 = pd.DataFrame(reviews[reviews['product_code']=='product_A0']).reset_index(drop=True)
product_A0

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2RGCW0QEZTS7X,B00003CWA0,5.0,1377475200,product_A0
1,A2XK0FM7JX8DUD,B00003CWA0,4.0,1395014400,product_A0
2,A21CUPIDTRSLA2,B00003CWA0,2.0,1160956800,product_A0
3,A238ORQTQ49E4T,B00004ZCA0,3.0,1126051200,product_A0
4,A3TXASO8B26DGW,B00004ZCA0,5.0,1402963200,product_A0
...,...,...,...,...,...
12983,AOMGV8H3X97FP,B00K3TJ9A0,5.0,1404172800,product_A0
12984,A1YWWMB3IOPFMY,B00K6T4LA0,4.0,1400803200,product_A0
12985,A2MFKKFK157FQL,B00KQ1JJA0,5.0,1404950400,product_A0
12986,A2LCY15KE79U9U,B00KQ1JJA0,2.0,1402444800,product_A0


In [74]:
len(product_A0.product_id.unique())

763

In [76]:
product_A0 = pd.DataFrame(reviews[reviews['product_code']=='product_A0']).reset_index(drop=True)
product_A0 =  pd.merge(product_A0, 
                       A0.iloc[:len(product_A0.product_id.unique())]\
                       .join(pd.Series(product_A0['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_A0.to_csv('product_A0.csv',index=False)
pd.read_csv('product_A0.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2RGCW0QEZTS7X,B00003CWA0,5.0,1377475200,product_A0,Antique Look Jewellery Silver Plated Tradition...,https://m.media-amazon.com/images/I/91zRc8gsan...,https://www.amazon.in//YouBella-Antique-Jewell...
1,A2XK0FM7JX8DUD,B00003CWA0,4.0,1395014400,product_A0,Antique Look Jewellery Silver Plated Tradition...,https://m.media-amazon.com/images/I/91zRc8gsan...,https://www.amazon.in//YouBella-Antique-Jewell...
2,A21CUPIDTRSLA2,B00003CWA0,2.0,1160956800,product_A0,Antique Look Jewellery Silver Plated Tradition...,https://m.media-amazon.com/images/I/91zRc8gsan...,https://www.amazon.in//YouBella-Antique-Jewell...
3,A238ORQTQ49E4T,B00004ZCA0,3.0,1126051200,product_A0,Oxidised Jewellery Choker Necklace Set for Wom...,https://m.media-amazon.com/images/I/81XMDZevzQ...,https://www.amazon.in//Total-Fashion-Oxidised-...
4,A3TXASO8B26DGW,B00004ZCA0,5.0,1402963200,product_A0,Oxidised Jewellery Choker Necklace Set for Wom...,https://m.media-amazon.com/images/I/81XMDZevzQ...,https://www.amazon.in//Total-Fashion-Oxidised-...
...,...,...,...,...,...,...,...,...
12983,AOMGV8H3X97FP,B00K3TJ9A0,5.0,1404172800,product_A0,Yuvaansh Creation Presents (THE ALL NEW) Human...,https://m.media-amazon.com/images/I/710x8zy1QH...,https://www.amazon.in//Yuvaansh-Creation-Showp...
12984,A1YWWMB3IOPFMY,B00K6T4LA0,4.0,1400803200,product_A0,SUPRAS Brass Antique Decorative Titanic Ship S...,https://m.media-amazon.com/images/I/81RTC43rvN...,https://www.amazon.in//SUPRAS-Brass-Antique-De...
12985,A2MFKKFK157FQL,B00KQ1JJA0,5.0,1404950400,product_A0,Money Collection Antique Look Polyresin (Cold ...,https://m.media-amazon.com/images/I/61J6aV0TjA...,https://www.amazon.in//Collection-Antique-Poly...
12986,A2LCY15KE79U9U,B00KQ1JJA0,2.0,1402444800,product_A0,Money Collection Antique Look Polyresin (Cold ...,https://m.media-amazon.com/images/I/61J6aV0TjA...,https://www.amazon.in//Collection-Antique-Poly...


### Product B0 dataframe

In [49]:
df = get_scraped('boat earphones')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Boat BassHeads 100 in-Ear Headphones with Mic ...,https://m.media-amazon.com/images/I/719elVA3Fv...,https://www.amazon.in//boAt-BassHeads-100-Head...
1,boAt Bassheads 225 Wired in Ear Earphone with ...,https://m.media-amazon.com/images/I/61iSV4o+X-...,https://www.amazon.in//boAt-Bassheads-225-Wire...
2,boAt Bassheads 242 in Ear Wired Earphones with...,https://m.media-amazon.com/images/I/61+tzvHZi+...,https://www.amazon.in//Boat-Bassheads-242-Earp...
3,boAt Bassheads 152: Made in India in Ear Wired...,https://m.media-amazon.com/images/I/61jCAtfvK-...,https://www.amazon.in//boAt-Bassheads-152-Earp...
4,Boat Bass Heads 225 in-Ear Headphones with Mic...,https://m.media-amazon.com/images/I/61sgEDKoIZ...,https://www.amazon.in//boAt-Bass-Heads-225-Hea...
...,...,...,...
200,High end Basic Earphone Blue (3),https://m.media-amazon.com/images/I/41Hfd8kjV9...,https://www.amazon.in//High-end-Basic-Earphone...
201,boAt Airdopes 391 Bluetooth Truly Wireless in ...,https://m.media-amazon.com/images/I/51EWsanbpV...,https://www.amazon.in//boAt-Airdopes-391-Techn...
202,(Renewed) boAt Airdopes 441 Truly Wireless Blu...,https://m.media-amazon.com/images/I/513oH0sSNg...,https://www.amazon.in//Renewed-boAt-Airdopes-W...
203,High end Basic Earphone Blue (2),https://m.media-amazon.com/images/I/41Hfd8kjV9...,https://www.amazon.in//High-end-Basic-Earphone...


In [46]:
B0 = df.copy()
B0

,product_name,product_img,product_link
0,"boAt Wave Call Smart Watch, Smart Talk with Ad...",https://m.media-amazon.com/images/I/614AipEWSI...,https://www.amazon.in//boAt-Wave-Call-Dedicate...
1,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/61gscZYmao...,https://www.amazon.in//boAt-Smartwatch-Multipl...
2,"boAt Wave Lite Smartwatch with 1.69"" HD Displa...",https://m.media-amazon.com/images/I/61DZclqQ4R...,https://www.amazon.in//boAt-Wave-Lite-Smartwat...
3,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/61IMRs+o0i...,https://www.amazon.in//boAt-Smartwatch-Multipl...
4,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/617ysOitci...,https://www.amazon.in//boAt-Display-Multiple-M...
...,...,...,...
301,MELFO 22mm Smart Watch Strap Compatible with B...,https://m.media-amazon.com/images/I/61HF-3jjvB...,https://www.amazon.in//MELFO-Smart-Compatible-...
302,boAt Flash Edition Smart Watch with Activity T...,https://m.media-amazon.com/images/I/61Ow5nqSvr...,https://www.amazon.in//boAt-Activity-Tracker-M...
303,HDSanvi Watch Strap Belt with Black Colour Met...,https://m.media-amazon.com/images/I/41DWPWvoS7...,https://www.amazon.in//HDSanvi-Strap-Colour-Bo...
304,boAt Flash Edition Smart Watch with Activity T...,https://m.media-amazon.com/images/I/71uuQ6W8f-...,https://www.amazon.in//boAt-Activity-Multiple-...


In [50]:
B0 = B0.append(df).reset_index(drop=True)
B0

,product_name,product_img,product_link
0,"boAt Wave Call Smart Watch, Smart Talk with Ad...",https://m.media-amazon.com/images/I/614AipEWSI...,https://www.amazon.in//boAt-Wave-Call-Dedicate...
1,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/61gscZYmao...,https://www.amazon.in//boAt-Smartwatch-Multipl...
2,"boAt Wave Lite Smartwatch with 1.69"" HD Displa...",https://m.media-amazon.com/images/I/61DZclqQ4R...,https://www.amazon.in//boAt-Wave-Lite-Smartwat...
3,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/61IMRs+o0i...,https://www.amazon.in//boAt-Smartwatch-Multipl...
4,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/617ysOitci...,https://www.amazon.in//boAt-Display-Multiple-M...
...,...,...,...
801,High end Basic Earphone Blue (3),https://m.media-amazon.com/images/I/41Hfd8kjV9...,https://www.amazon.in//High-end-Basic-Earphone...
802,boAt Airdopes 391 Bluetooth Truly Wireless in ...,https://m.media-amazon.com/images/I/51EWsanbpV...,https://www.amazon.in//boAt-Airdopes-391-Techn...
803,(Renewed) boAt Airdopes 441 Truly Wireless Blu...,https://m.media-amazon.com/images/I/513oH0sSNg...,https://www.amazon.in//Renewed-boAt-Airdopes-W...
804,High end Basic Earphone Blue (2),https://m.media-amazon.com/images/I/41Hfd8kjV9...,https://www.amazon.in//High-end-Basic-Earphone...


In [77]:
product_B0 = pd.DataFrame(reviews[reviews['product_code']=='product_B0']).reset_index(drop=True)
product_B0

,user_id,product_id,Ratings,Time_stamp,product_code
0,A73GTPVXJFWPP,B00000J0B0,5.0,1080000000,product_B0
1,A3S7KLS789FJB5,B00000J0B0,1.0,1048204800,product_B0
2,A38I8GLI1IWIFZ,B00000J0B0,1.0,1018828800,product_B0
3,AJZDH3LR4QS5,B00000J0B0,4.0,1075420800,product_B0
4,A309KI5ABYCD9N,B00000JMB0,1.0,1129161600,product_B0
...,...,...,...,...,...
11132,A2G87IJFG9T3VN,B00JPELAB0,5.0,1403568000,product_B0
11133,ARBKYIVNYWK3C,B00JPELAB0,4.0,1401753600,product_B0
11134,A8BBLN10ZC7EZ,B00JPELAB0,5.0,1405296000,product_B0
11135,AR21MJKTQ7FQF,B00K2WV9B0,5.0,1401926400,product_B0


In [78]:
len(product_B0.product_id.unique())

731

In [51]:
product_B0 = pd.DataFrame(reviews[reviews['product_code']=='product_B0']).reset_index(drop=True)
product_B0 =  pd.merge(product_B0, 
                       B0.iloc[:len(product_B0.product_id.unique())]\
                       .join(pd.Series(product_B0['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B0.to_csv('product_B0.csv',index=False)
pd.read_csv('product_B0.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A73GTPVXJFWPP,B00000J0B0,5.0,1080000000,product_B0,"boAt Wave Call Smart Watch, Smart Talk with Ad...",https://m.media-amazon.com/images/I/614AipEWSI...,https://www.amazon.in//boAt-Wave-Call-Dedicate...
1,A3S7KLS789FJB5,B00000J0B0,1.0,1048204800,product_B0,"boAt Wave Call Smart Watch, Smart Talk with Ad...",https://m.media-amazon.com/images/I/614AipEWSI...,https://www.amazon.in//boAt-Wave-Call-Dedicate...
2,A38I8GLI1IWIFZ,B00000J0B0,1.0,1018828800,product_B0,"boAt Wave Call Smart Watch, Smart Talk with Ad...",https://m.media-amazon.com/images/I/614AipEWSI...,https://www.amazon.in//boAt-Wave-Call-Dedicate...
3,AJZDH3LR4QS5,B00000J0B0,4.0,1075420800,product_B0,"boAt Wave Call Smart Watch, Smart Talk with Ad...",https://m.media-amazon.com/images/I/614AipEWSI...,https://www.amazon.in//boAt-Wave-Call-Dedicate...
4,A309KI5ABYCD9N,B00000JMB0,1.0,1129161600,product_B0,"boAt Xtend Smartwatch with Alexa Built-in, 1.6...",https://m.media-amazon.com/images/I/61gscZYmao...,https://www.amazon.in//boAt-Smartwatch-Multipl...
...,...,...,...,...,...,...,...,...
11132,A2G87IJFG9T3VN,B00JPELAB0,5.0,1403568000,product_B0,(Renewed) boAt Airdopes 281 Pro Truly Wireless...,https://m.media-amazon.com/images/I/51ypiYmDBR...,https://www.amazon.in//boAt-Airdopes-281-Wirel...
11133,ARBKYIVNYWK3C,B00JPELAB0,4.0,1401753600,product_B0,(Renewed) boAt Airdopes 281 Pro Truly Wireless...,https://m.media-amazon.com/images/I/51ypiYmDBR...,https://www.amazon.in//boAt-Airdopes-281-Wirel...
11134,A8BBLN10ZC7EZ,B00JPELAB0,5.0,1405296000,product_B0,(Renewed) boAt Airdopes 281 Pro Truly Wireless...,https://m.media-amazon.com/images/I/51ypiYmDBR...,https://www.amazon.in//boAt-Airdopes-281-Wirel...
11135,AR21MJKTQ7FQF,B00K2WV9B0,5.0,1401926400,product_B0,boAt Bassheads 104 in Ear Wired Earphones(Black),https://m.media-amazon.com/images/I/61CJVcjwV8...,https://www.amazon.in//boAt-Bassheads-104-Wire...


### Product B1 dataframe

In [88]:
df = get_scraped('b1 vitamin')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
1,Biotrex Nutraceuticals Thiamin (Vitamin B1) - ...,https://m.media-amazon.com/images/I/61qK8im98x...,https://www.amazon.in//Biotrex-Nutraceuticals-...
2,Himalayan Organics Plant Based Vitamin B Compl...,https://m.media-amazon.com/images/I/719aZ7Vt-S...,https://www.amazon.in//Himalayan-Organics-B-Co...
3,"Healthvit B-Vit Vitamin B complex with Biotin,...",https://m.media-amazon.com/images/I/61UxOMTZbT...,https://www.amazon.in//Healthvit-B-Vit-Vitamin...
4,"INLIFE Vitamin B12 with B1, B5, B6, Alpha Lipo...",https://m.media-amazon.com/images/I/71XVcpLNPl...,https://www.amazon.in//INLIFE-Vitamin-Lipoic-I...
...,...,...,...
207,Pronutrition Vitamin B Complex + Bone Strength,https://m.media-amazon.com/images/I/81wYTpy9Iu...,https://www.amazon.in//Pronutrition-Vitamin-Co...
208,Natures Velvet Lifecare Iron & Folic Acid with...,https://m.media-amazon.com/images/I/712zt8gd5e...,https://www.amazon.in//Natures-Velvet-Lifecare...
209,MegaBlend Hair Vitamin Gummy | 30 Count | Adva...,https://m.media-amazon.com/images/I/713ljEGqer...,https://www.amazon.in//MegaBlend-Advanced-Bion...
210,"Mason Vitamins B Complex Multivitamin Softgel,...",https://m.media-amazon.com/images/I/81u-+26CyY...,https://www.amazon.in//Mason-Vitamins-Complex-...


In [89]:
B1 = df.copy()
B1

,product_name,product_img,product_link
0,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
1,Biotrex Nutraceuticals Thiamin (Vitamin B1) - ...,https://m.media-amazon.com/images/I/61qK8im98x...,https://www.amazon.in//Biotrex-Nutraceuticals-...
2,Himalayan Organics Plant Based Vitamin B Compl...,https://m.media-amazon.com/images/I/719aZ7Vt-S...,https://www.amazon.in//Himalayan-Organics-B-Co...
3,"Healthvit B-Vit Vitamin B complex with Biotin,...",https://m.media-amazon.com/images/I/61UxOMTZbT...,https://www.amazon.in//Healthvit-B-Vit-Vitamin...
4,"INLIFE Vitamin B12 with B1, B5, B6, Alpha Lipo...",https://m.media-amazon.com/images/I/71XVcpLNPl...,https://www.amazon.in//INLIFE-Vitamin-Lipoic-I...
...,...,...,...
207,Pronutrition Vitamin B Complex + Bone Strength,https://m.media-amazon.com/images/I/81wYTpy9Iu...,https://www.amazon.in//Pronutrition-Vitamin-Co...
208,Natures Velvet Lifecare Iron & Folic Acid with...,https://m.media-amazon.com/images/I/712zt8gd5e...,https://www.amazon.in//Natures-Velvet-Lifecare...
209,MegaBlend Hair Vitamin Gummy | 30 Count | Adva...,https://m.media-amazon.com/images/I/713ljEGqer...,https://www.amazon.in//MegaBlend-Advanced-Bion...
210,"Mason Vitamins B Complex Multivitamin Softgel,...",https://m.media-amazon.com/images/I/81u-+26CyY...,https://www.amazon.in//Mason-Vitamins-Complex-...


In [ ]:
# B1 = B1.append(df).reset_index(drop=True)
# B1

In [86]:
product_B1 = pd.DataFrame(reviews[reviews['product_code']=='product_B1']).reset_index(drop=True)
product_B1

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2I3CLTPRXM57K,B00000JBB1,4.0,948326400,product_B1
1,A3MV25SJZC5493,B00000JBB1,4.0,939427200,product_B1
2,A1K9YK4JQ3Y7EE,B00000JBB1,5.0,946425600,product_B1
3,A1ZLP2VEK68YSC,B00000JBB1,4.0,943401600,product_B1
4,A2R56MW2Y54KUL,B00000JBB1,1.0,959212800,product_B1
...,...,...,...,...,...
113,A2FNVLR7558UOQ,B0000DC6B1,4.0,1110499200,product_B1
114,AB9KKW8K2MD5K,B0000DC6B1,3.0,1113004800,product_B1
115,A38MYVQYOG0KLK,B0000DC6B1,5.0,1095984000,product_B1
116,AV15W3FPLQPN0,B0000DC6B1,5.0,1091836800,product_B1


In [87]:
len(product_B1.product_id.unique())

19

In [90]:
product_B1 = pd.DataFrame(reviews[reviews['product_code']=='product_B1']).reset_index(drop=True)
product_B1 =  pd.merge(product_B1, 
                       B1.iloc[:len(product_B1.product_id.unique())]\
                       .join(pd.Series(product_B1['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B1.to_csv('product_B1.csv',index=False)
pd.read_csv('product_B1.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2I3CLTPRXM57K,B00000JBB1,4.0,948326400,product_B1,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
1,A3MV25SJZC5493,B00000JBB1,4.0,939427200,product_B1,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
2,A1K9YK4JQ3Y7EE,B00000JBB1,5.0,946425600,product_B1,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
3,A1ZLP2VEK68YSC,B00000JBB1,4.0,943401600,product_B1,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
4,A2R56MW2Y54KUL,B00000JBB1,1.0,959212800,product_B1,HealthyHey Nutrition Vitamin B1 Thiamine 1700 ...,https://m.media-amazon.com/images/I/51rXj9sjGh...,https://www.amazon.in//HealthyHey-Nutrition-Vi...
...,...,...,...,...,...,...,...,...
113,A2FNVLR7558UOQ,B0000DC6B1,4.0,1110499200,product_B1,VITARUHE® Vitamin B12 High Dose+ B6/Folic Acid...,https://m.media-amazon.com/images/I/61zk274t71...,https://www.amazon.in//VITARUHE%C2%AE-Vitamin-...
114,AB9KKW8K2MD5K,B0000DC6B1,3.0,1113004800,product_B1,VITARUHE® Vitamin B12 High Dose+ B6/Folic Acid...,https://m.media-amazon.com/images/I/61zk274t71...,https://www.amazon.in//VITARUHE%C2%AE-Vitamin-...
115,A38MYVQYOG0KLK,B0000DC6B1,5.0,1095984000,product_B1,VITARUHE® Vitamin B12 High Dose+ B6/Folic Acid...,https://m.media-amazon.com/images/I/61zk274t71...,https://www.amazon.in//VITARUHE%C2%AE-Vitamin-...
116,AV15W3FPLQPN0,B0000DC6B1,5.0,1091836800,product_B1,VITARUHE® Vitamin B12 High Dose+ B6/Folic Acid...,https://m.media-amazon.com/images/I/61zk274t71...,https://www.amazon.in//VITARUHE%C2%AE-Vitamin-...


### Product B2 dataframe

In [98]:
df = get_scraped('bajra flour')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,24 Mantra Organic Bajra Flour/Pearl Millet Flo...,https://m.media-amazon.com/images/I/71LtAovHIs...,https://www.amazon.in//24-Mantra-Organic-Bajra...
1,Natureland Organics Bajra / Pearl Millet Flour...,https://m.media-amazon.com/images/I/81+4-7l-YM...,https://www.amazon.in//Natureland-Organics-Baj...
2,"Organic Tattva, Organic Bajra (Pearl Millet) G...",https://m.media-amazon.com/images/I/713eU1NPnM...,https://www.amazon.in//Organic-Tattva-Bajra-Fl...
3,MASTER COOK Bajra Flour- (500 Gram x 2),https://m.media-amazon.com/images/I/711244aN4o...,https://www.amazon.in//Master-Cook-Bajra-Flour...
4,Trinetra Bajra Atta (Pearl Millet Flour)| 5Kg ...,https://m.media-amazon.com/images/I/71SlpvguPZ...,https://www.amazon.in//Trinetra-Bajra-Pearl-Mi...
...,...,...,...
301,Bajra Atta | Bajra Flour (Pearl Millet) - 2Kg,https://m.media-amazon.com/images/I/61LkcGAo-S...,https://www.amazon.in//Bajra-Atta-Flour-Pearl-...
302,Goshudh Rajgira Flour 500GM & Bajra/Pearl Mill...,https://m.media-amazon.com/images/I/816WyLoU+Q...,https://www.amazon.in//Goshudh-Rajgira-Flour-5...
303,Trinetra Bajra/Pearl Millet Atta 500GM & Multi...,https://m.media-amazon.com/images/I/71fJ39fZ3Y...,https://www.amazon.in//Trinetra-Bajra-Pearl-Mi...
304,Namajivi Healthy combo Pack of 3 Bajra Atta (P...,https://m.media-amazon.com/images/I/81rQs85GLg...,https://www.amazon.in//Namajivi-Healthy-combo-...


In [95]:
B2 = df.copy()
B2

,product_name,product_img,product_link
0,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
1,IAgriFarm® Bajra Seeds Pack of 1 Kg Multi Cut ...,https://m.media-amazon.com/images/I/71eBX5YqEF...,https://www.amazon.in//IAgriFarm%C2%AE-Bajra-S...
2,OrchidWala Bajra Seeds - Pearl Millet Seeds fo...,https://m.media-amazon.com/images/I/5150-8V2mL...,https://www.amazon.in//OrchidWala-Bajra-Seeds-...
3,A-Quality Store Bajra Seeds Pearl Millet Seeds...,https://m.media-amazon.com/images/I/61umDSA-zf...,https://www.amazon.in//Quality-Store-Bajra-See...
4,Pearl Millet Seed -5kg -( Bajra Seed ),https://m.media-amazon.com/images/I/41Pr-P65BI...,https://www.amazon.in//Pearl-Millet-Seed-5kg-B...
...,...,...,...
301,Organic Nature Raw Barley Seeds (Jau) Barley G...,https://m.media-amazon.com/images/I/71Rt0FTdGu...,https://www.amazon.in//Organic-Nature-Barley-S...
302,"Being Marwari Jau - [Raw Barley Seeds], 250g",https://m.media-amazon.com/images/I/71Y3nxMyrD...,https://www.amazon.in//Being-Marwari-Jau-Barle...
303,Pure & Sure Organic Jowar Flour | Healthy Food...,https://m.media-amazon.com/images/I/61xmERc61T...,https://www.amazon.in//Pure-Sure-Organic-Jowar...
304,"Jivabhumi Organic Pearl Millet, 1 Kg",https://m.media-amazon.com/images/I/61htVDR8Dr...,https://www.amazon.in//Jivabhumi-Organic-Pearl...


In [99]:
B2 = B2.append(df).reset_index(drop=True)
B2

,product_name,product_img,product_link
0,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
1,IAgriFarm® Bajra Seeds Pack of 1 Kg Multi Cut ...,https://m.media-amazon.com/images/I/71eBX5YqEF...,https://www.amazon.in//IAgriFarm%C2%AE-Bajra-S...
2,OrchidWala Bajra Seeds - Pearl Millet Seeds fo...,https://m.media-amazon.com/images/I/5150-8V2mL...,https://www.amazon.in//OrchidWala-Bajra-Seeds-...
3,A-Quality Store Bajra Seeds Pearl Millet Seeds...,https://m.media-amazon.com/images/I/61umDSA-zf...,https://www.amazon.in//Quality-Store-Bajra-See...
4,Pearl Millet Seed -5kg -( Bajra Seed ),https://m.media-amazon.com/images/I/41Pr-P65BI...,https://www.amazon.in//Pearl-Millet-Seed-5kg-B...
...,...,...,...
913,Bajra Atta | Bajra Flour (Pearl Millet) - 2Kg,https://m.media-amazon.com/images/I/61LkcGAo-S...,https://www.amazon.in//Bajra-Atta-Flour-Pearl-...
914,Goshudh Rajgira Flour 500GM & Bajra/Pearl Mill...,https://m.media-amazon.com/images/I/816WyLoU+Q...,https://www.amazon.in//Goshudh-Rajgira-Flour-5...
915,Trinetra Bajra/Pearl Millet Atta 500GM & Multi...,https://m.media-amazon.com/images/I/71fJ39fZ3Y...,https://www.amazon.in//Trinetra-Bajra-Pearl-Mi...
916,Namajivi Healthy combo Pack of 3 Bajra Atta (P...,https://m.media-amazon.com/images/I/81rQs85GLg...,https://www.amazon.in//Namajivi-Healthy-combo-...


In [92]:
product_B2 = pd.DataFrame(reviews[reviews['product_code']=='product_B2']).reset_index(drop=True)
product_B2

,user_id,product_id,Ratings,Time_stamp,product_code
0,A26115T1LX9JCL,B00000K4B2,3.0,1090281600,product_B2
1,AVPY9B8NYUMJG,B00000K4B2,3.0,971654400,product_B2
2,AD50TWQOM8W4G,B00000K4B2,5.0,945561600,product_B2
3,A2HDC502NCIXOW,B00000K4B2,5.0,983059200,product_B2
4,A39MUY7K72LXWH,B00000K4B2,1.0,1129507200,product_B2
...,...,...,...,...,...
14288,A17V9XL4CWTQ6G,B00KEZM6B2,5.0,1405728000,product_B2
14289,A3A4ZAIBQWKOZS,B00KEZM6B2,5.0,1405209600,product_B2
14290,AT53ZTTO707MB,B00KW2W6B2,5.0,1404086400,product_B2
14291,AOVTLYTHVDNUX,B00KW2W6B2,5.0,1404518400,product_B2


In [93]:
len(product_B2.product_id.unique())

757

In [100]:
product_B2 = pd.DataFrame(reviews[reviews['product_code']=='product_B2']).reset_index(drop=True)
product_B2 =  pd.merge(product_B2, 
                       B2.iloc[:len(product_B2.product_id.unique())]\
                       .join(pd.Series(product_B2['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B2.to_csv('product_B2.csv',index=False)
pd.read_csv('product_B2.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A26115T1LX9JCL,B00000K4B2,3.0,1090281600,product_B2,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
1,AVPY9B8NYUMJG,B00000K4B2,3.0,971654400,product_B2,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
2,AD50TWQOM8W4G,B00000K4B2,5.0,945561600,product_B2,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
3,A2HDC502NCIXOW,B00000K4B2,5.0,983059200,product_B2,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
4,A39MUY7K72LXWH,B00000K4B2,1.0,1129507200,product_B2,"3 S Bajra, 1kg",https://m.media-amazon.com/images/I/91aQAxn0dl...,https://www.amazon.in//3-S-Bajra-1kg/dp/B07TH9...
...,...,...,...,...,...,...,...,...
14288,A17V9XL4CWTQ6G,B00KEZM6B2,5.0,1405728000,product_B2,NATURELAND ORGANICS Barley Flour / Jau Flour 5...,https://m.media-amazon.com/images/I/81jUBkIG7i...,https://www.amazon.in//Natureland-Organic-Barl...
14289,A3A4ZAIBQWKOZS,B00KEZM6B2,5.0,1405209600,product_B2,NATURELAND ORGANICS Barley Flour / Jau Flour 5...,https://m.media-amazon.com/images/I/81jUBkIG7i...,https://www.amazon.in//Natureland-Organic-Barl...
14290,AT53ZTTO707MB,B00KW2W6B2,5.0,1404086400,product_B2,"Organic Tattva, Organic Jowar (Sorghum) Flour/...",https://m.media-amazon.com/images/I/71g0mgSC5o...,https://www.amazon.in//Organic-Tattva-Jowar-So...
14291,AOVTLYTHVDNUX,B00KW2W6B2,5.0,1404518400,product_B2,"Organic Tattva, Organic Jowar (Sorghum) Flour/...",https://m.media-amazon.com/images/I/71g0mgSC5o...,https://www.amazon.in//Organic-Tattva-Jowar-So...


### Product B3 dataframe

In [104]:
df = get_scraped('biotin')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
1,Myprotein Alpha Men Multivitamin Tablets | Cal...,https://m.media-amazon.com/images/I/31eCMNIDCX...,https://www.amazon.in//Myprotein-Multivitamin-...
2,Carbamide Forte Biotin 10000mcg for Hair Growt...,https://m.media-amazon.com/images/I/51vQFP666r...,https://www.amazon.in//Carbamide-Forte-Biotin-...
3,Carbamide Forte Biotin for Hair Growth with Am...,https://m.media-amazon.com/images/I/712D06p9cc...,https://www.amazon.in//Carbamide-Forte-Absorpt...
4,OZiva Plant Based Biotin for hair growth 10000...,https://m.media-amazon.com/images/I/61ZQejhk6K...,https://www.amazon.in//OZiva-Plant-Biotin-Sesb...
...,...,...,...
301,"Medy365's Hair, Skin & Nails Gummies | Biotin ...",https://m.media-amazon.com/images/I/51BtXD28RY...,https://www.amazon.in//Medy365s-Gummies-Biotin...
302,MegaBlend Hair Vitamin Tablet | Advanced Growt...,https://m.media-amazon.com/images/I/51YAQIaODY...,https://www.amazon.in//MegaBlend-Advanced-Enri...
303,HSN-Bio Tablet | Plant Based Biotin | Made wit...,https://m.media-amazon.com/images/I/61vsw0IB70...,https://www.amazon.in//HSN-Bio-Sesbania-Strong...
304,BSA PHARMA HASVIT Forte-Biotin Tablets for Hea...,https://m.media-amazon.com/images/I/51KGjpaqKD...,https://www.amazon.in//BSA-PHARMA-Forte-Biotin...


In [105]:
B3 = df.copy()
B3

,product_name,product_img,product_link
0,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
1,Myprotein Alpha Men Multivitamin Tablets | Cal...,https://m.media-amazon.com/images/I/31eCMNIDCX...,https://www.amazon.in//Myprotein-Multivitamin-...
2,Carbamide Forte Biotin 10000mcg for Hair Growt...,https://m.media-amazon.com/images/I/51vQFP666r...,https://www.amazon.in//Carbamide-Forte-Biotin-...
3,Carbamide Forte Biotin for Hair Growth with Am...,https://m.media-amazon.com/images/I/712D06p9cc...,https://www.amazon.in//Carbamide-Forte-Absorpt...
4,OZiva Plant Based Biotin for hair growth 10000...,https://m.media-amazon.com/images/I/61ZQejhk6K...,https://www.amazon.in//OZiva-Plant-Biotin-Sesb...
...,...,...,...
301,"Medy365's Hair, Skin & Nails Gummies | Biotin ...",https://m.media-amazon.com/images/I/51BtXD28RY...,https://www.amazon.in//Medy365s-Gummies-Biotin...
302,MegaBlend Hair Vitamin Tablet | Advanced Growt...,https://m.media-amazon.com/images/I/51YAQIaODY...,https://www.amazon.in//MegaBlend-Advanced-Enri...
303,HSN-Bio Tablet | Plant Based Biotin | Made wit...,https://m.media-amazon.com/images/I/61vsw0IB70...,https://www.amazon.in//HSN-Bio-Sesbania-Strong...
304,BSA PHARMA HASVIT Forte-Biotin Tablets for Hea...,https://m.media-amazon.com/images/I/51KGjpaqKD...,https://www.amazon.in//BSA-PHARMA-Forte-Biotin...


In [72]:
B3 = B3.append(df).reset_index(drop=True)
B3

,product_name,product_img,product_link
0,Godrej aer Matic Kit - Automatic Air Freshener...,https://m.media-amazon.com/images/I/51zk3-rGeq...,https://www.amazon.in//Godrej-Matic-Spray-Cool...
1,EXCLA® Car Air Freshener Double Loop Rotary Ai...,https://m.media-amazon.com/images/I/51WBai3npz...,https://www.amazon.in//EXCLA%C2%AE-Freshener-C...
2,Godrej Aer Matic Kit - Automatic Air Freshener...,https://m.media-amazon.com/images/I/6121tV2hW9...,https://www.amazon.in//Godrej-Matic-Spray-Viol...
3,Airwick Freshmatic Automatic Air Freshener Kit...,https://m.media-amazon.com/images/I/61efWAR9aT...,https://www.amazon.in//Airwick-Freshmatic-Comp...
4,"Godrej aer Matic, Automatic Air Freshener Kit ...",https://m.media-amazon.com/images/I/51AH3J9jWy...,https://www.amazon.in//Godrej-Matic-Petal-Crus...
...,...,...,...
769,HASTHIP® Automatic Drinking Water Pump - Water...,https://m.media-amazon.com/images/I/51wtkY9GdP...,https://www.amazon.in//HASTHIP%C2%AE-Automatic...
770,Stylewell Automatic Wireless Water Can Dispens...,https://m.media-amazon.com/images/I/51ks6BELV1...,https://www.amazon.in//Stylewell-Automatic-Wir...
771,SS Automatic Wireless Water Bottle Switch Rech...,https://m.media-amazon.com/images/I/41zn49Jqui...,https://www.amazon.in//Automatic-Wireless-Bott...
772,Automatic Wireless Water Can Dispenser Pump fo...,https://m.media-amazon.com/images/I/21CTyUqHjL...,https://www.amazon.in//Automatic-Wireless-Disp...


In [102]:
product_B3 = pd.DataFrame(reviews[reviews['product_code']=='product_B3']).reset_index(drop=True)
product_B3

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3S7P3292SRVIV,B00004ZCB3,5.0,1243987200,product_B3
1,A3M49MLZN9JI78,B00004ZCB3,5.0,1270512000,product_B3
2,A2CKQA93KUJUH2,B00004ZCB3,5.0,1321401600,product_B3
3,AE47VC8XN6342,B00004ZCB3,5.0,1266969600,product_B3
4,A2SI7VYCYSYNZX,B00004ZCB3,5.0,1358640000,product_B3
...,...,...,...,...,...
74,AVHVZF18BAN4X,B0000AE6B3,5.0,1389398400,product_B3
75,A3GCU493DGLSMA,B0000AE6B3,1.0,1300665600,product_B3
76,A171LKROHS66Q8,B0000AXPB3,5.0,1404691200,product_B3
77,A3EXJXHYQTW0XR,B0000AXPB3,3.0,1344384000,product_B3


In [103]:
len(product_B3.product_id.unique())

13

In [106]:
product_B3 = pd.DataFrame(reviews[reviews['product_code']=='product_B3']).reset_index(drop=True)
product_B3 =  pd.merge(product_B3, 
                       B3.iloc[:len(product_B3.product_id.unique())]\
                       .join(pd.Series(product_B3['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B3.to_csv('product_B3.csv', index=False)
pd.read_csv('product_B3.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3S7P3292SRVIV,B00004ZCB3,5.0,1243987200,product_B3,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
1,A3M49MLZN9JI78,B00004ZCB3,5.0,1270512000,product_B3,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
2,A2CKQA93KUJUH2,B00004ZCB3,5.0,1321401600,product_B3,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
3,AE47VC8XN6342,B00004ZCB3,5.0,1266969600,product_B3,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
4,A2SI7VYCYSYNZX,B00004ZCB3,5.0,1358640000,product_B3,"HealthKart HK Vitals Biotin 10000mcg, Suppleme...",https://m.media-amazon.com/images/I/51VvVBlS+2...,https://www.amazon.in//HealthKart-Biotin-Maxim...
...,...,...,...,...,...,...,...,...
74,AVHVZF18BAN4X,B0000AE6B3,5.0,1389398400,product_B3,Be Bodywise Biotin Hair Gummies | No Added Sug...,https://m.media-amazon.com/images/I/71ZnRjx0im...,https://www.amazon.in//Be-Bodywise-Supplements...
75,A3GCU493DGLSMA,B0000AE6B3,1.0,1300665600,product_B3,Be Bodywise Biotin Hair Gummies | No Added Sug...,https://m.media-amazon.com/images/I/71ZnRjx0im...,https://www.amazon.in//Be-Bodywise-Supplements...
76,A171LKROHS66Q8,B0000AXPB3,5.0,1404691200,product_B3,"SwisseMe Biotin Gummies With Vitamin B12, C & ...",https://m.media-amazon.com/images/I/515JgU-zjO...,https://www.amazon.in//SwisseMe-Biotin-Gummies...
77,A3EXJXHYQTW0XR,B0000AXPB3,3.0,1344384000,product_B3,"SwisseMe Biotin Gummies With Vitamin B12, C & ...",https://m.media-amazon.com/images/I/515JgU-zjO...,https://www.amazon.in//SwisseMe-Biotin-Gummies...


### Product B4 dataframe

In [121]:
df = get_scraped('biba kurta set for women')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Womens Round Neck Printed Kurta,https://m.media-amazon.com/images/I/81KdLd-DhX...,https://www.amazon.in//BIBA-Womens-Straight-MN...
1,Women Blue Cotton Kurta Salwar With Dupatta Set,https://m.media-amazon.com/images/I/41dAF47rST...,https://www.amazon.in//MATINE-Cotton-Salwar-Du...
2,Women's Cotton Regular Kurta,https://m.media-amazon.com/images/I/81Bd7rIL5P...,https://www.amazon.in//BIBA-Womens-Regular-Kur...
3,Women's Synthetic Salwar Suit Set,https://m.media-amazon.com/images/I/71LFI3kYkU...,https://www.amazon.in//BIBA-Womens-synthetic-S...
4,Women's Rayon Salwar Kurta Dupatta,https://m.media-amazon.com/images/I/81okUp7TGP...,https://www.amazon.in//BIBA-Womens-Salwar-Dupa...
...,...,...,...
301,Women's Cotton Suit Set,https://m.media-amazon.com/images/I/51L9ZXyusK...,https://www.amazon.in//BIBA-OFF-WHITE-SALWAR_S...
302,Women Salwar Suit Set,https://m.media-amazon.com/images/I/71himIpXB8...,https://www.amazon.in//BIBA-Womens-cotton-stra...
303,Women's Synthetic Salwar Suit Set,https://m.media-amazon.com/images/I/81rpHGB6ua...,https://www.amazon.in//BIBA-synthetic-SKDROSET...
304,Women's Cotton Kurta,https://m.media-amazon.com/images/I/71WPlcPuDN...,https://www.amazon.in//BIBA-Womens-cotton-TUDO...


In [118]:
B4 = df.copy()
B4

,product_name,product_img,product_link
0,PARUVI Latest Dress Material Cotton Salwar Sui...,https://m.media-amazon.com/images/I/71Y-PE2xFU...,https://www.amazon.in//Generic-Womens-Cotton-S...
1,Chikankari Beige Colour Designer Sleevless Kur...,https://m.media-amazon.com/images/I/31hcwpT2Zz...,https://www.amazon.in//Chikankari-Designer-Sle...
2,Flying Fashion Women Cotton Kurta Set with Dup...,https://m.media-amazon.com/images/I/614jTA0mSV...,https://www.amazon.in//Cotton-Dupatta-Designer...
3,Women Rayon Straight Golden Foil Printed Kurta...,https://m.media-amazon.com/images/I/61hJAB-8R5...,https://www.amazon.in//Straight-Sharara-Plazzo...
4,Pretty Night Women's Rayon Printed Kurta And P...,https://m.media-amazon.com/images/I/61yrvZRQqC...,https://www.amazon.in//NORAIN-Womens-Printed-P...
...,...,...,...
301,Royal Traderss || Men's Kurta Pyjama With Nehr...,https://m.media-amazon.com/images/I/61r2mbbrQZ...,https://www.amazon.in//Royal-Traderss-Pyjama-J...
302,Women Rayon Straight Golden Foil Printed Kurta...,https://m.media-amazon.com/images/I/31HCCxwVtF...,https://www.amazon.in//Straight-Golden-Printed...
303,Nikunj Majestic Creation 100% Rayon Kurti for ...,https://m.media-amazon.com/images/I/51fpWomnUa...,https://www.amazon.in//Majestic-Creation-Print...
304,Boy's Art Silk Blend Embroidered Multicolour D...,https://m.media-amazon.com/images/I/81OX3Ziz-E...,https://www.amazon.in//Blend-Embroidered-Multi...


In [122]:
B4 = B4.append(df).reset_index(drop=True)
B4

,product_name,product_img,product_link
0,PARUVI Latest Dress Material Cotton Salwar Sui...,https://m.media-amazon.com/images/I/71Y-PE2xFU...,https://www.amazon.in//Generic-Womens-Cotton-S...
1,Chikankari Beige Colour Designer Sleevless Kur...,https://m.media-amazon.com/images/I/31hcwpT2Zz...,https://www.amazon.in//Chikankari-Designer-Sle...
2,Flying Fashion Women Cotton Kurta Set with Dup...,https://m.media-amazon.com/images/I/614jTA0mSV...,https://www.amazon.in//Cotton-Dupatta-Designer...
3,Women Rayon Straight Golden Foil Printed Kurta...,https://m.media-amazon.com/images/I/61hJAB-8R5...,https://www.amazon.in//Straight-Sharara-Plazzo...
4,Pretty Night Women's Rayon Printed Kurta And P...,https://m.media-amazon.com/images/I/61yrvZRQqC...,https://www.amazon.in//NORAIN-Womens-Printed-P...
...,...,...,...
913,Women's Cotton Suit Set,https://m.media-amazon.com/images/I/51L9ZXyusK...,https://www.amazon.in//BIBA-OFF-WHITE-SALWAR_S...
914,Women Salwar Suit Set,https://m.media-amazon.com/images/I/71himIpXB8...,https://www.amazon.in//BIBA-Womens-cotton-stra...
915,Women's Synthetic Salwar Suit Set,https://m.media-amazon.com/images/I/81rpHGB6ua...,https://www.amazon.in//BIBA-synthetic-SKDROSET...
916,Women's Cotton Kurta,https://m.media-amazon.com/images/I/71WPlcPuDN...,https://www.amazon.in//BIBA-Womens-cotton-TUDO...


In [109]:
product_B4 = pd.DataFrame(reviews[reviews['product_code']=='product_B4']).reset_index(drop=True)
product_B4

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2ICGORA65UKD4,B00000K4B4,4.0,1064188800,product_B4
1,A2Q90UILXT8BNI,B00003CWB4,5.0,972432000,product_B4
2,A1B21OXFP340JJ,B00004U8B4,5.0,973814400,product_B4
3,A14JFUJ0YBRL5H,B00004U8B4,5.0,972777600,product_B4
4,ALXVAD9PU80WI,B00004ZCB4,1.0,1362096000,product_B4
...,...,...,...,...,...
11390,A2NAHGUQA2BAMJ,B00JQN2FB4,1.0,1397952000,product_B4
11391,A3W2YY20IEO266,B00K181KB4,5.0,1402963200,product_B4
11392,A1FAH48P58WRCE,B00KFPNCB4,1.0,1404950400,product_B4
11393,A1SQSWRPKJGYLW,B00KFPNCB4,5.0,1404691200,product_B4


In [110]:
len(product_B4.product_id.unique())

733

In [123]:
product_B4 = pd.DataFrame(reviews[reviews['product_code']=='product_B4']).reset_index(drop=True)
product_B4 =  pd.merge(product_B4, 
                       B4.iloc[:len(product_B4.product_id.unique())]\
                       .join(pd.Series(product_B4['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B4.to_csv('product_B4.csv',index=False)
pd.read_csv('product_B4.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2ICGORA65UKD4,B00000K4B4,4.0,1064188800,product_B4,PARUVI Latest Dress Material Cotton Salwar Sui...,https://m.media-amazon.com/images/I/71Y-PE2xFU...,https://www.amazon.in//Generic-Womens-Cotton-S...
1,A2Q90UILXT8BNI,B00003CWB4,5.0,972432000,product_B4,Chikankari Beige Colour Designer Sleevless Kur...,https://m.media-amazon.com/images/I/31hcwpT2Zz...,https://www.amazon.in//Chikankari-Designer-Sle...
2,A1B21OXFP340JJ,B00004U8B4,5.0,973814400,product_B4,Flying Fashion Women Cotton Kurta Set with Dup...,https://m.media-amazon.com/images/I/614jTA0mSV...,https://www.amazon.in//Cotton-Dupatta-Designer...
3,A14JFUJ0YBRL5H,B00004U8B4,5.0,972777600,product_B4,Flying Fashion Women Cotton Kurta Set with Dup...,https://m.media-amazon.com/images/I/614jTA0mSV...,https://www.amazon.in//Cotton-Dupatta-Designer...
4,ALXVAD9PU80WI,B00004ZCB4,1.0,1362096000,product_B4,Women Rayon Straight Golden Foil Printed Kurta...,https://m.media-amazon.com/images/I/61hJAB-8R5...,https://www.amazon.in//Straight-Sharara-Plazzo...
...,...,...,...,...,...,...,...,...
11390,A2NAHGUQA2BAMJ,B00JQN2FB4,1.0,1397952000,product_B4,Women's Regular Kurta,https://m.media-amazon.com/images/I/81OatsYRlH...,https://www.amazon.in//BIBA-Womens-Straight-Ku...
11391,A3W2YY20IEO266,B00K181KB4,5.0,1402963200,product_B4,Women's Cotton Salwar Suit Set,https://m.media-amazon.com/images/I/71EzKlPxlD...,https://www.amazon.in//BIBA-Womens-Achkan-SKD5...
11392,A1FAH48P58WRCE,B00KFPNCB4,1.0,1404950400,product_B4,Women's Cotton Regular Kurta,https://m.media-amazon.com/images/I/71BytYm-Cv...,https://www.amazon.in//BIBA-Womens-Regular-Sum...
11393,A1SQSWRPKJGYLW,B00KFPNCB4,5.0,1404691200,product_B4,Women's Cotton Regular Kurta,https://m.media-amazon.com/images/I/71BytYm-Cv...,https://www.amazon.in//BIBA-Womens-Regular-Sum...


### Product B5 dataframe

In [126]:
df = get_scraped('b4 file folder')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Amazinghind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//Amazinghind-Leatherette...
1,AmazingHind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/81fpuwDMPQ...,https://www.amazon.in//AmazingHind-Leatherette...
2,Ananda Collection Leather Executive File Folde...,https://m.media-amazon.com/images/I/51o16Kcp4X...,https://www.amazon.in//Ananda-Collection-Leath...
3,AmazingHind B4 Size Leatherette Material Profe...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//AmazingHind-Leatherette...
4,Ransh Professional Portfolio File Folder Organ...,https://m.media-amazon.com/images/I/612Xy92NaX...,https://www.amazon.in//Ransh-Professional-Port...
...,...,...,...
301,ZesTale Leatherette Material Professional File...,https://m.media-amazon.com/images/I/91h1e8RPoS...,https://www.amazon.in//ZesTale-Leatherette-Pro...
302,Artistic Leather Professional File Folder for ...,https://m.media-amazon.com/images/I/81KGsiqXfH...,https://www.amazon.in//Artistic-Professional-c...
303,GreatDio® Files and Folders Executive Files an...,https://m.media-amazon.com/images/I/71b3+T5ecy...,https://www.amazon.in//GreatDio%C2%AE-Executiv...
304,GreatDio Transparent Poly-Plastic A4 Documents...,https://m.media-amazon.com/images/I/71RKTNIWuA...,https://www.amazon.in//GreatDio-Transparent-Po...


In [127]:
B5 = df.copy()
B5

,product_name,product_img,product_link
0,Amazinghind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//Amazinghind-Leatherette...
1,AmazingHind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/81fpuwDMPQ...,https://www.amazon.in//AmazingHind-Leatherette...
2,Ananda Collection Leather Executive File Folde...,https://m.media-amazon.com/images/I/51o16Kcp4X...,https://www.amazon.in//Ananda-Collection-Leath...
3,AmazingHind B4 Size Leatherette Material Profe...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//AmazingHind-Leatherette...
4,Ransh Professional Portfolio File Folder Organ...,https://m.media-amazon.com/images/I/612Xy92NaX...,https://www.amazon.in//Ransh-Professional-Port...
...,...,...,...
301,ZesTale Leatherette Material Professional File...,https://m.media-amazon.com/images/I/91h1e8RPoS...,https://www.amazon.in//ZesTale-Leatherette-Pro...
302,Artistic Leather Professional File Folder for ...,https://m.media-amazon.com/images/I/81KGsiqXfH...,https://www.amazon.in//Artistic-Professional-c...
303,GreatDio® Files and Folders Executive Files an...,https://m.media-amazon.com/images/I/71b3+T5ecy...,https://www.amazon.in//GreatDio%C2%AE-Executiv...
304,GreatDio Transparent Poly-Plastic A4 Documents...,https://m.media-amazon.com/images/I/71RKTNIWuA...,https://www.amazon.in//GreatDio-Transparent-Po...


In [ ]:
# B5 = B5.append(df).reset_index(drop=True)
# B5

In [124]:
product_B5 = pd.DataFrame(reviews[reviews['product_code']=='product_B5']).reset_index(drop=True)
product_B5

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2N1K5RSK6N8S7,B00000K4B5,5.0,1321142400,product_B5
1,A3CFOKNN7DHY1V,B00000K4B5,5.0,1335484800,product_B5
2,A3AD28P9COKTFH,B00000K4B5,5.0,1348012800,product_B5
3,AISBJ2OX3YYB8,B00000K4B5,5.0,1322697600,product_B5
4,A2ENEGKNVXBANF,B00001QEB5,1.0,1032912000,product_B5
...,...,...,...,...,...
81,A1113IUPYQ5GQO,B0000CEPB5,1.0,1226361600,product_B5
82,AIBIYUPOC043B,B0000CEPB5,5.0,1106006400,product_B5
83,A27X4R84E1EK89,B0000CEPB5,5.0,1086998400,product_B5
84,A1S7BFT0HDF3HA,B0000CEPB5,5.0,1079568000,product_B5


In [125]:
len(product_B5.product_id.unique())

17

In [128]:
product_B5 = pd.DataFrame(reviews[reviews['product_code']=='product_B5']).reset_index(drop=True)
product_B5 =  pd.merge(product_B5, 
                       B5.iloc[:len(product_B5.product_id.unique())]\
                       .join(pd.Series(product_B5['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B5.to_csv('product_B5.csv',index=False)
pd.read_csv('product_B5.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2N1K5RSK6N8S7,B00000K4B5,5.0,1321142400,product_B5,Amazinghind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//Amazinghind-Leatherette...
1,A3CFOKNN7DHY1V,B00000K4B5,5.0,1335484800,product_B5,Amazinghind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//Amazinghind-Leatherette...
2,A3AD28P9COKTFH,B00000K4B5,5.0,1348012800,product_B5,Amazinghind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//Amazinghind-Leatherette...
3,AISBJ2OX3YYB8,B00000K4B5,5.0,1322697600,product_B5,Amazinghind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/915-s+pH2M...,https://www.amazon.in//Amazinghind-Leatherette...
4,A2ENEGKNVXBANF,B00001QEB5,1.0,1032912000,product_B5,AmazingHind Leatherette Material Professional ...,https://m.media-amazon.com/images/I/81fpuwDMPQ...,https://www.amazon.in//AmazingHind-Leatherette...
...,...,...,...,...,...,...,...,...
81,A1113IUPYQ5GQO,B0000CEPB5,1.0,1226361600,product_B5,NISUN PU Leather 4 Ring Professional File Fold...,https://m.media-amazon.com/images/I/816A-WnOFV...,https://www.amazon.in//NISUN-Professional-File...
82,AIBIYUPOC043B,B0000CEPB5,5.0,1106006400,product_B5,NISUN PU Leather 4 Ring Professional File Fold...,https://m.media-amazon.com/images/I/816A-WnOFV...,https://www.amazon.in//NISUN-Professional-File...
83,A27X4R84E1EK89,B0000CEPB5,5.0,1086998400,product_B5,NISUN PU Leather 4 Ring Professional File Fold...,https://m.media-amazon.com/images/I/816A-WnOFV...,https://www.amazon.in//NISUN-Professional-File...
84,A1S7BFT0HDF3HA,B0000CEPB5,5.0,1079568000,product_B5,NISUN PU Leather 4 Ring Professional File Fold...,https://m.media-amazon.com/images/I/816A-WnOFV...,https://www.amazon.in//NISUN-Professional-File...


### Product B6 dataframe

In [138]:
df = get_scraped('b660 motherboard')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"MSI PRO B660-A DDR4 Motherboard (ATX, 12th Gen...",https://m.media-amazon.com/images/I/91Xs3EA17+...,https://www.amazon.in//MSI-B660-Motherboard-In...
1,"MSI MAG B660M Mortar DDR4 Motherboard, Micro-A...",https://m.media-amazon.com/images/I/81+B1B7Cqh...,https://www.amazon.in//MSI-B660M-Mortar-Mother...
2,GIGABYTE B660M DS3H DDR4 (B660/ Intel/LGA 1700...,https://m.media-amazon.com/images/I/81BDHkrfgG...,https://www.amazon.in//GIGABYTE-B660M-DS3H-DDR...
3,ASUS TUF Gaming B660M-PLUS D4 Intel B660 (LGA ...,https://m.media-amazon.com/images/I/61A0Vv+d7+...,https://www.amazon.in//ASUS-TUF-Gaming-B660M-P...
4,ASUS Prime B660-PLUS D4 Intel B660 (LGA 1700) ...,https://m.media-amazon.com/images/I/81WGeVvsQu...,https://www.amazon.in//ASUS-B660-PLUS-Motherbo...
...,...,...,...
273,ASROCK H570M-ITX/AC Intel SOCKET1200 for Intel...,https://m.media-amazon.com/images/I/81NcFQQfMp...,https://www.amazon.in//ASROCK-H570M-ITX-AC-Int...
274,"Motherboard Case Base, 4mm Motherboard Acrylic...",https://m.media-amazon.com/images/I/41Yq3LWC03...,https://www.amazon.in//Motherboard-Acrylic-Fra...
275,ASUS ROG Strix Z690-A Gaming WiFi D4 LGA1700(I...,https://m.media-amazon.com/images/I/81qmjuAD-U...,https://www.amazon.in//ASUS-ROG-D4-Motherboard...
276,Generic 2-Digit Card PC Laptop Motherboard USB...,https://m.media-amazon.com/images/I/616ZmhhmW6...,https://www.amazon.in//Generic-2-Digit-Motherb...


In [135]:
B6 = df.copy()
B6

,product_name,product_img,product_link
0,Gigabyte B450M DS3H WiFi (AM4/ AMD B450/ SATA ...,https://m.media-amazon.com/images/I/81sKj8esIA...,https://www.amazon.in//Gigabyte-B450M-DS3H-Blu...
1,GIGABYTE B450M DS3H WiFi Motherboard with PCIe...,https://m.media-amazon.com/images/I/81sTiBMvmd...,https://www.amazon.in//GIGABYTE-B450M-DS3H-WiF...
2,ASRock B450M/ac (Wi-Fi) Motherboard (AMD Socke...,https://m.media-amazon.com/images/I/81+RJyfYdL...,https://www.amazon.in//ASRock-B450M-Motherboar...
3,GIGABYTE B450 AORUS Elite Motherboard with Hyb...,https://m.media-amazon.com/images/I/61PxT5Qnm+...,https://www.amazon.in//GIGABYTE-B450-Motherboa...
4,Gigabyte AMD B450 I AORUS PRO WiFi AORUS Mothe...,https://m.media-amazon.com/images/I/61WvRrHHQS...,https://www.amazon.in//Gigabyte-B450-Motherboa...
...,...,...,...
301,ASUS TUF Gaming B550-PRO AMD AM4(Ryzen 5000/30...,https://m.media-amazon.com/images/I/81RCTJ0tSr...,https://www.amazon.in//ASUS-TUF-B550-PRO-Mothe...
302,"Analyzer Diagnostic Card, 4‑in‑1 PCI PCI‑E 2LP...",https://m.media-amazon.com/images/I/71Zg5m0Ez2...,https://www.amazon.in//Analyzer-Diagnostic-Com...
303,MSI B450M-A PRO MAX ProSeries Motherboard (ATX...,https://m.media-amazon.com/images/I/61EdkALnjL...,https://www.amazon.in//MSI-B450M-ProSeries-Mot...
304,ASRock A520M-ITX/AC Supports 3rd Gen AMD AM4 R...,https://m.media-amazon.com/images/I/81SEG2FT7D...,https://www.amazon.in//ASRock-A520M-ITX-AC-Pro...


In [139]:
B6 = B6.append(df).reset_index(drop=True)
B6

,product_name,product_img,product_link
0,Gigabyte B450M DS3H WiFi (AM4/ AMD B450/ SATA ...,https://m.media-amazon.com/images/I/81sKj8esIA...,https://www.amazon.in//Gigabyte-B450M-DS3H-Blu...
1,GIGABYTE B450M DS3H WiFi Motherboard with PCIe...,https://m.media-amazon.com/images/I/81sTiBMvmd...,https://www.amazon.in//GIGABYTE-B450M-DS3H-WiF...
2,ASRock B450M/ac (Wi-Fi) Motherboard (AMD Socke...,https://m.media-amazon.com/images/I/81+RJyfYdL...,https://www.amazon.in//ASRock-B450M-Motherboar...
3,GIGABYTE B450 AORUS Elite Motherboard with Hyb...,https://m.media-amazon.com/images/I/61PxT5Qnm+...,https://www.amazon.in//GIGABYTE-B450-Motherboa...
4,Gigabyte AMD B450 I AORUS PRO WiFi AORUS Mothe...,https://m.media-amazon.com/images/I/61WvRrHHQS...,https://www.amazon.in//Gigabyte-B450-Motherboa...
...,...,...,...
885,ASROCK H570M-ITX/AC Intel SOCKET1200 for Intel...,https://m.media-amazon.com/images/I/81NcFQQfMp...,https://www.amazon.in//ASROCK-H570M-ITX-AC-Int...
886,"Motherboard Case Base, 4mm Motherboard Acrylic...",https://m.media-amazon.com/images/I/41Yq3LWC03...,https://www.amazon.in//Motherboard-Acrylic-Fra...
887,ASUS ROG Strix Z690-A Gaming WiFi D4 LGA1700(I...,https://m.media-amazon.com/images/I/81qmjuAD-U...,https://www.amazon.in//ASUS-ROG-D4-Motherboard...
888,Generic 2-Digit Card PC Laptop Motherboard USB...,https://m.media-amazon.com/images/I/616ZmhhmW6...,https://www.amazon.in//Generic-2-Digit-Motherb...


In [130]:
product_B6 = pd.DataFrame(reviews[reviews['product_code']=='product_B6']).reset_index(drop=True)
product_B6

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1R6HGNE1EDM1K,B00004TXB6,3.0,1047686400,product_B6
1,A3OE9B9RN13L8M,B00004TXB6,3.0,968112000,product_B6
2,AMQPW44OGQEIW,B000050FB6,5.0,999043200,product_B6
3,A3582LA00Q06JR,B000050FB6,4.0,1010793600,product_B6
4,AFMQBFSDQOTPO,B000050FB6,1.0,1071878400,product_B6
...,...,...,...,...,...
11689,A21UYA52ZCYBIW,B00KGMYQB6,5.0,1404950400,product_B6
11690,A11P6E1PLSHLA0,B00KGMYQB6,5.0,1404518400,product_B6
11691,AGWMKIATI25D,B00KGMYQB6,5.0,1405814400,product_B6
11692,A3F8NXX33WWF0D,B00KGMYQB6,5.0,1405728000,product_B6


In [131]:
len(product_B6.product_id.unique())

735

In [140]:
product_B6 = pd.DataFrame(reviews[reviews['product_code']=='product_B6']).reset_index(drop=True)
product_B6 =  pd.merge(product_B6, 
                       B6.iloc[:len(product_B6.product_id.unique())]\
                       .join(pd.Series(product_B6['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
c.to_csv('product_B6.csv',index=False)
pd.read_csv('product_B6.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1R6HGNE1EDM1K,B00004TXB6,3.0,1047686400,product_B6,Gigabyte B450M DS3H WiFi (AM4/ AMD B450/ SATA ...,https://m.media-amazon.com/images/I/81sKj8esIA...,https://www.amazon.in//Gigabyte-B450M-DS3H-Blu...
1,A3OE9B9RN13L8M,B00004TXB6,3.0,968112000,product_B6,Gigabyte B450M DS3H WiFi (AM4/ AMD B450/ SATA ...,https://m.media-amazon.com/images/I/81sKj8esIA...,https://www.amazon.in//Gigabyte-B450M-DS3H-Blu...
2,AMQPW44OGQEIW,B000050FB6,5.0,999043200,product_B6,GIGABYTE B450M DS3H WiFi Motherboard with PCIe...,https://m.media-amazon.com/images/I/81sTiBMvmd...,https://www.amazon.in//GIGABYTE-B450M-DS3H-WiF...
3,A3582LA00Q06JR,B000050FB6,4.0,1010793600,product_B6,GIGABYTE B450M DS3H WiFi Motherboard with PCIe...,https://m.media-amazon.com/images/I/81sTiBMvmd...,https://www.amazon.in//GIGABYTE-B450M-DS3H-WiF...
4,AFMQBFSDQOTPO,B000050FB6,1.0,1071878400,product_B6,GIGABYTE B450M DS3H WiFi Motherboard with PCIe...,https://m.media-amazon.com/images/I/81sTiBMvmd...,https://www.amazon.in//GIGABYTE-B450M-DS3H-WiF...
...,...,...,...,...,...,...,...,...
11689,A21UYA52ZCYBIW,B00KGMYQB6,5.0,1404950400,product_B6,ZEBRONICS H81 LGA 1150 Socket Motherboard,https://m.media-amazon.com/images/I/71trzCzWX3...,https://www.amazon.in//ZEBRONICS-H81-1150-Sock...
11690,A11P6E1PLSHLA0,B00KGMYQB6,5.0,1404518400,product_B6,ZEBRONICS H81 LGA 1150 Socket Motherboard,https://m.media-amazon.com/images/I/71trzCzWX3...,https://www.amazon.in//ZEBRONICS-H81-1150-Sock...
11691,AGWMKIATI25D,B00KGMYQB6,5.0,1405814400,product_B6,ZEBRONICS H81 LGA 1150 Socket Motherboard,https://m.media-amazon.com/images/I/71trzCzWX3...,https://www.amazon.in//ZEBRONICS-H81-1150-Sock...
11692,A3F8NXX33WWF0D,B00KGMYQB6,5.0,1405728000,product_B6,ZEBRONICS H81 LGA 1150 Socket Motherboard,https://m.media-amazon.com/images/I/71trzCzWX3...,https://www.amazon.in//ZEBRONICS-H81-1150-Sock...


### Product B7 dataframe

In [144]:
df = get_scraped('b75 motherboard')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,ELECTROPRIME B75 Desktop Motherboard Lga1155 f...,https://m.media-amazon.com/images/I/51vKHQJzY1...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
1,"Desktop Computer M-ATX Motherboard, DDR3 LGA11...",https://m.media-amazon.com/images/I/71kWSQjFgg...,https://www.amazon.in//Computer-Motherboard-Ma...
2,ELECTROPRIME Desktop Motherboard Main Board B7...,https://m.media-amazon.com/images/I/71QdO8+4Lz...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
3,Power-X B75 HN-61 DDR3,https://m.media-amazon.com/images/I/31HSLBqy0U...,https://www.amazon.in//Generic-Power-X-B75-HN-...
4,ELECTROPRIME B75 Desktop Motherboard Lga1155 f...,https://m.media-amazon.com/images/I/51jChgtSNX...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
...,...,...,...
174,ASRock H81 PRO BTC R2.0 LGA 1150 Intel H81 HDM...,https://m.media-amazon.com/images/I/817759oovf...,https://www.amazon.in//ASRock-H81-PRO-BTC-R2-0...
175,"Motherboard Case Base, 4mm Motherboard Acrylic...",https://m.media-amazon.com/images/I/41Yq3LWC03...,https://www.amazon.in//Motherboard-Acrylic-Fra...
176,Generic 2-Digit Card PC Laptop Motherboard USB...,https://m.media-amazon.com/images/I/616ZmhhmW6...,https://www.amazon.in//Generic-2-Digit-Motherb...
177,Generic 2 Digital PCI Diagnostic Card Motherbo...,https://m.media-amazon.com/images/I/51BEa-ptUx...,https://www.amazon.in//Generic-Digital-Diagnos...


In [145]:
B7 = df.copy()
B7

,product_name,product_img,product_link
0,ELECTROPRIME B75 Desktop Motherboard Lga1155 f...,https://m.media-amazon.com/images/I/51vKHQJzY1...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
1,"Desktop Computer M-ATX Motherboard, DDR3 LGA11...",https://m.media-amazon.com/images/I/71kWSQjFgg...,https://www.amazon.in//Computer-Motherboard-Ma...
2,ELECTROPRIME Desktop Motherboard Main Board B7...,https://m.media-amazon.com/images/I/71QdO8+4Lz...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
3,Power-X B75 HN-61 DDR3,https://m.media-amazon.com/images/I/31HSLBqy0U...,https://www.amazon.in//Generic-Power-X-B75-HN-...
4,ELECTROPRIME B75 Desktop Motherboard Lga1155 f...,https://m.media-amazon.com/images/I/51jChgtSNX...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
...,...,...,...
174,ASRock H81 PRO BTC R2.0 LGA 1150 Intel H81 HDM...,https://m.media-amazon.com/images/I/817759oovf...,https://www.amazon.in//ASRock-H81-PRO-BTC-R2-0...
175,"Motherboard Case Base, 4mm Motherboard Acrylic...",https://m.media-amazon.com/images/I/41Yq3LWC03...,https://www.amazon.in//Motherboard-Acrylic-Fra...
176,Generic 2-Digit Card PC Laptop Motherboard USB...,https://m.media-amazon.com/images/I/616ZmhhmW6...,https://www.amazon.in//Generic-2-Digit-Motherb...
177,Generic 2 Digital PCI Diagnostic Card Motherbo...,https://m.media-amazon.com/images/I/51BEa-ptUx...,https://www.amazon.in//Generic-Digital-Diagnos...


In [ ]:
# B7 = B7.append(df).reset_index(drop=True)
# B7

In [142]:
product_B7 = pd.DataFrame(reviews[reviews['product_code']=='product_B7']).reset_index(drop=True)
product_B7

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3LRS0D3K8CSHZ,B00004SAB7,3.0,972950400,product_B7
1,A2PS2HH1I4C8R0,B00004SAB7,4.0,991612800,product_B7
2,A34WQ8YT21HIWW,B00004SYB7,5.0,980035200,product_B7
3,AEH644IG7XZUI,B00004SYB7,5.0,1315353600,product_B7
4,AXK9O1OC2I2PD,B00004SYB7,5.0,1002240000,product_B7
...,...,...,...,...,...
330,AU3LDD1VCCQTA,B0000CD0B7,4.0,1126396800,product_B7
331,A2K3XAK8BP8LA,B0000CD0B7,5.0,1133049600,product_B7
332,A2BWZGD7S0JZQJ,B0000CNUB7,4.0,1081641600,product_B7
333,A299K5U9E7YVWM,B0000DCTB7,1.0,1397347200,product_B7


In [143]:
len(product_B7.product_id.unique())

17

In [146]:
product_B7 = pd.DataFrame(reviews[reviews['product_code']=='product_B7']).reset_index(drop=True)
product_B7 =  pd.merge(product_B7, 
                       B7.iloc[:len(product_B7.product_id.unique())]\
                       .join(pd.Series(product_B7['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B7.to_csv('product_B7.csv',index=False)
pd.read_csv('product_B7.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3LRS0D3K8CSHZ,B00004SAB7,3.0,972950400,product_B7,ELECTROPRIME B75 Desktop Motherboard Lga1155 f...,https://m.media-amazon.com/images/I/51vKHQJzY1...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
1,A2PS2HH1I4C8R0,B00004SAB7,4.0,991612800,product_B7,ELECTROPRIME B75 Desktop Motherboard Lga1155 f...,https://m.media-amazon.com/images/I/51vKHQJzY1...,https://www.amazon.in//ELECTROPRIME-Desktop-Mo...
2,A34WQ8YT21HIWW,B00004SYB7,5.0,980035200,product_B7,"Desktop Computer M-ATX Motherboard, DDR3 LGA11...",https://m.media-amazon.com/images/I/71kWSQjFgg...,https://www.amazon.in//Computer-Motherboard-Ma...
3,AEH644IG7XZUI,B00004SYB7,5.0,1315353600,product_B7,"Desktop Computer M-ATX Motherboard, DDR3 LGA11...",https://m.media-amazon.com/images/I/71kWSQjFgg...,https://www.amazon.in//Computer-Motherboard-Ma...
4,AXK9O1OC2I2PD,B00004SYB7,5.0,1002240000,product_B7,"Desktop Computer M-ATX Motherboard, DDR3 LGA11...",https://m.media-amazon.com/images/I/71kWSQjFgg...,https://www.amazon.in//Computer-Motherboard-Ma...
...,...,...,...,...,...,...,...,...
330,AU3LDD1VCCQTA,B0000CD0B7,4.0,1126396800,product_B7,Ubersweet® Imported 2X(Lga 1155 B75 P8B75-M P8...,https://m.media-amazon.com/images/I/51C8DHD0Qw...,https://www.amazon.in//Ubersweet%C2%AE-P8B75-M...
331,A2K3XAK8BP8LA,B0000CD0B7,5.0,1133049600,product_B7,Ubersweet® Imported 2X(Lga 1155 B75 P8B75-M P8...,https://m.media-amazon.com/images/I/51C8DHD0Qw...,https://www.amazon.in//Ubersweet%C2%AE-P8B75-M...
332,A2BWZGD7S0JZQJ,B0000CNUB7,4.0,1081641600,product_B7,𝗕𝗧𝗖-𝗕𝟳𝟱 𝟭𝟮-𝗚𝗣𝗨 𝗨𝗦𝗕 LGA1155 Mining Motherboard ...,https://m.media-amazon.com/images/I/71OysgiuC9...,https://www.amazon.in//%F0%9D%97%95%F0%9D%97%A...
333,A299K5U9E7YVWM,B0000DCTB7,1.0,1397347200,product_B7,P8H61-V Motherboard LGA 1155 DDR3 Intel H61(B3...,https://m.media-amazon.com/images/I/41Fn7-352w...,https://www.amazon.in//P8H61-V-Motherboard-Int...


### Product B8 dataframe

In [25]:
df = get_scraped('apple peeler')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Cheshta Multifunction Apple Peeler Manual Kitc...,https://m.media-amazon.com/images/I/51Zq4XqpRp...,https://www.amazon.in//Cheshta-Multifunction-K...
1,JALARAM Enterprise Corn Stripper Peeler Cutter...,https://m.media-amazon.com/images/I/61ocUlbu4-...,https://www.amazon.in//JALARAM-Enterprise-Stri...
2,Amazon Brand - Solimo Plastic Kitchen Peeler -...,https://m.media-amazon.com/images/I/51OqNLIe9n...,https://www.amazon.in//Amazon-Brand-Solimo-Sta...
3,Anup Apple Fruit Peeler Slicer Cutter Dicing K...,https://m.media-amazon.com/images/I/71l6ojPCEB...,https://www.amazon.in//Peeler-Slicer-Cutter-Ki...
4,Akiba store Stainless Steel Apple Fruit Peeler...,https://m.media-amazon.com/images/I/71hELm9pfA...,https://www.amazon.in//Akiba-store-Stainless-M...
...,...,...,...
301,"NILKANTH® - Vegetable Peelers for Kitchen, Y S...",https://m.media-amazon.com/images/I/31Nf423Hm8...,https://www.amazon.in//NIL-KANTH-Vegetable-Erg...
302,"Ikea ABS Plastic, Stainless Steel Potato Peele...",https://m.media-amazon.com/images/I/61V3Dmgaw6...,https://www.amazon.in//Ikea-Potato-Peeler-Gree...
303,MEGADALLAR Vegetable Peelers with Container Pl...,https://m.media-amazon.com/images/I/51PUVpyU2f...,https://www.amazon.in//MEGADALLAR-Vegetable-Co...
304,Parbrahm 3 in 1 Multipurpose Grater Shredder S...,https://m.media-amazon.com/images/I/51u4ud3fit...,https://www.amazon.in//Parbrahm-Multipurpose-S...


In [22]:
B8 = df.copy()
B8

,product_name,product_img,product_link
0,Bragg Raw Unfiltered Apple Cider Vinegar 473 ml,https://m.media-amazon.com/images/I/71bqv3D7zt...,https://www.amazon.in//Bragg-Apple-Cider-Vineg...
1,HealthVit Apple Cider Vinegar with Mother of V...,https://m.media-amazon.com/images/I/71pcIbMCDO...,https://www.amazon.in//HealthVit-Vinegar-Mothe...
2,WOW Life Science Organic Apple Cider Vinegar -...,https://m.media-amazon.com/images/I/81zyCiYjNj...,https://www.amazon.in//WOW-Raw-Apple-Cider-Vin...
3,PLIX - THE PLANT FIX World's First Apple Cider...,https://m.media-amazon.com/images/I/61mfzMeWJF...,https://www.amazon.in//PLIX-Vinegar-Effervesce...
4,Bragg Raw Unfiltered with the Mother Apple Cid...,https://m.media-amazon.com/images/I/71FDvndgYP...,https://www.amazon.in//Bragg-Organic-Unfiltere...
...,...,...,...
301,Organic life 100% Natural Apple Cider Vinegar ...,https://m.media-amazon.com/images/I/612PLofW3h...,https://www.amazon.in//Organic-Apple-Cider-Vin...
302,"Orgoze Apple Cider Vinegar with Ginger, Garlic...",https://m.media-amazon.com/images/I/71BEXoW-Fy...,https://www.amazon.in//Orgoze-Vinegar-Ginger-G...
303,Wellbeing Nutrition ACV Combo - USDA Organic H...,https://m.media-amazon.com/images/I/81CbRP3eT2...,https://www.amazon.in//Wellbeing-Nutrition-ACV...
304,Organic life Apple Cider Vinegar With Cinnamon...,https://m.media-amazon.com/images/I/61hADjTdLv...,https://www.amazon.in//Organic-Apple-Vinegar-C...


In [26]:
B8 = B8.append(df).reset_index(drop=True)
B8

,product_name,product_img,product_link
0,Bragg Raw Unfiltered Apple Cider Vinegar 473 ml,https://m.media-amazon.com/images/I/71bqv3D7zt...,https://www.amazon.in//Bragg-Apple-Cider-Vineg...
1,HealthVit Apple Cider Vinegar with Mother of V...,https://m.media-amazon.com/images/I/71pcIbMCDO...,https://www.amazon.in//HealthVit-Vinegar-Mothe...
2,WOW Life Science Organic Apple Cider Vinegar -...,https://m.media-amazon.com/images/I/81zyCiYjNj...,https://www.amazon.in//WOW-Raw-Apple-Cider-Vin...
3,PLIX - THE PLANT FIX World's First Apple Cider...,https://m.media-amazon.com/images/I/61mfzMeWJF...,https://www.amazon.in//PLIX-Vinegar-Effervesce...
4,Bragg Raw Unfiltered with the Mother Apple Cid...,https://m.media-amazon.com/images/I/71FDvndgYP...,https://www.amazon.in//Bragg-Organic-Unfiltere...
...,...,...,...
913,"NILKANTH® - Vegetable Peelers for Kitchen, Y S...",https://m.media-amazon.com/images/I/31Nf423Hm8...,https://www.amazon.in//NIL-KANTH-Vegetable-Erg...
914,"Ikea ABS Plastic, Stainless Steel Potato Peele...",https://m.media-amazon.com/images/I/61V3Dmgaw6...,https://www.amazon.in//Ikea-Potato-Peeler-Gree...
915,MEGADALLAR Vegetable Peelers with Container Pl...,https://m.media-amazon.com/images/I/51PUVpyU2f...,https://www.amazon.in//MEGADALLAR-Vegetable-Co...
916,Parbrahm 3 in 1 Multipurpose Grater Shredder S...,https://m.media-amazon.com/images/I/51u4ud3fit...,https://www.amazon.in//Parbrahm-Multipurpose-S...


In [6]:
product_B8 = pd.DataFrame(reviews[reviews['product_code']=='product_B8']).reset_index(drop=True)
product_B8

,user_id,product_id,Ratings,Time_stamp,product_code
0,ABRKIJ6D4V42O,B00000JBB8,5.0,951955200,product_B8
1,A3WOMVOTNLL67,B00000JBB8,3.0,1030320000,product_B8
2,A1TB2DSILODHQZ,B00000JBB8,5.0,1018224000,product_B8
3,A2GKJU6L0BKJRR,B00000JBB8,4.0,962323200,product_B8
4,A3HC9BXAM522M7,B00000K4B8,1.0,964569600,product_B8
...,...,...,...,...,...
12188,A13OTAXCMJP1IB,B00JIHAWB8,3.0,1401494400,product_B8
12189,A2UYF1SI0JIIM5,B00JKBMYB8,5.0,1401667200,product_B8
12190,A1IUJ9ASUBWNF5,B00JVJKCB8,5.0,1404777600,product_B8
12191,AZG5M25O8WZ9U,B00K6YD3B8,5.0,1404950400,product_B8


In [8]:
len(product_B8.product_id.unique())

727

In [27]:
product_B8 = pd.DataFrame(reviews[reviews['product_code']=='product_B8']).reset_index(drop=True)
product_B8 =  pd.merge(product_B8, 
                       B8.iloc[:len(product_B8.product_id.unique())]\
                       .join(pd.Series(product_B8['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B8.to_csv('product_B8.csv',index=False)
pd.read_csv('product_B8.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ABRKIJ6D4V42O,B00000JBB8,5.0,951955200,product_B8,Bragg Raw Unfiltered Apple Cider Vinegar 473 ml,https://m.media-amazon.com/images/I/71bqv3D7zt...,https://www.amazon.in//Bragg-Apple-Cider-Vineg...
1,A3WOMVOTNLL67,B00000JBB8,3.0,1030320000,product_B8,Bragg Raw Unfiltered Apple Cider Vinegar 473 ml,https://m.media-amazon.com/images/I/71bqv3D7zt...,https://www.amazon.in//Bragg-Apple-Cider-Vineg...
2,A1TB2DSILODHQZ,B00000JBB8,5.0,1018224000,product_B8,Bragg Raw Unfiltered Apple Cider Vinegar 473 ml,https://m.media-amazon.com/images/I/71bqv3D7zt...,https://www.amazon.in//Bragg-Apple-Cider-Vineg...
3,A2GKJU6L0BKJRR,B00000JBB8,4.0,962323200,product_B8,Bragg Raw Unfiltered Apple Cider Vinegar 473 ml,https://m.media-amazon.com/images/I/71bqv3D7zt...,https://www.amazon.in//Bragg-Apple-Cider-Vineg...
4,A3HC9BXAM522M7,B00000K4B8,1.0,964569600,product_B8,HealthVit Apple Cider Vinegar with Mother of V...,https://m.media-amazon.com/images/I/71pcIbMCDO...,https://www.amazon.in//HealthVit-Vinegar-Mothe...
...,...,...,...,...,...,...,...,...
12188,A13OTAXCMJP1IB,B00JIHAWB8,3.0,1401494400,product_B8,ONSUR Smart Peeler for Vegetables | Plastic Fr...,https://m.media-amazon.com/images/I/51+dUV9Dcx...,https://www.amazon.in//ONSUR-Smart-Peeler-Vege...
12189,A2UYF1SI0JIIM5,B00JKBMYB8,5.0,1401667200,product_B8,RKE Sharp Fast Fruit and Vegetables Peeler Sta...,https://m.media-amazon.com/images/I/71UsNu84dE...,https://www.amazon.in//RKE-Vegetables-Stainles...
12190,A1IUJ9ASUBWNF5,B00JVJKCB8,5.0,1404777600,product_B8,PANTH® Vegetable Peeler Julienne Peeler Slicer...,https://m.media-amazon.com/images/I/51VXVqka3G...,https://www.amazon.in//Vegetable-Julienne-Mult...
12191,AZG5M25O8WZ9U,B00K6YD3B8,5.0,1404950400,product_B8,Kavmart Smart Multifunctional All in One Veget...,https://m.media-amazon.com/images/I/51nEvjPvqM...,https://www.amazon.in//Kavmart-Multifunctional...


### Product B9 dataframe

In [30]:
df = get_scraped('bhimseni camphor original')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Mangalam Bhimseni Camphor Jar (Bhimseni Campho...,https://m.media-amazon.com/images/I/71LMMSYEsK...,https://www.amazon.in//Mangalam-Bhimseni-Camph...
1,Premier Kapoor Bhimseni Pure Camphor (Original...,https://m.media-amazon.com/images/I/31QqS0d3Qu...,https://www.amazon.in//Premier-Bhimseni-Origin...
2,"sri bhimseni -Pure,Organic, Baras,Certified &O...",https://m.media-amazon.com/images/I/61b0V+je0b...,https://www.amazon.in//sri-bhimseni-Certified-...
3,Shri Ganesh Premium Bhimseni Camphor (250g X 1...,https://m.media-amazon.com/images/I/71p0-Wnhl+...,https://www.amazon.in//Shri-Ganesh-Premium-Cam...
4,Shri Ganesh Premium Bhimseni Camphor (500g X 1...,https://m.media-amazon.com/images/I/71j3K6sj0x...,https://www.amazon.in//Shri-Ganesh-Premium-Bhi...
...,...,...,...
301,DS CREATIONS Bhimseni Energised Camphor 1KG | ...,https://m.media-amazon.com/images/I/41uKGZtrFC...,https://www.amazon.in//DS-CREATIONS-Bhimseni-E...
302,Mangalam CamPure Camphor Sticks-Wonderful Frag...,https://m.media-amazon.com/images/I/51BzkQI3BL...,https://www.amazon.in//Mangalam-Sticks-Wonderf...
303,Nature's Shadow - High Grade Edible Camphor / ...,https://m.media-amazon.com/images/I/81jgitgT-9...,https://www.amazon.in//Natures-Shadow-Karpoora...
304,90 Degree Double Colour Camphor Diffuser Elect...,https://m.media-amazon.com/images/I/61to5KnUwx...,https://www.amazon.in//90-Degree-Camphor-Diffu...


In [31]:
B9 = df.copy()
B9

,product_name,product_img,product_link
0,Mangalam Bhimseni Camphor Jar (Bhimseni Campho...,https://m.media-amazon.com/images/I/71LMMSYEsK...,https://www.amazon.in//Mangalam-Bhimseni-Camph...
1,Premier Kapoor Bhimseni Pure Camphor (Original...,https://m.media-amazon.com/images/I/31QqS0d3Qu...,https://www.amazon.in//Premier-Bhimseni-Origin...
2,"sri bhimseni -Pure,Organic, Baras,Certified &O...",https://m.media-amazon.com/images/I/61b0V+je0b...,https://www.amazon.in//sri-bhimseni-Certified-...
3,Shri Ganesh Premium Bhimseni Camphor (250g X 1...,https://m.media-amazon.com/images/I/71p0-Wnhl+...,https://www.amazon.in//Shri-Ganesh-Premium-Cam...
4,Shri Ganesh Premium Bhimseni Camphor (500g X 1...,https://m.media-amazon.com/images/I/71j3K6sj0x...,https://www.amazon.in//Shri-Ganesh-Premium-Bhi...
...,...,...,...
301,DS CREATIONS Bhimseni Energised Camphor 1KG | ...,https://m.media-amazon.com/images/I/41uKGZtrFC...,https://www.amazon.in//DS-CREATIONS-Bhimseni-E...
302,Mangalam CamPure Camphor Sticks-Wonderful Frag...,https://m.media-amazon.com/images/I/51BzkQI3BL...,https://www.amazon.in//Mangalam-Sticks-Wonderf...
303,Nature's Shadow - High Grade Edible Camphor / ...,https://m.media-amazon.com/images/I/81jgitgT-9...,https://www.amazon.in//Natures-Shadow-Karpoora...
304,90 Degree Double Colour Camphor Diffuser Elect...,https://m.media-amazon.com/images/I/61to5KnUwx...,https://www.amazon.in//90-Degree-Camphor-Diffu...


In [ ]:
# B9 = B9.append(df).reset_index(drop=True)
# B9

In [29]:
product_B9 = pd.DataFrame(reviews[reviews['product_code']=='product_B9']).reset_index(drop=True)
product_B9

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2HH1THR6DEIF1,B00003CWB9,5.0,1038268800,product_B9
1,AF463JB2N9AJI,B00003CWB9,4.0,1009843200,product_B9
2,A2T4YH12N1PAS,B00004SGB9,3.0,1156809600,product_B9
3,A2M5GKAGV88LWD,B00004SGB9,4.0,965952000,product_B9
4,A10WHIF3JI242T,B00004SYB9,2.0,1004313600,product_B9
...,...,...,...,...,...
268,A3HLILEELA8BPU,B0000CD0B9,5.0,1188864000,product_B9
269,A1CFD8WCH13PMV,B0000CEPB9,1.0,1140480000,product_B9
270,A3TNDVGV8A6T58,B0000CEPB9,5.0,1325894400,product_B9
271,A10NWPW88LOVJN,B0000CEPB9,2.0,1134604800,product_B9


In [8]:
len(product_B9.product_id.unique())

727

In [32]:
product_B9 = pd.DataFrame(reviews[reviews['product_code']=='product_B9']).reset_index(drop=True)
product_B9 =  pd.merge(product_B9, 
                       B9.iloc[:len(product_B9.product_id.unique())]\
                       .join(pd.Series(product_B9['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_B9.to_csv('product_B9.csv',index=False)
pd.read_csv('product_B9.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2HH1THR6DEIF1,B00003CWB9,5.0,1038268800,product_B9,Mangalam Bhimseni Camphor Jar (Bhimseni Campho...,https://m.media-amazon.com/images/I/71LMMSYEsK...,https://www.amazon.in//Mangalam-Bhimseni-Camph...
1,AF463JB2N9AJI,B00003CWB9,4.0,1009843200,product_B9,Mangalam Bhimseni Camphor Jar (Bhimseni Campho...,https://m.media-amazon.com/images/I/71LMMSYEsK...,https://www.amazon.in//Mangalam-Bhimseni-Camph...
2,A2T4YH12N1PAS,B00004SGB9,3.0,1156809600,product_B9,Premier Kapoor Bhimseni Pure Camphor (Original...,https://m.media-amazon.com/images/I/31QqS0d3Qu...,https://www.amazon.in//Premier-Bhimseni-Origin...
3,A2M5GKAGV88LWD,B00004SGB9,4.0,965952000,product_B9,Premier Kapoor Bhimseni Pure Camphor (Original...,https://m.media-amazon.com/images/I/31QqS0d3Qu...,https://www.amazon.in//Premier-Bhimseni-Origin...
4,A10WHIF3JI242T,B00004SYB9,2.0,1004313600,product_B9,"sri bhimseni -Pure,Organic, Baras,Certified &O...",https://m.media-amazon.com/images/I/61b0V+je0b...,https://www.amazon.in//sri-bhimseni-Certified-...
...,...,...,...,...,...,...,...,...
268,A3HLILEELA8BPU,B0000CD0B9,5.0,1188864000,product_B9,RADHA LAKSHMI® Bhimseni Camphor &kapoor (Bhims...,https://m.media-amazon.com/images/I/61ME-uvZnb...,https://www.amazon.in//RADHA-LAKSHMI%C2%AE-Bhi...
269,A1CFD8WCH13PMV,B0000CEPB9,1.0,1140480000,product_B9,Vasudev Premium Desi Bhimseni Camphor | Kapur ...,https://m.media-amazon.com/images/I/513G-wEaYl...,https://www.amazon.in//Vasudev-Bhimseni-Kappur...
270,A3TNDVGV8A6T58,B0000CEPB9,5.0,1325894400,product_B9,Vasudev Premium Desi Bhimseni Camphor | Kapur ...,https://m.media-amazon.com/images/I/513G-wEaYl...,https://www.amazon.in//Vasudev-Bhimseni-Kappur...
271,A10NWPW88LOVJN,B0000CEPB9,2.0,1134604800,product_B9,Vasudev Premium Desi Bhimseni Camphor | Kapur ...,https://m.media-amazon.com/images/I/513G-wEaYl...,https://www.amazon.in//Vasudev-Bhimseni-Kappur...


### Product BA dataframe

In [68]:
df = get_scraped('bag for men')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Half Moon 35 L Casual Waterproof Laptop Bag/Ba...,https://m.media-amazon.com/images/I/81oUvlsYcn...,https://www.amazon.in//Half-Moon-Waterproof-Ba...
1,American Tourister 32 Ltrs Grey Casual Backpac...,https://m.media-amazon.com/images/I/91PZUhSoOV...,https://www.amazon.in//American-Tourister-AMT-...
2,American Tourister Casual Backpack,https://m.media-amazon.com/images/I/91jPwUCqKG...,https://www.amazon.in//American-Tourister-AMT-...
3,Martucci 15.6 inch 30 L Casual Waterproof Lapt...,https://m.media-amazon.com/images/I/71i3dEmHwq...,https://www.amazon.in//Martucci-Casual-Travell...
4,Bagneeds Men's Black Synthetic Leather Briefca...,https://m.media-amazon.com/images/I/81EASQlMkS...,https://www.amazon.in//Bagneeds-Synthetic-Leat...
...,...,...,...
301,ICEMART 25 L Waterproof Laptop Bag/Backpack fo...,https://m.media-amazon.com/images/I/61OoxvwqtH...,https://www.amazon.in//ICEMART-Waterproof-Back...
302,Bagneeds Casual Crossbody Synthetic Leather Me...,https://m.media-amazon.com/images/I/91iDA-dn6u...,https://www.amazon.in//Bagneeds-Casual-Crossbo...
303,Northzone Backpacks for Boy Girls Stylish Men ...,https://m.media-amazon.com/images/I/71qOXmfYGO...,https://www.amazon.in//Northzone-Backpacks-Sty...
304,Wildcraft 44 Ltrs Casual Backpack (11629-Wolf_...,https://m.media-amazon.com/images/I/41Fz+sra7x...,https://www.amazon.in//Wildcraft-Wolf_Blk-Casu...


In [65]:
BA = df.copy()
BA

,product_name,product_img,product_link
0,Disney Princess Cinderella 25 litres Blue Poly...,https://m.media-amazon.com/images/I/91FOmJkpPW...,https://www.amazon.in//Priority-Disney-Princes...
1,Velvet Kids School Bag Soft Animal Cartoon Bac...,https://m.media-amazon.com/images/I/61tlq95ApE...,https://www.amazon.in//Frantic-Lightweight-Pre...
2,Velvet Kids School/Nursery/Picnic/Carry/Travel...,https://m.media-amazon.com/images/I/81Og6vP8x3...,https://www.amazon.in//Frantic-Velvet-School-N...
3,Disney Mickey Minnie Mouse 13 inch Polyster Sc...,https://m.media-amazon.com/images/I/91Y2yUbPE7...,https://www.amazon.in//Kuber-Industries-Polyst...
4,Kids School Bag Soft Plush Backpacks Cartoon B...,https://m.media-amazon.com/images/I/61143gRC6b...,https://www.amazon.in//DZert-Panda-School-Back...
...,...,...,...
301,"Kid's Mesh Car Shape Bag (Red, 16 Inch)",https://m.media-amazon.com/images/I/71TqZSHOs3...,https://www.amazon.in//RED-CAR-SHAPE-KIDS-INCH...
302,BhavGyan Cute Kids Backpack Toddler Bag Plush ...,https://m.media-amazon.com/images/I/61kEOdzNJy...,https://www.amazon.in//BhavGyan-Backpack-Toddl...
303,Bold 42 L College/ Casual/ Office/ Travel Back...,https://m.media-amazon.com/images/I/71bhrIjxD5...,https://www.amazon.in//POLESTAR-Casual-Backpac...
304,30 L Black Waterproof School Bag for Boys Girl...,https://m.media-amazon.com/images/I/71zPfG4us+...,https://www.amazon.in//Costura-Football-Printe...


In [69]:
BA = BA.append(df).reset_index(drop=True)
BA

,product_name,product_img,product_link
0,Disney Princess Cinderella 25 litres Blue Poly...,https://m.media-amazon.com/images/I/91FOmJkpPW...,https://www.amazon.in//Priority-Disney-Princes...
1,Velvet Kids School Bag Soft Animal Cartoon Bac...,https://m.media-amazon.com/images/I/61tlq95ApE...,https://www.amazon.in//Frantic-Lightweight-Pre...
2,Velvet Kids School/Nursery/Picnic/Carry/Travel...,https://m.media-amazon.com/images/I/81Og6vP8x3...,https://www.amazon.in//Frantic-Velvet-School-N...
3,Disney Mickey Minnie Mouse 13 inch Polyster Sc...,https://m.media-amazon.com/images/I/91Y2yUbPE7...,https://www.amazon.in//Kuber-Industries-Polyst...
4,Kids School Bag Soft Plush Backpacks Cartoon B...,https://m.media-amazon.com/images/I/61143gRC6b...,https://www.amazon.in//DZert-Panda-School-Back...
...,...,...,...
913,ICEMART 25 L Waterproof Laptop Bag/Backpack fo...,https://m.media-amazon.com/images/I/61OoxvwqtH...,https://www.amazon.in//ICEMART-Waterproof-Back...
914,Bagneeds Casual Crossbody Synthetic Leather Me...,https://m.media-amazon.com/images/I/91iDA-dn6u...,https://www.amazon.in//Bagneeds-Casual-Crossbo...
915,Northzone Backpacks for Boy Girls Stylish Men ...,https://m.media-amazon.com/images/I/71qOXmfYGO...,https://www.amazon.in//Northzone-Backpacks-Sty...
916,Wildcraft 44 Ltrs Casual Backpack (11629-Wolf_...,https://m.media-amazon.com/images/I/41Fz+sra7x...,https://www.amazon.in//Wildcraft-Wolf_Blk-Casu...


In [62]:
product_BA = pd.DataFrame(reviews[reviews['product_code']=='product_BA']).reset_index(drop=True)
product_BA

,user_id,product_id,Ratings,Time_stamp,product_code
0,AX9AGN3M9ZW01,B00000J4BA,3.0,967420800,product_BA
1,AEE2GJR0VF6R7,B00000J4BA,4.0,949622400,product_BA
2,A2DR0BDY9LHXG8,B0000225BA,5.0,1329177600,product_BA
3,A2UPAQB36G6TNB,B0000225BA,4.0,1387238400,product_BA
4,AXQ501LC74PKO,B0000225BA,5.0,1364428800,product_BA
...,...,...,...,...,...
11468,A3A4ZAIBQWKOZS,B00JXFHABA,5.0,1405555200,product_BA
11469,A13TQOLU1JHQ16,B00KFCR5BA,4.0,1404777600,product_BA
11470,A1PKOGRW0QUK7S,B00KXGJLBA,4.0,1404259200,product_BA
11471,A1W6QKYOT5GDV4,B00KXGJLBA,5.0,1405555200,product_BA


In [63]:
len(product_BA.product_id.unique())

674

In [70]:
product_BA = pd.DataFrame(reviews[reviews['product_code']=='product_BA']).reset_index(drop=True)
product_BA =  pd.merge(product_BA, 
                       BA.iloc[:len(product_BA.product_id.unique())]\
                       .join(pd.Series(product_BA['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BA.to_csv('product_BA.csv',index=False)
pd.read_csv('product_BA.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AX9AGN3M9ZW01,B00000J4BA,3.0,967420800,product_BA,Disney Princess Cinderella 25 litres Blue Poly...,https://m.media-amazon.com/images/I/91FOmJkpPW...,https://www.amazon.in//Priority-Disney-Princes...
1,AEE2GJR0VF6R7,B00000J4BA,4.0,949622400,product_BA,Disney Princess Cinderella 25 litres Blue Poly...,https://m.media-amazon.com/images/I/91FOmJkpPW...,https://www.amazon.in//Priority-Disney-Princes...
2,A2DR0BDY9LHXG8,B0000225BA,5.0,1329177600,product_BA,Velvet Kids School Bag Soft Animal Cartoon Bac...,https://m.media-amazon.com/images/I/61tlq95ApE...,https://www.amazon.in//Frantic-Lightweight-Pre...
3,A2UPAQB36G6TNB,B0000225BA,4.0,1387238400,product_BA,Velvet Kids School Bag Soft Animal Cartoon Bac...,https://m.media-amazon.com/images/I/61tlq95ApE...,https://www.amazon.in//Frantic-Lightweight-Pre...
4,AXQ501LC74PKO,B0000225BA,5.0,1364428800,product_BA,Velvet Kids School Bag Soft Animal Cartoon Bac...,https://m.media-amazon.com/images/I/61tlq95ApE...,https://www.amazon.in//Frantic-Lightweight-Pre...
...,...,...,...,...,...,...,...,...
11468,A3A4ZAIBQWKOZS,B00JXFHABA,5.0,1405555200,product_BA,Dein Kleider Nylon Sling Cross Body Travel Off...,https://m.media-amazon.com/images/I/81hvbJH6Sl...,https://www.amazon.in//Kleider-Travel-Business...
11469,A13TQOLU1JHQ16,B00KFCR5BA,4.0,1404777600,product_BA,Bennett Casual Laptop Bag school bags for boys...,https://m.media-amazon.com/images/I/91bzxvU2Zu...,https://www.amazon.in//Bennett-15-6-inch-Repel...
11470,A1PKOGRW0QUK7S,B00KXGJLBA,4.0,1404259200,product_BA,GEAR 25L SUAVE BUSINESS ANTI-THEFT OFFICE LAPT...,https://m.media-amazon.com/images/I/81zmPGt3DJ...,https://www.amazon.in//BUSINESS-ANTI-THEFT-OFF...
11471,A1W6QKYOT5GDV4,B00KXGJLBA,5.0,1405555200,product_BA,GEAR 25L SUAVE BUSINESS ANTI-THEFT OFFICE LAPT...,https://m.media-amazon.com/images/I/81zmPGt3DJ...,https://www.amazon.in//BUSINESS-ANTI-THEFT-OFF...


### Product BB dataframe

In [74]:
df = get_scraped('bean bag')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,GIGLICK 4XL Bean Bag with Footrest Ready to Us...,https://m.media-amazon.com/images/I/61ROdHOc6+...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
1,Couchette® Filled Bean Bag for Kids Disney Cha...,https://m.media-amazon.com/images/I/41P0Zp0utR...,https://www.amazon.in//Couchette%C2%AE-Filled-...
2,Kushuvi Faux Leather Bean Bag Filled with Bean...,https://m.media-amazon.com/images/I/61t0Db7Olw...,https://www.amazon.in//Kushuvi-Filled-Beans-Re...
3,Amazon Brand - Solimo XXXL Bean Bag Filled Wit...,https://m.media-amazon.com/images/I/61em4XfIdE...,https://www.amazon.in//Amazon-Brand-Solimo-Fil...
4,GIGLICK XXXL Bean Bag & Footrest Filled with B...,https://m.media-amazon.com/images/I/71rDRuE1q9...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
...,...,...,...
301,Urban Style Decore - Filled with Beans Footsto...,https://m.media-amazon.com/images/I/810uujO0BS...,https://www.amazon.in//Urban-Style-Decore-Fill...
302,Stylish Beans Refill Pack 2 KG Fillers for Bea...,https://m.media-amazon.com/images/I/61cjoAE7GU...,https://www.amazon.in//Stylish-Beans-Refill-Pa...
303,Pi Bean Bags Faux Leather Printed Bean Bag wit...,https://m.media-amazon.com/images/I/711XRXJYRp...,https://www.amazon.in//Pi-Bean-Bags-Leather-Pr...
304,Kushuvi Gaming Chair Filled with Beans Sofa Be...,https://m.media-amazon.com/images/I/51WhtQDLzY...,https://www.amazon.in//Kushuvi-Gaming-Filled-R...


In [75]:
BB = df.copy()
BB

,product_name,product_img,product_link
0,GIGLICK 4XL Bean Bag with Footrest Ready to Us...,https://m.media-amazon.com/images/I/61ROdHOc6+...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
1,Couchette® Filled Bean Bag for Kids Disney Cha...,https://m.media-amazon.com/images/I/41P0Zp0utR...,https://www.amazon.in//Couchette%C2%AE-Filled-...
2,Kushuvi Faux Leather Bean Bag Filled with Bean...,https://m.media-amazon.com/images/I/61t0Db7Olw...,https://www.amazon.in//Kushuvi-Filled-Beans-Re...
3,Amazon Brand - Solimo XXXL Bean Bag Filled Wit...,https://m.media-amazon.com/images/I/61em4XfIdE...,https://www.amazon.in//Amazon-Brand-Solimo-Fil...
4,GIGLICK XXXL Bean Bag & Footrest Filled with B...,https://m.media-amazon.com/images/I/71rDRuE1q9...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
...,...,...,...
301,Urban Style Decore - Filled with Beans Footsto...,https://m.media-amazon.com/images/I/810uujO0BS...,https://www.amazon.in//Urban-Style-Decore-Fill...
302,Stylish Beans Refill Pack 2 KG Fillers for Bea...,https://m.media-amazon.com/images/I/61cjoAE7GU...,https://www.amazon.in//Stylish-Beans-Refill-Pa...
303,Pi Bean Bags Faux Leather Printed Bean Bag wit...,https://m.media-amazon.com/images/I/711XRXJYRp...,https://www.amazon.in//Pi-Bean-Bags-Leather-Pr...
304,Kushuvi Gaming Chair Filled with Beans Sofa Be...,https://m.media-amazon.com/images/I/51WhtQDLzY...,https://www.amazon.in//Kushuvi-Gaming-Filled-R...


In [ ]:
# BB = BO.append(df).reset_index(drop=True)
# BB

In [72]:
product_BB = pd.DataFrame(reviews[reviews['product_code']=='product_BB']).reset_index(drop=True)
product_BB

,user_id,product_id,Ratings,Time_stamp,product_code
0,AEA0ME56LQ60Y,B00000J4BB,1.0,970531200,product_BB
1,A1SW1CR2I316PR,B00000J4BB,5.0,967161600,product_BB
2,A3S5827VSZE66O,B00000J4BB,3.0,985651200,product_BB
3,A21BR2RBZ35FHH,B00000K4BB,5.0,1001548800,product_BB
4,AD50TWQOM8W4G,B00000K4BB,5.0,945907200,product_BB
...,...,...,...,...,...
1191,A23HXH27QJXLJR,B0000C3WBB,5.0,1366156800,product_BB
1192,A2HT3SCIXV6GG,B0000C3WBB,5.0,1320796800,product_BB
1193,AJYONRYM1CPXN,B0000C3WBB,5.0,1309478400,product_BB
1194,A1ZRGIL94OHKOU,B0000C3WBB,5.0,1380499200,product_BB


In [73]:
len(product_BB.product_id.unique())

18

In [76]:
product_BB = pd.DataFrame(reviews[reviews['product_code']=='product_BB']).reset_index(drop=True)
product_BB =  pd.merge(product_BB, 
                       BB.iloc[:len(product_BB.product_id.unique())]\
                       .join(pd.Series(product_BB['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BB.to_csv('product_BB.csv',index=False)
pd.read_csv('product_BB.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AEA0ME56LQ60Y,B00000J4BB,1.0,970531200,product_BB,GIGLICK 4XL Bean Bag with Footrest Ready to Us...,https://m.media-amazon.com/images/I/61ROdHOc6+...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
1,A1SW1CR2I316PR,B00000J4BB,5.0,967161600,product_BB,GIGLICK 4XL Bean Bag with Footrest Ready to Us...,https://m.media-amazon.com/images/I/61ROdHOc6+...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
2,A3S5827VSZE66O,B00000J4BB,3.0,985651200,product_BB,GIGLICK 4XL Bean Bag with Footrest Ready to Us...,https://m.media-amazon.com/images/I/61ROdHOc6+...,https://www.amazon.in//GIGLICK-Bean-Bag-Footre...
3,A21BR2RBZ35FHH,B00000K4BB,5.0,1001548800,product_BB,Couchette® Filled Bean Bag for Kids Disney Cha...,https://m.media-amazon.com/images/I/41P0Zp0utR...,https://www.amazon.in//Couchette%C2%AE-Filled-...
4,AD50TWQOM8W4G,B00000K4BB,5.0,945907200,product_BB,Couchette® Filled Bean Bag for Kids Disney Cha...,https://m.media-amazon.com/images/I/41P0Zp0utR...,https://www.amazon.in//Couchette%C2%AE-Filled-...
...,...,...,...,...,...,...,...,...
1191,A23HXH27QJXLJR,B0000C3WBB,5.0,1366156800,product_BB,Amazon Brand - Solimo Luxe Fur XXXL Bean Bag C...,https://m.media-amazon.com/images/I/71BIT1eSEd...,https://www.amazon.in//Amazon-Brand-Solimo-Cov...
1192,A2HT3SCIXV6GG,B0000C3WBB,5.0,1320796800,product_BB,Amazon Brand - Solimo Luxe Fur XXXL Bean Bag C...,https://m.media-amazon.com/images/I/71BIT1eSEd...,https://www.amazon.in//Amazon-Brand-Solimo-Cov...
1193,AJYONRYM1CPXN,B0000C3WBB,5.0,1309478400,product_BB,Amazon Brand - Solimo Luxe Fur XXXL Bean Bag C...,https://m.media-amazon.com/images/I/71BIT1eSEd...,https://www.amazon.in//Amazon-Brand-Solimo-Cov...
1194,A1ZRGIL94OHKOU,B0000C3WBB,5.0,1380499200,product_BB,Amazon Brand - Solimo Luxe Fur XXXL Bean Bag C...,https://m.media-amazon.com/images/I/71BIT1eSEd...,https://www.amazon.in//Amazon-Brand-Solimo-Cov...


### Product BC dataframe

In [84]:
df = get_scraped('bcaa+supplement&i=hpc')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
1,"MuscleBlaze BCAA Pro, Powerful Intra Workout ,...",https://m.media-amazon.com/images/I/71kGR-9SBH...,https://www.amazon.in//MuscleBlaze-Amino-Powde...
2,"FAST&UP BCAA Basic (45 Servings, Orange Flavou...",https://m.media-amazon.com/images/I/61j+aSSusY...,https://www.amazon.in//FAST-UP-Essentials-Serv...
3,AS-IT-IS Nutrition BCAA 100% Pure Powder Pre/P...,https://m.media-amazon.com/images/I/61myEc6nd7...,https://www.amazon.in//AS-Nutrition-Powder-Wor...
4,Naturyz Instantized BCAA Supplement with 7000 ...,https://m.media-amazon.com/images/I/61ssPSCBpo...,https://www.amazon.in//Naturyz-Instantized-Glu...
...,...,...,...
931,Vogue Wellness Advance Nutrition Real Pre Work...,https://m.media-amazon.com/images/I/81T2NCyMYW...,https://www.amazon.in//Vogue-Nutrition-Supplem...
932,Doctor's Choice Shredz Pro Formula with 500mg ...,https://m.media-amazon.com/images/I/81PRNum40U...,https://www.amazon.in//Doctors-Choice-Acetyl-L...
933,"TrueBasics Multivit Daily, Multivitamin For Me...",https://m.media-amazon.com/images/I/61YTjBargS...,https://www.amazon.in//TrueBasics-Multivitamin...
934,MyFitFuel Citrulline Malate (1500 mg) 120 Caps...,https://m.media-amazon.com/images/I/71PiqRf1r5...,https://www.amazon.in//MyFitFuel-Citrulline-Ma...


In [85]:
BC = df.copy()
BC

,product_name,product_img,product_link
0,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
1,"MuscleBlaze BCAA Pro, Powerful Intra Workout ,...",https://m.media-amazon.com/images/I/71kGR-9SBH...,https://www.amazon.in//MuscleBlaze-Amino-Powde...
2,"FAST&UP BCAA Basic (45 Servings, Orange Flavou...",https://m.media-amazon.com/images/I/61j+aSSusY...,https://www.amazon.in//FAST-UP-Essentials-Serv...
3,AS-IT-IS Nutrition BCAA 100% Pure Powder Pre/P...,https://m.media-amazon.com/images/I/61myEc6nd7...,https://www.amazon.in//AS-Nutrition-Powder-Wor...
4,Naturyz Instantized BCAA Supplement with 7000 ...,https://m.media-amazon.com/images/I/61ssPSCBpo...,https://www.amazon.in//Naturyz-Instantized-Glu...
...,...,...,...
931,Vogue Wellness Advance Nutrition Real Pre Work...,https://m.media-amazon.com/images/I/81T2NCyMYW...,https://www.amazon.in//Vogue-Nutrition-Supplem...
932,Doctor's Choice Shredz Pro Formula with 500mg ...,https://m.media-amazon.com/images/I/81PRNum40U...,https://www.amazon.in//Doctors-Choice-Acetyl-L...
933,"TrueBasics Multivit Daily, Multivitamin For Me...",https://m.media-amazon.com/images/I/61YTjBargS...,https://www.amazon.in//TrueBasics-Multivitamin...
934,MyFitFuel Citrulline Malate (1500 mg) 120 Caps...,https://m.media-amazon.com/images/I/71PiqRf1r5...,https://www.amazon.in//MyFitFuel-Citrulline-Ma...


In [ ]:
# BC = BC.append(df).reset_index(drop=True)
# BC

In [78]:
product_BC = pd.DataFrame(reviews[reviews['product_code']=='product_BC']).reset_index(drop=True)
product_BC

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3L76RW9MBYJ84,B00000J4BC,1.0,982713600,product_BC
1,AHAJB6Y4WATM4,B00000J4BC,4.0,1082073600,product_BC
2,A2W9B62C2QAFU1,B00000J4BC,1.0,961200000,product_BC
3,A36LRSU3TECMVS,B00000J4BC,3.0,985392000,product_BC
4,A1H21C73JD0VYB,B00000J4BC,4.0,1010016000,product_BC
...,...,...,...,...,...
10846,A3AFCUHDZ0VX2Z,B00KCNARBC,4.0,1404000000,product_BC
10847,A7BVJOWKLASFB,B00KCNARBC,1.0,1404864000,product_BC
10848,A1PMW3A3WBYS23,B00KCNARBC,5.0,1403740800,product_BC
10849,AR5SRJ2Y380A0,B00KGLCXBC,5.0,1404691200,product_BC


In [79]:
len(product_BC.product_id.unique())

753

In [86]:
product_BC = pd.DataFrame(reviews[reviews['product_code']=='product_BC']).reset_index(drop=True)
product_BC =  pd.merge(product_BC, 
                       BC.iloc[:len(product_BC.product_id.unique())]\
                       .join(pd.Series(product_BC['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BC.to_csv('product_BC.csv',index=False)
pd.read_csv('product_BC.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3L76RW9MBYJ84,B00000J4BC,1.0,982713600,product_BC,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
1,AHAJB6Y4WATM4,B00000J4BC,4.0,1082073600,product_BC,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
2,A2W9B62C2QAFU1,B00000J4BC,1.0,961200000,product_BC,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
3,A36LRSU3TECMVS,B00000J4BC,3.0,985392000,product_BC,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
4,A1H21C73JD0VYB,B00000J4BC,4.0,1010016000,product_BC,Bigmuscles Nutrition Real BCAA [Pack of 50 Ser...,https://m.media-amazon.com/images/I/618v289E7A...,https://www.amazon.in//Bigmuscles-Nutrition-Se...
...,...,...,...,...,...,...,...,...
10846,A3AFCUHDZ0VX2Z,B00KCNARBC,4.0,1404000000,product_BC,HulkHolic Nutrition BCAA (Branched Chain Amino...,https://m.media-amazon.com/images/I/71nYV0IAEu...,https://www.amazon.in//HulkHolic-Nutrition-Bra...
10847,A7BVJOWKLASFB,B00KCNARBC,1.0,1404864000,product_BC,HulkHolic Nutrition BCAA (Branched Chain Amino...,https://m.media-amazon.com/images/I/71nYV0IAEu...,https://www.amazon.in//HulkHolic-Nutrition-Bra...
10848,A1PMW3A3WBYS23,B00KCNARBC,5.0,1403740800,product_BC,HulkHolic Nutrition BCAA (Branched Chain Amino...,https://m.media-amazon.com/images/I/71nYV0IAEu...,https://www.amazon.in//HulkHolic-Nutrition-Bra...
10849,AR5SRJ2Y380A0,B00KGLCXBC,5.0,1404691200,product_BC,PowerLift Muscle Mass Gainers Protein Powder (...,https://m.media-amazon.com/images/I/611uC-p+yn...,https://www.amazon.in//POWERLIFT-Protein-Digez...


### Product BD dataframe

In [90]:
df = get_scraped('bbetter')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
1,BBETTER Testosterone Booster Supplement for Me...,https://m.media-amazon.com/images/I/61j0j84Tmt...,https://www.amazon.in//BBETTER-Testosterone-Ar...
2,BBETTER Multivitamin Capsules for Men & Women ...,https://m.media-amazon.com/images/I/617pt0P7Ef...,https://www.amazon.in//BBETTER-Multivitamins-T...
3,BBETTER VEDA Purified Shilajit 500mg - Approve...,https://m.media-amazon.com/images/I/61EzBa2FC3...,https://www.amazon.in//BBETTER-Purified-Shilaj...
4,BBETTER Omega 3 Fish Oil 1000mg High Strength ...,https://m.media-amazon.com/images/I/71RcH-GkD6...,https://www.amazon.in//BBETTER-Omega3-Fish-Oil...
...,...,...,...
74,Plantigo Plant Based Vegan Collagen Powder 150...,https://m.media-amazon.com/images/I/71B7Lk7DCg...,https://www.amazon.in//Plantigo-Supplement-Irr...
75,Plantigo Plant Based Vegan Collagen Powder 150...,https://m.media-amazon.com/images/I/61EgFmIuQf...,https://www.amazon.in//Plantigo-Collagen-Suppl...
76,"Red Ribbon Polyresin Floral Window Curtain, 5 ...",https://m.media-amazon.com/images/I/51jcaevMAG...,https://www.amazon.in//Red-Ribbon-Floral-Polye...
77,The Sleep Company SmartGRID Luxe Mattress 10 i...,https://m.media-amazon.com/images/I/61TedluJTh...,https://www.amazon.in//Sleep-Company-SmartGRID...


In [91]:
BD = df.copy()
BD

,product_name,product_img,product_link
0,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
1,BBETTER Testosterone Booster Supplement for Me...,https://m.media-amazon.com/images/I/61j0j84Tmt...,https://www.amazon.in//BBETTER-Testosterone-Ar...
2,BBETTER Multivitamin Capsules for Men & Women ...,https://m.media-amazon.com/images/I/617pt0P7Ef...,https://www.amazon.in//BBETTER-Multivitamins-T...
3,BBETTER VEDA Purified Shilajit 500mg - Approve...,https://m.media-amazon.com/images/I/61EzBa2FC3...,https://www.amazon.in//BBETTER-Purified-Shilaj...
4,BBETTER Omega 3 Fish Oil 1000mg High Strength ...,https://m.media-amazon.com/images/I/71RcH-GkD6...,https://www.amazon.in//BBETTER-Omega3-Fish-Oil...
...,...,...,...
74,Plantigo Plant Based Vegan Collagen Powder 150...,https://m.media-amazon.com/images/I/71B7Lk7DCg...,https://www.amazon.in//Plantigo-Supplement-Irr...
75,Plantigo Plant Based Vegan Collagen Powder 150...,https://m.media-amazon.com/images/I/61EgFmIuQf...,https://www.amazon.in//Plantigo-Collagen-Suppl...
76,"Red Ribbon Polyresin Floral Window Curtain, 5 ...",https://m.media-amazon.com/images/I/51jcaevMAG...,https://www.amazon.in//Red-Ribbon-Floral-Polye...
77,The Sleep Company SmartGRID Luxe Mattress 10 i...,https://m.media-amazon.com/images/I/61TedluJTh...,https://www.amazon.in//Sleep-Company-SmartGRID...


In [ ]:
# BD = BD.append(df).reset_index(drop=True)
# BD

In [88]:
product_BD = pd.DataFrame(reviews[reviews['product_code']=='product_BD']).reset_index(drop=True)
product_BD

,user_id,product_id,Ratings,Time_stamp,product_code
0,ATNFV0GGIV5QY,B0000225BD,5.0,1360368000,product_BD
1,A13DX84TI2XR2A,B0000225BD,1.0,1371254400,product_BD
2,AXRFD4WAL7Y7T,B0000225BD,4.0,1329004800,product_BD
3,A1EFAJ4Q7XFJWD,B0000225BD,5.0,1386460800,product_BD
4,A1GGWEX2RW0IRW,B0000225BD,4.0,1363132800,product_BD
...,...,...,...,...,...
588,A10W40DOIE1EOP,B0000A9ZBD,4.0,1236988800,product_BD
589,AKT8TGIT6VVZ5,B0000C3WBD,5.0,1092528000,product_BD
590,A36MEN7YDDS3MV,B0000C3WBD,5.0,1085961600,product_BD
591,A2HA8ZYENIHLQU,B0000C3WBD,5.0,1377043200,product_BD


In [89]:
len(product_BD.product_id.unique())

19

In [92]:
product_BD = pd.DataFrame(reviews[reviews['product_code']=='product_BD']).reset_index(drop=True)
product_BD =  pd.merge(product_BD, 
                       BD.iloc[:len(product_BD.product_id.unique())]\
                       .join(pd.Series(product_BD['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BD.to_csv('product_BD.csv',index=False)
pd.read_csv('product_BD.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ATNFV0GGIV5QY,B0000225BD,5.0,1360368000,product_BD,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
1,A13DX84TI2XR2A,B0000225BD,1.0,1371254400,product_BD,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
2,AXRFD4WAL7Y7T,B0000225BD,4.0,1329004800,product_BD,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
3,A1EFAJ4Q7XFJWD,B0000225BD,5.0,1386460800,product_BD,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
4,A1GGWEX2RW0IRW,B0000225BD,4.0,1363132800,product_BD,BBETTER Garcinia Cambogia - With Green Coffee ...,https://m.media-amazon.com/images/I/61rXjIg2g9...,https://www.amazon.in//BBETTER-Garcinia-Cambog...
...,...,...,...,...,...,...,...,...
588,A10W40DOIE1EOP,B0000A9ZBD,4.0,1236988800,product_BD,BBETTER Stevia Drops - Sugar Free Stevia Liqui...,https://m.media-amazon.com/images/I/61hBm5pnbj...,https://www.amazon.in//BBETTER-Stevia-Drops-Li...
589,AKT8TGIT6VVZ5,B0000C3WBD,5.0,1092528000,product_BD,BBETTER Omega 3 6 9 Flaxseed Oil Softgels - Ve...,https://m.media-amazon.com/images/I/61dTF5KU12...,https://www.amazon.in//BBETTER-Omega-Flaxseed-...
590,A36MEN7YDDS3MV,B0000C3WBD,5.0,1085961600,product_BD,BBETTER Omega 3 6 9 Flaxseed Oil Softgels - Ve...,https://m.media-amazon.com/images/I/61dTF5KU12...,https://www.amazon.in//BBETTER-Omega-Flaxseed-...
591,A2HA8ZYENIHLQU,B0000C3WBD,5.0,1377043200,product_BD,BBETTER Omega 3 6 9 Flaxseed Oil Softgels - Ve...,https://m.media-amazon.com/images/I/61dTF5KU12...,https://www.amazon.in//BBETTER-Omega-Flaxseed-...


### Product BE dataframe

In [78]:
df = get_scraped('bluetooth earphones jabra')
BE = df.copy()
BE

  0%|          | 0/20 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Jabra Elite 4 Active in-Ear Bluetooth Earbuds ...,https://m.media-amazon.com/images/I/511ivbbkJd...,https://www.amazon.in//Jabra-Active-Bluetooth-...
1,Jabra Talk 25 SE Mono Bluetooth Headset - Wire...,https://m.media-amazon.com/images/I/51pHEb-eRT...,https://www.amazon.in//Jabra-Talk-Bluetooth-He...
2,Jabra Talk 65 Mono Bluetooth Headset - Premium...,https://m.media-amazon.com/images/I/413gds7MLw...,https://www.amazon.in//Jabra-Talk-Bluetooth-He...
3,Jabra Talk 5 Bluetooth Headset - Black,https://m.media-amazon.com/images/I/51YjVEQotV...,https://www.amazon.in//Jabra-Talk-Bluetooth-He...
4,Jabra Talk 25 Wireless Bluetooth On Ear Headse...,https://m.media-amazon.com/images/I/51d2qrFtYH...,https://www.amazon.in//Jabra-Definition-Hands-...
...,...,...,...
104,Jabra Link 230 USB Adapter,https://m.media-amazon.com/images/I/71loLX0bqs...,https://www.amazon.in//Jabra-Link-230-USB-Adap...
105,Jabra EVOLVE 20 MS Mono USB Headset,https://m.media-amazon.com/images/I/51K7YYdr+a...,https://www.amazon.in//Jabra-EVOLVE-Mono-USB-H...
106,Jabra Evolve 30 II Mono UC - Professional Unif...,https://m.media-amazon.com/images/I/61Y42N6DR0...,https://www.amazon.in//Jabra-Evolve-Mono-Profe...
107,Jabra REVO Wireless Bluetooth Stereo Headphone...,https://m.media-amazon.com/images/I/41dBtVssU3...,https://www.amazon.in//Jabra-REVO-Wireless-Blu...


In [202]:
product_BE = pd.DataFrame(reviews[reviews['product_code']=='product_BE']).reset_index(drop=True)
product_BE =  pd.merge(product_BE, 
                       BE.iloc[:len(product_BE.product_id.unique())]\
                       .join(pd.Series(product_BE['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BE.to_csv('product_BE.csv',index=False)
pd.read_csv('product_BE.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3H1IWF8FMJQ57,B00004Z0BE,5.0,1189555200,product_BE,boAt Rockerz 255 Pro+ Bluetooth Neckband with ...,https://m.media-amazon.com/images/I/515jdxLlma...,https://www.amazon.in//boAt-Rockerz-255-Pro-Ea...
1,A71MPFOKFW4JH,B00005MEBE,5.0,1080950400,product_BE,boAt Rockerz 255F Sports Wireless Headset with...,https://m.media-amazon.com/images/I/61N1qHNeeS...,https://www.amazon.in//Boat-Rockerz-255F-Wirel...
2,A2YBAHB747219P,B00005T3BE,2.0,1202428800,product_BE,boAt Airdopes 121v2 True Wireless Earbuds with...,https://m.media-amazon.com/images/I/510+wjzq-v...,https://www.amazon.in//Airdopes-121v2-Bluetoot...
3,A37PXFZXZZY639,B00005T3BE,5.0,1234656000,product_BE,boAt Airdopes 121v2 True Wireless Earbuds with...,https://m.media-amazon.com/images/I/510+wjzq-v...,https://www.amazon.in//Airdopes-121v2-Bluetoot...
4,A1EC9C6AP1GLH7,B0000645BE,4.0,1034553600,product_BE,boAt Airdopes 141 True Wireless Earbuds with 4...,https://m.media-amazon.com/images/I/51HBom8xz7...,https://www.amazon.in//Airdopes-141-Playtime-R...
...,...,...,...,...,...,...,...,...
10507,A1IX33FC4725C,B00KE80XBE,5.0,1405036800,product_BE,Jabra Evolve2 65 MS Wireless Headphones with L...,https://m.media-amazon.com/images/I/61RfvYU2+y...,https://www.amazon.in//Jabra-Evolve2-Wireless-...
10508,AHA0KZY165G7Q,B00KI2Y1BE,1.0,1404518400,product_BE,Jabra Evolve 65 Uc Duo Bluetooth Wireless On E...,https://m.media-amazon.com/images/I/71r0AlAXj+...,https://www.amazon.in//Jabra-Evolve-Bluetooth-...
10509,A3JBSY11NEUD0W,B00KIH14BE,5.0,1401926400,product_BE,Jabra 2400 II Duo USB MS BT Wireless In Ear Bl...,https://m.media-amazon.com/images/I/51yRZPyc2y...,https://www.amazon.in//Jabra-2400-Duo-Bluetoot...
10510,A2UCI9LMQV0VAP,B00KRZZNBE,1.0,1404950400,product_BE,Jabra Halo Free Wireless Bluetooth Stereo Earbuds,https://m.media-amazon.com/images/I/51o-xEpi05...,https://www.amazon.in//Jabra-Wireless-Bluetoot...


### Product BF dataframe

In [97]:
df = get_scraped('bfit adult diaper medium size')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
1,Nobel Hygiene B-Fit Diapers Adult Diaper Pull ...,https://m.media-amazon.com/images/I/71QfAPM4r9...,https://www.amazon.in//B-Fit-Diapers-Diaper-Me...
2,B-Fit Diapers Adult Diaper Pull Up Pants Mediu...,https://m.media-amazon.com/images/I/71QfAPM4r9...,https://www.amazon.in//B-Fit-Diapers-Diaper-Me...
3,Nobel Hygiene B-Fit Diapers Adult Diaper Pull ...,https://m.media-amazon.com/images/I/71QfAPM4r9...,https://www.amazon.in//Nobel-Hygiene-Diapers-D...
4,"B-Fit Diapers Adult Diaper Economy (Medium, 23...",https://m.media-amazon.com/images/I/41QX6eWXgS...,https://www.amazon.in//B-Fit-Diapers-Diaper-Ec...
...,...,...,...
114,KareMed Adult Diaper Pants for Incontinence,https://m.media-amazon.com/images/I/61ujSFB+bq...,https://www.amazon.in//KareMed-Adult-Diaper-Pa...
115,Leakex High Absorbency Senior Incontinence Pan...,https://m.media-amazon.com/images/I/71ts5NhCHS...,https://www.amazon.in//Leakex-Premium-Absorben...
116,Leakex High Absorbency Senior Incontinence Dia...,https://m.media-amazon.com/images/I/81UMo7Oe6r...,https://www.amazon.in//Leakex-Adult-Diapers-Un...
117,CARETEK Adult Diapers Size: Medium(-30),https://m.media-amazon.com/images/I/81-QyUw0B7...,https://www.amazon.in//CARETEK-Adult-Diapers-S...


In [98]:
BF = df.copy()
BF

,product_name,product_img,product_link
0,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
1,Nobel Hygiene B-Fit Diapers Adult Diaper Pull ...,https://m.media-amazon.com/images/I/71QfAPM4r9...,https://www.amazon.in//B-Fit-Diapers-Diaper-Me...
2,B-Fit Diapers Adult Diaper Pull Up Pants Mediu...,https://m.media-amazon.com/images/I/71QfAPM4r9...,https://www.amazon.in//B-Fit-Diapers-Diaper-Me...
3,Nobel Hygiene B-Fit Diapers Adult Diaper Pull ...,https://m.media-amazon.com/images/I/71QfAPM4r9...,https://www.amazon.in//Nobel-Hygiene-Diapers-D...
4,"B-Fit Diapers Adult Diaper Economy (Medium, 23...",https://m.media-amazon.com/images/I/41QX6eWXgS...,https://www.amazon.in//B-Fit-Diapers-Diaper-Ec...
...,...,...,...
114,KareMed Adult Diaper Pants for Incontinence,https://m.media-amazon.com/images/I/61ujSFB+bq...,https://www.amazon.in//KareMed-Adult-Diaper-Pa...
115,Leakex High Absorbency Senior Incontinence Pan...,https://m.media-amazon.com/images/I/71ts5NhCHS...,https://www.amazon.in//Leakex-Premium-Absorben...
116,Leakex High Absorbency Senior Incontinence Dia...,https://m.media-amazon.com/images/I/81UMo7Oe6r...,https://www.amazon.in//Leakex-Adult-Diapers-Un...
117,CARETEK Adult Diapers Size: Medium(-30),https://m.media-amazon.com/images/I/81-QyUw0B7...,https://www.amazon.in//CARETEK-Adult-Diapers-S...


In [ ]:
# BF = BF.append(df).reset_index(drop=True)
# BF

In [94]:
product_BF = pd.DataFrame(reviews[reviews['product_code']=='product_BF']).reset_index(drop=True)
product_BF

,user_id,product_id,Ratings,Time_stamp,product_code
0,AUCF61BNJ5ZX7,B00000IGBF,5.0,1355961600,product_BF
1,A2ZBB8G2KXBVLN,B00000IGBF,5.0,935020800,product_BF
2,A15U38AWJ3JDN5,B00000IGBF,3.0,957484800,product_BF
3,A7254GWKOPS7E,B00000IGBF,5.0,949881600,product_BF
4,A35VTCF8QVMYAH,B00000IGBF,4.0,949622400,product_BF
...,...,...,...,...,...
342,A294EX8F2EG9QN,B0000D17BF,4.0,1079654400,product_BF
343,A2H9YR7EG1LM4S,B0000D17BF,5.0,1129420800,product_BF
344,A38ACYSCVJXDHN,B0000D17BF,1.0,1114473600,product_BF
345,A32NDN7XDTMOTA,B0000D83BF,3.0,1391990400,product_BF


In [96]:
len(product_BF.product_id.unique())

19

In [99]:
product_BF = pd.DataFrame(reviews[reviews['product_code']=='product_BF']).reset_index(drop=True)
product_BF =  pd.merge(product_BF, 
                       BF.iloc[:len(product_BF.product_id.unique())]\
                       .join(pd.Series(product_BF['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BF.to_csv('product_BF.csv',index=False)
pd.read_csv('product_BF.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AUCF61BNJ5ZX7,B00000IGBF,5.0,1355961600,product_BF,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
1,A2ZBB8G2KXBVLN,B00000IGBF,5.0,935020800,product_BF,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
2,A15U38AWJ3JDN5,B00000IGBF,3.0,957484800,product_BF,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
3,A7254GWKOPS7E,B00000IGBF,5.0,949881600,product_BF,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
4,A35VTCF8QVMYAH,B00000IGBF,4.0,949622400,product_BF,"BFIT Adult Diaper Pants, Medium (White) - Pack...",https://m.media-amazon.com/images/I/71iIYNR27S...,https://www.amazon.in//BFIT-Adult-Diaper-Pants...
...,...,...,...,...,...,...,...,...
342,A294EX8F2EG9QN,B0000D17BF,4.0,1079654400,product_BF,Be Fit Adult Diapers (Medium/28-44-Inch) -10 P...,https://m.media-amazon.com/images/I/713xftpc1I...,https://www.amazon.in//BFIT-Adult-Diapers-Medi...
343,A2H9YR7EG1LM4S,B0000D17BF,5.0,1129420800,product_BF,Be Fit Adult Diapers (Medium/28-44-Inch) -10 P...,https://m.media-amazon.com/images/I/713xftpc1I...,https://www.amazon.in//BFIT-Adult-Diapers-Medi...
344,A38ACYSCVJXDHN,B0000D17BF,1.0,1114473600,product_BF,Be Fit Adult Diapers (Medium/28-44-Inch) -10 P...,https://m.media-amazon.com/images/I/713xftpc1I...,https://www.amazon.in//BFIT-Adult-Diapers-Medi...
345,A32NDN7XDTMOTA,B0000D83BF,3.0,1391990400,product_BF,"B-Fit Diapers Adult Pull Up Pants , Large 10 P...",https://m.media-amazon.com/images/I/71Rr6tZMWp...,https://www.amazon.in//B-Fit-Diapers-Adult-Pan...


### Product BG dataframe

In [107]:
df = get_scraped('bgmi gamepad')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Amkette Evo Gamepad Pro 4 for Android | Instan...,https://m.media-amazon.com/images/I/71Tg6JAz7S...,https://www.amazon.in//Amkette-Bluetooth-Exten...
1,"EvoFox Elite Play Wireless Controller for PS4,...",https://m.media-amazon.com/images/I/61H+rvpTyE...,https://www.amazon.in//EvoFox-Elite-Play-Contr...
2,Unixaa JL01 Pubg Mobile Gamepad controller Wit...,https://m.media-amazon.com/images/I/512rMKCGrV...,https://www.amazon.in//JL01-Gamepad-controller...
3,for Pro Player K21 Game Handle Trigger/Fast Sh...,https://m.media-amazon.com/images/I/41iPaflIsM...,https://www.amazon.in//K21-Trigger-Shooting-Co...
4,Lonaar PUBG Mobile Gamepad with L1R1 L2R2 Trig...,https://m.media-amazon.com/images/I/51SnXQVO07...,https://www.amazon.in//Lonaar-Gamepad-Controll...
...,...,...,...
301,TYROCX PUBG Shark Shape Trigger Mobile Gaming ...,https://m.media-amazon.com/images/I/41WVK543mE...,https://www.amazon.in//TYROCX-Controller-Enhan...
302,TYROCX Latest New PUBG Blue Shark Gaming Trigg...,https://m.media-amazon.com/images/I/41B4TBOD1Y...,https://www.amazon.in//TYROCX-Latest-Trigger-C...
303,Tyrocx New Pubg Gaming Sleeves Anti Sweat | On...,https://m.media-amazon.com/images/I/41B4TBOD1Y...,https://www.amazon.in//Tyrocx-Gaming-Sleeves-T...
304,TYROCX Games PUBG Blue Shark Trigger Mobile Ga...,https://m.media-amazon.com/images/I/41B4TBOD1Y...,https://www.amazon.in//TYROCX-Trigger-Controll...


In [104]:
BG = df.copy()
BG

,product_name,product_img,product_link
0,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
1,LIMESHOT BattleMods X1 Conductive Gaming Trigg...,https://m.media-amazon.com/images/I/517uSMcAVJ...,https://www.amazon.in//LIMESHOT-BattleMods-Con...
2,RPM Euro Games Pubg Gaming for Mobile Gaming. ...,https://m.media-amazon.com/images/I/511XhCr+Zm...,https://www.amazon.in//RPM-Euro-Games-Trigger-...
3,SpinBot BattleMods X1 | Mobile Gaming Triggers...,https://m.media-amazon.com/images/I/41lJSXPtaI...,https://www.amazon.in//SpinBot-BattleMods-Cond...
4,for Pro Player K21 Game Handle Trigger/Fast Sh...,https://m.media-amazon.com/images/I/41iPaflIsM...,https://www.amazon.in//K21-Trigger-Shooting-Co...
...,...,...,...
301,REGAR Mobile Phone USB Cooling Fan Cooler Adju...,https://m.media-amazon.com/images/I/41rRlR8IrU...,https://www.amazon.in//REGAR-Adjustable-Protab...
302,"AK66 Pubg Mobile Controller with 4 Triggers, W...",https://m.media-amazon.com/images/I/51br6ST7Dh...,https://www.amazon.in//Controller-Triggers-Wir...
303,Ceuta Retails� R11 Pubg Metal Triggers Gamepad...,https://m.media-amazon.com/images/I/71p6caRQiv...,https://www.amazon.in//Ceuta-Retails%C2%AE-Con...
304,AJES PUBG Trigger Joy-Stick Sensitive Shoot ai...,https://m.media-amazon.com/images/I/31rXCFVTYN...,https://www.amazon.in//Joy-Stick-Sensitive-Con...


In [108]:
BG = BG.append(df).reset_index(drop=True)
BG

,product_name,product_img,product_link
0,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
1,LIMESHOT BattleMods X1 Conductive Gaming Trigg...,https://m.media-amazon.com/images/I/517uSMcAVJ...,https://www.amazon.in//LIMESHOT-BattleMods-Con...
2,RPM Euro Games Pubg Gaming for Mobile Gaming. ...,https://m.media-amazon.com/images/I/511XhCr+Zm...,https://www.amazon.in//RPM-Euro-Games-Trigger-...
3,SpinBot BattleMods X1 | Mobile Gaming Triggers...,https://m.media-amazon.com/images/I/41lJSXPtaI...,https://www.amazon.in//SpinBot-BattleMods-Cond...
4,for Pro Player K21 Game Handle Trigger/Fast Sh...,https://m.media-amazon.com/images/I/41iPaflIsM...,https://www.amazon.in//K21-Trigger-Shooting-Co...
...,...,...,...
913,TYROCX PUBG Shark Shape Trigger Mobile Gaming ...,https://m.media-amazon.com/images/I/41WVK543mE...,https://www.amazon.in//TYROCX-Controller-Enhan...
914,TYROCX Latest New PUBG Blue Shark Gaming Trigg...,https://m.media-amazon.com/images/I/41B4TBOD1Y...,https://www.amazon.in//TYROCX-Latest-Trigger-C...
915,Tyrocx New Pubg Gaming Sleeves Anti Sweat | On...,https://m.media-amazon.com/images/I/41B4TBOD1Y...,https://www.amazon.in//Tyrocx-Gaming-Sleeves-T...
916,TYROCX Games PUBG Blue Shark Trigger Mobile Ga...,https://m.media-amazon.com/images/I/41B4TBOD1Y...,https://www.amazon.in//TYROCX-Trigger-Controll...


In [101]:
product_BG = pd.DataFrame(reviews[reviews['product_code']=='product_BG']).reset_index(drop=True)
product_BG

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3BYE9L14X9NXV,B00000J4BG,2.0,1010102400,product_BG
1,A2AVV4LX0U104K,B00000J4BG,3.0,944352000,product_BG
2,A2PK5OYQ3F5J8U,B00000J4BG,2.0,955411200,product_BG
3,A2ZGSHBH47H9KL,B00000J4BG,3.0,944784000,product_BG
4,AXRWJQX8BYK7C,B00000J4BG,3.0,945734400,product_BG
...,...,...,...,...,...
9667,ANF3N6NAHCXNX,B00JTMRQBG,5.0,1402876800,product_BG
9668,APMOPIXTU9MVO,B00KCLPZBG,1.0,1402617600,product_BG
9669,A2CO31MDHBZZP6,B00KCLPZBG,5.0,1404518400,product_BG
9670,A1SX3QWCPBO5KP,B00KCLPZBG,1.0,1405987200,product_BG


In [102]:
len(product_BG.product_id.unique())

730

In [109]:
product_BG = pd.DataFrame(reviews[reviews['product_code']=='product_BG']).reset_index(drop=True)
product_BG =  pd.merge(product_BG, 
                       BG.iloc[:len(product_BG.product_id.unique())]\
                       .join(pd.Series(product_BG['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BG.to_csv('product_BG.csv',index=False)
pd.read_csv('product_BG.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3BYE9L14X9NXV,B00000J4BG,2.0,1010102400,product_BG,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
1,A2AVV4LX0U104K,B00000J4BG,3.0,944352000,product_BG,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
2,A2PK5OYQ3F5J8U,B00000J4BG,2.0,955411200,product_BG,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
3,A2ZGSHBH47H9KL,B00000J4BG,3.0,944784000,product_BG,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
4,AXRWJQX8BYK7C,B00000J4BG,3.0,945734400,product_BG,SpinBot BattleMods NinjaX | Mobile Gaming Trig...,https://m.media-amazon.com/images/I/612G+KX3bZ...,https://www.amazon.in//SpinBot-BattleMods-Cont...
...,...,...,...,...,...,...,...,...
9667,ANF3N6NAHCXNX,B00JTMRQBG,5.0,1402876800,product_BG,SPYKART Shark Shape Blue Pubg Triggers Cum Gam...,https://m.media-amazon.com/images/I/51Vu+0slmP...,https://www.amazon.in//SPYKART-Triggers-Joysti...
9668,APMOPIXTU9MVO,B00KCLPZBG,1.0,1402617600,product_BG,(Renewed) Redgear Elite Wireless Gamepad (Black),https://m.media-amazon.com/images/I/61Uac6U0zm...,https://www.amazon.in//CERTIFIED-REFURBISHED-R...
9669,A2CO31MDHBZZP6,B00KCLPZBG,5.0,1404518400,product_BG,(Renewed) Redgear Elite Wireless Gamepad (Black),https://m.media-amazon.com/images/I/61Uac6U0zm...,https://www.amazon.in//CERTIFIED-REFURBISHED-R...
9670,A1SX3QWCPBO5KP,B00KCLPZBG,1.0,1405987200,product_BG,(Renewed) Redgear Elite Wireless Gamepad (Black),https://m.media-amazon.com/images/I/61Uac6U0zm...,https://www.amazon.in//CERTIFIED-REFURBISHED-R...


### Product BH dataframe

In [114]:
df = get_scraped('bhringraj powder')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Alps Goodness Bhringraj Powder For Hair, 250 g...",https://m.media-amazon.com/images/I/5101RooFgP...,https://www.amazon.in//Alps-Goodness-Bhringraj...
1,Khadi Omorose Bhringraj Powder For Nourishment...,https://m.media-amazon.com/images/I/81IUSuOeFb...,https://www.amazon.in//KHADI-Omorose-Bhringraj...
2,Elecious Bhringraj Powder for hair growth (200...,https://m.media-amazon.com/images/I/61U7tVPjtg...,https://www.amazon.in//Elecious-Bhringraj-Powd...
3,Attar Ayurveda Natural Bhringraj Powder for ha...,https://m.media-amazon.com/images/I/713Rjwxmgf...,https://www.amazon.in//Ayurveda-Natural-Bhring...
4,Forest Herbs 100% Natural Organic Bhringraj Po...,https://m.media-amazon.com/images/I/61UY9PkS5Y...,https://www.amazon.in//Forest-Herbs-Natural-Or...
...,...,...,...
301,Khadi Omorose Amla Powder for Skin & Hair Care...,https://m.media-amazon.com/images/I/81lYluruAk...,https://www.amazon.in//Khadi-Omorose-Powder-Na...
302,HomBorn Pure Natural & Organic Bhringraj Powde...,https://m.media-amazon.com/images/I/61eOvjhcIA...,https://www.amazon.in//HomBorn-Natural-Organic...
303,"khadihub Herbal Amla, Hibiscus, Bhringraj powd...",https://m.media-amazon.com/images/I/61REcgiU1g...,https://www.amazon.in//khadihub-Herbal-Hibiscu...
304,Vedicherbs Amla Bhringraj Powder - Hair Pack f...,https://m.media-amazon.com/images/I/81WEuST1Ff...,https://www.amazon.in//Vedicherbs-Amla-Bhringr...


In [115]:
BH = df.copy()
BH

,product_name,product_img,product_link
0,"Alps Goodness Bhringraj Powder For Hair, 250 g...",https://m.media-amazon.com/images/I/5101RooFgP...,https://www.amazon.in//Alps-Goodness-Bhringraj...
1,Khadi Omorose Bhringraj Powder For Nourishment...,https://m.media-amazon.com/images/I/81IUSuOeFb...,https://www.amazon.in//KHADI-Omorose-Bhringraj...
2,Elecious Bhringraj Powder for hair growth (200...,https://m.media-amazon.com/images/I/61U7tVPjtg...,https://www.amazon.in//Elecious-Bhringraj-Powd...
3,Attar Ayurveda Natural Bhringraj Powder for ha...,https://m.media-amazon.com/images/I/713Rjwxmgf...,https://www.amazon.in//Ayurveda-Natural-Bhring...
4,Forest Herbs 100% Natural Organic Bhringraj Po...,https://m.media-amazon.com/images/I/61UY9PkS5Y...,https://www.amazon.in//Forest-Herbs-Natural-Or...
...,...,...,...
301,Khadi Omorose Amla Powder for Skin & Hair Care...,https://m.media-amazon.com/images/I/81lYluruAk...,https://www.amazon.in//Khadi-Omorose-Powder-Na...
302,HomBorn Pure Natural & Organic Bhringraj Powde...,https://m.media-amazon.com/images/I/61eOvjhcIA...,https://www.amazon.in//HomBorn-Natural-Organic...
303,"khadihub Herbal Amla, Hibiscus, Bhringraj powd...",https://m.media-amazon.com/images/I/61REcgiU1g...,https://www.amazon.in//khadihub-Herbal-Hibiscu...
304,Vedicherbs Amla Bhringraj Powder - Hair Pack f...,https://m.media-amazon.com/images/I/81WEuST1Ff...,https://www.amazon.in//Vedicherbs-Amla-Bhringr...


In [ ]:
# BH = BH.append(df).reset_index(drop=True)
# BH

In [112]:
product_BH = pd.DataFrame(reviews[reviews['product_code']=='product_BH']).reset_index(drop=True)
product_BH

,user_id,product_id,Ratings,Time_stamp,product_code
0,AW6YFXN47WRWA,B00001QEBH,4.0,998611200,product_BH
1,A1K6CFURUHHCGS,B00001QEBH,4.0,1002585600,product_BH
2,A1CIU572HUUE5A,B000038ABH,4.0,966988800,product_BH
3,A2BEW179R7FI0F,B000038ABH,5.0,970704000,product_BH
4,A2E4LHUSURH3H4,B000038ABH,5.0,1071446400,product_BH
...,...,...,...,...,...
143,A3V9MKLNX25MHI,B00009RTBH,4.0,1128038400,product_BH
144,ANRMH1UJP7U1U,B0000A0VBH,4.0,1093996800,product_BH
145,A31P05L40GZZ59,B0000A2QBH,1.0,1275004800,product_BH
146,A19U8NGKXOY5T1,B0000A2QBH,4.0,1185580800,product_BH


In [113]:
len(product_BH.product_id.unique())

16

In [116]:
product_BH = pd.DataFrame(reviews[reviews['product_code']=='product_BH']).reset_index(drop=True)
product_BH =  pd.merge(product_BH, 
                       BH.iloc[:len(product_BH.product_id.unique())]\
                       .join(pd.Series(product_BH['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BH.to_csv('product_BH.csv',index=False)
pd.read_csv('product_BH.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AW6YFXN47WRWA,B00001QEBH,4.0,998611200,product_BH,"Alps Goodness Bhringraj Powder For Hair, 250 g...",https://m.media-amazon.com/images/I/5101RooFgP...,https://www.amazon.in//Alps-Goodness-Bhringraj...
1,A1K6CFURUHHCGS,B00001QEBH,4.0,1002585600,product_BH,"Alps Goodness Bhringraj Powder For Hair, 250 g...",https://m.media-amazon.com/images/I/5101RooFgP...,https://www.amazon.in//Alps-Goodness-Bhringraj...
2,A1CIU572HUUE5A,B000038ABH,4.0,966988800,product_BH,Khadi Omorose Bhringraj Powder For Nourishment...,https://m.media-amazon.com/images/I/81IUSuOeFb...,https://www.amazon.in//KHADI-Omorose-Bhringraj...
3,A2BEW179R7FI0F,B000038ABH,5.0,970704000,product_BH,Khadi Omorose Bhringraj Powder For Nourishment...,https://m.media-amazon.com/images/I/81IUSuOeFb...,https://www.amazon.in//KHADI-Omorose-Bhringraj...
4,A2E4LHUSURH3H4,B000038ABH,5.0,1071446400,product_BH,Khadi Omorose Bhringraj Powder For Nourishment...,https://m.media-amazon.com/images/I/81IUSuOeFb...,https://www.amazon.in//KHADI-Omorose-Bhringraj...
...,...,...,...,...,...,...,...,...
143,A3V9MKLNX25MHI,B00009RTBH,4.0,1128038400,product_BH,OREAYU Organic Handmade Bhringraj Powder | Ecl...,https://m.media-amazon.com/images/I/71N4C+beqB...,https://www.amazon.in//OREAYU-Organic-Handmade...
144,ANRMH1UJP7U1U,B0000A0VBH,4.0,1093996800,product_BH,"HEILEN BIOPHARM Bhringraj Powder, 200g Food Gr...",https://m.media-amazon.com/images/I/71cwg6azjL...,https://www.amazon.in//Heilen-Biopharm-Bhringr...
145,A31P05L40GZZ59,B0000A2QBH,1.0,1275004800,product_BH,Bhringraj Powder for Eating | False Daisy | Ec...,https://m.media-amazon.com/images/I/61SEnFVQ9f...,https://www.amazon.in//Bhringraj-Eclipta-Prost...
146,A19U8NGKXOY5T1,B0000A2QBH,4.0,1185580800,product_BH,Bhringraj Powder for Eating | False Daisy | Ec...,https://m.media-amazon.com/images/I/61SEnFVQ9f...,https://www.amazon.in//Bhringraj-Eclipta-Prost...


### Product BI dataframe

In [136]:
df = get_scraped('biotique toner')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Biotique Cucumber Pore Tightening Refreshing T...,https://m.media-amazon.com/images/I/61u4iIRAX7...,https://www.amazon.in//Biotique-Cucumber-Tight...
1,Biotique Honey Water Pore Tightening Brighteni...,https://m.media-amazon.com/images/I/61LnUALNOy...,https://www.amazon.in//Biotique-Honey-Clarifyi...
2,Biotique Tea Tree Skin Clearing Mattifying Fac...,https://m.media-amazon.com/images/I/61zC9cOLjz...,https://www.amazon.in//Biotique-Clearing-Matti...
3,"Biotique Bio Honey Water Clarifying Toner, 120...",https://m.media-amazon.com/images/I/612VnxAlwF...,https://www.amazon.in//Biotique-Clarifying-Mor...
4,"Biotique Honey Water Pore Tightening Toner, 12...",https://m.media-amazon.com/images/I/61DkU1mjMp...,https://www.amazon.in//Biotique-Honey-Water-Ti...
...,...,...,...
56,"Biotique Bio Cucumber Pore Tightening Toner, 1...",https://m.media-amazon.com/images/I/51bBgNCY6u...,https://www.amazon.in//Biotique-Cucumber-Tight...
57,Biotique Unisex Sustainable Set of Sandalwood ...,https://m.media-amazon.com/images/I/51XgMdYY3I...,https://www.amazon.in//Biotique-Sustainable-Sa...
58,"Biotique Bio Honey Water Clarifying Toner, 120...",https://m.media-amazon.com/images/I/51-LraqLA0...,https://www.amazon.in//Biotique-Clarifying-Dan...
59,Biotique combo Toner & Cream_title2,https://m.media-amazon.com/images/I/71DicQk9Ch...,https://www.amazon.in//Biotique-combo-Toner-Cr...


In [127]:
BI = df.copy()
BI

,product_name,product_img,product_link
0,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
1,"Biotique Almond Oil Ultra Rich Body Wash, Bota...",https://m.media-amazon.com/images/I/41T4DXsqrH...,https://www.amazon.in//Biotique-Almond-Ultra-B...
2,Biotique Bio Basil & Parsley Purifying Body Wa...,https://m.media-amazon.com/images/I/610Ynrx-JM...,https://www.amazon.in//Biotique-Basil-Parsley-...
3,"Biotique Bio Apricot Refreshing Body Wash, 200ml",https://m.media-amazon.com/images/I/51to5HxtrS...,https://www.amazon.in//Biotique-Apricot-Body-W...
4,Biotique Bio Morning Nectar (Visibly Flawless ...,https://m.media-amazon.com/images/I/41rAQM662b...,https://www.amazon.in//Biotique-Morning-Visibl...
...,...,...,...
80,Biotique Basil & Parsley Revitalizing Body Was...,https://m.media-amazon.com/images/I/711M25afq7...,https://www.amazon.in//Biotique-Revitalizing-M...
81,"Biotique Bio Dandelion Visibly Ageless Serum, ...",https://m.media-amazon.com/images/I/41YTH1JfTH...,https://www.amazon.in//Biotique-Dandelion-Visi...
82,"Biotique Bio Apricot Refreshing Body Wash, 190...",https://m.media-amazon.com/images/I/41gAIYcFtj...,https://www.amazon.in//Biotique-Apricot-Refres...
83,Biotique Morning Nectar Nourish & Hydrate Body...,https://m.media-amazon.com/images/I/51Yoz7v4nA...,https://www.amazon.in//Biotique-Morning-Nouris...


In [137]:
BI = BI.append(df).reset_index(drop=True)
BI

,product_name,product_img,product_link
0,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
1,"Biotique Almond Oil Ultra Rich Body Wash, Bota...",https://m.media-amazon.com/images/I/41T4DXsqrH...,https://www.amazon.in//Biotique-Almond-Ultra-B...
2,Biotique Bio Basil & Parsley Purifying Body Wa...,https://m.media-amazon.com/images/I/610Ynrx-JM...,https://www.amazon.in//Biotique-Basil-Parsley-...
3,"Biotique Bio Apricot Refreshing Body Wash, 200ml",https://m.media-amazon.com/images/I/51to5HxtrS...,https://www.amazon.in//Biotique-Apricot-Body-W...
4,Biotique Bio Morning Nectar (Visibly Flawless ...,https://m.media-amazon.com/images/I/41rAQM662b...,https://www.amazon.in//Biotique-Morning-Visibl...
...,...,...,...
767,"Biotique Bio Cucumber Pore Tightening Toner, 1...",https://m.media-amazon.com/images/I/51bBgNCY6u...,https://www.amazon.in//Biotique-Cucumber-Tight...
768,Biotique Unisex Sustainable Set of Sandalwood ...,https://m.media-amazon.com/images/I/51XgMdYY3I...,https://www.amazon.in//Biotique-Sustainable-Sa...
769,"Biotique Bio Honey Water Clarifying Toner, 120...",https://m.media-amazon.com/images/I/51-LraqLA0...,https://www.amazon.in//Biotique-Clarifying-Dan...
770,Biotique combo Toner & Cream_title2,https://m.media-amazon.com/images/I/71DicQk9Ch...,https://www.amazon.in//Biotique-combo-Toner-Cr...


In [118]:
product_BI = pd.DataFrame(reviews[reviews['product_code']=='product_BI']).reset_index(drop=True)
product_BI

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1VYL2UPD89IQF,B00000JBBI,5.0,1294531200,product_BI
1,A28ZWH3WYC7ZLD,B00000JBBI,5.0,981763200,product_BI
2,A3D2K4VTDZVBRZ,B00000JBBI,5.0,966384000,product_BI
3,A1J2TBEWQ7YBJO,B00000JBBI,2.0,991267200,product_BI
4,A2KWCYC5PD2VMA,B00000JBBI,3.0,1018483200,product_BI
...,...,...,...,...,...
12109,A6KEOXTMFWGH5,B00K69MYBI,5.0,1405382400,product_BI
12110,A3IGBDCPZSJ8T6,B00K69MYBI,1.0,1404604800,product_BI
12111,A3BRIDOGFG7BGP,B00K69MYBI,4.0,1404086400,product_BI
12112,A8FYUGQNKRB5D,B00K69MYBI,5.0,1404000000,product_BI


In [119]:
len(product_BI.product_id.unique())

754

In [138]:
product_BI = pd.DataFrame(reviews[reviews['product_code']=='product_BI']).reset_index(drop=True)
product_BI =  pd.merge(product_BI, 
                       BI.iloc[:len(product_BI.product_id.unique())]\
                       .join(pd.Series(product_BI['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BI.to_csv('product_BI.csv',index=False)
pd.read_csv('product_BI.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1VYL2UPD89IQF,B00000JBBI,5.0,1294531200,product_BI,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
1,A28ZWH3WYC7ZLD,B00000JBBI,5.0,981763200,product_BI,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
2,A3D2K4VTDZVBRZ,B00000JBBI,5.0,966384000,product_BI,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
3,A1J2TBEWQ7YBJO,B00000JBBI,2.0,991267200,product_BI,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
4,A2KWCYC5PD2VMA,B00000JBBI,3.0,1018483200,product_BI,"Biotique Bio Apricot Refreshing Body Wash, 190ml",https://m.media-amazon.com/images/I/414oXMDwo8...,https://www.amazon.in//Biotique-Apricot-Refres...
...,...,...,...,...,...,...,...,...
12109,A6KEOXTMFWGH5,B00K69MYBI,5.0,1405382400,product_BI,Biotique Unisex Bio Sustainable Set Of Skin Lo...,https://m.media-amazon.com/images/I/51aEHMJ9qL...,https://www.amazon.in//Biotique-Unisex-Sustain...
12110,A3IGBDCPZSJ8T6,B00K69MYBI,1.0,1404604800,product_BI,Biotique Unisex Bio Sustainable Set Of Skin Lo...,https://m.media-amazon.com/images/I/51aEHMJ9qL...,https://www.amazon.in//Biotique-Unisex-Sustain...
12111,A3BRIDOGFG7BGP,B00K69MYBI,4.0,1404086400,product_BI,Biotique Unisex Bio Sustainable Set Of Skin Lo...,https://m.media-amazon.com/images/I/51aEHMJ9qL...,https://www.amazon.in//Biotique-Unisex-Sustain...
12112,A8FYUGQNKRB5D,B00K69MYBI,5.0,1404000000,product_BI,Biotique Unisex Bio Sustainable Set Of Skin Lo...,https://m.media-amazon.com/images/I/51aEHMJ9qL...,https://www.amazon.in//Biotique-Unisex-Sustain...


### Product BJ dataframe

In [144]:
df = get_scraped('bangles juda')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Tripya Arts Glass Bangles Kada Festival Workpl...,https://m.media-amazon.com/images/I/61mybQVB72...,https://www.amazon.in//Bangles-Festival-Workpl...
1,Shining Diva Fashion,https://m.media-amazon.com/images/I/71K5YnNlcW...,https://www.amazon.in//Shining-Diva-Fashion-Br...
2,American Diamond Full White Plated CZ Openable...,https://m.media-amazon.com/images/I/81-8xAG6Up...,https://www.amazon.in//Ratnavali-Jewels-Americ...
3,Bangle Gold Plated American Studded Royal Styl...,https://m.media-amazon.com/images/I/91a9Zeim6a...,https://www.amazon.in//Zeneme-Plated-Antique-B...
4,Multi Color Glossy Glass Plain Bangle Kada Set...,https://m.media-amazon.com/images/I/413lz6BLR6...,https://www.amazon.in//Virya-Glass-Women-Tradi...
...,...,...,...
301,Fuschianet Brass Kada Bangle Large 1 Piece for...,https://m.media-amazon.com/images/I/51CCBUHzx9...,https://www.amazon.in//Fuschianet-Brass-Bangle...
302,Rozy Style Creation Women's Oxidised Metal Sil...,https://m.media-amazon.com/images/I/81++2I4qSt...,https://www.amazon.in//Creation-Womens-Oxidise...
303,Latest fashion Rajwadi polish copper bangles/k...,https://m.media-amazon.com/images/I/81vDkHUWTI...,https://www.amazon.in//Panini-Jewels-Rajwadi-P...
304,Bhavyaj Bangle Gold Plated Royal Style Kada Ba...,https://m.media-amazon.com/images/I/51icwQcI7P...,https://www.amazon.in//Bhavyaj-Bangle-Plated-J...


In [143]:
BJ = df.copy()
BJ

,product_name,product_img,product_link
0,"Rich Club BJP Flag Rectangle Car Window Flag, ...",https://m.media-amazon.com/images/I/41svcBLegS...,https://www.amazon.in//Rich-Club-Rectangle-Win...
1,h.m Aluminium 3D Flag Bharatiya Janata Party B...,https://m.media-amazon.com/images/I/31NMv6haXq...,https://www.amazon.in//Aluminium-Bharatiya-Jan...
2,BJP Outdoor Silk 60x90silk Flag by Sheela Ad M...,https://m.media-amazon.com/images/I/51-T1WTNqK...,https://www.amazon.in//Outdoor-60x90silk-Sheel...
3,Sheela Ad Makers BJP Car Flag,https://m.media-amazon.com/images/I/51EYeelYAn...,https://www.amazon.in//Sheela-Ad-Makers-BJP-Fl...
4,OMG Flag Flag Reshmi Cloth BJP Flag (40X60-inc...,https://m.media-amazon.com/images/I/71lXwa0J8i...,https://www.amazon.in//OMG-Flag-Reshmi-40X60-i...
...,...,...,...
301,The Flag Corporation Indus Ind Bank Car Dashbo...,https://m.media-amazon.com/images/I/71-MhWSbAg...,https://www.amazon.in//Flag-Corporation-Dashbo...
302,The Flag Corporation BOI Bank of India Car Das...,https://m.media-amazon.com/images/I/71A06ZNliO...,https://www.amazon.in//Flag-Corporation-India-...
303,The Flag Corporation Afghanistan Car Dashboard...,https://m.media-amazon.com/images/I/71mlIX0l0A...,https://www.amazon.in//Flag-Corporation-Afghan...
304,The Flag Corporation SRH Sunrisers Hyderabad C...,https://m.media-amazon.com/images/I/71KwM8u8C-...,https://www.amazon.in//Flag-Corporation-Sunris...


In [ ]:
# BJ = BJ.append(df).reset_index(drop=True)
# BJ

In [140]:
product_BJ = pd.DataFrame(reviews[reviews['product_code']=='product_BJ']).reset_index(drop=True)
product_BJ

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2AEVHN77DR3NN,B00003CWBJ,5.0,950054400,product_BJ
1,A13T2XWEGL7DU7,B00003CWBJ,5.0,1247270400,product_BJ
2,A9SNI7VP04Z9M,B00004SABJ,5.0,1358121600,product_BJ
3,AYK7KLKHFHS89,B00004SABJ,5.0,1367020800,product_BJ
4,AGZP8G6JTKQJC,B00004SABJ,4.0,1232582400,product_BJ
...,...,...,...,...,...
900,A2VG364UUGYFHI,B0000ALPBJ,4.0,1119312000,product_BJ
901,A2G0IXGK2CAU25,B0000ALPBJ,2.0,1142640000,product_BJ
902,A2QYII9G0SB68A,B0000ALPBJ,5.0,1222300800,product_BJ
903,A3TXXMGAHHVEWW,B0000ALPBJ,4.0,1220054400,product_BJ


In [141]:
len(product_BJ.product_id.unique())

24

In [145]:
product_BJ = pd.DataFrame(reviews[reviews['product_code']=='product_BJ']).reset_index(drop=True)
product_BJ =  pd.merge(product_BJ, 
                       BJ.iloc[:len(product_BJ.product_id.unique())]\
                       .join(pd.Series(product_BJ['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BJ.to_csv('product_BJ.csv',index=False)
pd.read_csv('product_BJ.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2AEVHN77DR3NN,B00003CWBJ,5.0,950054400,product_BJ,"Rich Club BJP Flag Rectangle Car Window Flag, ...",https://m.media-amazon.com/images/I/41svcBLegS...,https://www.amazon.in//Rich-Club-Rectangle-Win...
1,A13T2XWEGL7DU7,B00003CWBJ,5.0,1247270400,product_BJ,"Rich Club BJP Flag Rectangle Car Window Flag, ...",https://m.media-amazon.com/images/I/41svcBLegS...,https://www.amazon.in//Rich-Club-Rectangle-Win...
2,A9SNI7VP04Z9M,B00004SABJ,5.0,1358121600,product_BJ,h.m Aluminium 3D Flag Bharatiya Janata Party B...,https://m.media-amazon.com/images/I/31NMv6haXq...,https://www.amazon.in//Aluminium-Bharatiya-Jan...
3,AYK7KLKHFHS89,B00004SABJ,5.0,1367020800,product_BJ,h.m Aluminium 3D Flag Bharatiya Janata Party B...,https://m.media-amazon.com/images/I/31NMv6haXq...,https://www.amazon.in//Aluminium-Bharatiya-Jan...
4,AGZP8G6JTKQJC,B00004SABJ,4.0,1232582400,product_BJ,h.m Aluminium 3D Flag Bharatiya Janata Party B...,https://m.media-amazon.com/images/I/31NMv6haXq...,https://www.amazon.in//Aluminium-Bharatiya-Jan...
...,...,...,...,...,...,...,...,...
900,A2VG364UUGYFHI,B0000ALPBJ,4.0,1119312000,product_BJ,KIING Car Aluminium 3D Flag Bharatiya Janata P...,https://m.media-amazon.com/images/I/41ffvuXtm3...,https://www.amazon.in//KIING-Aluminium-Bharati...
901,A2G0IXGK2CAU25,B0000ALPBJ,2.0,1142640000,product_BJ,KIING Car Aluminium 3D Flag Bharatiya Janata P...,https://m.media-amazon.com/images/I/41ffvuXtm3...,https://www.amazon.in//KIING-Aluminium-Bharati...
902,A2QYII9G0SB68A,B0000ALPBJ,5.0,1222300800,product_BJ,KIING Car Aluminium 3D Flag Bharatiya Janata P...,https://m.media-amazon.com/images/I/41ffvuXtm3...,https://www.amazon.in//KIING-Aluminium-Bharati...
903,A3TXXMGAHHVEWW,B0000ALPBJ,4.0,1220054400,product_BJ,KIING Car Aluminium 3D Flag Bharatiya Janata P...,https://m.media-amazon.com/images/I/41ffvuXtm3...,https://www.amazon.in//KIING-Aluminium-Bharati...


### Product BK dataframe

In [153]:
df = get_scraped('bread box')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,HAZEL Bread Box with Handle Lid | Bread Storag...,https://m.media-amazon.com/images/I/51XrZRfQO3...,https://www.amazon.in//HAZEL-Bread-Storage-Kit...
1,"Signoraware Big Bread Box, Lemon Yellow",https://m.media-amazon.com/images/I/319fdoZ+rq...,https://www.amazon.in//Signoraware-Big-Bread-L...
2,Pamirate ® Unbreakable Plastic Fridge Multi St...,https://m.media-amazon.com/images/I/61joWIWurU...,https://www.amazon.in//Pamirate-Unbreakable-Pl...
3,FLODET Bread Box Big Size Air Tight Container ...,https://m.media-amazon.com/images/I/71xvLgfbng...,https://www.amazon.in//FLODET-Bread-Tight-Cont...
4,Ellementry Twine Wire Bread Box,https://m.media-amazon.com/images/I/81czrp+hyi...,https://www.amazon.in//Ellementry-Twine-Wire-B...
...,...,...,...
301,Aluminium Bread Box with sliding lid 250g - 6x...,https://m.media-amazon.com/images/I/61c2F4vvJ4...,https://www.amazon.in//Bread-Box-sliding-lid-2...
302,MosQuick Mosquick- Set Of 3 Food Storage Conta...,https://m.media-amazon.com/images/I/71reQddq0f...,https://www.amazon.in//MosQuick%C2%AE-Storage-...
303,Intactmedia Aluminium Silver Foil Container 45...,https://m.media-amazon.com/images/I/71RdAbAFM0...,https://www.amazon.in//Intactmedia-Aluminium-C...
304,"HPMEAPPY Plastic Bread Box, Fresh Dry Food Sto...",https://m.media-amazon.com/images/I/61-xzbq6nV...,https://www.amazon.in//HPMEAPPY-Plastic-Storag...


In [150]:
BK = df.copy()
BK

,product_name,product_img,product_link
0,"Cartini Godrej Classic Bread Knife, 355 mm, Black",https://m.media-amazon.com/images/I/21CeZYyikB...,https://www.amazon.in//Cartini-Godrej-Classic-...
1,Flair Stainless Steel Serrated Professional Br...,https://m.media-amazon.com/images/I/41kjl3FXR4...,https://www.amazon.in//Professional-Bread-Knif...
2,"Godrej Cartini Classis Bread Knife Large - 16""...",https://m.media-amazon.com/images/I/51speP6chi...,https://www.amazon.in//Godrej-Cartini-Classic-...
3,Amazon Brand - Solimo Stainless Steel Bread Kn...,https://m.media-amazon.com/images/I/71vF0uuJsz...,https://www.amazon.in//Amazon-Brand-Solimo-Sta...
4,Home republic -14 Inch Cake Knife Multiform Br...,https://m.media-amazon.com/images/I/51fWeigLQZ...,https://www.amazon.in//republic-Multiform-Stai...
...,...,...,...
301,Nixby Toast Bread Slicer Bread Well-Distribute...,https://m.media-amazon.com/images/I/612HF3X4o1...,https://www.amazon.in//Nixby-Slicer-Well-Distr...
302,APURK Bread Slicer Cutter | Foldable Practical...,https://m.media-amazon.com/images/I/61HowjitkL...,https://www.amazon.in//APURK-Foldable-Practica...
303,STORE99® 3 PCS Cake Knife Server Pastry Bread ...,https://m.media-amazon.com/images/I/21xwoRLv2o...,https://www.amazon.in//STORE99%C2%AE-Knife-Ser...
304,"Shree ANADAYE ENTERPRISE Bread Slicer, Bread S...",https://m.media-amazon.com/images/I/712+W9xnCg...,https://www.amazon.in//ENTERPRISE-Homemade-Mat...


In [154]:
BK = BK.append(df).reset_index(drop=True)
BK

,product_name,product_img,product_link
0,"Cartini Godrej Classic Bread Knife, 355 mm, Black",https://m.media-amazon.com/images/I/21CeZYyikB...,https://www.amazon.in//Cartini-Godrej-Classic-...
1,Flair Stainless Steel Serrated Professional Br...,https://m.media-amazon.com/images/I/41kjl3FXR4...,https://www.amazon.in//Professional-Bread-Knif...
2,"Godrej Cartini Classis Bread Knife Large - 16""...",https://m.media-amazon.com/images/I/51speP6chi...,https://www.amazon.in//Godrej-Cartini-Classic-...
3,Amazon Brand - Solimo Stainless Steel Bread Kn...,https://m.media-amazon.com/images/I/71vF0uuJsz...,https://www.amazon.in//Amazon-Brand-Solimo-Sta...
4,Home republic -14 Inch Cake Knife Multiform Br...,https://m.media-amazon.com/images/I/51fWeigLQZ...,https://www.amazon.in//republic-Multiform-Stai...
...,...,...,...
913,Aluminium Bread Box with sliding lid 250g - 6x...,https://m.media-amazon.com/images/I/61c2F4vvJ4...,https://www.amazon.in//Bread-Box-sliding-lid-2...
914,MosQuick Mosquick- Set Of 3 Food Storage Conta...,https://m.media-amazon.com/images/I/71reQddq0f...,https://www.amazon.in//MosQuick%C2%AE-Storage-...
915,Intactmedia Aluminium Silver Foil Container 45...,https://m.media-amazon.com/images/I/71RdAbAFM0...,https://www.amazon.in//Intactmedia-Aluminium-C...
916,"HPMEAPPY Plastic Bread Box, Fresh Dry Food Sto...",https://m.media-amazon.com/images/I/61-xzbq6nV...,https://www.amazon.in//HPMEAPPY-Plastic-Storag...


In [147]:
product_BK = pd.DataFrame(reviews[reviews['product_code']=='product_BK']).reset_index(drop=True)
product_BK

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2DAGS7N1NFEVE,B00003CWBK,5.0,963360000,product_BK
1,A2CAY0PA1O54VV,B00004YMBK,5.0,997401600,product_BK
2,ANNTCUP956KMM,B00004YMBK,5.0,980294400,product_BK
3,A1YXGJIY3ZYS0T,B00004YMBK,5.0,993945600,product_BK
4,A3OIYA9U8ZHCDW,B00004YMBK,4.0,977961600,product_BK
...,...,...,...,...,...
9514,A2NX6PABLIJLP1,B00JPITXBK,5.0,1403827200,product_BK
9515,A3U5HKDHPYBAK3,B00JPITXBK,5.0,1405728000,product_BK
9516,A2R6F2NILZ9IDH,B00JUY4CBK,4.0,1403740800,product_BK
9517,A3ANO8ZDNQ9KX7,B00KAJRXBK,4.0,1404777600,product_BK


In [148]:
len(product_BK.product_id.unique())

705

In [155]:
product_BK = pd.DataFrame(reviews[reviews['product_code']=='product_BK']).reset_index(drop=True)
product_BK =  pd.merge(product_BK, 
                       BK.iloc[:len(product_BK.product_id.unique())]\
                       .join(pd.Series(product_BK['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BK.to_csv('product_BK.csv',index=False)
pd.read_csv('product_BK.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2DAGS7N1NFEVE,B00003CWBK,5.0,963360000,product_BK,"Cartini Godrej Classic Bread Knife, 355 mm, Black",https://m.media-amazon.com/images/I/21CeZYyikB...,https://www.amazon.in//Cartini-Godrej-Classic-...
1,A2CAY0PA1O54VV,B00004YMBK,5.0,997401600,product_BK,Flair Stainless Steel Serrated Professional Br...,https://m.media-amazon.com/images/I/41kjl3FXR4...,https://www.amazon.in//Professional-Bread-Knif...
2,ANNTCUP956KMM,B00004YMBK,5.0,980294400,product_BK,Flair Stainless Steel Serrated Professional Br...,https://m.media-amazon.com/images/I/41kjl3FXR4...,https://www.amazon.in//Professional-Bread-Knif...
3,A1YXGJIY3ZYS0T,B00004YMBK,5.0,993945600,product_BK,Flair Stainless Steel Serrated Professional Br...,https://m.media-amazon.com/images/I/41kjl3FXR4...,https://www.amazon.in//Professional-Bread-Knif...
4,A3OIYA9U8ZHCDW,B00004YMBK,4.0,977961600,product_BK,Flair Stainless Steel Serrated Professional Br...,https://m.media-amazon.com/images/I/41kjl3FXR4...,https://www.amazon.in//Professional-Bread-Knif...
...,...,...,...,...,...,...,...,...
9514,A2NX6PABLIJLP1,B00JPITXBK,5.0,1403827200,product_BK,PENADIA Plastic Food Storage Container for Fri...,https://m.media-amazon.com/images/I/71bc1iyQgy...,https://www.amazon.in//PENADIA-Plastic-Contain...
9515,A3U5HKDHPYBAK3,B00JPITXBK,5.0,1405728000,product_BK,PENADIA Plastic Food Storage Container for Fri...,https://m.media-amazon.com/images/I/71bc1iyQgy...,https://www.amazon.in//PENADIA-Plastic-Contain...
9516,A2R6F2NILZ9IDH,B00JUY4CBK,4.0,1403740800,product_BK,G Crystal Bread Box Plastic for Kitchen Storag...,https://m.media-amazon.com/images/I/61yuifIHYZ...,https://www.amazon.in//Crystal-Plastic-Kitchen...
9517,A3ANO8ZDNQ9KX7,B00KAJRXBK,4.0,1404777600,product_BK,Amazon Brand - Solimo Set of 2 Storage Contain...,https://m.media-amazon.com/images/I/71BrOwgkYj...,https://www.amazon.in//Amazon-Brand-Kitchen-St...


### Product BL dataframe

In [159]:
df = get_scraped('blender')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Orpat HHB-100E WOB 250-Watt Hand Blender (White),https://m.media-amazon.com/images/I/21OWOIM1wM...,https://www.amazon.in//Orpat-HHB-100E-WOB-250-...
1,"PHILIPS HL1655/00 Hand Blender, White Jar 250W",https://m.media-amazon.com/images/I/41-5nd-r6b...,https://www.amazon.in//Philips-Collection-HL16...
2,NutriPro Juicer Mixer Grinder - Smoothie Maker...,https://m.media-amazon.com/images/I/51y8XGTdDQ...,https://www.amazon.in//NutriPro-Bullet-Juicer-...
3,"Wonderchef Nutri-blend Mixer, Grinder & Blende...",https://m.media-amazon.com/images/I/61BGNSBBwY...,https://www.amazon.in//Wonderchef-Nutri-Blend-...
4,KENT 16044 Hand Blender Stainless Steel 400 W ...,https://m.media-amazon.com/images/I/51x7cJG02P...,https://www.amazon.in//Kent-KENT-Hand-Blender/...
...,...,...,...
301,Zoreo Mini Blender Fruit Mixer Machine Portabl...,https://m.media-amazon.com/images/I/51AkG5J4w3...,https://www.amazon.in//Zoreo-Mixer-Juicer-Rech...
302,Usha Sure Blend 125 Watts Hand Blender (Magent...,https://m.media-amazon.com/images/I/413VobI8+3...,https://www.amazon.in//Sure-Blend-Blender-Mage...
303,"MAULION Powered by Milk Frother Coffee Beater,...",https://m.media-amazon.com/images/I/71fJLf1mF7...,https://www.amazon.in//MAULIONTM-Powered-Premi...
304,S V Enterprise Power Free Hand Blender & Beate...,https://m.media-amazon.com/images/I/51DflImJyi...,https://www.amazon.in//Enterprise-Hand-Blender...


In [160]:
BL = df.copy()
BL

,product_name,product_img,product_link
0,Orpat HHB-100E WOB 250-Watt Hand Blender (White),https://m.media-amazon.com/images/I/21OWOIM1wM...,https://www.amazon.in//Orpat-HHB-100E-WOB-250-...
1,"PHILIPS HL1655/00 Hand Blender, White Jar 250W",https://m.media-amazon.com/images/I/41-5nd-r6b...,https://www.amazon.in//Philips-Collection-HL16...
2,NutriPro Juicer Mixer Grinder - Smoothie Maker...,https://m.media-amazon.com/images/I/51y8XGTdDQ...,https://www.amazon.in//NutriPro-Bullet-Juicer-...
3,"Wonderchef Nutri-blend Mixer, Grinder & Blende...",https://m.media-amazon.com/images/I/61BGNSBBwY...,https://www.amazon.in//Wonderchef-Nutri-Blend-...
4,KENT 16044 Hand Blender Stainless Steel 400 W ...,https://m.media-amazon.com/images/I/51x7cJG02P...,https://www.amazon.in//Kent-KENT-Hand-Blender/...
...,...,...,...
301,Zoreo Mini Blender Fruit Mixer Machine Portabl...,https://m.media-amazon.com/images/I/51AkG5J4w3...,https://www.amazon.in//Zoreo-Mixer-Juicer-Rech...
302,Usha Sure Blend 125 Watts Hand Blender (Magent...,https://m.media-amazon.com/images/I/413VobI8+3...,https://www.amazon.in//Sure-Blend-Blender-Mage...
303,"MAULION Powered by Milk Frother Coffee Beater,...",https://m.media-amazon.com/images/I/71fJLf1mF7...,https://www.amazon.in//MAULIONTM-Powered-Premi...
304,S V Enterprise Power Free Hand Blender & Beate...,https://m.media-amazon.com/images/I/51DflImJyi...,https://www.amazon.in//Enterprise-Hand-Blender...


In [ ]:
# BL = BL.append(df).reset_index(drop=True)
# BL

In [157]:
product_BL = pd.DataFrame(reviews[reviews['product_code']=='product_BL']).reset_index(drop=True)
product_BL

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3EQI2CEEO3ZIO,B00000J4BL,1.0,1101772800,product_BL
1,AJX0Q5B0QEP57,B00000J4BL,5.0,947203200,product_BL
2,A19Y9WEUU8YHH4,B00000J4BL,5.0,1003363200,product_BL
3,A2ECHN6P37UUV4,B00004YMBL,1.0,1016150400,product_BL
4,AW003H7Z3OCHA,B00004YMBL,5.0,1001376000,product_BL
...,...,...,...,...,...
276,A1PW2HIA2JF65O,B0000D93BL,4.0,1167091200,product_BL
277,A9GYUG4TU9HDC,B0000D93BL,2.0,1089676800,product_BL
278,AKO2394W11V66,B0000D93BL,5.0,1084838400,product_BL
279,A18YFGWNSK2VVU,B0000D93BL,3.0,1082332800,product_BL


In [158]:
len(product_BL.product_id.unique())

29

In [161]:
product_BL = pd.DataFrame(reviews[reviews['product_code']=='product_BL']).reset_index(drop=True)
product_BL =  pd.merge(product_BL, 
                       BL.iloc[:len(product_BL.product_id.unique())]\
                       .join(pd.Series(product_BL['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BL.to_csv('product_BL.csv',index=False)
pd.read_csv('product_BL.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3EQI2CEEO3ZIO,B00000J4BL,1.0,1101772800,product_BL,Orpat HHB-100E WOB 250-Watt Hand Blender (White),https://m.media-amazon.com/images/I/21OWOIM1wM...,https://www.amazon.in//Orpat-HHB-100E-WOB-250-...
1,AJX0Q5B0QEP57,B00000J4BL,5.0,947203200,product_BL,Orpat HHB-100E WOB 250-Watt Hand Blender (White),https://m.media-amazon.com/images/I/21OWOIM1wM...,https://www.amazon.in//Orpat-HHB-100E-WOB-250-...
2,A19Y9WEUU8YHH4,B00000J4BL,5.0,1003363200,product_BL,Orpat HHB-100E WOB 250-Watt Hand Blender (White),https://m.media-amazon.com/images/I/21OWOIM1wM...,https://www.amazon.in//Orpat-HHB-100E-WOB-250-...
3,A2ECHN6P37UUV4,B00004YMBL,1.0,1016150400,product_BL,"PHILIPS HL1655/00 Hand Blender, White Jar 250W",https://m.media-amazon.com/images/I/41-5nd-r6b...,https://www.amazon.in//Philips-Collection-HL16...
4,AW003H7Z3OCHA,B00004YMBL,5.0,1001376000,product_BL,"PHILIPS HL1655/00 Hand Blender, White Jar 250W",https://m.media-amazon.com/images/I/41-5nd-r6b...,https://www.amazon.in//Philips-Collection-HL16...
...,...,...,...,...,...,...,...,...
276,A1PW2HIA2JF65O,B0000D93BL,4.0,1167091200,product_BL,Coffee frother Coffee Beater Electric Handheld...,https://m.media-amazon.com/images/I/31zqHTvu32...,https://www.amazon.in//frother-Electric-Handhe...
277,A9GYUG4TU9HDC,B0000D93BL,2.0,1089676800,product_BL,Coffee frother Coffee Beater Electric Handheld...,https://m.media-amazon.com/images/I/31zqHTvu32...,https://www.amazon.in//frother-Electric-Handhe...
278,AKO2394W11V66,B0000D93BL,5.0,1084838400,product_BL,Coffee frother Coffee Beater Electric Handheld...,https://m.media-amazon.com/images/I/31zqHTvu32...,https://www.amazon.in//frother-Electric-Handhe...
279,A18YFGWNSK2VVU,B0000D93BL,3.0,1082332800,product_BL,Coffee frother Coffee Beater Electric Handheld...,https://m.media-amazon.com/images/I/31zqHTvu32...,https://www.amazon.in//frother-Electric-Handhe...


### Product BM dataframe

In [170]:
df = get_scraped('boots for kids')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Boys PU Printed Snow Boots in Dark Blue Color,https://m.media-amazon.com/images/I/61NKp2rjO+...,https://www.amazon.in//Hopscotch-Boys-Printed-...
1,Women's Spencer Ankle Boot,https://m.media-amazon.com/images/I/51iuW0bln0...,https://www.amazon.in//Aerosoles-Womens-Spence...
2,Boy's & Girl's Casual Stylish Boots for 15 Mon...,https://m.media-amazon.com/images/I/81nUZMgAMZ...,https://www.amazon.in//Edee-Girls-Casual-Styli...
3,Kids Casual Boots for boys,https://m.media-amazon.com/images/I/61Ncve6q6O...,https://www.amazon.in//kids-boots-boys-6-7-yea...
4,Kids Casual Boots,https://m.media-amazon.com/images/I/71syeu9PHT...,https://www.amazon.in//Onbeat-Casual-Boots-Bla...
...,...,...,...
301,Unisex-Child Burke YTH Fashion Boot,https://m.media-amazon.com/images/I/71gKgvHQUz...,https://www.amazon.in//Dr-Scholls-Kids-Unisex-...
302,Unisex-Child Moab Voyager Mid Alternative Clos...,https://m.media-amazon.com/images/I/71Z6QMlHf8...,https://www.amazon.in//Merrell-Voyager-Alterna...
303,Vipanshi Footwear Pink Color Girl's Boots of S...,https://m.media-amazon.com/images/I/41CzRWENZq...,https://www.amazon.in//Vipanshi-Footwear-Color...
304,Vipanshi Footwear Pinck Color Girl's Boots of ...,https://m.media-amazon.com/images/I/41CzRWENZq...,https://www.amazon.in//Vipanshi-Footwear-Pinck...


In [167]:
BM = df.copy()
BM

,product_name,product_img,product_link
0,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
1,Bebop Men Boots,https://m.media-amazon.com/images/I/71eQ+C0QIC...,https://www.amazon.in//Kraasa-Bebop-Boots-Men-...
2,Men's Synthetic Boots,https://m.media-amazon.com/images/I/71csCqLdKy...,https://www.amazon.in//Amazon-Brand-Symbol-Boo...
3,Flame Original 7-Eye Moto Inspired Mild Water ...,https://m.media-amazon.com/images/I/81esw-C0gl...,https://www.amazon.in//Bacca-Bucci-Flame-Origi...
4,mens Rte3083 Chelsea Boot,https://m.media-amazon.com/images/I/61vKUYLUfG...,https://www.amazon.in//Red-Tape-Leather-Chelse...
...,...,...,...
301,Omee Gold NCC/Military/Army Genuine Leather Boots,https://m.media-amazon.com/images/I/71PxJLQ72H...,https://www.amazon.in//Commando-Military-Genui...
302,Men Chelsea Boots Handcrafted Luxury Leather S...,https://m.media-amazon.com/images/I/51xP4Kq69y...,https://www.amazon.in//MUTAQINOTI-Chelsea-Amer...
303,OMEE GOLD Mens Black Tactical Combat Army Boot...,https://m.media-amazon.com/images/I/71Sw7XDkdU...,https://www.amazon.in//Commando-Tactical-Comba...
304,Men Black Synthetic Leather Suede Chelsea Boots,https://m.media-amazon.com/images/I/71vphSa1XT...,https://www.amazon.in//FENTACIA-Black-Syntheti...


In [171]:
BM = BM.append(df).reset_index(drop=True)
BM

,product_name,product_img,product_link
0,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
1,Bebop Men Boots,https://m.media-amazon.com/images/I/71eQ+C0QIC...,https://www.amazon.in//Kraasa-Bebop-Boots-Men-...
2,Men's Synthetic Boots,https://m.media-amazon.com/images/I/71csCqLdKy...,https://www.amazon.in//Amazon-Brand-Symbol-Boo...
3,Flame Original 7-Eye Moto Inspired Mild Water ...,https://m.media-amazon.com/images/I/81esw-C0gl...,https://www.amazon.in//Bacca-Bucci-Flame-Origi...
4,mens Rte3083 Chelsea Boot,https://m.media-amazon.com/images/I/61vKUYLUfG...,https://www.amazon.in//Red-Tape-Leather-Chelse...
...,...,...,...
913,Unisex-Child Burke YTH Fashion Boot,https://m.media-amazon.com/images/I/71gKgvHQUz...,https://www.amazon.in//Dr-Scholls-Kids-Unisex-...
914,Unisex-Child Moab Voyager Mid Alternative Clos...,https://m.media-amazon.com/images/I/71Z6QMlHf8...,https://www.amazon.in//Merrell-Voyager-Alterna...
915,Vipanshi Footwear Pink Color Girl's Boots of S...,https://m.media-amazon.com/images/I/41CzRWENZq...,https://www.amazon.in//Vipanshi-Footwear-Color...
916,Vipanshi Footwear Pinck Color Girl's Boots of ...,https://m.media-amazon.com/images/I/41CzRWENZq...,https://www.amazon.in//Vipanshi-Footwear-Pinck...


In [164]:
product_BM = pd.DataFrame(reviews[reviews['product_code']=='product_BM']).reset_index(drop=True)
product_BM

,user_id,product_id,Ratings,Time_stamp,product_code
0,A368NPNYPY3GAX,B00000J0BM,2.0,963619200,product_BM
1,A3RIN9UUT53ADQ,B00000J0BM,5.0,1080172800,product_BM
2,A1CAL36PFWCOC1,B00000J0BM,4.0,979603200,product_BM
3,ANSR0LCACFPRV,B00000J0BM,3.0,1001376000,product_BM
4,A2JH9HH1OZMU67,B00000J0BM,3.0,1040947200,product_BM
...,...,...,...,...,...
14732,A3T7WRWUJL0E70,B00K0QVFBM,5.0,1405209600,product_BM
14733,A15ECFQ2OX6YTJ,B00K1DHBBM,5.0,1399680000,product_BM
14734,A2PJ4FAENU8OQD,B00K1DHBBM,5.0,1403913600,product_BM
14735,AFV6EBY9590S,B00K1DHBBM,4.0,1405987200,product_BM


In [165]:
len(product_BM.product_id.unique())

784

In [172]:
product_BM = pd.DataFrame(reviews[reviews['product_code']=='product_BM']).reset_index(drop=True)
product_BM =  pd.merge(product_BM, 
                       BM.iloc[:len(product_BM.product_id.unique())]\
                       .join(pd.Series(product_BM['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BM.to_csv('product_BM.csv',index=False)
pd.read_csv('product_BM.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A368NPNYPY3GAX,B00000J0BM,2.0,963619200,product_BM,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
1,A3RIN9UUT53ADQ,B00000J0BM,5.0,1080172800,product_BM,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
2,A1CAL36PFWCOC1,B00000J0BM,4.0,979603200,product_BM,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
3,ANSR0LCACFPRV,B00000J0BM,3.0,1001376000,product_BM,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
4,A2JH9HH1OZMU67,B00000J0BM,3.0,1040947200,product_BM,Men's Rte319a Leather Ankle Boot,https://m.media-amazon.com/images/I/71vgrQNoGZ...,https://www.amazon.in//Red-Tape-Ankle-Length-B...
...,...,...,...,...,...,...,...,...
14732,A3T7WRWUJL0E70,B00K0QVFBM,5.0,1405209600,product_BM,Girl's Boots,https://m.media-amazon.com/images/I/71DTvdgkef...,https://www.amazon.in//Cienta-Girls-Marino-Boo...
14733,A15ECFQ2OX6YTJ,B00K1DHBBM,5.0,1399680000,product_BM,Baby Boys and Baby Girls Fabric Applique Buckl...,https://m.media-amazon.com/images/I/51gOcxb4t2...,https://www.amazon.in//Hopscotch-Girls-Fabric-...
14734,A2PJ4FAENU8OQD,B00K1DHBBM,5.0,1403913600,product_BM,Baby Boys and Baby Girls Fabric Applique Buckl...,https://m.media-amazon.com/images/I/51gOcxb4t2...,https://www.amazon.in//Hopscotch-Girls-Fabric-...
14735,AFV6EBY9590S,B00K1DHBBM,4.0,1405987200,product_BM,Baby Boys and Baby Girls Fabric Applique Buckl...,https://m.media-amazon.com/images/I/51gOcxb4t2...,https://www.amazon.in//Hopscotch-Girls-Fabric-...


### Product BN dataframe

In [176]:
df = get_scraped('binocular')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
1,Krevia Binocular for Long Distance | Telescopi...,https://m.media-amazon.com/images/I/71qRGLQBTB...,https://www.amazon.in//Krevia-Powerful-Binocul...
2,KREVIA Binocular for Long Distance | Professio...,https://m.media-amazon.com/images/I/61ki5jVTlN...,https://www.amazon.in//Professional-Telescopic...
3,HOME BUY Compact 10x25 Mini Binoculars Telesco...,https://m.media-amazon.com/images/I/61soMphe0k...,https://www.amazon.in//Tiny-Deal-Binoculars-Te...
4,CASON (DEVICE OF C)-Professional 8 X 40 HD Bin...,https://m.media-amazon.com/images/I/61vng2BIoS...,https://www.amazon.in//CASON-DEVICE-Profession...
...,...,...,...
301,Layfuz BinocularsTelescope 180 * 100 Low-Light...,https://m.media-amazon.com/images/I/61K3lnb-Go...,https://www.amazon.in//Layfuz-BinocularsTelesc...
302,BIGWIN Camman Mini 2.5 x 26 Children Binocular...,https://m.media-amazon.com/images/I/31SYMqjLPv...,https://www.amazon.in//AV-TOYS-Binocular-Teles...
303,Smitex HD Professional Lightweight Pocket Size...,https://m.media-amazon.com/images/I/61dXNxW8s2...,https://www.amazon.in//Smitex-Professional-Lig...
304,Winoso Sakura 30X60 Foldable HD Professional B...,https://m.media-amazon.com/images/I/61Wt-ZprdM...,https://www.amazon.in//Winoso-Foldable-Profess...


In [177]:
BN = df.copy()
BN

,product_name,product_img,product_link
0,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
1,Krevia Binocular for Long Distance | Telescopi...,https://m.media-amazon.com/images/I/71qRGLQBTB...,https://www.amazon.in//Krevia-Powerful-Binocul...
2,KREVIA Binocular for Long Distance | Professio...,https://m.media-amazon.com/images/I/61ki5jVTlN...,https://www.amazon.in//Professional-Telescopic...
3,HOME BUY Compact 10x25 Mini Binoculars Telesco...,https://m.media-amazon.com/images/I/61soMphe0k...,https://www.amazon.in//Tiny-Deal-Binoculars-Te...
4,CASON (DEVICE OF C)-Professional 8 X 40 HD Bin...,https://m.media-amazon.com/images/I/61vng2BIoS...,https://www.amazon.in//CASON-DEVICE-Profession...
...,...,...,...
301,Layfuz BinocularsTelescope 180 * 100 Low-Light...,https://m.media-amazon.com/images/I/61K3lnb-Go...,https://www.amazon.in//Layfuz-BinocularsTelesc...
302,BIGWIN Camman Mini 2.5 x 26 Children Binocular...,https://m.media-amazon.com/images/I/31SYMqjLPv...,https://www.amazon.in//AV-TOYS-Binocular-Teles...
303,Smitex HD Professional Lightweight Pocket Size...,https://m.media-amazon.com/images/I/61dXNxW8s2...,https://www.amazon.in//Smitex-Professional-Lig...
304,Winoso Sakura 30X60 Foldable HD Professional B...,https://m.media-amazon.com/images/I/61Wt-ZprdM...,https://www.amazon.in//Winoso-Foldable-Profess...


In [ ]:
# BN = BO.append(df).reset_index(drop=True)
# BN

In [174]:
product_BN = pd.DataFrame(reviews[reviews['product_code']=='product_BN']).reset_index(drop=True)
product_BN

,user_id,product_id,Ratings,Time_stamp,product_code
0,A13PACGL8P9ZZ,B00004Z0BN,5.0,1139961600,product_BN
1,A3PM4QTTDC3N32,B00004Z0BN,5.0,1170720000,product_BN
2,A3JZBBAYKUJP9Z,B00004Z0BN,5.0,1193702400,product_BN
3,AIF81Y6GP42O6,B00004Z0BN,5.0,1143676800,product_BN
4,A22WFCWDH6PRAY,B00004Z0BN,5.0,1262822400,product_BN
...,...,...,...,...,...
387,A1DL8PN4MVO8ZC,B0000BZLBN,5.0,1236816000,product_BN
388,AGQSJFQLW22RS,B0000BZLBN,5.0,1342137600,product_BN
389,A1452IWUPOIWRT,B0000BZLBN,5.0,1262995200,product_BN
390,A1HGOTMS9H27PB,B0000BZLBN,5.0,1235088000,product_BN


In [175]:
len(product_BN.product_id.unique())

25

In [178]:
product_BN = pd.DataFrame(reviews[reviews['product_code']=='product_BN']).reset_index(drop=True)
product_BN =  pd.merge(product_BN, 
                       BN.iloc[:len(product_BN.product_id.unique())]\
                       .join(pd.Series(product_BN['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BN.to_csv('product_BN.csv',index=False)
pd.read_csv('product_BN.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A13PACGL8P9ZZ,B00004Z0BN,5.0,1139961600,product_BN,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
1,A3PM4QTTDC3N32,B00004Z0BN,5.0,1170720000,product_BN,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
2,A3JZBBAYKUJP9Z,B00004Z0BN,5.0,1193702400,product_BN,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
3,AIF81Y6GP42O6,B00004Z0BN,5.0,1143676800,product_BN,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
4,A22WFCWDH6PRAY,B00004Z0BN,5.0,1262822400,product_BN,Inditradition 30x60 High Powered Binoculars | ...,https://m.media-amazon.com/images/I/61-iRwBY37...,https://www.amazon.in//Inditradition-Powered-B...
...,...,...,...,...,...,...,...,...
387,A1DL8PN4MVO8ZC,B0000BZLBN,5.0,1236816000,product_BN,"ANAB GI Binoculars (Doorbeen) for Kids Gifts, ...",https://m.media-amazon.com/images/I/61ayvOxleE...,https://www.amazon.in//ANAB-GI-Binoculars-Door...
388,AGQSJFQLW22RS,B0000BZLBN,5.0,1342137600,product_BN,"ANAB GI Binoculars (Doorbeen) for Kids Gifts, ...",https://m.media-amazon.com/images/I/61ayvOxleE...,https://www.amazon.in//ANAB-GI-Binoculars-Door...
389,A1452IWUPOIWRT,B0000BZLBN,5.0,1262995200,product_BN,"ANAB GI Binoculars (Doorbeen) for Kids Gifts, ...",https://m.media-amazon.com/images/I/61ayvOxleE...,https://www.amazon.in//ANAB-GI-Binoculars-Door...
390,A1HGOTMS9H27PB,B0000BZLBN,5.0,1235088000,product_BN,"ANAB GI Binoculars (Doorbeen) for Kids Gifts, ...",https://m.media-amazon.com/images/I/61ayvOxleE...,https://www.amazon.in//ANAB-GI-Binoculars-Door...


### Product BO dataframe

In [58]:
df = get_scraped('bowl with lid')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Cello Ornella Glass Mixing Bowl Set with Lid (...,https://m.media-amazon.com/images/I/610OThs7YV...,https://www.amazon.in//Cello-Ornella-Glass-Mix...
1,"Yera B16FL Glass 4-Piece Polo Bowl with Lid, 4...",https://m.media-amazon.com/images/I/51jOA4hY7j...,https://www.amazon.in//Yera-B16FL-Glass-4-Piec...
2,Cello Opalware Twilight Floral Mixing Bowls wi...,https://m.media-amazon.com/images/I/71Af3G8Br+...,https://www.amazon.in//Cello-Twilight-Opalware...
3,Borosil Basic Glass Mixing & Serving Bowls Wit...,https://m.media-amazon.com/images/I/611TUEXcnT...,https://www.amazon.in//Borosil-Basics-Glass-Mi...
4,Borosil Glass Mixing & Serving Bowls With Lids...,https://m.media-amazon.com/images/I/61qjnnQS52...,https://www.amazon.in//Borosil-Glass-Mixing-Bo...
...,...,...,...
301,LMR3 6Pcs Reusable Universal Super Stretch Foo...,https://m.media-amazon.com/images/I/41yaSZzK4k...,https://www.amazon.in//kitchengram-Skycandle-R...
302,MOXWARE Modular BPA Free Plastic Mixing and Se...,https://m.media-amazon.com/images/I/51DDcmGNcq...,https://www.amazon.in//MOXWARE-Modular-Plastic...
303,"Treo Glass Mixing Bowl with Lid - 440ml, Set o...",https://m.media-amazon.com/images/I/71QHo5vyoP...,https://www.amazon.in//Treo-Milton-Mixing-Bowl...
304,Treo By Milton Mixing Bowl With Eazy Lid Set o...,https://m.media-amazon.com/images/I/71v24UDeMM...,https://www.amazon.in//Treo-Milton-Mixing-Bowl...


In [55]:
BO = df.copy()
BO

,product_name,product_img,product_link
0,Sumeet Stainless Steel SoLid Apple Bowl - 220m...,https://m.media-amazon.com/images/I/81r5+pDM4c...,https://www.amazon.in//Sumeet-Stainless-Katori...
1,PHORE Ceramic Floral Hand Painted Dessert/Dinn...,https://m.media-amazon.com/images/I/41InSpOFM3...,https://www.amazon.in//PHORE-Ceramic-Painted-D...
2,HAZEL Stainless Steel Mixing Bowl | Mixing Bow...,https://m.media-amazon.com/images/I/716G1XVbK6...,https://www.amazon.in//HAZEL-Stainless-Mixing-...
3,Sumeet Stainless Steel Heavy Gauge Solid Bowl ...,https://m.media-amazon.com/images/I/811k8Hluca...,https://www.amazon.in//Sumeet-Stainless-Mirror...
4,"Sumeet Stainless Steel Solid Bowl - 0.350 L, S...",https://m.media-amazon.com/images/I/81thuY18lA...,https://www.amazon.in//Sumeet-Stainless-Steel-...
...,...,...,...
301,HANDICRAFT-PALACE Steel Kitchenware Bowls Beau...,https://m.media-amazon.com/images/I/71JAULpbFR...,https://www.amazon.in//HANDICRAFT-PALACE-Kitch...
302,Sharda Metals Stainless Steel Solid Mini Bowls...,https://m.media-amazon.com/images/I/51ggORYYZW...,https://www.amazon.in//Sharda-Metals-Stainless...
303,Generic Noodle Bowl with Lid and Spoon - Noodl...,https://m.media-amazon.com/images/I/51ojwDl-Ni...,https://www.amazon.in//Generic-Noodle-Bowl-Lid...
304,Sharda Metals Sleek Polished Stainless Steel C...,https://m.media-amazon.com/images/I/71zSP8YQuW...,https://www.amazon.in//Sharda-Metals-Polished-...


In [59]:
BO = BO.append(df).reset_index(drop=True)
BO

,product_name,product_img,product_link
0,Sumeet Stainless Steel SoLid Apple Bowl - 220m...,https://m.media-amazon.com/images/I/81r5+pDM4c...,https://www.amazon.in//Sumeet-Stainless-Katori...
1,PHORE Ceramic Floral Hand Painted Dessert/Dinn...,https://m.media-amazon.com/images/I/41InSpOFM3...,https://www.amazon.in//PHORE-Ceramic-Painted-D...
2,HAZEL Stainless Steel Mixing Bowl | Mixing Bow...,https://m.media-amazon.com/images/I/716G1XVbK6...,https://www.amazon.in//HAZEL-Stainless-Mixing-...
3,Sumeet Stainless Steel Heavy Gauge Solid Bowl ...,https://m.media-amazon.com/images/I/811k8Hluca...,https://www.amazon.in//Sumeet-Stainless-Mirror...
4,"Sumeet Stainless Steel Solid Bowl - 0.350 L, S...",https://m.media-amazon.com/images/I/81thuY18lA...,https://www.amazon.in//Sumeet-Stainless-Steel-...
...,...,...,...
913,LMR3 6Pcs Reusable Universal Super Stretch Foo...,https://m.media-amazon.com/images/I/41yaSZzK4k...,https://www.amazon.in//kitchengram-Skycandle-R...
914,MOXWARE Modular BPA Free Plastic Mixing and Se...,https://m.media-amazon.com/images/I/51DDcmGNcq...,https://www.amazon.in//MOXWARE-Modular-Plastic...
915,"Treo Glass Mixing Bowl with Lid - 440ml, Set o...",https://m.media-amazon.com/images/I/71QHo5vyoP...,https://www.amazon.in//Treo-Milton-Mixing-Bowl...
916,Treo By Milton Mixing Bowl With Eazy Lid Set o...,https://m.media-amazon.com/images/I/71v24UDeMM...,https://www.amazon.in//Treo-Milton-Mixing-Bowl...


In [52]:
product_BO = pd.DataFrame(reviews[reviews['product_code']=='product_BO']).reset_index(drop=True)
product_BO

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2YA18JC0YGZL8,B00004TYBO,5.0,1356825600,product_BO
1,ATAH976DMJNHN,B00004TYBO,5.0,1207612800,product_BO
2,A1DH42DYTFYVOA,B00004TYBO,5.0,1225065600,product_BO
3,A1BD8V3ARDX4NA,B00004Z0BO,4.0,1162684800,product_BO
4,A1KOBN9IRFNF7W,B00004Z0BO,5.0,1121904000,product_BO
...,...,...,...,...,...
9633,A3A3EHOI058999,B00JW56DBO,5.0,1402531200,product_BO
9634,AAEOXQUJNG9NS,B00JZDMVBO,5.0,1404172800,product_BO
9635,A1NSE3A82V9LNO,B00K7MF6BO,5.0,1403913600,product_BO
9636,A2RYPNAZQRE8GI,B00K7MF6BO,4.0,1404777600,product_BO


In [53]:
len(product_BO.product_id.unique())

706

In [60]:
product_BO = pd.DataFrame(reviews[reviews['product_code']=='product_BO']).reset_index(drop=True)
product_BO =  pd.merge(product_BO, 
                       BO.iloc[:len(product_BO.product_id.unique())]\
                       .join(pd.Series(product_BO['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BO.to_csv('product_BO.csv',index=False)
pd.read_csv('product_BO.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2YA18JC0YGZL8,B00004TYBO,5.0,1356825600,product_BO,Sumeet Stainless Steel SoLid Apple Bowl - 220m...,https://m.media-amazon.com/images/I/81r5+pDM4c...,https://www.amazon.in//Sumeet-Stainless-Katori...
1,ATAH976DMJNHN,B00004TYBO,5.0,1207612800,product_BO,Sumeet Stainless Steel SoLid Apple Bowl - 220m...,https://m.media-amazon.com/images/I/81r5+pDM4c...,https://www.amazon.in//Sumeet-Stainless-Katori...
2,A1DH42DYTFYVOA,B00004TYBO,5.0,1225065600,product_BO,Sumeet Stainless Steel SoLid Apple Bowl - 220m...,https://m.media-amazon.com/images/I/81r5+pDM4c...,https://www.amazon.in//Sumeet-Stainless-Katori...
3,A1BD8V3ARDX4NA,B00004Z0BO,4.0,1162684800,product_BO,PHORE Ceramic Floral Hand Painted Dessert/Dinn...,https://m.media-amazon.com/images/I/41InSpOFM3...,https://www.amazon.in//PHORE-Ceramic-Painted-D...
4,A1KOBN9IRFNF7W,B00004Z0BO,5.0,1121904000,product_BO,PHORE Ceramic Floral Hand Painted Dessert/Dinn...,https://m.media-amazon.com/images/I/41InSpOFM3...,https://www.amazon.in//PHORE-Ceramic-Painted-D...
...,...,...,...,...,...,...,...,...
9633,A3A3EHOI058999,B00JW56DBO,5.0,1402531200,product_BO,QERINKLE® Microwave Safe Silicone Stretch Lids...,https://m.media-amazon.com/images/I/61rjAE3Rfe...,https://www.amazon.in//QERINKLE%C2%AE-Microwav...
9634,AAEOXQUJNG9NS,B00JZDMVBO,5.0,1404172800,product_BO,Larah by Borosil Green Herbs Set of 2 Mixing &...,https://m.media-amazon.com/images/I/51FyII2BqP...,https://www.amazon.in//Larah-Borosil-Microwave...
9635,A1NSE3A82V9LNO,B00K7MF6BO,5.0,1403913600,product_BO,S&S QUALITY INTEGRITY INNOVATION 550ml Microwa...,https://m.media-amazon.com/images/I/51o4Pf34vG...,https://www.amazon.in//Microwave-Stainless-Pla...
9636,A2RYPNAZQRE8GI,B00K7MF6BO,4.0,1404777600,product_BO,S&S QUALITY INTEGRITY INNOVATION 550ml Microwa...,https://m.media-amazon.com/images/I/51o4Pf34vG...,https://www.amazon.in//Microwave-Stainless-Pla...


### Product BP dataframe

In [180]:
df = get_scraped('blood pressure checking machine')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Omron HEM 7124 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81JcZIx4qJ...,https://www.amazon.in//Omron-HEM-7124-Blood-Pr...
1,Omron HEM 7120 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81eIfnfHiI...,https://www.amazon.in//Omron-Automatic-Intelli...
2,Dr Trust Smart Dual Talking Automatic Digital ...,https://m.media-amazon.com/images/I/61Wz7zrENc...,https://www.amazon.in//Dr-Trust-Talking-Automa...
3,Dr. Morepen Bp02 Automatic Blood Pressure Moni...,https://m.media-amazon.com/images/I/61PnHkFPUM...,https://www.amazon.in//Dr-Morepen-Automatic-Pr...
4,Omron Most Advance Digital Blood Pressure Moni...,https://m.media-amazon.com/images/I/71TMiB94AS...,https://www.amazon.in//Omron-Pressure-Accuracy...
...,...,...,...
280,BPL Medical Technologies Automatic Blood Press...,https://m.media-amazon.com/images/I/31oBF1AIFR...,https://www.amazon.in//BPL-Medical-Technologie...
281,"Shock-resistant Blood Pressure Monitor Bag, We...",https://m.media-amazon.com/images/I/61zT3GA5FN...,https://www.amazon.in//Shock-resistant-Pressur...
282,Omron CD-CS9 Small D-Ring Cuff for BP710N/BP74...,https://m.media-amazon.com/images/I/41FOxi0koH...,https://www.amazon.in//Omron-CD-CS9-Small-Cuff...
283,Mowell Desktop Electronic Sphygmomanometer Blo...,https://m.media-amazon.com/images/I/61qo-7R8XW...,https://www.amazon.in//Mowell-Desktop-Electron...


In [181]:
BP = df.copy()
BP

,product_name,product_img,product_link
0,Omron HEM 7124 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81JcZIx4qJ...,https://www.amazon.in//Omron-HEM-7124-Blood-Pr...
1,Omron HEM 7120 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81eIfnfHiI...,https://www.amazon.in//Omron-Automatic-Intelli...
2,Dr Trust Smart Dual Talking Automatic Digital ...,https://m.media-amazon.com/images/I/61Wz7zrENc...,https://www.amazon.in//Dr-Trust-Talking-Automa...
3,Dr. Morepen Bp02 Automatic Blood Pressure Moni...,https://m.media-amazon.com/images/I/61PnHkFPUM...,https://www.amazon.in//Dr-Morepen-Automatic-Pr...
4,Omron Most Advance Digital Blood Pressure Moni...,https://m.media-amazon.com/images/I/71TMiB94AS...,https://www.amazon.in//Omron-Pressure-Accuracy...
...,...,...,...
280,BPL Medical Technologies Automatic Blood Press...,https://m.media-amazon.com/images/I/31oBF1AIFR...,https://www.amazon.in//BPL-Medical-Technologie...
281,"Shock-resistant Blood Pressure Monitor Bag, We...",https://m.media-amazon.com/images/I/61zT3GA5FN...,https://www.amazon.in//Shock-resistant-Pressur...
282,Omron CD-CS9 Small D-Ring Cuff for BP710N/BP74...,https://m.media-amazon.com/images/I/41FOxi0koH...,https://www.amazon.in//Omron-CD-CS9-Small-Cuff...
283,Mowell Desktop Electronic Sphygmomanometer Blo...,https://m.media-amazon.com/images/I/61qo-7R8XW...,https://www.amazon.in//Mowell-Desktop-Electron...


In [ ]:
# BP = BP.append(df).reset_index(drop=True)
# BP

In [179]:
product_BP = pd.DataFrame(reviews[reviews['product_code']=='product_BP']).reset_index(drop=True)
product_BP

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1RE2QZZKXREQJ,B00000J4BP,2.0,1139616000,product_BP
1,A1Y5QNZ622N5B3,B00004WCBP,4.0,1019001600,product_BP
2,A23IQKCIU19BDW,B00004WCBP,4.0,1016409600,product_BP
3,A1KUXLF19YQPN6,B00004WCBP,5.0,1248566400,product_BP
4,A19YY1LKXQJSVR,B00004WCBP,1.0,1091577600,product_BP
...,...,...,...,...,...
523,AC43PJUZ95NTS,B0000AHOBP,1.0,1136160000,product_BP
524,A3MYGACI4XV51Q,B0000AKABP,5.0,1197504000,product_BP
525,A18U8BUMK2XEUG,B0000AKABP,5.0,1233360000,product_BP
526,AN4E1CBDBSJSX,B0000AKABP,5.0,1382918400,product_BP


In [53]:
len(product_BP.product_id.unique())

706

In [182]:
product_BP = pd.DataFrame(reviews[reviews['product_code']=='product_BP']).reset_index(drop=True)
product_BP =  pd.merge(product_BP, 
                       BP.iloc[:len(product_BP.product_id.unique())]\
                       .join(pd.Series(product_BP['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BP.to_csv('product_BP.csv',index=False)
pd.read_csv('product_BP.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1RE2QZZKXREQJ,B00000J4BP,2.0,1139616000,product_BP,Omron HEM 7124 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81JcZIx4qJ...,https://www.amazon.in//Omron-HEM-7124-Blood-Pr...
1,A1Y5QNZ622N5B3,B00004WCBP,4.0,1019001600,product_BP,Omron HEM 7120 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81eIfnfHiI...,https://www.amazon.in//Omron-Automatic-Intelli...
2,A23IQKCIU19BDW,B00004WCBP,4.0,1016409600,product_BP,Omron HEM 7120 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81eIfnfHiI...,https://www.amazon.in//Omron-Automatic-Intelli...
3,A1KUXLF19YQPN6,B00004WCBP,5.0,1248566400,product_BP,Omron HEM 7120 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81eIfnfHiI...,https://www.amazon.in//Omron-Automatic-Intelli...
4,A19YY1LKXQJSVR,B00004WCBP,1.0,1091577600,product_BP,Omron HEM 7120 Fully Automatic Digital Blood P...,https://m.media-amazon.com/images/I/81eIfnfHiI...,https://www.amazon.in//Omron-Automatic-Intelli...
...,...,...,...,...,...,...,...,...
523,AC43PJUZ95NTS,B0000AHOBP,1.0,1136160000,product_BP,Rossmax GB102 Aneroid Blood Pressure Monitor (...,https://m.media-amazon.com/images/I/71naP5ODpj...,https://www.amazon.in//Rossmax-GB102-Aneroid-P...
524,A3MYGACI4XV51Q,B0000AKABP,5.0,1197504000,product_BP,AccuSure Blood Pressure Monitor 3 Color Smart ...,https://m.media-amazon.com/images/I/61lZI5eJPl...,https://www.amazon.in//AccuSure-Pressure-Techn...
525,A18U8BUMK2XEUG,B0000AKABP,5.0,1233360000,product_BP,AccuSure Blood Pressure Monitor 3 Color Smart ...,https://m.media-amazon.com/images/I/61lZI5eJPl...,https://www.amazon.in//AccuSure-Pressure-Techn...
526,AN4E1CBDBSJSX,B0000AKABP,5.0,1382918400,product_BP,AccuSure Blood Pressure Monitor 3 Color Smart ...,https://m.media-amazon.com/images/I/61lZI5eJPl...,https://www.amazon.in//AccuSure-Pressure-Techn...


### Product BQ dataframe

In [195]:
df = get_scraped('bbq grill net')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Livzing Portable Barbecue BBQ Grill Net Basket...,https://m.media-amazon.com/images/I/815kivFb8C...,https://www.amazon.in//Livzing-Portable-Barbec...
1,Freshome Mild Steel Portable Barbecue BBQ Gril...,https://m.media-amazon.com/images/I/81tsKFOFIe...,https://www.amazon.in//Freshome-Portable-Barbe...
2,ZINZA Stainless Steel Portable Barbecue BBQ Gr...,https://m.media-amazon.com/images/I/716NP0058m...,https://www.amazon.in//ZINZA-Stainless-Portabl...
3,OMANZA Portable Barbecue BBQ Grill Net Basket ...,https://m.media-amazon.com/images/I/71eOvyp6Mn...,https://www.amazon.in//OMANZA-Portable-Barbecu...
4,VIZORA Mild Steel Portable Barbecue BBQ Grill ...,https://m.media-amazon.com/images/I/61m-FqRzHR...,https://www.amazon.in//VIZORA-Portable-Barbecu...
...,...,...,...
301,Home Bee Barbeque Jali Roti Roast Grill Papad ...,https://m.media-amazon.com/images/I/31AmdMer61...,https://www.amazon.in//Barbeque-Roaster-Chapat...
302,Harshiv Barbeque grilling Basket | Steel Porta...,https://m.media-amazon.com/images/I/417ywZT2LP...,https://www.amazon.in//Barbeque-grilling-Porta...
303,Moradiya Fresh Portable Barbecue Grill Net Bas...,https://m.media-amazon.com/images/I/712FPAuv6T...,https://www.amazon.in//Moradiya-Portable-Barbe...
304,"Gnanex BBQ Grill Net Basket for Fish, Vegetabl...",https://m.media-amazon.com/images/I/61Fp6VUrhq...,https://www.amazon.in//Gnanex-Basket-Vegetable...


In [192]:
BQ = df.copy()
BQ

,product_name,product_img,product_link
0,TOMdoxx 2000W Electric Smokeless Portable Adju...,https://m.media-amazon.com/images/I/81YlQ889QD...,https://www.amazon.in//TOMdoxx-Electric-Temper...
1,AMAZING STUFF 2000W Electric Portable Barbeque...,https://m.media-amazon.com/images/I/81Bhn7QKR+...,https://www.amazon.in//AMAZING-STUFF-Adjustabl...
2,Orbit Electric Barbeque Grill 2000w Tandoori M...,https://m.media-amazon.com/images/I/81IJR4huYs...,https://www.amazon.in//Orbit-Electric-Barbeque...
3,UDEETA MART Electric Portable Barbeque Grill S...,https://m.media-amazon.com/images/I/711hP6rV1F...,https://www.amazon.in//Electric-Smokeless-Adju...
4,Sisliya Powder Coated 2000 Watts Electric Barb...,https://m.media-amazon.com/images/I/61hJZnkxe9...,https://www.amazon.in//Sisliya-Electric-Barbeq...
...,...,...,...
301,KIRFIZ 3 in1 Heart-shaped Egg Omelette Machine...,https://m.media-amazon.com/images/I/51BZauAKz3...,https://www.amazon.in//KIRFIZ-Heart-shaped-Ome...
302,Ritmo Smokeless Indoor Outdoor Electric Grill ...,https://m.media-amazon.com/images/I/61fMcbOZ8Y...,https://www.amazon.in//Ritmo-Smokeless-Adjusta...
303,Wellberg Style Electric Charcoal Barbeque Gril...,https://m.media-amazon.com/images/I/41Pq4Fo1eQ...,https://www.amazon.in//Wellberg-Electric-Charc...
304,Wellberg Style Electric Charcoal Barbeque Gril...,https://m.media-amazon.com/images/I/41Pq4Fo1eQ...,https://www.amazon.in//Wellberg-Electric-Charc...


In [196]:
BQ = BQ.append(df).reset_index(drop=True)
BQ

,product_name,product_img,product_link
0,TOMdoxx 2000W Electric Smokeless Portable Adju...,https://m.media-amazon.com/images/I/81YlQ889QD...,https://www.amazon.in//TOMdoxx-Electric-Temper...
1,AMAZING STUFF 2000W Electric Portable Barbeque...,https://m.media-amazon.com/images/I/81Bhn7QKR+...,https://www.amazon.in//AMAZING-STUFF-Adjustabl...
2,Orbit Electric Barbeque Grill 2000w Tandoori M...,https://m.media-amazon.com/images/I/81IJR4huYs...,https://www.amazon.in//Orbit-Electric-Barbeque...
3,UDEETA MART Electric Portable Barbeque Grill S...,https://m.media-amazon.com/images/I/711hP6rV1F...,https://www.amazon.in//Electric-Smokeless-Adju...
4,Sisliya Powder Coated 2000 Watts Electric Barb...,https://m.media-amazon.com/images/I/61hJZnkxe9...,https://www.amazon.in//Sisliya-Electric-Barbeq...
...,...,...,...
913,Home Bee Barbeque Jali Roti Roast Grill Papad ...,https://m.media-amazon.com/images/I/31AmdMer61...,https://www.amazon.in//Barbeque-Roaster-Chapat...
914,Harshiv Barbeque grilling Basket | Steel Porta...,https://m.media-amazon.com/images/I/417ywZT2LP...,https://www.amazon.in//Barbeque-grilling-Porta...
915,Moradiya Fresh Portable Barbecue Grill Net Bas...,https://m.media-amazon.com/images/I/712FPAuv6T...,https://www.amazon.in//Moradiya-Portable-Barbe...
916,"Gnanex BBQ Grill Net Basket for Fish, Vegetabl...",https://m.media-amazon.com/images/I/61Fp6VUrhq...,https://www.amazon.in//Gnanex-Basket-Vegetable...


In [185]:
product_BQ = pd.DataFrame(reviews[reviews['product_code']=='product_BQ']).reset_index(drop=True)
product_BQ

,user_id,product_id,Ratings,Time_stamp,product_code
0,A36GF79DI8HD5J,B00003CWBQ,5.0,948844800,product_BQ
1,A3FMF9QZ8RFMZ8,B00004Z0BQ,3.0,1071792000,product_BQ
2,ANLS8YJVVBZ5F,B00004Z0BQ,3.0,1072051200,product_BQ
3,A19WBNIMDXXW51,B00004Z0BQ,4.0,1063065600,product_BQ
4,A2MYRC7PQGC41V,B00004Z0BQ,2.0,1174780800,product_BQ
...,...,...,...,...,...
14178,A1IX1VPX2NQWRJ,B00KJGYOBQ,5.0,1404259200,product_BQ
14179,AC041JY54KEGC,B00KJGYOBQ,5.0,1403308800,product_BQ
14180,A2XKMBOPK6DI5T,B00KJGYOBQ,5.0,1405209600,product_BQ
14181,A31Z5G3FRNVEW5,B00KJGYOBQ,4.0,1405382400,product_BQ


In [186]:
len(product_BQ.product_id.unique())

687

In [197]:
product_BQ = pd.DataFrame(reviews[reviews['product_code']=='product_BQ']).reset_index(drop=True)
product_BQ =  pd.merge(product_BQ, 
                       BQ.iloc[:len(product_BQ.product_id.unique())]\
                       .join(pd.Series(product_BQ['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BQ.to_csv('product_BQ.csv',index=False)
pd.read_csv('product_BQ.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A36GF79DI8HD5J,B00003CWBQ,5.0,948844800,product_BQ,TOMdoxx 2000W Electric Smokeless Portable Adju...,https://m.media-amazon.com/images/I/81YlQ889QD...,https://www.amazon.in//TOMdoxx-Electric-Temper...
1,A3FMF9QZ8RFMZ8,B00004Z0BQ,3.0,1071792000,product_BQ,AMAZING STUFF 2000W Electric Portable Barbeque...,https://m.media-amazon.com/images/I/81Bhn7QKR+...,https://www.amazon.in//AMAZING-STUFF-Adjustabl...
2,ANLS8YJVVBZ5F,B00004Z0BQ,3.0,1072051200,product_BQ,AMAZING STUFF 2000W Electric Portable Barbeque...,https://m.media-amazon.com/images/I/81Bhn7QKR+...,https://www.amazon.in//AMAZING-STUFF-Adjustabl...
3,A19WBNIMDXXW51,B00004Z0BQ,4.0,1063065600,product_BQ,AMAZING STUFF 2000W Electric Portable Barbeque...,https://m.media-amazon.com/images/I/81Bhn7QKR+...,https://www.amazon.in//AMAZING-STUFF-Adjustabl...
4,A2MYRC7PQGC41V,B00004Z0BQ,2.0,1174780800,product_BQ,AMAZING STUFF 2000W Electric Portable Barbeque...,https://m.media-amazon.com/images/I/81Bhn7QKR+...,https://www.amazon.in//AMAZING-STUFF-Adjustabl...
...,...,...,...,...,...,...,...,...
14178,A1IX1VPX2NQWRJ,B00KJGYOBQ,5.0,1404259200,product_BQ,Replex Steel Plating Barbecue Grill Net Basket...,https://m.media-amazon.com/images/I/71sQVwzcm2...,https://www.amazon.in//Replex-Plating-Barbecue...
14179,AC041JY54KEGC,B00KJGYOBQ,5.0,1403308800,product_BQ,Replex Steel Plating Barbecue Grill Net Basket...,https://m.media-amazon.com/images/I/71sQVwzcm2...,https://www.amazon.in//Replex-Plating-Barbecue...
14180,A2XKMBOPK6DI5T,B00KJGYOBQ,5.0,1405209600,product_BQ,Replex Steel Plating Barbecue Grill Net Basket...,https://m.media-amazon.com/images/I/71sQVwzcm2...,https://www.amazon.in//Replex-Plating-Barbecue...
14181,A31Z5G3FRNVEW5,B00KJGYOBQ,4.0,1405382400,product_BQ,Replex Steel Plating Barbecue Grill Net Basket...,https://m.media-amazon.com/images/I/71sQVwzcm2...,https://www.amazon.in//Replex-Plating-Barbecue...


### Product BR dataframe

In [200]:
df = get_scraped('brush pen')
df

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
1,"Camlin 4019272 Kokuyo Brush Pen, 12 Shades (Mu...",https://m.media-amazon.com/images/I/7135Wmtesc...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
2,"Doms Brush Pens- 14 Shades, Multicolor (DM8137a)",https://m.media-amazon.com/images/I/612tdy9Qd+...,https://www.amazon.in//DOMS-Brush-PENS/dp/B08B...
3,Pathos India 24 Colors Dual Tip Brush Pens Fin...,https://m.media-amazon.com/images/I/81-5gzAssd...,https://www.amazon.in//Dual-Fineliners-Waterco...
4,Vishal 3 Sizes Water Brush Pen for Watercolor ...,https://m.media-amazon.com/images/I/51Sqyb6hxl...,https://www.amazon.in//Vishal-Watercolor-Calli...
...,...,...,...
301,Pentel Brush Sign Pen SES15C (Turquoise Green),https://m.media-amazon.com/images/I/51kmj3KaKS...,https://www.amazon.in//Pentel-Brush-SES15C-Tur...
302,PENTEL ARTS XGFL Colour Brush Pen-Pale Orange,https://m.media-amazon.com/images/I/71OuuBskYh...,https://www.amazon.in//PENTEL-Colour-Brush-Pen...
303,"Levin 4 size Writing, Signature, Drawing,Sketc...",https://m.media-amazon.com/images/I/61ckfBXsu1...,https://www.amazon.in//Levin-Writing-Signature...
304,"Levin Water Brush Pens, Premium Nylon Brushs S...",https://m.media-amazon.com/images/I/61ha1D2wf3...,https://www.amazon.in//Levin-Painting-Watercol...


In [201]:
BR = df.copy()
BR

,product_name,product_img,product_link
0,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
1,"Camlin 4019272 Kokuyo Brush Pen, 12 Shades (Mu...",https://m.media-amazon.com/images/I/7135Wmtesc...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
2,"Doms Brush Pens- 14 Shades, Multicolor (DM8137a)",https://m.media-amazon.com/images/I/612tdy9Qd+...,https://www.amazon.in//DOMS-Brush-PENS/dp/B08B...
3,Pathos India 24 Colors Dual Tip Brush Pens Fin...,https://m.media-amazon.com/images/I/81-5gzAssd...,https://www.amazon.in//Dual-Fineliners-Waterco...
4,Vishal 3 Sizes Water Brush Pen for Watercolor ...,https://m.media-amazon.com/images/I/51Sqyb6hxl...,https://www.amazon.in//Vishal-Watercolor-Calli...
...,...,...,...
301,Pentel Brush Sign Pen SES15C (Turquoise Green),https://m.media-amazon.com/images/I/51kmj3KaKS...,https://www.amazon.in//Pentel-Brush-SES15C-Tur...
302,PENTEL ARTS XGFL Colour Brush Pen-Pale Orange,https://m.media-amazon.com/images/I/71OuuBskYh...,https://www.amazon.in//PENTEL-Colour-Brush-Pen...
303,"Levin 4 size Writing, Signature, Drawing,Sketc...",https://m.media-amazon.com/images/I/61ckfBXsu1...,https://www.amazon.in//Levin-Writing-Signature...
304,"Levin Water Brush Pens, Premium Nylon Brushs S...",https://m.media-amazon.com/images/I/61ha1D2wf3...,https://www.amazon.in//Levin-Painting-Watercol...


In [ ]:
# BR = BR.append(df).reset_index(drop=True)
# BR

In [198]:
product_BR = pd.DataFrame(reviews[reviews['product_code']=='product_BR']).reset_index(drop=True)
product_BR

,user_id,product_id,Ratings,Time_stamp,product_code
0,AUXTH8T9QRNN6,B00002EQBR,5.0,1201737600,product_BR
1,A3NRM2VALN8NT7,B00002EQBR,5.0,1043280000,product_BR
2,ANH7R4JHOGTH,B00002EQBR,5.0,1114905600,product_BR
3,A379OJA9HIGER3,B00002EQBR,5.0,1017619200,product_BR
4,ACT9T2DUBC8W8,B00002EQBR,5.0,1073520000,product_BR
...,...,...,...,...,...
123,A3UZR6DNZE51J9,B0000D90BR,4.0,1075680000,product_BR
124,A3UT0254HXNUO7,B0000D90BR,5.0,1093651200,product_BR
125,A11A8GWG0IXBZH,B0000D90BR,5.0,1070496000,product_BR
126,ADELY2BKK4GJ0,B0000D90BR,5.0,1102982400,product_BR


In [199]:
len(product_BR.product_id.unique())

24

In [202]:
product_BR = pd.DataFrame(reviews[reviews['product_code']=='product_BR']).reset_index(drop=True)
product_BR =  pd.merge(product_BR, 
                       BR.iloc[:len(product_BR.product_id.unique())]\
                       .join(pd.Series(product_BR['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BR.to_csv('product_BR.csv',index=False)
pd.read_csv('product_BR.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,AUXTH8T9QRNN6,B00002EQBR,5.0,1201737600,product_BR,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
1,A3NRM2VALN8NT7,B00002EQBR,5.0,1043280000,product_BR,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
2,ANH7R4JHOGTH,B00002EQBR,5.0,1114905600,product_BR,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
3,A379OJA9HIGER3,B00002EQBR,5.0,1017619200,product_BR,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
4,ACT9T2DUBC8W8,B00002EQBR,5.0,1073520000,product_BR,"Camlin Kokuyo Brush Pens, 24 Shades (Multicolo...",https://m.media-amazon.com/images/I/81biip6Sxz...,https://www.amazon.in//Camlin-Kokuyo-Brush-Sha...
...,...,...,...,...,...,...,...,...
123,A3UZR6DNZE51J9,B0000D90BR,4.0,1075680000,product_BR,"Levin Brush Pen, 6 pcs Water Colour Brush, Wat...",https://m.media-amazon.com/images/I/71keDHsCO0...,https://www.amazon.in//Levin-Watercolor-Callig...
124,A3UT0254HXNUO7,B0000D90BR,5.0,1093651200,product_BR,"Levin Brush Pen, 6 pcs Water Colour Brush, Wat...",https://m.media-amazon.com/images/I/71keDHsCO0...,https://www.amazon.in//Levin-Watercolor-Callig...
125,A11A8GWG0IXBZH,B0000D90BR,5.0,1070496000,product_BR,"Levin Brush Pen, 6 pcs Water Colour Brush, Wat...",https://m.media-amazon.com/images/I/71keDHsCO0...,https://www.amazon.in//Levin-Watercolor-Callig...
126,ADELY2BKK4GJ0,B0000D90BR,5.0,1102982400,product_BR,"Levin Brush Pen, 6 pcs Water Colour Brush, Wat...",https://m.media-amazon.com/images/I/71keDHsCO0...,https://www.amazon.in//Levin-Watercolor-Callig...


### Product BT dataframe

In [12]:
df = get_scraped('bts bag')
df

 15%|████████████▊                                                                      | 6/39 [00:12<01:09,  2.11s/it]


,product_name,product_img,product_link
0,Kpop BTS Bangtan Boys Casual Backpack Daypack ...,https://m.media-amazon.com/images/I/61Smyg9Vr3...,https://www.amazon.in//Fancyku-Bangtan-Backpac...
1,BTS School Backpack Kpop Theme BTS Bangtan Gir...,https://m.media-amazon.com/images/I/412gcFpVjY...,https://www.amazon.in//PALAY%C2%AE-Backpack-Ba...
2,BTS Teens School Backpack for Women College Gi...,https://m.media-amazon.com/images/I/61IguiMbln...,https://www.amazon.in//Backpack-College-Bookba...
3,GOODWAVE® School Backpack Kpop Theme BTS Bangt...,https://m.media-amazon.com/images/I/31QkTAzRG0...,https://www.amazon.in//GOODWAVE%C2%AE-Backpack...
4,Kpop BTS Gradient Cute Rabbit Ear Waterproof B...,https://m.media-amazon.com/images/I/51DVeXe5vL...,https://www.amazon.in//Gradient-Waterproof-Bac...
...,...,...,...
301,BTS Bangtan Boys Kpop Fan Art Printed Laptop B...,https://m.media-amazon.com/images/I/41NIbrU34V...,https://www.amazon.in//HeartInk-Bangtan-Printe...
302,AirCase Vegan Leather Laptop Sleeve with Handl...,https://m.media-amazon.com/images/I/518UqU+fY2...,https://www.amazon.in//AirCase-14-1-Inch-Think...
303,BTS Bangtan Boys KPOP Theme Fan Art Rose Gold ...,https://m.media-amazon.com/images/I/61dmUNz1Y-...,https://www.amazon.in//HeartInk-Bangtan-Theme-...
304,Office Laptop Vegan Leather Executive Formal 1...,https://m.media-amazon.com/images/I/91Uk9gQMvz...,https://www.amazon.in//ZIPLINE-Synthetic-Execu...


In [13]:
BT = df.copy()
BT

,product_name,product_img,product_link
0,Kpop BTS Bangtan Boys Casual Backpack Daypack ...,https://m.media-amazon.com/images/I/61Smyg9Vr3...,https://www.amazon.in//Fancyku-Bangtan-Backpac...
1,BTS School Backpack Kpop Theme BTS Bangtan Gir...,https://m.media-amazon.com/images/I/412gcFpVjY...,https://www.amazon.in//PALAY%C2%AE-Backpack-Ba...
2,BTS Teens School Backpack for Women College Gi...,https://m.media-amazon.com/images/I/61IguiMbln...,https://www.amazon.in//Backpack-College-Bookba...
3,GOODWAVE® School Backpack Kpop Theme BTS Bangt...,https://m.media-amazon.com/images/I/31QkTAzRG0...,https://www.amazon.in//GOODWAVE%C2%AE-Backpack...
4,Kpop BTS Gradient Cute Rabbit Ear Waterproof B...,https://m.media-amazon.com/images/I/51DVeXe5vL...,https://www.amazon.in//Gradient-Waterproof-Bac...
...,...,...,...
301,BTS Bangtan Boys Kpop Fan Art Printed Laptop B...,https://m.media-amazon.com/images/I/41NIbrU34V...,https://www.amazon.in//HeartInk-Bangtan-Printe...
302,AirCase Vegan Leather Laptop Sleeve with Handl...,https://m.media-amazon.com/images/I/518UqU+fY2...,https://www.amazon.in//AirCase-14-1-Inch-Think...
303,BTS Bangtan Boys KPOP Theme Fan Art Rose Gold ...,https://m.media-amazon.com/images/I/61dmUNz1Y-...,https://www.amazon.in//HeartInk-Bangtan-Theme-...
304,Office Laptop Vegan Leather Executive Formal 1...,https://m.media-amazon.com/images/I/91Uk9gQMvz...,https://www.amazon.in//ZIPLINE-Synthetic-Execu...


In [ ]:
# BT = BT.append(df).reset_index(drop=True)
# BT

In [10]:
product_BT = pd.DataFrame(reviews[reviews['product_code']=='product_BT']).reset_index(drop=True)
product_BT

,user_id,product_id,Ratings,Time_stamp,product_code
0,A6TBHW5LNOHRJ,B00002EQBT,5.0,959558400,product_BT
1,AISMWE2H33VWM,B00002EQBT,5.0,951696000,product_BT
2,AQTPZGVCD2U1,B00002EQBT,5.0,954201600,product_BT
3,A340LIADLJ165T,B00002EQBT,5.0,953596800,product_BT
4,A3QIZEQ74K1I6Z,B00004WCBT,4.0,977270400,product_BT
...,...,...,...,...,...
774,A45TEJY3SJQAL,B0000BZLBT,5.0,1270252800,product_BT
775,A3SKJ70P46YFN5,B0000DB4BT,1.0,1252195200,product_BT
776,A1PM8ZYEAO3W7O,B0000DB4BT,1.0,1204070400,product_BT
777,AN23NX3RXES13,B0000DB4BT,5.0,1155513600,product_BT


In [11]:
len(product_BT.product_id.unique())

21

In [14]:
product_BT = pd.DataFrame(reviews[reviews['product_code']=='product_BT']).reset_index(drop=True)
product_BT =  pd.merge(product_BT, 
                       BT.iloc[:len(product_BT.product_id.unique())]\
                       .join(pd.Series(product_BT['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BT.to_csv('product_BT.csv',index=False)
pd.read_csv('product_BT.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A6TBHW5LNOHRJ,B00002EQBT,5.0,959558400,product_BT,Kpop BTS Bangtan Boys Casual Backpack Daypack ...,https://m.media-amazon.com/images/I/61Smyg9Vr3...,https://www.amazon.in//Fancyku-Bangtan-Backpac...
1,AISMWE2H33VWM,B00002EQBT,5.0,951696000,product_BT,Kpop BTS Bangtan Boys Casual Backpack Daypack ...,https://m.media-amazon.com/images/I/61Smyg9Vr3...,https://www.amazon.in//Fancyku-Bangtan-Backpac...
2,AQTPZGVCD2U1,B00002EQBT,5.0,954201600,product_BT,Kpop BTS Bangtan Boys Casual Backpack Daypack ...,https://m.media-amazon.com/images/I/61Smyg9Vr3...,https://www.amazon.in//Fancyku-Bangtan-Backpac...
3,A340LIADLJ165T,B00002EQBT,5.0,953596800,product_BT,Kpop BTS Bangtan Boys Casual Backpack Daypack ...,https://m.media-amazon.com/images/I/61Smyg9Vr3...,https://www.amazon.in//Fancyku-Bangtan-Backpac...
4,A3QIZEQ74K1I6Z,B00004WCBT,4.0,977270400,product_BT,BTS School Backpack Kpop Theme BTS Bangtan Gir...,https://m.media-amazon.com/images/I/412gcFpVjY...,https://www.amazon.in//PALAY%C2%AE-Backpack-Ba...
...,...,...,...,...,...,...,...,...
774,A45TEJY3SJQAL,B0000BZLBT,5.0,1270252800,product_BT,35 ltrs(45 Cms)BACKPACK (BB-001 Black ),https://m.media-amazon.com/images/I/51efyngSZy...,https://www.amazon.in//NewYouth-Bookbag-Backpa...
775,A3SKJ70P46YFN5,B0000DB4BT,1.0,1252195200,product_BT,BTS Bags for Boys School Backpack Kpop Bangtan...,https://m.media-amazon.com/images/I/51nols3EEK...,https://www.amazon.in//Backpack-Bangtan-Daypac...
776,A1PM8ZYEAO3W7O,B0000DB4BT,1.0,1204070400,product_BT,BTS Bags for Boys School Backpack Kpop Bangtan...,https://m.media-amazon.com/images/I/51nols3EEK...,https://www.amazon.in//Backpack-Bangtan-Daypac...
777,AN23NX3RXES13,B0000DB4BT,5.0,1155513600,product_BT,BTS Bags for Boys School Backpack Kpop Bangtan...,https://m.media-amazon.com/images/I/51nols3EEK...,https://www.amazon.in//Backpack-Bangtan-Daypac...


### Product BU dataframe

In [22]:
df = get_scraped('bubble gum')
df

 15%|████████████▊                                                                      | 6/39 [00:10<00:56,  1.70s/it]


,product_name,product_img,product_link
0,Fusen gum Bubble Gum,https://m.media-amazon.com/images/I/71BzTQQo2u...,https://www.amazon.in//Fusen-gum-Bubble-Gum/dp...
1,"Marukawa Fusen Bubble Gum 60 Pcs Box, 249g",https://m.media-amazon.com/images/I/71BzTQQo2u...,https://www.amazon.in//Marukawa-Fusen-Bubble-G...
2,"Trident Bubble Gum, 14 Sticks, 20 g (Pack of 1)",https://m.media-amazon.com/images/I/51shganl4c...,https://www.amazon.in//Trident-Xylitol-Bubble-...
3,Wrigley's Hubba Bubba Bubble Tape Fancy Fruit ...,https://m.media-amazon.com/images/I/81CQc5EyXc...,https://www.amazon.in//Wrigleys-Hubba-Bubba-Bu...
4,Hubba Bubba Chunky and Bubbly Bubble Gum Origi...,https://m.media-amazon.com/images/I/71I9ivRF+s...,https://www.amazon.in//Hubba-Bubba-Chunky-Orig...
...,...,...,...
301,WRIGLEYS 5 Sugar-free Gum Mint Variety Box - 1...,https://m.media-amazon.com/images/I/61F+5xF64M...,https://www.amazon.in//WRIGLEYS-Sugar-free-Gum...
302,NATURE YARD Babool Gond for Ladoo Making - 250...,https://m.media-amazon.com/images/I/71D78NU45Q...,https://www.amazon.in//NATURE-YARD-Babool-Lado...
303,Tripathi Products Xanthan Gum (100 Grams),https://m.media-amazon.com/images/I/61Iei3NHur...,https://www.amazon.in//Tripathi-Products-Xanth...
304,"Spearmint : Orbit Spearmint Sugarfree Gum, 14 ...",https://m.media-amazon.com/images/I/71L0aRkOAQ...,https://www.amazon.in//Spearmint-Orbit-Sugarfr...


In [19]:
BU = df.copy()
BU

,product_name,product_img,product_link
0,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
1,PLASTO WORLD Bubbles Gatling Machine 8-Hole Ba...,https://m.media-amazon.com/images/I/81slnTf2UH...,https://www.amazon.in//PLASTO-WORLD-Operated-E...
2,"TECHERE Bubble Gun, Gatlin Bubble Machine with...",https://m.media-amazon.com/images/I/51btY+fcGr...,https://www.amazon.in//TECHERE-Machine-25-Hole...
3,First Play 8-Hole Electric Bubbles Gun for Tod...,https://m.media-amazon.com/images/I/51YifgGtH3...,https://www.amazon.in//Electric-Gun-Toddlers-B...
4,"GURBIK Gatling Bubble Machine Gun, 8 Hole Auto...",https://m.media-amazon.com/images/I/61FdNsNCXV...,https://www.amazon.in//GURBIK-Automatic-Outdoo...
...,...,...,...
301,SHENKY Bubble Gun with Light & Music | Bubble ...,https://m.media-amazon.com/images/I/61P-58ZZUp...,https://www.amazon.in//SHENKY-Automatic-Lighti...
302,"BROWNWAVE Gatling Bubble Machine Gun, 8 Hole A...",https://m.media-amazon.com/images/I/7135CcWTab...,https://www.amazon.in//BROWNWAVE-Automatic-Out...
303,32 Holes Big Bubbles Gatling Machine Battery O...,https://m.media-amazon.com/images/I/51lLFtiHPy...,https://www.amazon.in//Operated-Electric-Toy-T...
304,BabyGo Gatling Bubble Machine Gun Outdoor & In...,https://m.media-amazon.com/images/I/51LXwq9lfW...,https://www.amazon.in//BabyGo-Gatling-Machine-...


In [23]:
BU = BU.append(df).reset_index(drop=True)
BU

,product_name,product_img,product_link
0,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
1,PLASTO WORLD Bubbles Gatling Machine 8-Hole Ba...,https://m.media-amazon.com/images/I/81slnTf2UH...,https://www.amazon.in//PLASTO-WORLD-Operated-E...
2,"TECHERE Bubble Gun, Gatlin Bubble Machine with...",https://m.media-amazon.com/images/I/51btY+fcGr...,https://www.amazon.in//TECHERE-Machine-25-Hole...
3,First Play 8-Hole Electric Bubbles Gun for Tod...,https://m.media-amazon.com/images/I/51YifgGtH3...,https://www.amazon.in//Electric-Gun-Toddlers-B...
4,"GURBIK Gatling Bubble Machine Gun, 8 Hole Auto...",https://m.media-amazon.com/images/I/61FdNsNCXV...,https://www.amazon.in//GURBIK-Automatic-Outdoo...
...,...,...,...
913,WRIGLEYS 5 Sugar-free Gum Mint Variety Box - 1...,https://m.media-amazon.com/images/I/61F+5xF64M...,https://www.amazon.in//WRIGLEYS-Sugar-free-Gum...
914,NATURE YARD Babool Gond for Ladoo Making - 250...,https://m.media-amazon.com/images/I/71D78NU45Q...,https://www.amazon.in//NATURE-YARD-Babool-Lado...
915,Tripathi Products Xanthan Gum (100 Grams),https://m.media-amazon.com/images/I/61Iei3NHur...,https://www.amazon.in//Tripathi-Products-Xanth...
916,"Spearmint : Orbit Spearmint Sugarfree Gum, 14 ...",https://m.media-amazon.com/images/I/71L0aRkOAQ...,https://www.amazon.in//Spearmint-Orbit-Sugarfr...


In [16]:
product_BU = pd.DataFrame(reviews[reviews['product_code']=='product_BU']).reset_index(drop=True)
product_BU

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1BMHFWEZH0FAB,B00002EQBU,5.0,1033084800,product_BU
1,A3TP0YU6KALPVP,B00002EQBU,5.0,958867200,product_BU
2,A2GJOQUYFF07HC,B00002EQBU,5.0,961804800,product_BU
3,AUXTH8T9QRNN6,B00002EQBU,5.0,1268006400,product_BU
4,A1D5WG289X5GVE,B00002EQBU,5.0,1153699200,product_BU
...,...,...,...,...,...
11813,A2BVBRNHIK0E4E,B00KC2JVBU,1.0,1404086400,product_BU
11814,A25JALI19RM4LY,B00KC2JVBU,5.0,1404777600,product_BU
11815,A3LYR92ALM5AB2,B00KC2JVBU,5.0,1404172800,product_BU
11816,A1K08DMDVNJDQ9,B00KC2JVBU,5.0,1401753600,product_BU


In [17]:
len(product_BU.product_id.unique())

729

In [24]:
product_BU = pd.DataFrame(reviews[reviews['product_code']=='product_BU']).reset_index(drop=True)
product_BU =  pd.merge(product_BU, 
                       BU.iloc[:len(product_BU.product_id.unique())]\
                       .join(pd.Series(product_BU['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BU.to_csv('product_BU.csv',index=False)
pd.read_csv('product_BU.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1BMHFWEZH0FAB,B00002EQBU,5.0,1033084800,product_BU,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
1,A3TP0YU6KALPVP,B00002EQBU,5.0,958867200,product_BU,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
2,A2GJOQUYFF07HC,B00002EQBU,5.0,961804800,product_BU,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
3,AUXTH8T9QRNN6,B00002EQBU,5.0,1268006400,product_BU,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
4,A1D5WG289X5GVE,B00002EQBU,5.0,1153699200,product_BU,First Play 25-Hole Electric Rocket Bubbles Gun...,https://m.media-amazon.com/images/I/51nquKGxPx...,https://www.amazon.in//Electric-Gun-Toddlers-G...
...,...,...,...,...,...,...,...,...
11813,A2BVBRNHIK0E4E,B00KC2JVBU,1.0,1404086400,product_BU,Wrigley's Extra Sweet Watermelon Sugarfree Gum...,https://m.media-amazon.com/images/I/61A23ftaKT...,https://www.amazon.in//Wrigleys-Extra-Watermel...
11814,A25JALI19RM4LY,B00KC2JVBU,5.0,1404777600,product_BU,Wrigley's Extra Sweet Watermelon Sugarfree Gum...,https://m.media-amazon.com/images/I/61A23ftaKT...,https://www.amazon.in//Wrigleys-Extra-Watermel...
11815,A3LYR92ALM5AB2,B00KC2JVBU,5.0,1404172800,product_BU,Wrigley's Extra Sweet Watermelon Sugarfree Gum...,https://m.media-amazon.com/images/I/61A23ftaKT...,https://www.amazon.in//Wrigleys-Extra-Watermel...
11816,A1K08DMDVNJDQ9,B00KC2JVBU,5.0,1401753600,product_BU,Wrigley's Extra Sweet Watermelon Sugarfree Gum...,https://m.media-amazon.com/images/I/61A23ftaKT...,https://www.amazon.in//Wrigleys-Extra-Watermel...


### Product BW dataframe

In [59]:
df = get_scraped('body weighing scale')
df

 15%|████████████▊                                                                      | 6/39 [00:09<00:54,  1.65s/it]


,product_name,product_img,product_link
0,Hoffen (India) Digital Electronic LCD Personal...,https://m.media-amazon.com/images/I/61wrwvM1a6...,https://www.amazon.in//Hoffen-Digital-Elecroni...
1,beatXP Wave Digital Bathroom Weighing Scale wi...,https://m.media-amazon.com/images/I/61geSK9K2y...,https://www.amazon.in//beatXP-Wave-Bathroom-We...
2,beatXP Floral Digital Bathroom Weighing Scale ...,https://m.media-amazon.com/images/I/71VQE4ocHD...,https://www.amazon.in//beatXP-Bathroom-Weighin...
3,MEDITIVE Bluetooth Digital BMI Weight Scale wi...,https://m.media-amazon.com/images/I/71v5o3vzuJ...,https://www.amazon.in//MEDITIVE-Bluetooth-Digi...
4,"HealthSense Weight Machine for Body Weight, Di...",https://m.media-amazon.com/images/I/61+T+tuO3-...,https://www.amazon.in//Health-Sense-Ultra-Lite...
...,...,...,...
301,JM SELLER Electronic Thick Tempered Glass LCD ...,https://m.media-amazon.com/images/I/61QUIYErGo...,https://www.amazon.in//JM-SELLER-Electronic-Te...
302,MEDITIVE Digital Human Weighing Scale for Body...,https://m.media-amazon.com/images/I/512RO2W6nP...,https://www.amazon.in//MEDITIVE-Digital-Weighi...
303,Dr. Odin Electronic Personal Digital Weighing ...,https://m.media-amazon.com/images/I/71hhumg5+n...,https://www.amazon.in//Dr-Odin-Electronic-Tech...
304,"Zyomatiq Digital Electronic Body Weight Scale,...",https://m.media-amazon.com/images/I/51fa2WARQ6...,https://www.amazon.in//Zyomatiq-Electronic-Per...


In [56]:
BW = df.copy()
BW

,product_name,product_img,product_link
0,"Dove Deeply Nourishing Body Wash, With Exfolia...",https://m.media-amazon.com/images/I/419w4PBH11...,https://www.amazon.in//Dove-Deeply-Nourishing-...
1,Bodywise 5% Lactic Acid Body Lotion- 200ml & B...,https://m.media-amazon.com/images/I/61qwyTfaJq...,https://www.amazon.in//Bodywise-Lotion-Bodywas...
2,Pears Naturale Brightening Pomegranate Bodywas...,https://m.media-amazon.com/images/I/61Jtp1HqzQ...,https://www.amazon.in//Pears-Naturale-Brighten...
3,"Palmolive Aroma Absolute Relax Body Wash, 750m...",https://m.media-amazon.com/images/I/61Rq0wFz7L...,https://www.amazon.in//Palmolive-Aroma-Therapy...
4,Lux Body Wash Soft Skin French Rose & Almond O...,https://m.media-amazon.com/images/I/515OxbUeCB...,https://www.amazon.in//Lux-SuperSaver-Lasting-...
...,...,...,...
301,St. Ives Hydrating Body Wash/ Shower gel for w...,https://m.media-amazon.com/images/I/51zZkXqaz0...,https://www.amazon.in//St-Ives-Hydrating-Moist...
302,KHADI NATURAL Sandal and Turmeric Herbal Body ...,https://m.media-amazon.com/images/I/41hTjXnx6b...,https://www.amazon.in//Khadi-Natural-Sandal-Tu...
303,Biotique Advanced Organics Clarity Cleanser Cl...,https://m.media-amazon.com/images/I/61sH3vM6tf...,https://www.amazon.in//Biotique-Advanced-Organ...
304,Kimiska 1% Salicylic Acid Body Wash - Enriched...,https://m.media-amazon.com/images/I/61k9nw3Xlf...,https://www.amazon.in//Kimiska-Salicylic-Acid-...


In [60]:
BW = BW.append(df).reset_index(drop=True)
BW

,product_name,product_img,product_link
0,"Dove Deeply Nourishing Body Wash, With Exfolia...",https://m.media-amazon.com/images/I/419w4PBH11...,https://www.amazon.in//Dove-Deeply-Nourishing-...
1,Bodywise 5% Lactic Acid Body Lotion- 200ml & B...,https://m.media-amazon.com/images/I/61qwyTfaJq...,https://www.amazon.in//Bodywise-Lotion-Bodywas...
2,Pears Naturale Brightening Pomegranate Bodywas...,https://m.media-amazon.com/images/I/61Jtp1HqzQ...,https://www.amazon.in//Pears-Naturale-Brighten...
3,"Palmolive Aroma Absolute Relax Body Wash, 750m...",https://m.media-amazon.com/images/I/61Rq0wFz7L...,https://www.amazon.in//Palmolive-Aroma-Therapy...
4,Lux Body Wash Soft Skin French Rose & Almond O...,https://m.media-amazon.com/images/I/515OxbUeCB...,https://www.amazon.in//Lux-SuperSaver-Lasting-...
...,...,...,...
913,JM SELLER Electronic Thick Tempered Glass LCD ...,https://m.media-amazon.com/images/I/61QUIYErGo...,https://www.amazon.in//JM-SELLER-Electronic-Te...
914,MEDITIVE Digital Human Weighing Scale for Body...,https://m.media-amazon.com/images/I/512RO2W6nP...,https://www.amazon.in//MEDITIVE-Digital-Weighi...
915,Dr. Odin Electronic Personal Digital Weighing ...,https://m.media-amazon.com/images/I/71hhumg5+n...,https://www.amazon.in//Dr-Odin-Electronic-Tech...
916,"Zyomatiq Digital Electronic Body Weight Scale,...",https://m.media-amazon.com/images/I/51fa2WARQ6...,https://www.amazon.in//Zyomatiq-Electronic-Per...


In [32]:
product_BW = pd.DataFrame(reviews[reviews['product_code']=='product_BW']).reset_index(drop=True)
product_BW

,user_id,product_id,Ratings,Time_stamp,product_code
0,ANWQXHNGBIRBK,B00002EQBW,2.0,980985600,product_BW
1,A3FTNG39Y1FXAJ,B00002EQBW,4.0,960940800,product_BW
2,A3GAB6L55URQ8V,B00002EQBW,5.0,1001980800,product_BW
3,A27TC4PZ9OQN3Y,B00002EQBW,5.0,970531200,product_BW
4,A4FY8L6M7NL6K,B00003CWBW,1.0,1070841600,product_BW
...,...,...,...,...,...
11534,ACASVMQYUPGQH,B00JCCWMBW,5.0,1403049600,product_BW
11535,A1ZOT6YBD18M3O,B00JCCWMBW,5.0,1405900800,product_BW
11536,A2N86ONDC5AFEL,B00JO6KNBW,4.0,1404691200,product_BW
11537,A253CDOMI1O4GE,B00LBEUBBW,5.0,1404950400,product_BW


In [33]:
len(product_BW.product_id.unique())

696

In [61]:
product_BW = pd.DataFrame(reviews[reviews['product_code']=='product_BW']).reset_index(drop=True)
product_BW =  pd.merge(product_BW, 
                       BW.iloc[:len(product_BW.product_id.unique())]\
                       .join(pd.Series(product_BW['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BW.to_csv('product_BW.csv',index=False)
pd.read_csv('product_BW.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ANWQXHNGBIRBK,B00002EQBW,2.0,980985600,product_BW,"Dove Deeply Nourishing Body Wash, With Exfolia...",https://m.media-amazon.com/images/I/419w4PBH11...,https://www.amazon.in//Dove-Deeply-Nourishing-...
1,A3FTNG39Y1FXAJ,B00002EQBW,4.0,960940800,product_BW,"Dove Deeply Nourishing Body Wash, With Exfolia...",https://m.media-amazon.com/images/I/419w4PBH11...,https://www.amazon.in//Dove-Deeply-Nourishing-...
2,A3GAB6L55URQ8V,B00002EQBW,5.0,1001980800,product_BW,"Dove Deeply Nourishing Body Wash, With Exfolia...",https://m.media-amazon.com/images/I/419w4PBH11...,https://www.amazon.in//Dove-Deeply-Nourishing-...
3,A27TC4PZ9OQN3Y,B00002EQBW,5.0,970531200,product_BW,"Dove Deeply Nourishing Body Wash, With Exfolia...",https://m.media-amazon.com/images/I/419w4PBH11...,https://www.amazon.in//Dove-Deeply-Nourishing-...
4,A4FY8L6M7NL6K,B00003CWBW,1.0,1070841600,product_BW,Bodywise 5% Lactic Acid Body Lotion- 200ml & B...,https://m.media-amazon.com/images/I/61qwyTfaJq...,https://www.amazon.in//Bodywise-Lotion-Bodywas...
...,...,...,...,...,...,...,...,...
11534,ACASVMQYUPGQH,B00JCCWMBW,5.0,1403049600,product_BW,MCP Digital Glass Weighing Machine Round Perso...,https://m.media-amazon.com/images/I/61Y9jA4AeG...,https://www.amazon.in//MCP-Round-Digital-Weigh...
11535,A1ZOT6YBD18M3O,B00JCCWMBW,5.0,1405900800,product_BW,MCP Digital Glass Weighing Machine Round Perso...,https://m.media-amazon.com/images/I/61Y9jA4AeG...,https://www.amazon.in//MCP-Round-Digital-Weigh...
11536,A2N86ONDC5AFEL,B00JO6KNBW,4.0,1404691200,product_BW,Healthgenie Digital Weight Machine with Magic ...,https://m.media-amazon.com/images/I/41TP5hHOGO...,https://www.amazon.in//Healthgenie-Magically-W...
11537,A253CDOMI1O4GE,B00LBEUBBW,5.0,1404950400,product_BW,Mantavya Digital Personal Bathroom Health Body...,https://m.media-amazon.com/images/I/31MTQesVp9...,https://www.amazon.in//Mantavya-Personal-Bathr...


### Product BX dataframe

In [65]:
df = get_scraped('boxers for men')
df

 15%|████████████▊                                                                      | 6/39 [00:15<01:27,  2.64s/it]


,product_name,product_img,product_link
0,Men Boxer Shorts,https://m.media-amazon.com/images/I/81-bYC402D...,https://www.amazon.in//Longies-Mens-Printed-Bo...
1,Men's Graphic Blue Pure Cotton Boxers,https://m.media-amazon.com/images/I/51zLEut4pj...,https://www.amazon.in//Bewakoof-Mens-Graphic-C...
2,Men Boxer Shorts,https://m.media-amazon.com/images/I/81HjYrvxPk...,https://www.amazon.in//Amazon-Brand-Printed-SY...
3,Men Boxer Shorts (Color & Print May Vary),https://m.media-amazon.com/images/I/71Mz4I8CiS...,https://www.amazon.in//Amazon-Brand-Printed-SY...
4,Men Boxer Shorts,https://m.media-amazon.com/images/I/81S-iDVmtI...,https://www.amazon.in//Jockey-Cotton-Boxers-12...
...,...,...,...
301,Men Boxer Shorts,https://m.media-amazon.com/images/I/81hqOwVHd5...,https://www.amazon.in//Marks-Spencer-Shorts-T1...
302,Dkosta Mens Checked Boxers,https://m.media-amazon.com/images/I/514BJx84vq...,https://www.amazon.in//Dkosta-Mens-Checked-Box...
303,Men Boxers,https://m.media-amazon.com/images/I/91eCBLETw7...,https://www.amazon.in//Max-Mens-Checkered-Boxe...
304,Men Boxers,https://m.media-amazon.com/images/I/91uJN7JCHH...,https://www.amazon.in//Diverse-Mens-Printed-Bo...


In [66]:
BX = df.copy()
BX

,product_name,product_img,product_link
0,Men Boxer Shorts,https://m.media-amazon.com/images/I/81-bYC402D...,https://www.amazon.in//Longies-Mens-Printed-Bo...
1,Men's Graphic Blue Pure Cotton Boxers,https://m.media-amazon.com/images/I/51zLEut4pj...,https://www.amazon.in//Bewakoof-Mens-Graphic-C...
2,Men Boxer Shorts,https://m.media-amazon.com/images/I/81HjYrvxPk...,https://www.amazon.in//Amazon-Brand-Printed-SY...
3,Men Boxer Shorts (Color & Print May Vary),https://m.media-amazon.com/images/I/71Mz4I8CiS...,https://www.amazon.in//Amazon-Brand-Printed-SY...
4,Men Boxer Shorts,https://m.media-amazon.com/images/I/81S-iDVmtI...,https://www.amazon.in//Jockey-Cotton-Boxers-12...
...,...,...,...
301,Men Boxer Shorts,https://m.media-amazon.com/images/I/81hqOwVHd5...,https://www.amazon.in//Marks-Spencer-Shorts-T1...
302,Dkosta Mens Checked Boxers,https://m.media-amazon.com/images/I/514BJx84vq...,https://www.amazon.in//Dkosta-Mens-Checked-Box...
303,Men Boxers,https://m.media-amazon.com/images/I/91eCBLETw7...,https://www.amazon.in//Max-Mens-Checkered-Boxe...
304,Men Boxers,https://m.media-amazon.com/images/I/91uJN7JCHH...,https://www.amazon.in//Diverse-Mens-Printed-Bo...


In [ ]:
# BX = BX.append(df).reset_index(drop=True)
# BX

In [63]:
product_BX = pd.DataFrame(reviews[reviews['product_code']=='product_BX']).reset_index(drop=True)
product_BX

,user_id,product_id,Ratings,Time_stamp,product_code
0,A3OFCP5DTLDG6Y,B00003CWBX,4.0,1034985600,product_BX
1,A3652F8SOQGOIU,B00003CWBX,1.0,1232496000,product_BX
2,A10C84Y38RT22P,B00003CWBX,5.0,1008201600,product_BX
3,A36ZS6A88KAG8F,B00004SPBX,4.0,974937600,product_BX
4,A2MSEP19A7X5PL,B00004SPBX,4.0,963964800,product_BX
...,...,...,...,...,...
630,ARDNBELWO5NZ3,B0000DYVBX,5.0,1338422400,product_BX
631,A2GRXKP57LXEK,B0000DYVBX,2.0,1299024000,product_BX
632,A3SKUTTERYIOQE,B0000DYVBX,5.0,1379462400,product_BX
633,A2CNPKRMOY7YX1,B0000DYVBX,3.0,1391731200,product_BX


In [64]:
len(product_BX.product_id.unique())

20

In [67]:
product_BX = pd.DataFrame(reviews[reviews['product_code']=='product_BX']).reset_index(drop=True)
product_BX =  pd.merge(product_BX, 
                       BX.iloc[:len(product_BX.product_id.unique())]\
                       .join(pd.Series(product_BX['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BX.to_csv('product_BX.csv',index=False)
pd.read_csv('product_BX.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3OFCP5DTLDG6Y,B00003CWBX,4.0,1034985600,product_BX,Men Boxer Shorts,https://m.media-amazon.com/images/I/81-bYC402D...,https://www.amazon.in//Longies-Mens-Printed-Bo...
1,A3652F8SOQGOIU,B00003CWBX,1.0,1232496000,product_BX,Men Boxer Shorts,https://m.media-amazon.com/images/I/81-bYC402D...,https://www.amazon.in//Longies-Mens-Printed-Bo...
2,A10C84Y38RT22P,B00003CWBX,5.0,1008201600,product_BX,Men Boxer Shorts,https://m.media-amazon.com/images/I/81-bYC402D...,https://www.amazon.in//Longies-Mens-Printed-Bo...
3,A36ZS6A88KAG8F,B00004SPBX,4.0,974937600,product_BX,Men's Graphic Blue Pure Cotton Boxers,https://m.media-amazon.com/images/I/51zLEut4pj...,https://www.amazon.in//Bewakoof-Mens-Graphic-C...
4,A2MSEP19A7X5PL,B00004SPBX,4.0,963964800,product_BX,Men's Graphic Blue Pure Cotton Boxers,https://m.media-amazon.com/images/I/51zLEut4pj...,https://www.amazon.in//Bewakoof-Mens-Graphic-C...
...,...,...,...,...,...,...,...,...
630,ARDNBELWO5NZ3,B0000DYVBX,5.0,1338422400,product_BX,Men's Printed Boxers (Pack of 2),https://m.media-amazon.com/images/I/91jIlYYKYJ...,https://www.amazon.in//Amazon-Brand-Printed-SY...
631,A2GRXKP57LXEK,B0000DYVBX,2.0,1299024000,product_BX,Men's Printed Boxers (Pack of 2),https://m.media-amazon.com/images/I/91jIlYYKYJ...,https://www.amazon.in//Amazon-Brand-Printed-SY...
632,A3SKUTTERYIOQE,B0000DYVBX,5.0,1379462400,product_BX,Men's Printed Boxers (Pack of 2),https://m.media-amazon.com/images/I/91jIlYYKYJ...,https://www.amazon.in//Amazon-Brand-Printed-SY...
633,A2CNPKRMOY7YX1,B0000DYVBX,3.0,1391731200,product_BX,Men's Printed Boxers (Pack of 2),https://m.media-amazon.com/images/I/91jIlYYKYJ...,https://www.amazon.in//Amazon-Brand-Printed-SY...


### Product BY dataframe

In [75]:
df = get_scraped('bicycle for kids')
df

 15%|████████████▊                                                                      | 6/39 [00:10<00:56,  1.72s/it]


,product_name,product_img,product_link
0,Lifelong LLBC2001 Tribe 20T Cycle (Yellow and ...,https://m.media-amazon.com/images/I/61StKt0X+t...,https://www.amazon.in//Lifelong-LLBC2001-Tribe...
1,Vector 91 Space Fighter 16T Red Single Speed K...,https://m.media-amazon.com/images/I/51VUbUBIui...,https://www.amazon.in//VECTOR-91-Fighter-Singl...
2,Lifelong LLBC1401 Juniors Ride Kids Cycle 14T ...,https://m.media-amazon.com/images/I/61ogmfeZm6...,https://www.amazon.in//Lifelong-LLBC1401-%E2%8...
3,Hi-Fast 14 inch Sports Kids Cycle for Boys & G...,https://m.media-amazon.com/images/I/81+WS-0Pve...,https://www.amazon.in//HI-FAST-Sports-Training...
4,Leader Kids Buddy 20T Inches Carbon Steel Fram...,https://m.media-amazon.com/images/I/71Z6WKWWxB...,https://www.amazon.in//Leader-Buddy-Carbon-ste...
...,...,...,...
301,VESCO Kick Pro 16 Inch Sporty Look Bicycle's F...,https://m.media-amazon.com/images/I/71Y9o-5F-q...,https://www.amazon.in//VESCO-Cycles-Adjustable...
302,Roadster Bike Studio Presents Unisex-Youth Her...,https://m.media-amazon.com/images/I/41b2wdPJvW...,https://www.amazon.in//Roadster-Studio-Present...
303,BSA Champ Doodle 20T Kids Bicycle with 13 Inch...,https://m.media-amazon.com/images/I/718bKxY+Ve...,https://www.amazon.in//BSA-Champ-Doodle-Kids-B...
304,URBAN STAR 20T Suspension (85% Assembled) for ...,https://m.media-amazon.com/images/I/916ObWcKe-...,https://www.amazon.in//URBAN-Suspension-Assemb...


In [72]:
BY = df.copy()
BY

,product_name,product_img,product_link
0,Hero Sprint Growler 27.5T 21 Speed with Dual Disc,https://m.media-amazon.com/images/I/71WgIo+Qk2...,https://www.amazon.in//Hero-Sprint-Growler-27-...
1,Bicycle,https://m.media-amazon.com/images/I/71jYJ7D2Db...,https://www.amazon.in//CRADIAC-Bicycle/dp/B0BC...
2,Leader Gladiator 26t Multispeed (21 Speed) Gea...,https://m.media-amazon.com/images/I/71Qd9DaJYd...,https://www.amazon.in//Leader-Gladiator-Multis...
3,"Urban Terrain UT1000 Series, Steel MTB 27.5 Mo...",https://m.media-amazon.com/images/I/61NiR2F2uM...,https://www.amazon.in//Urban-Terrain-UT1000-MT...
4,Vector 91 Athens 26T 21 Speed Black Grey Cycle,https://m.media-amazon.com/images/I/71KmVlrIoz...,https://www.amazon.in//Vector-91-Athens-Speed-...
...,...,...,...
301,Ahoy! Bounce Mountain Cycle Non Gear Cycle 29T...,https://m.media-amazon.com/images/I/61jN3yhpd3...,https://www.amazon.in//Bounce-Mountain-Carbon-...
302,Firefox Bikes Bicycle 27.5 (Torreto D 21S) Blk...,https://m.media-amazon.com/images/I/81m4bFRbjC...,https://www.amazon.in//Firefox-Bikes-Bicycle-2...
303,Ahoy! Hover Mountain Cycle Non Gear Cycle 27.5...,https://m.media-amazon.com/images/I/61GRGyS4Qb...,https://www.amazon.in//Hover-Mountain-Height-C...
304,Ahoy! Bounce Mountain Cycle Non Gear Cycle 29T...,https://m.media-amazon.com/images/I/61jN3yhpd3...,https://www.amazon.in//Bounce-Mountain-Carbon-...


In [76]:
BY = BY.append(df).reset_index(drop=True)
BY

,product_name,product_img,product_link
0,Hero Sprint Growler 27.5T 21 Speed with Dual Disc,https://m.media-amazon.com/images/I/71WgIo+Qk2...,https://www.amazon.in//Hero-Sprint-Growler-27-...
1,Bicycle,https://m.media-amazon.com/images/I/71jYJ7D2Db...,https://www.amazon.in//CRADIAC-Bicycle/dp/B0BC...
2,Leader Gladiator 26t Multispeed (21 Speed) Gea...,https://m.media-amazon.com/images/I/71Qd9DaJYd...,https://www.amazon.in//Leader-Gladiator-Multis...
3,"Urban Terrain UT1000 Series, Steel MTB 27.5 Mo...",https://m.media-amazon.com/images/I/61NiR2F2uM...,https://www.amazon.in//Urban-Terrain-UT1000-MT...
4,Vector 91 Athens 26T 21 Speed Black Grey Cycle,https://m.media-amazon.com/images/I/71KmVlrIoz...,https://www.amazon.in//Vector-91-Athens-Speed-...
...,...,...,...
913,VESCO Kick Pro 16 Inch Sporty Look Bicycle's F...,https://m.media-amazon.com/images/I/71Y9o-5F-q...,https://www.amazon.in//VESCO-Cycles-Adjustable...
914,Roadster Bike Studio Presents Unisex-Youth Her...,https://m.media-amazon.com/images/I/41b2wdPJvW...,https://www.amazon.in//Roadster-Studio-Present...
915,BSA Champ Doodle 20T Kids Bicycle with 13 Inch...,https://m.media-amazon.com/images/I/718bKxY+Ve...,https://www.amazon.in//BSA-Champ-Doodle-Kids-B...
916,URBAN STAR 20T Suspension (85% Assembled) for ...,https://m.media-amazon.com/images/I/916ObWcKe-...,https://www.amazon.in//URBAN-Suspension-Assemb...


In [69]:
product_BY = pd.DataFrame(reviews[reviews['product_code']=='product_BY']).reset_index(drop=True)
product_BY

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2AXT3H52X6P9T,B00004SPBY,5.0,962064000,product_BY
1,A3O4F0C47C2FEY,B00004SPBY,5.0,981504000,product_BY
2,A26LSQYSF47YF5,B00004YMBY,5.0,981504000,product_BY
3,A3NJ1W51XSRC29,B00004YMBY,5.0,981763200,product_BY
4,A1LYN3ZK230TGE,B00004YMBY,2.0,1051228800,product_BY
...,...,...,...,...,...
12802,A7ZVWYFM4XO1O,B00J2LIFBY,5.0,1402963200,product_BY
12803,A18JG6ITKGJLPT,B00J3693BY,5.0,1401062400,product_BY
12804,AR3HV5WH2I4B2,B00J3693BY,5.0,1401321600,product_BY
12805,A3FY1BFGQ5TFK,B00J3693BY,4.0,1404086400,product_BY


In [70]:
len(product_BY.product_id.unique())

690

In [77]:
product_BY = pd.DataFrame(reviews[reviews['product_code']=='product_BY']).reset_index(drop=True)
product_BY =  pd.merge(product_BY, 
                       BY.iloc[:len(product_BY.product_id.unique())]\
                       .join(pd.Series(product_BY['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BY.to_csv('product_BY.csv',index=False)
pd.read_csv('product_BY.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2AXT3H52X6P9T,B00004SPBY,5.0,962064000,product_BY,Hero Sprint Growler 27.5T 21 Speed with Dual Disc,https://m.media-amazon.com/images/I/71WgIo+Qk2...,https://www.amazon.in//Hero-Sprint-Growler-27-...
1,A3O4F0C47C2FEY,B00004SPBY,5.0,981504000,product_BY,Hero Sprint Growler 27.5T 21 Speed with Dual Disc,https://m.media-amazon.com/images/I/71WgIo+Qk2...,https://www.amazon.in//Hero-Sprint-Growler-27-...
2,A26LSQYSF47YF5,B00004YMBY,5.0,981504000,product_BY,Bicycle,https://m.media-amazon.com/images/I/71jYJ7D2Db...,https://www.amazon.in//CRADIAC-Bicycle/dp/B0BC...
3,A3NJ1W51XSRC29,B00004YMBY,5.0,981763200,product_BY,Bicycle,https://m.media-amazon.com/images/I/71jYJ7D2Db...,https://www.amazon.in//CRADIAC-Bicycle/dp/B0BC...
4,A1LYN3ZK230TGE,B00004YMBY,2.0,1051228800,product_BY,Bicycle,https://m.media-amazon.com/images/I/71jYJ7D2Db...,https://www.amazon.in//CRADIAC-Bicycle/dp/B0BC...
...,...,...,...,...,...,...,...,...
12802,A7ZVWYFM4XO1O,B00J2LIFBY,5.0,1402963200,product_BY,"Norman Jr,RR24 Collection -Bike for Boys and G...",https://m.media-amazon.com/images/I/71NY76V0ie...,https://www.amazon.in//Collection-Bicycle-Trai...
12803,A18JG6ITKGJLPT,B00J3693BY,5.0,1401062400,product_BY,Speed Bird Cooper Kids Cycle 16-T Baby Cycle f...,https://m.media-amazon.com/images/I/81UoRellK9...,https://www.amazon.in//Speed-Bird-Cooper-Cycle...
12804,AR3HV5WH2I4B2,B00J3693BY,5.0,1401321600,product_BY,Speed Bird Cooper Kids Cycle 16-T Baby Cycle f...,https://m.media-amazon.com/images/I/81UoRellK9...,https://www.amazon.in//Speed-Bird-Cooper-Cycle...
12805,A3FY1BFGQ5TFK,B00J3693BY,4.0,1404086400,product_BY,Speed Bird Cooper Kids Cycle 16-T Baby Cycle f...,https://m.media-amazon.com/images/I/81UoRellK9...,https://www.amazon.in//Speed-Bird-Cooper-Cycle...


### Product BZ dataframe

In [85]:
df = get_scraped('bag zipper puller')
df

 15%|████████████▊                                                                      | 6/39 [00:09<00:50,  1.52s/it]


,product_name,product_img,product_link
0,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
1,Generic 10pcs Non-slip Zipper Pull Cord Rope E...,https://m.media-amazon.com/images/I/61-ia3xran...,https://www.amazon.in//Generic-Non-slip-Fasten...
2,Zipper Puller Replacement Quick Snap Bag Zippe...,https://m.media-amazon.com/images/I/61+NmpezoZ...,https://www.amazon.in//HASTHIP%C2%AE-Replaceme...
3,R K Y Durable Zipper Pulls Zipper Cord Pulls Z...,https://m.media-amazon.com/images/I/61M6-SpN7W...,https://www.amazon.in//Durable-Extension-Backp...
4,Durable Zipper Pulls Zipper Tab Zipper Tags Co...,https://m.media-amazon.com/images/I/71Bp7TR4Sk...,https://www.amazon.in//FunMart-Durable-Extensi...
...,...,...,...
301,32 Pcs Zipper Pull Replacement Universal Zippe...,https://m.media-amazon.com/images/I/71kTt1HHIq...,https://www.amazon.in//Replacement-Universal-E...
302,Kush 20 Pieces Zipper Puller with Webbing Cord...,https://m.media-amazon.com/images/I/510RQhWIjx...,https://www.amazon.in//Pieces-Zipper-Webbing-S...
303,Kush 20 Pieces Zipper Puller with Webbing Cord...,https://m.media-amazon.com/images/I/51KDD-nj0o...,https://www.amazon.in//Pieces-Zipper-Webbing-S...
304,JMD Bearing Puller Heavy Chrome Vanadium Steel...,https://m.media-amazon.com/images/I/61cBaNzkQ9...,https://www.amazon.in//Bearing-Puller-Chrome-V...


In [86]:
BZ = df.copy()
BZ

,product_name,product_img,product_link
0,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
1,Generic 10pcs Non-slip Zipper Pull Cord Rope E...,https://m.media-amazon.com/images/I/61-ia3xran...,https://www.amazon.in//Generic-Non-slip-Fasten...
2,Zipper Puller Replacement Quick Snap Bag Zippe...,https://m.media-amazon.com/images/I/61+NmpezoZ...,https://www.amazon.in//HASTHIP%C2%AE-Replaceme...
3,R K Y Durable Zipper Pulls Zipper Cord Pulls Z...,https://m.media-amazon.com/images/I/61M6-SpN7W...,https://www.amazon.in//Durable-Extension-Backp...
4,Durable Zipper Pulls Zipper Tab Zipper Tags Co...,https://m.media-amazon.com/images/I/71Bp7TR4Sk...,https://www.amazon.in//FunMart-Durable-Extensi...
...,...,...,...
301,32 Pcs Zipper Pull Replacement Universal Zippe...,https://m.media-amazon.com/images/I/71kTt1HHIq...,https://www.amazon.in//Replacement-Universal-E...
302,Kush 20 Pieces Zipper Puller with Webbing Cord...,https://m.media-amazon.com/images/I/510RQhWIjx...,https://www.amazon.in//Pieces-Zipper-Webbing-S...
303,Kush 20 Pieces Zipper Puller with Webbing Cord...,https://m.media-amazon.com/images/I/51KDD-nj0o...,https://www.amazon.in//Pieces-Zipper-Webbing-S...
304,JMD Bearing Puller Heavy Chrome Vanadium Steel...,https://m.media-amazon.com/images/I/61cBaNzkQ9...,https://www.amazon.in//Bearing-Puller-Chrome-V...


In [ ]:
# BZ = BZ.append(df).reset_index(drop=True)
# BZ

In [84]:
product_BZ = pd.DataFrame(reviews[reviews['product_code']=='product_BZ']).reset_index(drop=True)
product_BZ

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2KWTTOGXSWUUJ,B00002EQBZ,4.0,961113600,product_BZ
1,A1LSVNW99NVZ82,B00002EQBZ,4.0,967075200,product_BZ
2,A18D1RLW38LVLW,B00002EQBZ,5.0,1031961600,product_BZ
3,AQGSLUMPQKD8S,B00002EQBZ,5.0,977529600,product_BZ
4,A3IUUQ538VCZZ3,B00002EQBZ,4.0,1174953600,product_BZ
...,...,...,...,...,...
140,AX17355P8ZRMR,B0000DYVBZ,5.0,1371168000,product_BZ
141,A30ATV92LKJG3G,B0000DYVBZ,5.0,1383004800,product_BZ
142,A3DHJ0UCEKRU93,B0000DYVBZ,5.0,1390176000,product_BZ
143,A3M7BCQKSQAK4L,B0000DYVBZ,1.0,1316563200,product_BZ


In [10]:
len(product_BZ.product_id.unique())

21

In [87]:
product_BZ = pd.DataFrame(reviews[reviews['product_code']=='product_BZ']).reset_index(drop=True)
product_BZ =  pd.merge(product_BZ, 
                       BZ.iloc[:len(product_BZ.product_id.unique())]\
                       .join(pd.Series(product_BZ['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_BZ.to_csv('product_BZ.csv',index=False)
pd.read_csv('product_BZ.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2KWTTOGXSWUUJ,B00002EQBZ,4.0,961113600,product_BZ,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
1,A1LSVNW99NVZ82,B00002EQBZ,4.0,967075200,product_BZ,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
2,A18D1RLW38LVLW,B00002EQBZ,5.0,1031961600,product_BZ,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
3,AQGSLUMPQKD8S,B00002EQBZ,5.0,977529600,product_BZ,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
4,A3IUUQ538VCZZ3,B00002EQBZ,4.0,1174953600,product_BZ,Durable Zipper Pulls Zipper Cord Pulls Zipper ...,https://m.media-amazon.com/images/I/61Bewi0hGj...,https://www.amazon.in//Risan-Durable-Extension...
...,...,...,...,...,...,...,...,...
140,AX17355P8ZRMR,B0000DYVBZ,5.0,1371168000,product_BZ,Zeniso® 15 Pieces 3 Different Size Zipper Slid...,https://m.media-amazon.com/images/I/41CMLxGoyI...,https://www.amazon.in//Zeniso%C2%AE-Pieces-Dif...
141,A30ATV92LKJG3G,B0000DYVBZ,5.0,1383004800,product_BZ,Zeniso® 15 Pieces 3 Different Size Zipper Slid...,https://m.media-amazon.com/images/I/41CMLxGoyI...,https://www.amazon.in//Zeniso%C2%AE-Pieces-Dif...
142,A3DHJ0UCEKRU93,B0000DYVBZ,5.0,1390176000,product_BZ,Zeniso® 15 Pieces 3 Different Size Zipper Slid...,https://m.media-amazon.com/images/I/41CMLxGoyI...,https://www.amazon.in//Zeniso%C2%AE-Pieces-Dif...
143,A3M7BCQKSQAK4L,B0000DYVBZ,1.0,1316563200,product_BZ,Zeniso® 15 Pieces 3 Different Size Zipper Slid...,https://m.media-amazon.com/images/I/41CMLxGoyI...,https://www.amazon.in//Zeniso%C2%AE-Pieces-Dif...


### Product C0 dataframe

In [95]:
df = get_scraped('chameli oil')
df

 15%|████████████▊                                                                      | 6/39 [00:08<00:49,  1.49s/it]


,product_name,product_img,product_link
0,Jioo Organics Chameli Oil for Hanuman Puja | C...,https://m.media-amazon.com/images/I/31-vKQTdpq...,https://www.amazon.in//Jioo-Organics-Chameli-H...
1,PNC Handicrafts Shubhshree Chola Chameli Oil (...,https://m.media-amazon.com/images/I/61m03fgqiq...,https://www.amazon.in//PNC-Handicrafts-Shubhsh...
2,J Organics Chameli Oil For Hanuman Puja Chola ...,https://m.media-amazon.com/images/I/61Y2Zz1MM7...,https://www.amazon.in//Organics-Chameli-Hanuma...
3,WQT - Chola Chameli Pooja Oil Hanuman JI For R...,https://m.media-amazon.com/images/I/712L+Rlvg0...,https://www.amazon.in//WQT-Chameli-Hanuman-Rel...
4,Chameli (Jasmine) Oil 900 ML |Special chameli ...,https://m.media-amazon.com/images/I/71Ip6tKKmX...,https://www.amazon.in//Chameli-Jasmine-Special...
...,...,...,...
301,All Naturals 100% Pure Jasmine Essential Oil -...,https://m.media-amazon.com/images/I/61dObrBSZS...,https://www.amazon.in//All-Naturals-100-Jasmin...
302,Reed diffuser refill oil 250ml. reed diffuser ...,https://m.media-amazon.com/images/I/61JC9LAQ95...,https://www.amazon.in//diffuser-sandalwood-alu...
303,FnP CL Ethnic Ceramic Electric Aroma Diffuser ...,https://m.media-amazon.com/images/I/71kZNfdjcJ...,https://www.amazon.in//Ethnic-Electric-Aroma-D...
304,Reliable Combo Pack OFLOBANDANI & Guggal Sambr...,https://m.media-amazon.com/images/I/916Pu7WzIg...,https://www.amazon.in//Reliable-OFLOBANDANI-Sa...


In [92]:
C0 = df.copy()
C0

,product_name,product_img,product_link
0,"Paper Boat Chikki Jar, Peanut Bar, No Added Pr...",https://m.media-amazon.com/images/I/711aMV+x2-...,https://www.amazon.in//Paper-Boat-Chikki-800g-...
1,"Paper Boat Sesame Chikki Jar, Nutritious Bar, ...",https://m.media-amazon.com/images/I/717rOcm1ia...,https://www.amazon.in//Paperboat-Sesame-Chikki...
2,"Paper Boat Crushed Peanut Chikki Jar, No Added...",https://m.media-amazon.com/images/I/71acma9-xc...,https://www.amazon.in//Paper-Boat-Crushed-Pean...
3,"Paper Boat Chikki Family Pack, Peanut Bar, No ...",https://m.media-amazon.com/images/I/614OTQygFI...,https://www.amazon.in//Paper-Boat-Chikki-Pouch...
4,"Paper Boat Dry Fruit Chikki, No Added Preserva...",https://m.media-amazon.com/images/I/61mVf4sMqO...,https://www.amazon.in//Paper-Boat-Chikki-Prese...
...,...,...,...
301,Govindjee Hazelnut Butter Chikki | Soft & Tast...,https://m.media-amazon.com/images/I/61oOzQxl3t...,https://www.amazon.in//Govindjee-Hazelnut-Butt...
302,P-Link Mix Dry Fruit Chikki (Kaju - Badam - Pi...,https://m.media-amazon.com/images/I/61-Ts5vglv...,https://www.amazon.in//P-Link-Mix-Fruit-Chikki...
303,WAY 4 WELLNESS Herbal Peanut Candy | No Added ...,https://m.media-amazon.com/images/I/61qYTOphe-...,https://www.amazon.in//WAY-WELLNESS-Preservati...
304,Avni Sesame Balls (Pack Of 3X200 Grams),https://m.media-amazon.com/images/I/91Tt0ZdAek...,https://www.amazon.in//Avni-Sesame-Balls-3X200...


In [96]:
C0 = C0.append(df).reset_index(drop=True)
C0

,product_name,product_img,product_link
0,"Paper Boat Chikki Jar, Peanut Bar, No Added Pr...",https://m.media-amazon.com/images/I/711aMV+x2-...,https://www.amazon.in//Paper-Boat-Chikki-800g-...
1,"Paper Boat Sesame Chikki Jar, Nutritious Bar, ...",https://m.media-amazon.com/images/I/717rOcm1ia...,https://www.amazon.in//Paperboat-Sesame-Chikki...
2,"Paper Boat Crushed Peanut Chikki Jar, No Added...",https://m.media-amazon.com/images/I/71acma9-xc...,https://www.amazon.in//Paper-Boat-Crushed-Pean...
3,"Paper Boat Chikki Family Pack, Peanut Bar, No ...",https://m.media-amazon.com/images/I/614OTQygFI...,https://www.amazon.in//Paper-Boat-Chikki-Pouch...
4,"Paper Boat Dry Fruit Chikki, No Added Preserva...",https://m.media-amazon.com/images/I/61mVf4sMqO...,https://www.amazon.in//Paper-Boat-Chikki-Prese...
...,...,...,...
913,All Naturals 100% Pure Jasmine Essential Oil -...,https://m.media-amazon.com/images/I/61dObrBSZS...,https://www.amazon.in//All-Naturals-100-Jasmin...
914,Reed diffuser refill oil 250ml. reed diffuser ...,https://m.media-amazon.com/images/I/61JC9LAQ95...,https://www.amazon.in//diffuser-sandalwood-alu...
915,FnP CL Ethnic Ceramic Electric Aroma Diffuser ...,https://m.media-amazon.com/images/I/71kZNfdjcJ...,https://www.amazon.in//Ethnic-Electric-Aroma-D...
916,Reliable Combo Pack OFLOBANDANI & Guggal Sambr...,https://m.media-amazon.com/images/I/916Pu7WzIg...,https://www.amazon.in//Reliable-OFLOBANDANI-Sa...


In [89]:
product_C0 = pd.DataFrame(reviews[reviews['product_code']=='product_C0']).reset_index(drop=True)
product_C0

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2X61KKXXR6WNP,B00003CWC0,5.0,1086480000,product_C0
1,A1T34GV7S9M9PZ,B00003CWC0,5.0,1025136000,product_C0
2,A1ORYZATNANH5L,B00004RFC0,5.0,1293235200,product_C0
3,A3GJOLUSU1UX3G,B00004RFC0,4.0,955584000,product_C0
4,A1LG7243001X88,B00004RFC0,4.0,962755200,product_C0
...,...,...,...,...,...
15508,ACQQNOPWFVSHG,B00JX321C0,5.0,1402358400,product_C0
15509,A3K58VTCPASFAI,B00JX321C0,5.0,1402358400,product_C0
15510,A1IOQYMOAUG97W,B00JX321C0,5.0,1402358400,product_C0
15511,A1PQHZNMZ8422F,B00KATOPC0,5.0,1405987200,product_C0


In [90]:
len(product_C0.product_id.unique())

661

In [97]:
product_C0 = pd.DataFrame(reviews[reviews['product_code']=='product_C0']).reset_index(drop=True)
product_C0 =  pd.merge(product_C0, 
                       C0.iloc[:len(product_C0.product_id.unique())]\
                       .join(pd.Series(product_C0['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C0.to_csv('product_C0.csv',index=False)
pd.read_csv('product_C0.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2X61KKXXR6WNP,B00003CWC0,5.0,1086480000,product_C0,"Paper Boat Chikki Jar, Peanut Bar, No Added Pr...",https://m.media-amazon.com/images/I/711aMV+x2-...,https://www.amazon.in//Paper-Boat-Chikki-800g-...
1,A1T34GV7S9M9PZ,B00003CWC0,5.0,1025136000,product_C0,"Paper Boat Chikki Jar, Peanut Bar, No Added Pr...",https://m.media-amazon.com/images/I/711aMV+x2-...,https://www.amazon.in//Paper-Boat-Chikki-800g-...
2,A1ORYZATNANH5L,B00004RFC0,5.0,1293235200,product_C0,"Paper Boat Sesame Chikki Jar, Nutritious Bar, ...",https://m.media-amazon.com/images/I/717rOcm1ia...,https://www.amazon.in//Paperboat-Sesame-Chikki...
3,A3GJOLUSU1UX3G,B00004RFC0,4.0,955584000,product_C0,"Paper Boat Sesame Chikki Jar, Nutritious Bar, ...",https://m.media-amazon.com/images/I/717rOcm1ia...,https://www.amazon.in//Paperboat-Sesame-Chikki...
4,A1LG7243001X88,B00004RFC0,4.0,962755200,product_C0,"Paper Boat Sesame Chikki Jar, Nutritious Bar, ...",https://m.media-amazon.com/images/I/717rOcm1ia...,https://www.amazon.in//Paperboat-Sesame-Chikki...
...,...,...,...,...,...,...,...,...
15508,ACQQNOPWFVSHG,B00JX321C0,5.0,1402358400,product_C0,"Fortune Premium Kachi Ghani Pure Mustard Oil, ...",https://m.media-amazon.com/images/I/41YDq-tchU...,https://www.amazon.in//Fortune-Kachi-Ghani-Pur...
15509,A3K58VTCPASFAI,B00JX321C0,5.0,1402358400,product_C0,"Fortune Premium Kachi Ghani Pure Mustard Oil, ...",https://m.media-amazon.com/images/I/41YDq-tchU...,https://www.amazon.in//Fortune-Kachi-Ghani-Pur...
15510,A1IOQYMOAUG97W,B00JX321C0,5.0,1402358400,product_C0,"Fortune Premium Kachi Ghani Pure Mustard Oil, ...",https://m.media-amazon.com/images/I/41YDq-tchU...,https://www.amazon.in//Fortune-Kachi-Ghani-Pur...
15511,A1PQHZNMZ8422F,B00KATOPC0,5.0,1405987200,product_C0,My God Gift Shree Hanuman Ji Chola with Silver...,https://m.media-amazon.com/images/I/71XTvKD6qN...,https://www.amazon.in//Hanuman-Sindoor-Chameli...


### Product C1 dataframe

In [106]:
df = get_scraped('c11 realme cover')
df

 49%|███████████████████████████████████████▉                                          | 19/39 [00:32<00:34,  1.70s/it]


,product_name,product_img,product_link
0,Pikkme Realme C11 Leather Flip Cover Wallet Ca...,https://m.media-amazon.com/images/I/71KP1CB1rt...,https://www.amazon.in//Pikkme-Realme-C11-Leath...
1,Jkobi Back Cover Case for Realme C11 (2021) (T...,https://m.media-amazon.com/images/I/51-Sl8CaCl...,https://www.amazon.in//Rubberised-Micromax-Can...
2,Fashionury Back Cover for Realme C25 / C25s / ...,https://m.media-amazon.com/images/I/51ZfXFdtDD...,https://www.amazon.in//Fashionury-Realme-Anti-...
3,Pikkme Realme C11 Leather Flip Cover Wallet Ca...,https://m.media-amazon.com/images/I/71zWNFObDW...,https://www.amazon.in//Pikkme-Realme-C11-Leath...
4,SPAZY CASE ® Silicone 360 Degree Rotation Dura...,https://m.media-amazon.com/images/I/61+EGoEVtz...,https://www.amazon.in//Realme-Shockproof-Kicks...
...,...,...,...
301,Fashionury Rugged Armor Designer Rubberized Pr...,https://m.media-amazon.com/images/I/61r5Lcr+1K...,https://www.amazon.in//Fashionury-Designer-Rub...
302,Casotec Premium Leather Kickstand Wallet Flip ...,https://m.media-amazon.com/images/I/71qgfV3E7a...,https://www.amazon.in//Casotec-Premium-Leather...
303,CaseRepublic Thermoplastic Polyurethane Carbon...,https://m.media-amazon.com/images/I/51WvrmEdDH...,https://www.amazon.in//CaseRepublic-Cover-real...
304,Casotec Blocks Rainbow 3D Graphics Design 3D P...,https://m.media-amazon.com/images/I/71DWo+XPcC...,https://www.amazon.in//Casotec-Blocks-Rainbow-...


In [107]:
C1 = df.copy()
C1

,product_name,product_img,product_link
0,Pikkme Realme C11 Leather Flip Cover Wallet Ca...,https://m.media-amazon.com/images/I/71KP1CB1rt...,https://www.amazon.in//Pikkme-Realme-C11-Leath...
1,Jkobi Back Cover Case for Realme C11 (2021) (T...,https://m.media-amazon.com/images/I/51-Sl8CaCl...,https://www.amazon.in//Rubberised-Micromax-Can...
2,Fashionury Back Cover for Realme C25 / C25s / ...,https://m.media-amazon.com/images/I/51ZfXFdtDD...,https://www.amazon.in//Fashionury-Realme-Anti-...
3,Pikkme Realme C11 Leather Flip Cover Wallet Ca...,https://m.media-amazon.com/images/I/71zWNFObDW...,https://www.amazon.in//Pikkme-Realme-C11-Leath...
4,SPAZY CASE ® Silicone 360 Degree Rotation Dura...,https://m.media-amazon.com/images/I/61+EGoEVtz...,https://www.amazon.in//Realme-Shockproof-Kicks...
...,...,...,...
301,Fashionury Rugged Armor Designer Rubberized Pr...,https://m.media-amazon.com/images/I/61r5Lcr+1K...,https://www.amazon.in//Fashionury-Designer-Rub...
302,Casotec Premium Leather Kickstand Wallet Flip ...,https://m.media-amazon.com/images/I/71qgfV3E7a...,https://www.amazon.in//Casotec-Premium-Leather...
303,CaseRepublic Thermoplastic Polyurethane Carbon...,https://m.media-amazon.com/images/I/51WvrmEdDH...,https://www.amazon.in//CaseRepublic-Cover-real...
304,Casotec Blocks Rainbow 3D Graphics Design 3D P...,https://m.media-amazon.com/images/I/71DWo+XPcC...,https://www.amazon.in//Casotec-Blocks-Rainbow-...


In [ ]:
# C1 = C1.append(df).reset_index(drop=True)
# C1

In [104]:
product_C1 = pd.DataFrame(reviews[reviews['product_code']=='product_C1']).reset_index(drop=True)
product_C1

,user_id,product_id,Ratings,Time_stamp,product_code
0,A33D2WC7HOD9S2,B00003GPC1,5.0,1227139200,product_C1
1,A31AH7RPW17NI5,B00004RFC1,4.0,975888000,product_C1
2,AU9ZCY8ALVKSI,B00004RFC1,4.0,958953600,product_C1
3,A1SXGLGV41TJIB,B00004RFC1,1.0,977011200,product_C1
4,APOROTTY2J7QD,B00004RFC1,4.0,994982400,product_C1
...,...,...,...,...,...
247,A16VHFP9MZQ4WJ,B0000A2UC1,5.0,1103068800,product_C1
248,A15GNP22XF1KGI,B0000A2UC1,1.0,1110931200,product_C1
249,A16Y8FP66KOXCK,B0000AE6C1,4.0,1223510400,product_C1
250,A1PZ5N0AS1QES2,B0000AE6C1,5.0,1214697600,product_C1


In [105]:
len(product_C1.product_id.unique())

18

In [108]:
product_C1 = pd.DataFrame(reviews[reviews['product_code']=='product_C1']).reset_index(drop=True)
product_C1 =  pd.merge(product_C1, 
                       C1.iloc[:len(product_C1.product_id.unique())]\
                       .join(pd.Series(product_C1['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C1.to_csv('product_C1.csv',index=False)
pd.read_csv('product_C1.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A33D2WC7HOD9S2,B00003GPC1,5.0,1227139200,product_C1,Pikkme Realme C11 Leather Flip Cover Wallet Ca...,https://m.media-amazon.com/images/I/71KP1CB1rt...,https://www.amazon.in//Pikkme-Realme-C11-Leath...
1,A31AH7RPW17NI5,B00004RFC1,4.0,975888000,product_C1,Jkobi Back Cover Case for Realme C11 (2021) (T...,https://m.media-amazon.com/images/I/51-Sl8CaCl...,https://www.amazon.in//Rubberised-Micromax-Can...
2,AU9ZCY8ALVKSI,B00004RFC1,4.0,958953600,product_C1,Jkobi Back Cover Case for Realme C11 (2021) (T...,https://m.media-amazon.com/images/I/51-Sl8CaCl...,https://www.amazon.in//Rubberised-Micromax-Can...
3,A1SXGLGV41TJIB,B00004RFC1,1.0,977011200,product_C1,Jkobi Back Cover Case for Realme C11 (2021) (T...,https://m.media-amazon.com/images/I/51-Sl8CaCl...,https://www.amazon.in//Rubberised-Micromax-Can...
4,APOROTTY2J7QD,B00004RFC1,4.0,994982400,product_C1,Jkobi Back Cover Case for Realme C11 (2021) (T...,https://m.media-amazon.com/images/I/51-Sl8CaCl...,https://www.amazon.in//Rubberised-Micromax-Can...
...,...,...,...,...,...,...,...,...
247,A16VHFP9MZQ4WJ,B0000A2UC1,5.0,1103068800,product_C1,Fashionury Back Cover Case for Realme C20 / Re...,https://m.media-amazon.com/images/I/61yr6FQErY...,https://www.amazon.in//Fashionury-Realme-Carbo...
248,A15GNP22XF1KGI,B0000A2UC1,1.0,1110931200,product_C1,Fashionury Back Cover Case for Realme C20 / Re...,https://m.media-amazon.com/images/I/61yr6FQErY...,https://www.amazon.in//Fashionury-Realme-Carbo...
249,A16Y8FP66KOXCK,B0000AE6C1,4.0,1223510400,product_C1,"Fashionury Realme C20 Back Cover, Silicone & A...",https://m.media-amazon.com/images/I/51SVAQbS7a...,https://www.amazon.in//Fashionury-Silicone-Acr...
250,A1PZ5N0AS1QES2,B0000AE6C1,5.0,1214697600,product_C1,"Fashionury Realme C20 Back Cover, Silicone & A...",https://m.media-amazon.com/images/I/51SVAQbS7a...,https://www.amazon.in//Fashionury-Silicone-Acr...


### Product C2 dataframe

In [127]:
df = get_scraped('chandan tika')
df

 15%|████████████▊                                                                      | 6/39 [00:10<00:59,  1.81s/it]


,product_name,product_img,product_link
0,Hari Darshan Premium Chandan Tika -100g | Pure...,https://m.media-amazon.com/images/I/619wqK2y4d...,https://www.amazon.in//Darshan-Premium-Chandan...
1,Hari Darshan Chandan Tika - Pure Sandal Wood P...,https://m.media-amazon.com/images/I/61x6X3hOeB...,https://www.amazon.in//Haridarshan-Chandan-Tik...
2,Hari Darshan - Tika Combo - Sandalwood Chandan...,https://m.media-amazon.com/images/I/61w+wWGCPv...,https://www.amazon.in//HARI-DARSHAN-Sandalwood...
3,Hari Darshan Chandan Tika - Pure Sandal Wood P...,https://m.media-amazon.com/images/I/71Fakq0dd+...,https://www.amazon.in//Hari-Darshan-Sandalwood...
4,"Shree Ji Chandan Tika for Navratri, Shiv Pujan...",https://m.media-amazon.com/images/I/419Dr5+QSp...,https://www.amazon.in//Shree-Chandan-Tika-Navr...
...,...,...,...
301,Akshayshree Sales Sandlewood Powder/Chandan Ti...,https://m.media-amazon.com/images/I/41Atd3OlXB...,https://www.amazon.in//Akshayshree-Sales-Sandl...
302,Vidha Impex Bhai Dooj Set for Brother Ceramic ...,https://m.media-amazon.com/images/I/61X889tk2h...,https://www.amazon.in//Vidha-Impex-Bhai-Dooj-M...
303,Spherulemuster Rudraksha 5 Mukhi (108+1) Beads...,https://m.media-amazon.com/images/I/61me3B3f0Q...,https://www.amazon.in//Spherulemuster-Rudraksh...
304,"ME&YOU Chandan Tika for Navratri, Shiv Pujan, ...",https://m.media-amazon.com/images/I/71qC8hPIXA...,https://www.amazon.in//ME-YOU-Chandan-Navratri...


In [124]:
C2 = df.copy()
C2

,product_name,product_img,product_link
0,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
1,Sandle Incense Stick (Pack of 6),https://m.media-amazon.com/images/I/71hyrzYaz0...,https://www.amazon.in//Sandle-Incense-Stick-Pa...
2,Krisah® 1 pcs Lab Tested ISI Seal Original San...,https://m.media-amazon.com/images/I/61DcvGB107...,https://www.amazon.in//Krisah%C2%AE-Original-S...
3,RDK Original Sandalwood Sticks Scented Process...,https://m.media-amazon.com/images/I/71RRPtyIY3...,https://www.amazon.in//RDK-Original-Sandalwood...
4,Betala Fragrance Chandan Flavour Dhoop Sticks ...,https://m.media-amazon.com/images/I/81c4gHv62n...,https://www.amazon.in//BETALA-FRAGRANCE-Chanda...
...,...,...,...
301,Top Trends® Natural Sandalwood Chandan Pata Bo...,https://m.media-amazon.com/images/I/81EywFYBhi...,https://www.amazon.in//Top-Trends%C2%AE-Natura...
302,AKHIL BHARTIYA JYOTISH VASTU SANSTHAN RAKT Cha...,https://m.media-amazon.com/images/I/71c3J5l-6x...,https://www.amazon.in//AKHIL-BHARTIYA-JYOTISH-...
303,Agarwal Rudraksha Natural Malyagiri Sandalwood...,https://m.media-amazon.com/images/I/41FQcYJRg+...,https://www.amazon.in//Agarwal-Rudraksha-Natur...
304,Around Pure White Sandalwood Stick/SAFED Chand...,https://m.media-amazon.com/images/I/71pYns925P...,https://www.amazon.in//Around-Sandalwood-Chand...


In [128]:
C2 = C2.append(df).reset_index(drop=True)
C2

,product_name,product_img,product_link
0,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
1,Sandle Incense Stick (Pack of 6),https://m.media-amazon.com/images/I/71hyrzYaz0...,https://www.amazon.in//Sandle-Incense-Stick-Pa...
2,Krisah® 1 pcs Lab Tested ISI Seal Original San...,https://m.media-amazon.com/images/I/61DcvGB107...,https://www.amazon.in//Krisah%C2%AE-Original-S...
3,RDK Original Sandalwood Sticks Scented Process...,https://m.media-amazon.com/images/I/71RRPtyIY3...,https://www.amazon.in//RDK-Original-Sandalwood...
4,Betala Fragrance Chandan Flavour Dhoop Sticks ...,https://m.media-amazon.com/images/I/81c4gHv62n...,https://www.amazon.in//BETALA-FRAGRANCE-Chanda...
...,...,...,...
913,Akshayshree Sales Sandlewood Powder/Chandan Ti...,https://m.media-amazon.com/images/I/41Atd3OlXB...,https://www.amazon.in//Akshayshree-Sales-Sandl...
914,Vidha Impex Bhai Dooj Set for Brother Ceramic ...,https://m.media-amazon.com/images/I/61X889tk2h...,https://www.amazon.in//Vidha-Impex-Bhai-Dooj-M...
915,Spherulemuster Rudraksha 5 Mukhi (108+1) Beads...,https://m.media-amazon.com/images/I/61me3B3f0Q...,https://www.amazon.in//Spherulemuster-Rudraksh...
916,"ME&YOU Chandan Tika for Navratri, Shiv Pujan, ...",https://m.media-amazon.com/images/I/71qC8hPIXA...,https://www.amazon.in//ME-YOU-Chandan-Navratri...


In [110]:
product_C2 = pd.DataFrame(reviews[reviews['product_code']=='product_C2']).reset_index(drop=True)
product_C2

,user_id,product_id,Ratings,Time_stamp,product_code
0,A2IEJCU2FG5HD6,B00002EQC2,5.0,1190851200,product_C2
1,A2KGO4E9CXPF10,B00002EQC2,5.0,1120262400,product_C2
2,AIQ5EY5CAHBWO,B00002EQC2,5.0,964656000,product_C2
3,A14J3OCHQ9GH8B,B00002EQC2,5.0,1358294400,product_C2
4,A3TAS1AG6FMBQW,B00002EQC2,5.0,1350604800,product_C2
...,...,...,...,...,...
9572,A38U4YKRF161JX,B00JKCHDC2,2.0,1405555200,product_C2
9573,A1N4DDOJBGGDT1,B00JKCHDC2,4.0,1405987200,product_C2
9574,A3X0AKFN364QN,B00JKCHDC2,5.0,1400803200,product_C2
9575,A2CIN8XSS1WIYU,B00JKCHDC2,1.0,1405900800,product_C2


In [111]:
len(product_C2.product_id.unique())

766

In [129]:
product_C2 = pd.DataFrame(reviews[reviews['product_code']=='product_C2']).reset_index(drop=True)
product_C2 =  pd.merge(product_C2, 
                       C2.iloc[:len(product_C2.product_id.unique())]\
                       .join(pd.Series(product_C2['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C2.to_csv('product_C2.csv',index=False)
pd.read_csv('product_C2.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2IEJCU2FG5HD6,B00002EQC2,5.0,1190851200,product_C2,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
1,A2KGO4E9CXPF10,B00002EQC2,5.0,1120262400,product_C2,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
2,AIQ5EY5CAHBWO,B00002EQC2,5.0,964656000,product_C2,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
3,A14J3OCHQ9GH8B,B00002EQC2,5.0,1358294400,product_C2,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
4,A3TAS1AG6FMBQW,B00002EQC2,5.0,1350604800,product_C2,Lord krishna's Original Chandan Stick with Cha...,https://m.media-amazon.com/images/I/41moXPF8LV...,https://www.amazon.in//Lord-krishnas-Original-...
...,...,...,...,...,...,...,...,...
9572,A38U4YKRF161JX,B00JKCHDC2,2.0,1405555200,product_C2,Bekner Chandan Roli/Tika Chopda Set/Chawal Aks...,https://m.media-amazon.com/images/I/31ZAvnoHuj...,https://www.amazon.in//Bekner-Chandan-Chopda-C...
9573,A1N4DDOJBGGDT1,B00JKCHDC2,4.0,1405987200,product_C2,Bekner Chandan Roli/Tika Chopda Set/Chawal Aks...,https://m.media-amazon.com/images/I/31ZAvnoHuj...,https://www.amazon.in//Bekner-Chandan-Chopda-C...
9574,A3X0AKFN364QN,B00JKCHDC2,5.0,1400803200,product_C2,Bekner Chandan Roli/Tika Chopda Set/Chawal Aks...,https://m.media-amazon.com/images/I/31ZAvnoHuj...,https://www.amazon.in//Bekner-Chandan-Chopda-C...
9575,A2CIN8XSS1WIYU,B00JKCHDC2,1.0,1405900800,product_C2,Bekner Chandan Roli/Tika Chopda Set/Chawal Aks...,https://m.media-amazon.com/images/I/31ZAvnoHuj...,https://www.amazon.in//Bekner-Chandan-Chopda-C...


### Product C3 dataframe

In [133]:
df = get_scraped('c35 realme back cover')
df

 49%|███████████████████████████████████████▉                                          | 19/39 [00:23<00:24,  1.25s/it]


,product_name,product_img,product_link
0,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
1,ERIT Realme C35 Back Cover Ultra Slim Soft Rub...,https://m.media-amazon.com/images/I/51lbsmpqRY...,https://www.amazon.in//ERIT-Rubberized-Silicon...
2,Newlike Silicone Perfect Fitting Hybrid Edge t...,https://m.media-amazon.com/images/I/51MDLJuslw...,https://www.amazon.in//Newlike-Perfect-Fitting...
3,CEDO Realme Narzo 50A Prime/Realme C35 Back Co...,https://m.media-amazon.com/images/I/61hB5mmlFz...,https://www.amazon.in//CEDO-Realme-Narzo-50A-P...
4,Fashionury Protective Soft Silicone Shockproof...,https://m.media-amazon.com/images/I/51Z0m8FDNl...,https://www.amazon.in//Fashionury-Protective-S...
...,...,...,...
301,Fashionury Back Cover for Realme C35 Designer ...,https://m.media-amazon.com/images/I/513ymRWDhm...,https://www.amazon.in//Fashionury-Designer-Tra...
302,NDCOM Tu 13 Dekh Printed Hard Mobile Back Cove...,https://m.media-amazon.com/images/I/51rrQ8QVLq...,https://www.amazon.in//NDCOM-Printed-Mobile-Re...
303,Fashionury Silicone Designer Printed Back Case...,https://m.media-amazon.com/images/I/512W1Kpy8u...,https://www.amazon.in//Fashionury-Silicone-Des...
304,NDCOM for Kakashii Hatakee Printed Hard Mobile...,https://m.media-amazon.com/images/I/61MQgLwuul...,https://www.amazon.in//NDCOM-Kakashii-Hatakee-...


In [134]:
C3 = df.copy()
C3

,product_name,product_img,product_link
0,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
1,ERIT Realme C35 Back Cover Ultra Slim Soft Rub...,https://m.media-amazon.com/images/I/51lbsmpqRY...,https://www.amazon.in//ERIT-Rubberized-Silicon...
2,Newlike Silicone Perfect Fitting Hybrid Edge t...,https://m.media-amazon.com/images/I/51MDLJuslw...,https://www.amazon.in//Newlike-Perfect-Fitting...
3,CEDO Realme Narzo 50A Prime/Realme C35 Back Co...,https://m.media-amazon.com/images/I/61hB5mmlFz...,https://www.amazon.in//CEDO-Realme-Narzo-50A-P...
4,Fashionury Protective Soft Silicone Shockproof...,https://m.media-amazon.com/images/I/51Z0m8FDNl...,https://www.amazon.in//Fashionury-Protective-S...
...,...,...,...
301,Fashionury Back Cover for Realme C35 Designer ...,https://m.media-amazon.com/images/I/513ymRWDhm...,https://www.amazon.in//Fashionury-Designer-Tra...
302,NDCOM Tu 13 Dekh Printed Hard Mobile Back Cove...,https://m.media-amazon.com/images/I/51rrQ8QVLq...,https://www.amazon.in//NDCOM-Printed-Mobile-Re...
303,Fashionury Silicone Designer Printed Back Case...,https://m.media-amazon.com/images/I/512W1Kpy8u...,https://www.amazon.in//Fashionury-Silicone-Des...
304,NDCOM for Kakashii Hatakee Printed Hard Mobile...,https://m.media-amazon.com/images/I/61MQgLwuul...,https://www.amazon.in//NDCOM-Kakashii-Hatakee-...


In [ ]:
# C3 = C3.append(df).reset_index(drop=True)
# C3

In [131]:
product_C3 = pd.DataFrame(reviews[reviews['product_code']=='product_C3']).reset_index(drop=True)
product_C3

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1ORYZATNANH5L,B00004RFC3,5.0,1293235200,product_C3
1,A36EPPVML57KU0,B00004RFC3,5.0,1341619200,product_C3
2,A14G35BFC2NAU9,B00004RFC3,5.0,1187740800,product_C3
3,A29GFFF2VAH8MB,B00004RFC3,1.0,1374192000,product_C3
4,A22I5C2UYG0AET,B00004RFC3,4.0,1353369600,product_C3
...,...,...,...,...,...
147,A30ZZDY1QSY6U8,B00009RUC3,5.0,1073606400,product_C3
148,A39C4C1ZA2SGP3,B00009W3C3,5.0,1366156800,product_C3
149,AB9TBYP8HWEHJ,B0000A2QC3,4.0,1109721600,product_C3
150,A1FR68QH6Z4YZM,B0000AE6C3,5.0,1175817600,product_C3


In [132]:
len(product_C3.product_id.unique())

19

In [135]:
product_C3 = pd.DataFrame(reviews[reviews['product_code']=='product_C3']).reset_index(drop=True)
product_C3 =  pd.merge(product_C3, 
                       C3.iloc[:len(product_C3.product_id.unique())]\
                       .join(pd.Series(product_C3['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C3.to_csv('product_C3.csv',index=False)
pd.read_csv('product_C3.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1ORYZATNANH5L,B00004RFC3,5.0,1293235200,product_C3,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
1,A36EPPVML57KU0,B00004RFC3,5.0,1341619200,product_C3,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
2,A14G35BFC2NAU9,B00004RFC3,5.0,1187740800,product_C3,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
3,A29GFFF2VAH8MB,B00004RFC3,1.0,1374192000,product_C3,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
4,A22I5C2UYG0AET,B00004RFC3,4.0,1353369600,product_C3,Fashionury Realme C35/ Realme Narzo 50A Prime ...,https://m.media-amazon.com/images/I/51wZHdTGix...,https://www.amazon.in//Fashionury-Realme-Hybri...
...,...,...,...,...,...,...,...,...
147,A30ZZDY1QSY6U8,B00009RUC3,5.0,1073606400,product_C3,U&E Back Cover for REALME C35 Back Cover | Sho...,https://m.media-amazon.com/images/I/41bWPpkLCs...,https://www.amazon.in//REALME-Shockproof-Insid...
148,A39C4C1ZA2SGP3,B00009W3C3,5.0,1366156800,product_C3,Fashionury Back Cover for Realme C35 Designer ...,https://m.media-amazon.com/images/I/511wG0Y2FK...,https://www.amazon.in//Fashionury-Designer-Tra...
149,AB9TBYP8HWEHJ,B0000A2QC3,4.0,1109721600,product_C3,Fashionury Back Cover for Realme C35 Designer ...,https://m.media-amazon.com/images/I/41fGub3Ip5...,https://www.amazon.in//Fashionury-Designer-Tra...
150,A1FR68QH6Z4YZM,B0000AE6C3,5.0,1175817600,product_C3,Fashionury Designer Case for Realme C35 Back C...,https://m.media-amazon.com/images/I/51xcbv8Ii8...,https://www.amazon.in//Fashionury-Designer-Rea...


### Product C4 dataframe

In [19]:
df = get_scraped('chicory powder')
df

 15%|████████████▊                                                                      | 6/39 [00:11<01:01,  1.87s/it]


,product_name,product_img,product_link
0,"Urban Platter Roasted Chicory Root Powder, 250...",https://m.media-amazon.com/images/I/81vRFwGrTJ...,https://www.amazon.in//Urban-Platter-Chicory-P...
1,NATURE'S GIFT - FOR THOSE WHO CARE'S Roasted C...,https://m.media-amazon.com/images/I/71a2jCJe3N...,https://www.amazon.in//NATURES-GIFT-Roasted-Ch...
2,"Urban Platter Instant Chicory Root Powder, 150...",https://m.media-amazon.com/images/I/71+i75yNHO...,https://www.amazon.in//Platter-Caffeine-Instan...
3,Organic Prime Roasted Chicory Root Powder - 20...,https://m.media-amazon.com/images/I/51-8NNh-eG...,https://www.amazon.in//Organic-Prime-Roasted-C...
4,B Naturall Instant Chicory Root Powder - 100 G...,https://m.media-amazon.com/images/I/61DEuZ6KkA...,https://www.amazon.in//Naturall-Roasted-Chicor...
...,...,...,...
301,Perfetto Hazelnut Flavoured Instant Coffee Pow...,https://m.media-amazon.com/images/I/61mpnNmM6l...,https://www.amazon.in//Perfetto-Hazelnut-Flavo...
302,TGL Co. The Good Life Company Euphoria Instant...,https://m.media-amazon.com/images/I/61cz59IlSY...,https://www.amazon.in//TGL-Co-Luxury-Coffee-Eu...
303,Nova Nova Instant Flavoured Coffee Powder - Cl...,https://m.media-amazon.com/images/I/71jC0rDVhb...,https://www.amazon.in//Nova-Instant-Flavoured-...
304,Narasus Coffee Filter Coffee Udhayam - 500 Grams,https://m.media-amazon.com/images/I/91s2QYxwSW...,https://www.amazon.in//Narasus-Coffee-Filter-U...


In [15]:
C4 = df.copy()
C4

,product_name,product_img,product_link
0,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
1,True Elements Chia Seeds 150g - Diet Food | Fi...,https://m.media-amazon.com/images/I/61ZsHU5URM...,https://www.amazon.in//True-Elements-Chia-Seed...
2,True Elements Chia Seeds 500gm - Premium Raw C...,https://m.media-amazon.com/images/I/51GB3smjJg...,https://www.amazon.in//True-Elements-Chia-Seed...
3,JIWA healthy by nature Organic Chia Seeds 200 ...,https://m.media-amazon.com/images/I/81kcoiUpZu...,https://www.amazon.in//Jiwa-Organic-Chia-Seeds...
4,"Amazon Brand - Vedaka Premium Raw Chia Seeds, ...",https://m.media-amazon.com/images/I/A1novFlMUG...,https://www.amazon.in//Vedaka-Premium-Chia-See...
...,...,...,...
301,"By Nature White Chia Seeds, 100 Gm",https://m.media-amazon.com/images/I/91bfww354Q...,https://www.amazon.in//Nature-White-Chia-Seeds...
302,ECONBIO ROOTS Certified 100% Organic Combo Raw...,https://m.media-amazon.com/images/I/71MZ0ZQB3b...,https://www.amazon.in//Econbioroots-Certified-...
303,YUVIKA Chia Seeds - Omega 3 - Anti Oxidant - G...,https://m.media-amazon.com/images/I/61d-V0+4Et...,https://www.amazon.in//YUVIKA-Chia-Seeds-Peru-...
304,Medo Specia - Raw Unroasted Chia Seeds 350 gm ...,https://m.media-amazon.com/images/I/61N-t7vxF-...,https://www.amazon.in//Medo-Specia-Unroasted-P...


In [20]:
C4 = C4.append(df).reset_index(drop=True)
C4

,product_name,product_img,product_link
0,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
1,True Elements Chia Seeds 150g - Diet Food | Fi...,https://m.media-amazon.com/images/I/61ZsHU5URM...,https://www.amazon.in//True-Elements-Chia-Seed...
2,True Elements Chia Seeds 500gm - Premium Raw C...,https://m.media-amazon.com/images/I/51GB3smjJg...,https://www.amazon.in//True-Elements-Chia-Seed...
3,JIWA healthy by nature Organic Chia Seeds 200 ...,https://m.media-amazon.com/images/I/81kcoiUpZu...,https://www.amazon.in//Jiwa-Organic-Chia-Seeds...
4,"Amazon Brand - Vedaka Premium Raw Chia Seeds, ...",https://m.media-amazon.com/images/I/A1novFlMUG...,https://www.amazon.in//Vedaka-Premium-Chia-See...
...,...,...,...
913,Perfetto Hazelnut Flavoured Instant Coffee Pow...,https://m.media-amazon.com/images/I/61mpnNmM6l...,https://www.amazon.in//Perfetto-Hazelnut-Flavo...
914,TGL Co. The Good Life Company Euphoria Instant...,https://m.media-amazon.com/images/I/61cz59IlSY...,https://www.amazon.in//TGL-Co-Luxury-Coffee-Eu...
915,Nova Nova Instant Flavoured Coffee Powder - Cl...,https://m.media-amazon.com/images/I/71jC0rDVhb...,https://www.amazon.in//Nova-Instant-Flavoured-...
916,Narasus Coffee Filter Coffee Udhayam - 500 Grams,https://m.media-amazon.com/images/I/91s2QYxwSW...,https://www.amazon.in//Narasus-Coffee-Filter-U...


In [12]:
product_C4 = pd.DataFrame(reviews[reviews['product_code']=='product_C4']).reset_index(drop=True)
product_C4

,user_id,product_id,Ratings,Time_stamp,product_code
0,A31PN9Z2PBBVLA,B00000JGC4,4.0,955497600,product_C4
1,A1UV3PNF5TXAG6,B00000JGC4,1.0,940809600,product_C4
2,A2XG6UP4BQ9IEP,B00000JGC4,1.0,945561600,product_C4
3,ALDXG1061L8NV,B00000JGC4,1.0,958176000,product_C4
4,A2RWKDQXCU9KVT,B00000JGC4,5.0,962755200,product_C4
...,...,...,...,...,...
17321,A3ONT6755CDU4H,B00JNDPTC4,5.0,1403308800,product_C4
17322,A3E9K9G57L86CR,B00JNDPTC4,2.0,1404518400,product_C4
17323,A1A67UVYFQTM6R,B00JNDPTC4,5.0,1401148800,product_C4
17324,AAWQT1EK55X2G,B00JZQX1C4,1.0,1403222400,product_C4


In [13]:
len(product_C4.product_id.unique())

691

In [21]:
product_C4 = pd.DataFrame(reviews[reviews['product_code']=='product_C4']).reset_index(drop=True)
product_C4 =  pd.merge(product_C4, 
                       C4.iloc[:len(product_C4.product_id.unique())]\
                       .join(pd.Series(product_C4['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C4.to_csv('product_C4.csv',index=False)
pd.read_csv('product_C4.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A31PN9Z2PBBVLA,B00000JGC4,4.0,955497600,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
1,A1UV3PNF5TXAG6,B00000JGC4,1.0,940809600,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
2,A2XG6UP4BQ9IEP,B00000JGC4,1.0,945561600,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
3,ALDXG1061L8NV,B00000JGC4,1.0,958176000,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
4,A2RWKDQXCU9KVT,B00000JGC4,5.0,962755200,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
...,...,...,...,...,...,...,...,...
17321,A3ONT6755CDU4H,B00JNDPTC4,5.0,1403308800,product_C4,Organic Infinity Roasted Chicory Root Powder -...,https://m.media-amazon.com/images/I/71pVF1Uzbp...,https://www.amazon.in//Organic-Infinity-Roaste...
17322,A3E9K9G57L86CR,B00JNDPTC4,2.0,1404518400,product_C4,Organic Infinity Roasted Chicory Root Powder -...,https://m.media-amazon.com/images/I/71pVF1Uzbp...,https://www.amazon.in//Organic-Infinity-Roaste...
17323,A1A67UVYFQTM6R,B00JNDPTC4,5.0,1401148800,product_C4,Organic Infinity Roasted Chicory Root Powder -...,https://m.media-amazon.com/images/I/71pVF1Uzbp...,https://www.amazon.in//Organic-Infinity-Roaste...
17324,AAWQT1EK55X2G,B00JZQX1C4,1.0,1403222400,product_C4,Mysore Concerns | Brindavan Bold | South India...,https://m.media-amazon.com/images/I/715kWlBq3d...,https://www.amazon.in//Mysore-Concerns-Indian-...


### Product C5 dataframe

In [27]:
df = get_scraped('chindi rugs')
df

 15%|████████████▊                                                                      | 6/39 [00:10<00:57,  1.75s/it]


,product_name,product_img,product_link
0,vasac Round Braided Jute & Chindi Area Rug | H...,https://m.media-amazon.com/images/I/71GBe0JNQ5...,https://www.amazon.in//vasac-Braided-Handmade-...
1,"SYN STOR Home Floor Mat, Dari, Chindi Rug, for...",https://m.media-amazon.com/images/I/81ts2D3hy-...,https://www.amazon.in//SYN-STOR-Beautifully-Ki...
2,"EKEFFY Handwoven Natural Fibres, Braided Rever...",https://m.media-amazon.com/images/I/81lopFj6Wo...,https://www.amazon.in//EKEFFY-Handwoven-Natura...
3,HOMADORN Handmade Sustainable Printed Modern D...,https://m.media-amazon.com/images/I/91XCqfwkzL...,https://www.amazon.in//HOMADORN-Handmade-Enhan...
4,M TAKE Handwoven Jute Rug.Braided Reversible C...,https://m.media-amazon.com/images/I/61sofgEGwZ...,https://www.amazon.in//Handwoven-Rug-Braided-R...
...,...,...,...
301,Vanee Handloom Jute & Chindi Hand Wovan Rug fo...,https://m.media-amazon.com/images/I/71BZGrqxg8...,https://www.amazon.in//Vanee-Handloom-Chindi-W...
302,"Spinrific Handwoven chindi Round Rug, Natural ...",https://m.media-amazon.com/images/I/71xmcbgS99...,https://www.amazon.in//Spinrific-Handwoven-Nat...
303,Kasper Collections Classic Design Carpet Natur...,https://m.media-amazon.com/images/I/715dLpQk18...,https://www.amazon.in//Kasper-Collections-Clas...
304,Beautiful Home Cotton Chindi Braided Floor Rug...,https://m.media-amazon.com/images/I/91MsvGoOV3...,https://www.amazon.in//Beautiful-Home-Cotton-B...


In [28]:
C5 = df.copy()
C5

,product_name,product_img,product_link
0,vasac Round Braided Jute & Chindi Area Rug | H...,https://m.media-amazon.com/images/I/71GBe0JNQ5...,https://www.amazon.in//vasac-Braided-Handmade-...
1,"SYN STOR Home Floor Mat, Dari, Chindi Rug, for...",https://m.media-amazon.com/images/I/81ts2D3hy-...,https://www.amazon.in//SYN-STOR-Beautifully-Ki...
2,"EKEFFY Handwoven Natural Fibres, Braided Rever...",https://m.media-amazon.com/images/I/81lopFj6Wo...,https://www.amazon.in//EKEFFY-Handwoven-Natura...
3,HOMADORN Handmade Sustainable Printed Modern D...,https://m.media-amazon.com/images/I/91XCqfwkzL...,https://www.amazon.in//HOMADORN-Handmade-Enhan...
4,M TAKE Handwoven Jute Rug.Braided Reversible C...,https://m.media-amazon.com/images/I/61sofgEGwZ...,https://www.amazon.in//Handwoven-Rug-Braided-R...
...,...,...,...
301,Vanee Handloom Jute & Chindi Hand Wovan Rug fo...,https://m.media-amazon.com/images/I/71BZGrqxg8...,https://www.amazon.in//Vanee-Handloom-Chindi-W...
302,"Spinrific Handwoven chindi Round Rug, Natural ...",https://m.media-amazon.com/images/I/71xmcbgS99...,https://www.amazon.in//Spinrific-Handwoven-Nat...
303,Kasper Collections Classic Design Carpet Natur...,https://m.media-amazon.com/images/I/715dLpQk18...,https://www.amazon.in//Kasper-Collections-Clas...
304,Beautiful Home Cotton Chindi Braided Floor Rug...,https://m.media-amazon.com/images/I/91MsvGoOV3...,https://www.amazon.in//Beautiful-Home-Cotton-B...


In [ ]:
# C5 = C5.append(df).reset_index(drop=True)
# C5

In [25]:
product_C5 = pd.DataFrame(reviews[reviews['product_code']=='product_C5']).reset_index(drop=True)
product_C5

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1PK1QCQAU0KY6,B00002EQC5,5.0,1170460800,product_C5
1,AVR1PAA0FYE3,B00002EQC5,4.0,960681600,product_C5
2,A36QY2QCCS16E2,B00004RFC5,2.0,1212969600,product_C5
3,A298UEB02VH9OO,B00004RFC5,5.0,1403136000,product_C5
4,AJ4EJQAKP058L,B00004RFC5,5.0,1374537600,product_C5
...,...,...,...,...,...
585,A11KW5S7UT7PGK,B00009V3C5,4.0,1368403200,product_C5
586,A1T7EHCMZ92TKC,B0000D15C5,4.0,1070668800,product_C5
587,A1KWBSH5VN8WFR,B0000D15C5,5.0,1068940800,product_C5
588,ABF0ARHORHUUC,B0000D15C5,4.0,1073433600,product_C5


In [26]:
len(product_C5.product_id.unique())

22

In [29]:
product_C5 = pd.DataFrame(reviews[reviews['product_code']=='product_C5']).reset_index(drop=True)
product_C5 =  pd.merge(product_C5, 
                       C5.iloc[:len(product_C5.product_id.unique())]\
                       .join(pd.Series(product_C5['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C5.to_csv('product_C5.csv',index=False)
pd.read_csv('product_C5.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1PK1QCQAU0KY6,B00002EQC5,5.0,1170460800,product_C5,vasac Round Braided Jute & Chindi Area Rug | H...,https://m.media-amazon.com/images/I/71GBe0JNQ5...,https://www.amazon.in//vasac-Braided-Handmade-...
1,AVR1PAA0FYE3,B00002EQC5,4.0,960681600,product_C5,vasac Round Braided Jute & Chindi Area Rug | H...,https://m.media-amazon.com/images/I/71GBe0JNQ5...,https://www.amazon.in//vasac-Braided-Handmade-...
2,A36QY2QCCS16E2,B00004RFC5,2.0,1212969600,product_C5,"SYN STOR Home Floor Mat, Dari, Chindi Rug, for...",https://m.media-amazon.com/images/I/81ts2D3hy-...,https://www.amazon.in//SYN-STOR-Beautifully-Ki...
3,A298UEB02VH9OO,B00004RFC5,5.0,1403136000,product_C5,"SYN STOR Home Floor Mat, Dari, Chindi Rug, for...",https://m.media-amazon.com/images/I/81ts2D3hy-...,https://www.amazon.in//SYN-STOR-Beautifully-Ki...
4,AJ4EJQAKP058L,B00004RFC5,5.0,1374537600,product_C5,"SYN STOR Home Floor Mat, Dari, Chindi Rug, for...",https://m.media-amazon.com/images/I/81ts2D3hy-...,https://www.amazon.in//SYN-STOR-Beautifully-Ki...
...,...,...,...,...,...,...,...,...
585,A11KW5S7UT7PGK,B00009V3C5,4.0,1368403200,product_C5,Brand Zone Brand Zone Handmade Jute Chindi Rou...,https://m.media-amazon.com/images/I/71ocfNzQfg...,https://www.amazon.in//Brand-Zone-Handmade-Nat...
586,A1T7EHCMZ92TKC,B0000D15C5,4.0,1070668800,product_C5,"SASHAA WORLD Modern Chindi Rug for Kitchen,Liv...",https://m.media-amazon.com/images/I/818ybd0G0m...,https://www.amazon.in//SASHAA-Modern-Kitchen-B...
587,A1KWBSH5VN8WFR,B0000D15C5,5.0,1068940800,product_C5,"SASHAA WORLD Modern Chindi Rug for Kitchen,Liv...",https://m.media-amazon.com/images/I/818ybd0G0m...,https://www.amazon.in//SASHAA-Modern-Kitchen-B...
588,ABF0ARHORHUUC,B0000D15C5,4.0,1073433600,product_C5,"SASHAA WORLD Modern Chindi Rug for Kitchen,Liv...",https://m.media-amazon.com/images/I/818ybd0G0m...,https://www.amazon.in//SASHAA-Modern-Kitchen-B...


### Product C6 dataframe

In [37]:
df = get_scraped('chana dal')
df

 15%|████████████▊                                                                      | 6/39 [00:10<00:55,  1.68s/it]


,product_name,product_img,product_link
0,"Tata Sampann Unpolished Chana Dal, 1kg",https://m.media-amazon.com/images/I/71nx6ZI0Pm...,https://www.amazon.in//Tata-Sampann-Chana-Dal-...
1,"Vedaka Popular Chana Dal, 1kg",https://m.media-amazon.com/images/I/91+85gJpsh...,https://www.amazon.in//Amazon-Brand-Vedaka-Pop...
2,More Loose Chana Dal (1 kg),https://m.media-amazon.com/images/I/711F7wLBto...,https://www.amazon.in//More-Chana-Dal-Loose-50...
3,"Organic Tattva, Organic Chana (Bengalgram) Dal...",https://m.media-amazon.com/images/I/714C9ISGNs...,https://www.amazon.in//Organic-Tattva-Chana-Da...
4,"Amazon Brand - Vedaka Popular Chana Dal, 500 g",https://m.media-amazon.com/images/I/91+85gJpsh...,https://www.amazon.in//Amazon-Brand-Vedaka-Pop...
...,...,...,...
301,SONI FARMS Organic Unpolished Pure Chana Dal /...,https://m.media-amazon.com/images/I/61e0Hxiqrt...,https://www.amazon.in//SONI-FARMS-Organic-Unpo...
302,Chana dal,https://m.media-amazon.com/images/I/71P88oROJe...,https://www.amazon.in//Kiran-Chana-dal/dp/B084...
303,USDA Organic Certified Simply Jaivik Chana Dal...,https://m.media-amazon.com/images/I/818Iw8fsoq...,https://www.amazon.in//Organic-Certified-Simpl...
304,"DESI AAHAR Chana Dal, 1KG",https://m.media-amazon.com/images/I/71zyrvUIs8...,https://www.amazon.in//DESI-AAHAR-Chana-Dal-1K...


In [34]:
C6 = df.copy()
C6

,product_name,product_img,product_link
0,Women's Rayon & Cotton Readymade Lehenga Choli,https://m.media-amazon.com/images/I/71XIZ3aa1O...,https://www.amazon.in//Saudeep-Ethnic-Gujrati-...
1,Navaratri Specia royalty with this dola silk l...,https://m.media-amazon.com/images/I/91JHSCt95Z...,https://www.amazon.in//MISHILL-CREATORS-Navara...
2,Kkalakriti Gujrati Garba Dance Chaniya Choli L...,https://m.media-amazon.com/images/I/61GEL7UcP9...,https://www.amazon.in//Kkalakriti-Gujrati-Trad...
3,Women's Maslin Cotton Diwali Wedding Party Wea...,https://m.media-amazon.com/images/I/819CEETq+3...,https://www.amazon.in//Divine-International-Tr...
4,Women's Semi-Stitched Floral Printed Satin Leh...,https://m.media-amazon.com/images/I/61E+Exjc0M...,https://www.amazon.in//Stitched-Floral-Printed...
...,...,...,...
301,Women's Net Semi-Stitched Lehenga Choli With B...,https://m.media-amazon.com/images/I/61rsLe0nS9...,https://www.amazon.in//sojitra-enterprise-Wome...
302,Pink Green Navratri Special Woman Festival wea...,https://m.media-amazon.com/images/I/61Q+72Fe65...,https://www.amazon.in//Navratri-Special-Festiv...
303,Women's Silk Semi-stitched Lehenga Choli with ...,https://m.media-amazon.com/images/I/51003b7c7j...,https://www.amazon.in//Drashti-Villa-Taffeta-S...
304,Women's Pink Net Embroidered Semi Stictehd Leh...,https://m.media-amazon.com/images/I/71fJkbhNJe...,https://www.amazon.in//Mordenfab-Womens-Embroi...


In [38]:
C6 = C6.append(df).reset_index(drop=True)
C6

,product_name,product_img,product_link
0,Women's Rayon & Cotton Readymade Lehenga Choli,https://m.media-amazon.com/images/I/71XIZ3aa1O...,https://www.amazon.in//Saudeep-Ethnic-Gujrati-...
1,Navaratri Specia royalty with this dola silk l...,https://m.media-amazon.com/images/I/91JHSCt95Z...,https://www.amazon.in//MISHILL-CREATORS-Navara...
2,Kkalakriti Gujrati Garba Dance Chaniya Choli L...,https://m.media-amazon.com/images/I/61GEL7UcP9...,https://www.amazon.in//Kkalakriti-Gujrati-Trad...
3,Women's Maslin Cotton Diwali Wedding Party Wea...,https://m.media-amazon.com/images/I/819CEETq+3...,https://www.amazon.in//Divine-International-Tr...
4,Women's Semi-Stitched Floral Printed Satin Leh...,https://m.media-amazon.com/images/I/61E+Exjc0M...,https://www.amazon.in//Stitched-Floral-Printed...
...,...,...,...
913,SONI FARMS Organic Unpolished Pure Chana Dal /...,https://m.media-amazon.com/images/I/61e0Hxiqrt...,https://www.amazon.in//SONI-FARMS-Organic-Unpo...
914,Chana dal,https://m.media-amazon.com/images/I/71P88oROJe...,https://www.amazon.in//Kiran-Chana-dal/dp/B084...
915,USDA Organic Certified Simply Jaivik Chana Dal...,https://m.media-amazon.com/images/I/818Iw8fsoq...,https://www.amazon.in//Organic-Certified-Simpl...
916,"DESI AAHAR Chana Dal, 1KG",https://m.media-amazon.com/images/I/71zyrvUIs8...,https://www.amazon.in//DESI-AAHAR-Chana-Dal-1K...


In [31]:
product_C6 = pd.DataFrame(reviews[reviews['product_code']=='product_C6']).reset_index(drop=True)
product_C6

,user_id,product_id,Ratings,Time_stamp,product_code
0,A23KUCXWHBCPT6,B00001X5C6,5.0,946512000,product_C6
1,A9BXSJ8H4115U,B00001X5C6,5.0,950400000,product_C6
2,AFLUB7EK21DAH,B00001X5C6,5.0,948844800,product_C6
3,A216H9L5WUAUO6,B00003CWC6,5.0,965865600,product_C6
4,A2ARE6Q0B7C8UC,B00004RFC6,2.0,1400371200,product_C6
...,...,...,...,...,...
11317,A1JTTI1KH3DOCY,B00JKM0GC6,2.0,1400544000,product_C6
11318,A2J8NCGR14OSJG,B00K5ZJDC6,5.0,1405036800,product_C6
11319,A13SAVYJLEH4D1,B00K7ZYQC6,5.0,1404950400,product_C6
11320,A1FQG1NNNKHN9U,B00KEI6EC6,2.0,1405728000,product_C6


In [32]:
len(product_C6.product_id.unique())

763

In [39]:
product_C6 = pd.DataFrame(reviews[reviews['product_code']=='product_C6']).reset_index(drop=True)
product_C6 =  pd.merge(product_C6, 
                       C6.iloc[:len(product_C6.product_id.unique())]\
                       .join(pd.Series(product_C6['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C6.to_csv('product_C6.csv',index=False)
pd.read_csv('product_C6.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A23KUCXWHBCPT6,B00001X5C6,5.0,946512000,product_C6,Women's Rayon & Cotton Readymade Lehenga Choli,https://m.media-amazon.com/images/I/71XIZ3aa1O...,https://www.amazon.in//Saudeep-Ethnic-Gujrati-...
1,A9BXSJ8H4115U,B00001X5C6,5.0,950400000,product_C6,Women's Rayon & Cotton Readymade Lehenga Choli,https://m.media-amazon.com/images/I/71XIZ3aa1O...,https://www.amazon.in//Saudeep-Ethnic-Gujrati-...
2,AFLUB7EK21DAH,B00001X5C6,5.0,948844800,product_C6,Women's Rayon & Cotton Readymade Lehenga Choli,https://m.media-amazon.com/images/I/71XIZ3aa1O...,https://www.amazon.in//Saudeep-Ethnic-Gujrati-...
3,A216H9L5WUAUO6,B00003CWC6,5.0,965865600,product_C6,Navaratri Specia royalty with this dola silk l...,https://m.media-amazon.com/images/I/91JHSCt95Z...,https://www.amazon.in//MISHILL-CREATORS-Navara...
4,A2ARE6Q0B7C8UC,B00004RFC6,2.0,1400371200,product_C6,Kkalakriti Gujrati Garba Dance Chaniya Choli L...,https://m.media-amazon.com/images/I/61GEL7UcP9...,https://www.amazon.in//Kkalakriti-Gujrati-Trad...
...,...,...,...,...,...,...,...,...
11317,A1JTTI1KH3DOCY,B00JKM0GC6,2.0,1400544000,product_C6,"Gropure Organic Chana Dal (Bengal Gram Split),...",https://m.media-amazon.com/images/I/81KDS2NtQj...,https://www.amazon.in//Gropure-Organic-Chana-B...
11318,A2J8NCGR14OSJG,B00K5ZJDC6,5.0,1405036800,product_C6,Bubble Organic Chana dal 500 Gram / Chemical F...,https://m.media-amazon.com/images/I/41SbGgrBja...,https://www.amazon.in//Bubble-Organic-Chana-Ch...
11319,A13SAVYJLEH4D1,B00K7ZYQC6,5.0,1404950400,product_C6,"Raunak Unpolished Chana Dal, 500g",https://m.media-amazon.com/images/I/81lRg0rbqt...,https://www.amazon.in//Raunak-Unpolished-Chana...
11320,A1FQG1NNNKHN9U,B00KEI6EC6,2.0,1405728000,product_C6,BLACK PARTRIDGE FARMS Organic Pulses Chana Dal...,https://m.media-amazon.com/images/I/71JJLlro9a...,https://www.amazon.in//BLACK-PARTRIDGE-FARMS-O...


### Product C7 dataframe

In [41]:
df = get_scraped('c7 pro back cover samsung')
df

 49%|███████████████████████████████████████▉                                          | 19/39 [00:23<00:24,  1.22s/it]


,product_name,product_img,product_link
0,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71ohREwnYb...,https://www.amazon.in//Rubberised-Samsung-Gala...
1,Hello Zone Exclusive Matte Finish Soft Back Ca...,https://m.media-amazon.com/images/I/51XExwNnzz...,https://www.amazon.in//Exclusive-Samsung-Galax...
2,Plus Protective Soft Transparent Shockproof Hy...,https://m.media-amazon.com/images/I/413-OIJ1hV...,https://www.amazon.in//Plus-Protective-Shockpr...
3,Amazon Brand - Solimo Mobile Cover (Soft & Fle...,https://m.media-amazon.com/images/I/516zOyHKDY...,https://www.amazon.in//Amazon-Brand-Shockproof...
4,ANKIRANT Back Cover For Samsung Galaxy C7 Pro ...,https://m.media-amazon.com/images/I/81PvvOvhb4...,https://www.amazon.in//Ankirant-Silicon-Cover-...
...,...,...,...
301,Oye Stuff Pebbles Butterfly Designer Slim Hard...,https://m.media-amazon.com/images/I/61CTAW-2Yh...,https://www.amazon.in//Butterfly-Designer-Sams...
302,Oye Stuff Stripe Hearts Designer Slim Hard Pla...,https://m.media-amazon.com/images/I/61ye+i7893...,https://www.amazon.in//Designer-Plastic-Samsun...
303,RGSVCases White Polka Dots with Pink Bow Patte...,https://m.media-amazon.com/images/I/51u3-5Z2E2...,https://www.amazon.in//RGSVCases-Pattern-Print...
304,Generic Floral Painting Background Printed Des...,https://m.media-amazon.com/images/I/61UBRxRcUl...,https://www.amazon.in//Generic-Painting-Backgr...


In [42]:
C7 = df.copy()
C7

,product_name,product_img,product_link
0,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71ohREwnYb...,https://www.amazon.in//Rubberised-Samsung-Gala...
1,Hello Zone Exclusive Matte Finish Soft Back Ca...,https://m.media-amazon.com/images/I/51XExwNnzz...,https://www.amazon.in//Exclusive-Samsung-Galax...
2,Plus Protective Soft Transparent Shockproof Hy...,https://m.media-amazon.com/images/I/413-OIJ1hV...,https://www.amazon.in//Plus-Protective-Shockpr...
3,Amazon Brand - Solimo Mobile Cover (Soft & Fle...,https://m.media-amazon.com/images/I/516zOyHKDY...,https://www.amazon.in//Amazon-Brand-Shockproof...
4,ANKIRANT Back Cover For Samsung Galaxy C7 Pro ...,https://m.media-amazon.com/images/I/81PvvOvhb4...,https://www.amazon.in//Ankirant-Silicon-Cover-...
...,...,...,...
301,Oye Stuff Pebbles Butterfly Designer Slim Hard...,https://m.media-amazon.com/images/I/61CTAW-2Yh...,https://www.amazon.in//Butterfly-Designer-Sams...
302,Oye Stuff Stripe Hearts Designer Slim Hard Pla...,https://m.media-amazon.com/images/I/61ye+i7893...,https://www.amazon.in//Designer-Plastic-Samsun...
303,RGSVCases White Polka Dots with Pink Bow Patte...,https://m.media-amazon.com/images/I/51u3-5Z2E2...,https://www.amazon.in//RGSVCases-Pattern-Print...
304,Generic Floral Painting Background Printed Des...,https://m.media-amazon.com/images/I/61UBRxRcUl...,https://www.amazon.in//Generic-Painting-Backgr...


In [ ]:
# C7 = C7.append(df).reset_index(drop=True)
# C7

In [40]:
product_C7 = pd.DataFrame(reviews[reviews['product_code']=='product_C7']).reset_index(drop=True)
product_C7

,user_id,product_id,Ratings,Time_stamp,product_code
0,A1YEHGDRYNQRIS,B00002EQC7,4.0,964396800,product_C7
1,A1PFV1JGK2NC3N,B00002EQC7,5.0,1051747200,product_C7
2,A1SMP7MFF2YUXF,B00004YMC7,1.0,1401148800,product_C7
3,A1T2VVTCYK0MOS,B00004YMC7,5.0,1012521600,product_C7
4,A17YZDMBJH2XVU,B00004YMC7,1.0,1305331200,product_C7
...,...,...,...,...,...
312,A2IKA32YQXLDUF,B0000AE6C7,4.0,1264032000,product_C7
313,A1BF8P83D76RY3,B0000BZHC7,5.0,1156723200,product_C7
314,A23O9WCKXAMPJS,B0000BZHC7,5.0,1132531200,product_C7
315,A318VRHUBIUMB3,B0000DCTC7,5.0,1365897600,product_C7


In [13]:
len(product_C7.product_id.unique())

691

In [43]:
product_C7 = pd.DataFrame(reviews[reviews['product_code']=='product_C7']).reset_index(drop=True)
product_C7 =  pd.merge(product_C7, 
                       C7.iloc[:len(product_C7.product_id.unique())]\
                       .join(pd.Series(product_C7['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C7.to_csv('product_C7.csv',index=False)
pd.read_csv('product_C7.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1YEHGDRYNQRIS,B00002EQC7,4.0,964396800,product_C7,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71ohREwnYb...,https://www.amazon.in//Rubberised-Samsung-Gala...
1,A1PFV1JGK2NC3N,B00002EQC7,5.0,1051747200,product_C7,Hello Zone Exclusive Dotted Matte Finish Soft ...,https://m.media-amazon.com/images/I/71ohREwnYb...,https://www.amazon.in//Rubberised-Samsung-Gala...
2,A1SMP7MFF2YUXF,B00004YMC7,1.0,1401148800,product_C7,Hello Zone Exclusive Matte Finish Soft Back Ca...,https://m.media-amazon.com/images/I/51XExwNnzz...,https://www.amazon.in//Exclusive-Samsung-Galax...
3,A1T2VVTCYK0MOS,B00004YMC7,5.0,1012521600,product_C7,Hello Zone Exclusive Matte Finish Soft Back Ca...,https://m.media-amazon.com/images/I/51XExwNnzz...,https://www.amazon.in//Exclusive-Samsung-Galax...
4,A17YZDMBJH2XVU,B00004YMC7,1.0,1305331200,product_C7,Hello Zone Exclusive Matte Finish Soft Back Ca...,https://m.media-amazon.com/images/I/51XExwNnzz...,https://www.amazon.in//Exclusive-Samsung-Galax...
...,...,...,...,...,...,...,...,...
312,A2IKA32YQXLDUF,B0000AE6C7,4.0,1264032000,product_C7,"KC Back Cover for Samsung Galaxy C7 Pro, Soft ...",https://m.media-amazon.com/images/I/71LxRUvL-n...,https://www.amazon.in//Flower-Standing-Girl-Ga...
313,A1BF8P83D76RY3,B0000BZHC7,5.0,1156723200,product_C7,Casotec Plastic Moon Pattern Print Design 3D P...,https://m.media-amazon.com/images/I/713YvEbgtI...,https://www.amazon.in//Casotec-Pattern-Design-...
314,A23O9WCKXAMPJS,B0000BZHC7,5.0,1132531200,product_C7,Casotec Plastic Moon Pattern Print Design 3D P...,https://m.media-amazon.com/images/I/713YvEbgtI...,https://www.amazon.in//Casotec-Pattern-Design-...
315,A318VRHUBIUMB3,B0000DCTC7,5.0,1365897600,product_C7,Casotec Puzzle Pattern 3D Printed Designer Har...,https://m.media-amazon.com/images/I/717Y6InRn2...,https://www.amazon.in//Casotec-Pattern-Printed...


### Product C4 dataframe

In [19]:
df = get_scraped('chicory powder')
df

 15%|████████████▊                                                                      | 6/39 [00:11<01:01,  1.87s/it]


,product_name,product_img,product_link
0,"Urban Platter Roasted Chicory Root Powder, 250...",https://m.media-amazon.com/images/I/81vRFwGrTJ...,https://www.amazon.in//Urban-Platter-Chicory-P...
1,NATURE'S GIFT - FOR THOSE WHO CARE'S Roasted C...,https://m.media-amazon.com/images/I/71a2jCJe3N...,https://www.amazon.in//NATURES-GIFT-Roasted-Ch...
2,"Urban Platter Instant Chicory Root Powder, 150...",https://m.media-amazon.com/images/I/71+i75yNHO...,https://www.amazon.in//Platter-Caffeine-Instan...
3,Organic Prime Roasted Chicory Root Powder - 20...,https://m.media-amazon.com/images/I/51-8NNh-eG...,https://www.amazon.in//Organic-Prime-Roasted-C...
4,B Naturall Instant Chicory Root Powder - 100 G...,https://m.media-amazon.com/images/I/61DEuZ6KkA...,https://www.amazon.in//Naturall-Roasted-Chicor...
...,...,...,...
301,Perfetto Hazelnut Flavoured Instant Coffee Pow...,https://m.media-amazon.com/images/I/61mpnNmM6l...,https://www.amazon.in//Perfetto-Hazelnut-Flavo...
302,TGL Co. The Good Life Company Euphoria Instant...,https://m.media-amazon.com/images/I/61cz59IlSY...,https://www.amazon.in//TGL-Co-Luxury-Coffee-Eu...
303,Nova Nova Instant Flavoured Coffee Powder - Cl...,https://m.media-amazon.com/images/I/71jC0rDVhb...,https://www.amazon.in//Nova-Instant-Flavoured-...
304,Narasus Coffee Filter Coffee Udhayam - 500 Grams,https://m.media-amazon.com/images/I/91s2QYxwSW...,https://www.amazon.in//Narasus-Coffee-Filter-U...


In [15]:
C4 = df.copy()
C4

,product_name,product_img,product_link
0,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
1,True Elements Chia Seeds 150g - Diet Food | Fi...,https://m.media-amazon.com/images/I/61ZsHU5URM...,https://www.amazon.in//True-Elements-Chia-Seed...
2,True Elements Chia Seeds 500gm - Premium Raw C...,https://m.media-amazon.com/images/I/51GB3smjJg...,https://www.amazon.in//True-Elements-Chia-Seed...
3,JIWA healthy by nature Organic Chia Seeds 200 ...,https://m.media-amazon.com/images/I/81kcoiUpZu...,https://www.amazon.in//Jiwa-Organic-Chia-Seeds...
4,"Amazon Brand - Vedaka Premium Raw Chia Seeds, ...",https://m.media-amazon.com/images/I/A1novFlMUG...,https://www.amazon.in//Vedaka-Premium-Chia-See...
...,...,...,...
301,"By Nature White Chia Seeds, 100 Gm",https://m.media-amazon.com/images/I/91bfww354Q...,https://www.amazon.in//Nature-White-Chia-Seeds...
302,ECONBIO ROOTS Certified 100% Organic Combo Raw...,https://m.media-amazon.com/images/I/71MZ0ZQB3b...,https://www.amazon.in//Econbioroots-Certified-...
303,YUVIKA Chia Seeds - Omega 3 - Anti Oxidant - G...,https://m.media-amazon.com/images/I/61d-V0+4Et...,https://www.amazon.in//YUVIKA-Chia-Seeds-Peru-...
304,Medo Specia - Raw Unroasted Chia Seeds 350 gm ...,https://m.media-amazon.com/images/I/61N-t7vxF-...,https://www.amazon.in//Medo-Specia-Unroasted-P...


In [20]:
C4 = C4.append(df).reset_index(drop=True)
C4

,product_name,product_img,product_link
0,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
1,True Elements Chia Seeds 150g - Diet Food | Fi...,https://m.media-amazon.com/images/I/61ZsHU5URM...,https://www.amazon.in//True-Elements-Chia-Seed...
2,True Elements Chia Seeds 500gm - Premium Raw C...,https://m.media-amazon.com/images/I/51GB3smjJg...,https://www.amazon.in//True-Elements-Chia-Seed...
3,JIWA healthy by nature Organic Chia Seeds 200 ...,https://m.media-amazon.com/images/I/81kcoiUpZu...,https://www.amazon.in//Jiwa-Organic-Chia-Seeds...
4,"Amazon Brand - Vedaka Premium Raw Chia Seeds, ...",https://m.media-amazon.com/images/I/A1novFlMUG...,https://www.amazon.in//Vedaka-Premium-Chia-See...
...,...,...,...
913,Perfetto Hazelnut Flavoured Instant Coffee Pow...,https://m.media-amazon.com/images/I/61mpnNmM6l...,https://www.amazon.in//Perfetto-Hazelnut-Flavo...
914,TGL Co. The Good Life Company Euphoria Instant...,https://m.media-amazon.com/images/I/61cz59IlSY...,https://www.amazon.in//TGL-Co-Luxury-Coffee-Eu...
915,Nova Nova Instant Flavoured Coffee Powder - Cl...,https://m.media-amazon.com/images/I/71jC0rDVhb...,https://www.amazon.in//Nova-Instant-Flavoured-...
916,Narasus Coffee Filter Coffee Udhayam - 500 Grams,https://m.media-amazon.com/images/I/91s2QYxwSW...,https://www.amazon.in//Narasus-Coffee-Filter-U...


In [12]:
product_C4 = pd.DataFrame(reviews[reviews['product_code']=='product_C4']).reset_index(drop=True)
product_C4

,user_id,product_id,Ratings,Time_stamp,product_code
0,A31PN9Z2PBBVLA,B00000JGC4,4.0,955497600,product_C4
1,A1UV3PNF5TXAG6,B00000JGC4,1.0,940809600,product_C4
2,A2XG6UP4BQ9IEP,B00000JGC4,1.0,945561600,product_C4
3,ALDXG1061L8NV,B00000JGC4,1.0,958176000,product_C4
4,A2RWKDQXCU9KVT,B00000JGC4,5.0,962755200,product_C4
...,...,...,...,...,...
17321,A3ONT6755CDU4H,B00JNDPTC4,5.0,1403308800,product_C4
17322,A3E9K9G57L86CR,B00JNDPTC4,2.0,1404518400,product_C4
17323,A1A67UVYFQTM6R,B00JNDPTC4,5.0,1401148800,product_C4
17324,AAWQT1EK55X2G,B00JZQX1C4,1.0,1403222400,product_C4


In [13]:
len(product_C4.product_id.unique())

691

In [21]:
product_C4 = pd.DataFrame(reviews[reviews['product_code']=='product_C4']).reset_index(drop=True)
product_C4 =  pd.merge(product_C4, 
                       C4.iloc[:len(product_C4.product_id.unique())]\
                       .join(pd.Series(product_C4['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_C4.to_csv('product_C4.csv',index=False)
pd.read_csv('product_C4.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A31PN9Z2PBBVLA,B00000JGC4,4.0,955497600,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
1,A1UV3PNF5TXAG6,B00000JGC4,1.0,940809600,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
2,A2XG6UP4BQ9IEP,B00000JGC4,1.0,945561600,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
3,ALDXG1061L8NV,B00000JGC4,1.0,958176000,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
4,A2RWKDQXCU9KVT,B00000JGC4,5.0,962755200,product_C4,Neuherbs Raw Unroasted Chia Seeds for eating w...,https://m.media-amazon.com/images/I/61suasFilD...,https://www.amazon.in//Neuherbs-Unroasted-Chia...
...,...,...,...,...,...,...,...,...
17321,A3ONT6755CDU4H,B00JNDPTC4,5.0,1403308800,product_C4,Organic Infinity Roasted Chicory Root Powder -...,https://m.media-amazon.com/images/I/71pVF1Uzbp...,https://www.amazon.in//Organic-Infinity-Roaste...
17322,A3E9K9G57L86CR,B00JNDPTC4,2.0,1404518400,product_C4,Organic Infinity Roasted Chicory Root Powder -...,https://m.media-amazon.com/images/I/71pVF1Uzbp...,https://www.amazon.in//Organic-Infinity-Roaste...
17323,A1A67UVYFQTM6R,B00JNDPTC4,5.0,1401148800,product_C4,Organic Infinity Roasted Chicory Root Powder -...,https://m.media-amazon.com/images/I/71pVF1Uzbp...,https://www.amazon.in//Organic-Infinity-Roaste...
17324,AAWQT1EK55X2G,B00JZQX1C4,1.0,1403222400,product_C4,Mysore Concerns | Brindavan Bold | South India...,https://m.media-amazon.com/images/I/715kWlBq3d...,https://www.amazon.in//Mysore-Concerns-Indian-...


### Product MI dataframe

In [251]:
df = get_scraped('mi products&i=electronics')
MI = df.copy()
MI

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Mi Corded & Cordless Waterproof Beard Trimmer ...,https://m.media-amazon.com/images/I/61ApUe3q-P...,https://www.amazon.in//Mi-Corded-Cordless-Wate...
1,MI Rechargeable Electric Toothbrush T100 for A...,https://m.media-amazon.com/images/I/61R3a2YTMm...,https://www.amazon.in//MI-Rechargeable-Electri...
2,"Mi Step Out 12 L Mini Backpack (Small Size, Da...",https://m.media-amazon.com/images/I/71FRRQKKSS...,https://www.amazon.in//Step-Backpack-Small-Wat...
3,Xiaomi Selfie Stick with Micro USB Rechargeabl...,https://m.media-amazon.com/images/I/41jv7D7b8U...,https://www.amazon.in//Selfie-Tripod-Rechargea...
4,"Mi Robot Vacuum-Mop P, Best-in-class Laser Nav...",https://m.media-amazon.com/images/I/61yy90wpYL...,https://www.amazon.in//Vacuum-Mop-Intelligent-...
...,...,...,...
931,Liddu Wristband Band Straps for Xiaomi Origina...,https://m.media-amazon.com/images/I/6119Y83Gsz...,https://www.amazon.in//Liddu-Wristband-Band-St...
932,JugaaduStore Back Case Cover for Xiaomi Mi 11X...,https://m.media-amazon.com/images/I/71WbsEUjVh...,https://www.amazon.in//JugaaduStore-Xiaomi-Mi-...
933,Camlin Kokuyo Artist Acrylic Colour 500 ml Ser...,https://m.media-amazon.com/images/I/61Ag9zQ9Ky...,https://www.amazon.in//Camlin-Kokuyo-Artist-Ac...
934,Hianjoo Soft Silicone Case Compatible for Redm...,https://m.media-amazon.com/images/I/51O5yQ3g8M...,https://www.amazon.in//Hianjoo-Silicone-Compat...


In [13]:
df = get_scraped('mi products&i=electronics')
MI = df.copy()
MI

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Rapidotzz Band Straps Compatible for Mi Band 4...,https://m.media-amazon.com/images/I/51iHAUQxv7...,https://www.amazon.in//Rapidotzz-Band-Straps-C...
1,MI Original 27W Superfast Charging Adapter (20...,https://m.media-amazon.com/images/I/61r+hFh5ZB...,https://www.amazon.in//Original-Superfast-Char...
2,Amazon Brand - Solimo Back Cover for Mi 10T / ...,https://m.media-amazon.com/images/I/61HlsrBh5H...,https://www.amazon.in//Amazon-Brand-Solimo-Fle...
3,Xiaomi Mi 4A Dual_Band Ethernet 1200Mbps Speed...,https://m.media-amazon.com/images/I/51CerdrNfI...,https://www.amazon.in//Dualband-1200Mbps-Frequ...
4,Noise ColorFit Pulse Spo2 Smart Watch with 10 ...,https://m.media-amazon.com/images/I/61epn29QG0...,https://www.amazon.in//Noise-ColorFit-Smartwat...
...,...,...,...
259,Boxxo 9H+ High Definition [HD] Tempered Glass ...,https://m.media-amazon.com/images/I/81QoPM7riN...,https://www.amazon.in//Definition-Redmi-Note-P...
260,Boxxo 9H+ High Definition [HD] Tempered Glass ...,https://m.media-amazon.com/images/I/81Y-vvJAjC...,https://www.amazon.in//Definition-Mi-Redmi-Not...
261,Amazon Brand - Solimo Designer Product Photogr...,https://m.media-amazon.com/images/I/713YWYjhjI...,https://www.amazon.in//Amazon-Brand-Designer-P...
262,Mi Complete Protect (1year) - Brand Authorised...,https://m.media-amazon.com/images/I/31yES2I6Ip...,https://www.amazon.in//Complete-Protect-1year-...


In [256]:
product_MI = pd.DataFrame(reviews[reviews['product_code']=='product_MI']).reset_index(drop=True)
product_MI =  pd.merge(product_MI, 
                       MI.iloc[:len(product_MI.product_id.unique())]\
                       .join(pd.Series(product_MI['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_MI.to_csv('product_MI.csv',index=False)
pd.read_csv('product_MI.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1JJRKEOKBNNOX,B000001OMI,1.0,1158710400,product_MI,Mi Corded & Cordless Waterproof Beard Trimmer ...,https://m.media-amazon.com/images/I/61ApUe3q-P...,https://www.amazon.in//Mi-Corded-Cordless-Wate...
1,AAOTEO8XB48SL,B000001OMI,5.0,1404172800,product_MI,Mi Corded & Cordless Waterproof Beard Trimmer ...,https://m.media-amazon.com/images/I/61ApUe3q-P...,https://www.amazon.in//Mi-Corded-Cordless-Wate...
2,A6TK95DKOTVBI,B000001OMI,1.0,1390176000,product_MI,Mi Corded & Cordless Waterproof Beard Trimmer ...,https://m.media-amazon.com/images/I/61ApUe3q-P...,https://www.amazon.in//Mi-Corded-Cordless-Wate...
3,A38DZGZZ7RN5MR,B000001OMI,5.0,1394582400,product_MI,Mi Corded & Cordless Waterproof Beard Trimmer ...,https://m.media-amazon.com/images/I/61ApUe3q-P...,https://www.amazon.in//Mi-Corded-Cordless-Wate...
4,A22KW5YWAI5YDO,B000001OMI,4.0,1385251200,product_MI,Mi Corded & Cordless Waterproof Beard Trimmer ...,https://m.media-amazon.com/images/I/61ApUe3q-P...,https://www.amazon.in//Mi-Corded-Cordless-Wate...
...,...,...,...,...,...,...,...,...
10519,A2NI008JYXV8NO,B00KQRFMMI,5.0,1405641600,product_MI,Amazon Brand - Solimo Designer Product Photogr...,https://m.media-amazon.com/images/I/713YWYjhjI...,https://www.amazon.in//Amazon-Brand-Designer-P...
10520,A161NOSKB13Y70,B00KQRFMMI,5.0,1405036800,product_MI,Amazon Brand - Solimo Designer Product Photogr...,https://m.media-amazon.com/images/I/713YWYjhjI...,https://www.amazon.in//Amazon-Brand-Designer-P...
10521,ARAKTT5M3DDYA,B00KQRFMMI,5.0,1405468800,product_MI,Amazon Brand - Solimo Designer Product Photogr...,https://m.media-amazon.com/images/I/713YWYjhjI...,https://www.amazon.in//Amazon-Brand-Designer-P...
10522,AJXTCPC7ER84L,B00KQRFMMI,5.0,1403913600,product_MI,Amazon Brand - Solimo Designer Product Photogr...,https://m.media-amazon.com/images/I/713YWYjhjI...,https://www.amazon.in//Amazon-Brand-Designer-P...


### Product SI dataframe

In [22]:
df = get_scraped('slime&i=toys')
SI = df.copy()
SI

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,HOTKEI (Pack of 6 Slime) Multicolor Fruit Scen...,https://m.media-amazon.com/images/I/81WLIWanEC...,https://www.amazon.in//HOTKEI-Slime-Multicolor...
1,"Asterin Sales Slime kit, Colors Non-Toxic Clea...",https://m.media-amazon.com/images/I/61O+EGspJ-...,https://www.amazon.in//Asterin-Sales-Non-Toxic...
2,Webby Science Slime Putty Toy for Kids (6 Pieces),https://m.media-amazon.com/images/I/61oDonVJ1m...,https://www.amazon.in//Webby-Science-Slime-Put...
3,Royals Soft Slime Toy for Kids (Pearl - Pack o...,https://m.media-amazon.com/images/I/51lSPZYcgP...,https://www.amazon.in//ROYALS-Rubber-Pearl-Non...
4,HOTKEI (10 Slime kit) DIY Colored and Clear Sc...,https://m.media-amazon.com/images/I/71hXs9xNn7...,https://www.amazon.in//HOTKEI-Slime-Colored-Ki...
...,...,...,...
931,Pumpkin House DIY Slime Kit Putty Glitter Slim...,https://m.media-amazon.com/images/I/51xKAG7lN+...,https://www.amazon.in//Pumpkin-House-Glitter-U...
932,JULIANA Glittery Stress Relief Non Toxic Fluff...,https://m.media-amazon.com/images/I/91g+PlSBVT...,https://www.amazon.in//JULIANA-Glittery-Activa...
933,Poovin Crystal Clear Non Toxic Glitter Slime f...,https://m.media-amazon.com/images/I/81ArsBsoGp...,https://www.amazon.in//Poovin-Crystal-Glitter-...
934,JULIANA Combo- Fluffy Slime Pack Kids Adults t...,https://m.media-amazon.com/images/I/71Ercen1KK...,https://www.amazon.in//JULIANA-Exploring-Creat...


In [20]:
product_SI = pd.DataFrame(reviews[reviews['product_code']=='product_SI']).reset_index(drop=True)
product_SI

,user_id,product_id,Ratings,Time_stamp,product_code
0,ARQBN70XQND3O,B00000J1SI,1.0,958694400,product_SI
1,A1B10WSCRZTPPT,B00000J1SI,5.0,932169600,product_SI
2,AYF026D7GCM5Y,B00000J1SI,4.0,962582400,product_SI
3,A4SJ9B8S0FYDC,B00000J1SI,1.0,937180800,product_SI
4,A1SCIWWJCB1RLQ,B00000J1SI,1.0,947203200,product_SI
...,...,...,...,...,...
12199,A2JTUYVQFB1VTA,B00KEIRFSI,3.0,1405641600,product_SI
12200,A2OEXK11JIKDDW,B00KEIRFSI,5.0,1405036800,product_SI
12201,A3FYB8SU4M73U6,B00KEIRFSI,5.0,1404432000,product_SI
12202,A3MUNF55YPOMVT,B00KEIRFSI,2.0,1403395200,product_SI


In [21]:
len(product_SI.product_id.unique())

712

In [23]:
product_SI = pd.DataFrame(reviews[reviews['product_code']=='product_SI']).reset_index(drop=True)
product_SI =  pd.merge(product_SI, 
                       SI.iloc[:len(product_SI.product_id.unique())]\
                       .join(pd.Series(product_SI['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_SI.to_csv('product_SI.csv',index=False)
pd.read_csv('product_SI.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,ARQBN70XQND3O,B00000J1SI,1.0,958694400,product_SI,HOTKEI (Pack of 6 Slime) Multicolor Fruit Scen...,https://m.media-amazon.com/images/I/81WLIWanEC...,https://www.amazon.in//HOTKEI-Slime-Multicolor...
1,A1B10WSCRZTPPT,B00000J1SI,5.0,932169600,product_SI,HOTKEI (Pack of 6 Slime) Multicolor Fruit Scen...,https://m.media-amazon.com/images/I/81WLIWanEC...,https://www.amazon.in//HOTKEI-Slime-Multicolor...
2,AYF026D7GCM5Y,B00000J1SI,4.0,962582400,product_SI,HOTKEI (Pack of 6 Slime) Multicolor Fruit Scen...,https://m.media-amazon.com/images/I/81WLIWanEC...,https://www.amazon.in//HOTKEI-Slime-Multicolor...
3,A4SJ9B8S0FYDC,B00000J1SI,1.0,937180800,product_SI,HOTKEI (Pack of 6 Slime) Multicolor Fruit Scen...,https://m.media-amazon.com/images/I/81WLIWanEC...,https://www.amazon.in//HOTKEI-Slime-Multicolor...
4,A1SCIWWJCB1RLQ,B00000J1SI,1.0,947203200,product_SI,HOTKEI (Pack of 6 Slime) Multicolor Fruit Scen...,https://m.media-amazon.com/images/I/81WLIWanEC...,https://www.amazon.in//HOTKEI-Slime-Multicolor...
...,...,...,...,...,...,...,...,...
12199,A2JTUYVQFB1VTA,B00KEIRFSI,3.0,1405641600,product_SI,Juliana Fluffy Butter Combo Slime Stretchy Non...,https://m.media-amazon.com/images/I/71X1EAvJmB...,https://www.amazon.in//JULIANA-Scented-Fluffy-...
12200,A2OEXK11JIKDDW,B00KEIRFSI,5.0,1405036800,product_SI,Juliana Fluffy Butter Combo Slime Stretchy Non...,https://m.media-amazon.com/images/I/71X1EAvJmB...,https://www.amazon.in//JULIANA-Scented-Fluffy-...
12201,A3FYB8SU4M73U6,B00KEIRFSI,5.0,1404432000,product_SI,Juliana Fluffy Butter Combo Slime Stretchy Non...,https://m.media-amazon.com/images/I/71X1EAvJmB...,https://www.amazon.in//JULIANA-Scented-Fluffy-...
12202,A3MUNF55YPOMVT,B00KEIRFSI,2.0,1403395200,product_SI,Juliana Fluffy Butter Combo Slime Stretchy Non...,https://m.media-amazon.com/images/I/71X1EAvJmB...,https://www.amazon.in//JULIANA-Scented-Fluffy-...


### Product SN dataframe

In [278]:
df = get_scraped('sony products')
SN = df.copy()
SN

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71+GXwLe23...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
1,Sony Wh-Ch510 Bluetooth Wireless On Ear Headph...,https://m.media-amazon.com/images/I/41i+KMBAFj...,https://www.amazon.in//SONY-WH-CH510-Wireless-...
2,Sony MDR-ZX110A Wired On Ear Headphone without...,https://m.media-amazon.com/images/I/41BoLKMYjn...,https://www.amazon.in//Sony-MDR-ZX110A-Stereo-...
3,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71JaIPh39S...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
4,Sony Srs-Xb13 Wireless Extra Bass Portable Com...,https://m.media-amazon.com/images/I/71mRVqt2rX...,https://www.amazon.in//Sony-SRS-XB13-Bluetooth...
...,...,...,...
219,Sony 4X DVD RW Media,https://m.media-amazon.com/images/I/51KwUrY0+e...,https://www.amazon.in//Sony-4X-DVD-RW-Media/dp...
220,Sony DLT1 VS Cleaning Cartridge (VS80 and VS16...,https://m.media-amazon.com/images/I/41TUNTMkrP...,https://www.amazon.in//Sony-Cleaning-Cartridge...
221,LTO Ultrium 5 with Labels Data,https://m.media-amazon.com/images/I/51CZCQo8Se...,https://www.amazon.in//LTO-Ultrium-5-Labels-Da...
222,Sony MDR-E9LP White,https://m.media-amazon.com/images/I/51O6ADYghB...,https://www.amazon.in//Sony-MDR-E9LP-WICU-Whit...


In [279]:
product_SN = pd.DataFrame(reviews[reviews['product_code']=='product_SN']).reset_index(drop=True)
product_SN =  pd.merge(product_SN, 
                       SN.iloc[:len(product_SN.product_id.unique())]\
                       .join(pd.Series(product_SN['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_SN.to_csv('product_SN.csv',index=False)
pd.read_csv('product_SN.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A2EOZLWEXG9T7D,B00000JBSN,5.0,932083200,product_SN,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71+GXwLe23...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
1,A2NE1GWSMKGC3P,B00000JBSN,3.0,1315180800,product_SN,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71+GXwLe23...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
2,A1YDYMXPHK5B13,B00000JBSN,5.0,1332892800,product_SN,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71+GXwLe23...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
3,A2OMFF9EAJ7KY9,B00000JBSN,4.0,945129600,product_SN,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71+GXwLe23...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
4,A1PSG8DOO9YMLM,B00000JBSN,3.0,931824000,product_SN,Sony WI-XB400 Wireless Extra Bass in-Ear Headp...,https://m.media-amazon.com/images/I/71+GXwLe23...,https://www.amazon.in//Sony-WI-XB400-Wireless-...
...,...,...,...,...,...,...,...,...
443,A1E3GSE9DOBLWI,B0000C20SN,3.0,1129766400,product_SN,Sony WF-1000XM3 Industry Leading Active Noise ...,https://m.media-amazon.com/images/I/61zKkP36kD...,https://www.amazon.in//Sony-WF-1000XM3-Wireles...
444,A29C114ZRGZLT3,B0000DIESN,1.0,1110844800,product_SN,Sony SEL16F28 E Mount - APS-C 16mm F2.8 Prime ...,https://m.media-amazon.com/images/I/61bBCoYdjp...,https://www.amazon.in//Sony-SEL16F28-Mount-APS...
445,AWWFGGMR2L0IL,B0000DIESN,2.0,1096848000,product_SN,Sony SEL16F28 E Mount - APS-C 16mm F2.8 Prime ...,https://m.media-amazon.com/images/I/61bBCoYdjp...,https://www.amazon.in//Sony-SEL16F28-Mount-APS...
446,A3QBHMMDRSC5VY,B0000DIESN,1.0,1121731200,product_SN,Sony SEL16F28 E Mount - APS-C 16mm F2.8 Prime ...,https://m.media-amazon.com/images/I/61bBCoYdjp...,https://www.amazon.in//Sony-SEL16F28-Mount-APS...


### Product SO dataframe

In [6]:
df = get_scraped('soft toys&i=toys')
SO = df.copy()
SO

  0%|          | 0/39 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Babique Unicorn Teddy Bear Plush Soft Toy Cute...,https://m.media-amazon.com/images/I/31l9X1z7+8...,https://www.amazon.in//Babique-Unicorn-Stuffed...
1,Babique Penguin Teddy Bear Plush Soft Toy Cute...,https://m.media-amazon.com/images/I/411KgE3-I3...,https://www.amazon.in//Babique-Penguin-Stuffed...
2,Babique Octopus Sitting Plush Soft Toy Cute Ki...,https://m.media-amazon.com/images/I/41c83CqCMN...,https://www.amazon.in//Babique-Octopus-Sitting...
3,"Webby Soft Animal Plush Elephant Toy 20cm, Blue",https://m.media-amazon.com/images/I/711DBUR31b...,https://www.amazon.in//Webby-Soft-Animal-Plush...
4,odinbirds Combo of 4 Animals Soft Toy Kids (Ba...,https://m.media-amazon.com/images/I/81cYBsgOHR...,https://www.amazon.in//odinbirds-Animals-Eleph...
...,...,...,...
931,Shivay International- Soft Toys Cartoon Charac...,https://m.media-amazon.com/images/I/41aChQBwWH...,https://www.amazon.in//Shivay-International-Ch...
932,"HUG 'N' FEEL SOFT TOYS Ball, baby toys, kids t...",https://m.media-amazon.com/images/I/5139P6T8Bl...,https://www.amazon.in//HUG-FEEL-SOFT-TOYS-chil...
933,ZYEPE Dinosaur Soft / Stuff Toy | Green Soft C...,https://m.media-amazon.com/images/I/51HusaRMUw...,https://www.amazon.in//ZYEPE-Dinosaur-Cartoon-...
934,"Tiny Miny Dora Monkey Soft Teddy Toy, Kids Big...",https://m.media-amazon.com/images/I/61DnFtnoct...,https://www.amazon.in//Tiny-Miny-Decoration-Fe...


In [289]:
product_SO = pd.DataFrame(reviews[reviews['product_code']=='product_SO']).reset_index(drop=True)
product_SO

,user_id,product_id,Ratings,Time_stamp,product_code
0,A5JLAU2ARJ0BO,B00004TVSO,2.0,1107129600,product_SO
1,A2BKGT9VAJM8HB,B00004YNSO,5.0,975715200,product_SO
2,A3JK8M9P3HMH9Y,B00004YNSO,5.0,1285113600,product_SO
3,A224FTD4WTOHA8,B00005A8SO,5.0,1079827200,product_SO
4,A3ONNS8GBOGKGX,B00005RKSO,5.0,1343260800,product_SO
...,...,...,...,...,...
11336,AP0AKQ7F092N1,B00K8942SO,5.0,1404172800,product_SO
11337,A27XOE502ZTASY,B00KKAMVSO,1.0,1405900800,product_SO
11338,A2BRMO5N6V2EE5,B00KKAMVSO,1.0,1405382400,product_SO
11339,A2IKFCPB451V27,B00KKAMVSO,1.0,1405814400,product_SO


In [291]:
len(product_SO.product_id.unique())

715

In [7]:
product_SO = pd.DataFrame(reviews[reviews['product_code']=='product_SO']).reset_index(drop=True)
product_SO =  pd.merge(product_SO, 
                       SO.iloc[:len(product_SO.product_id.unique())]\
                       .join(pd.Series(product_SO['product_id'].unique(),name='product_id'))\
                       .reset_index(drop=True)
                       , on='product_id')
product_SO.to_csv('product_SO.csv',index=False)
pd.read_csv('product_SO.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A5JLAU2ARJ0BO,B00004TVSO,2.0,1107129600,product_SO,Babique Unicorn Teddy Bear Plush Soft Toy Cute...,https://m.media-amazon.com/images/I/31l9X1z7+8...,https://www.amazon.in//Babique-Unicorn-Stuffed...
1,A2BKGT9VAJM8HB,B00004YNSO,5.0,975715200,product_SO,Babique Penguin Teddy Bear Plush Soft Toy Cute...,https://m.media-amazon.com/images/I/411KgE3-I3...,https://www.amazon.in//Babique-Penguin-Stuffed...
2,A3JK8M9P3HMH9Y,B00004YNSO,5.0,1285113600,product_SO,Babique Penguin Teddy Bear Plush Soft Toy Cute...,https://m.media-amazon.com/images/I/411KgE3-I3...,https://www.amazon.in//Babique-Penguin-Stuffed...
3,A224FTD4WTOHA8,B00005A8SO,5.0,1079827200,product_SO,Babique Octopus Sitting Plush Soft Toy Cute Ki...,https://m.media-amazon.com/images/I/41c83CqCMN...,https://www.amazon.in//Babique-Octopus-Sitting...
4,A3ONNS8GBOGKGX,B00005RKSO,5.0,1343260800,product_SO,"Webby Soft Animal Plush Elephant Toy 20cm, Blue",https://m.media-amazon.com/images/I/711DBUR31b...,https://www.amazon.in//Webby-Soft-Animal-Plush...
...,...,...,...,...,...,...,...,...
11336,AP0AKQ7F092N1,B00K8942SO,5.0,1404172800,product_SO,Urban hub Stuffed Soft Animal Toys (Simba Lion...,https://m.media-amazon.com/images/I/41PDT8vnU7...,https://www.amazon.in//Urban-Stuffed-Soft-Anim...
11337,A27XOE502ZTASY,B00KKAMVSO,1.0,1405900800,product_SO,Tickles Penguin Animal Soft Stuffed Plush Toy ...,https://m.media-amazon.com/images/I/51NWPXL1SK...,https://www.amazon.in//Tickles-Penguin-Stuffed...
11338,A2BRMO5N6V2EE5,B00KKAMVSO,1.0,1405382400,product_SO,Tickles Penguin Animal Soft Stuffed Plush Toy ...,https://m.media-amazon.com/images/I/51NWPXL1SK...,https://www.amazon.in//Tickles-Penguin-Stuffed...
11339,A2IKFCPB451V27,B00KKAMVSO,1.0,1405814400,product_SO,Tickles Penguin Animal Soft Stuffed Plush Toy ...,https://m.media-amazon.com/images/I/51NWPXL1SK...,https://www.amazon.in//Tickles-Penguin-Stuffed...


### Product WD dataframe

In [48]:
df = get_scraped('wd')
df

  0%|          | 0/20 [00:00<?, ?it/s]

,product_name,product_img,product_link
0,Western Digital WD 1.5TB Elements Portable Har...,https://m.media-amazon.com/images/I/81WFWi9sKl...,https://www.amazon.in//Elements-Portable-Exter...
1,Seagate IronWolf 4TB NAS Internal Hard Drive H...,https://m.media-amazon.com/images/I/71y0UqWIiS...,https://www.amazon.in//Seagate-IronWolf-Intern...
2,Western Digital WD 2TB My Passport Portable Ha...,https://m.media-amazon.com/images/I/615dYaRCSL...,https://www.amazon.in//Passport-Portable-Exter...
3,Western Digital WD 1TB Elements Portable Hard ...,https://m.media-amazon.com/images/I/81u47A2BCh...,https://www.amazon.in//Elements-Portable-Exter...
4,Western Digital WD Green SATA 240GB Internal S...,https://m.media-amazon.com/images/I/71yOeCQVTP...,https://www.amazon.in//Western-Digital-Green-2...
...,...,...,...
301,"WD 18TB Elements Desktop Hard Drive, USB 3.0 -...",https://m.media-amazon.com/images/I/61LoGZq-4N...,https://www.amazon.in//18TB-Elements-Desktop-D...
302,WD Elements 1TB USB 3.0 Portable External Hard...,https://m.media-amazon.com/images/I/41oaYfHyBW...,https://www.amazon.in//Elements-Portable-Exter...
303,Western Digital Ultrastar DC HC310 HUS726T4TAL...,https://m.media-amazon.com/images/I/61XVI6On5h...,https://www.amazon.in//Western-Digital-Ultrast...
304,WD Red Pro 2TB Internal NAS Hard Drive (WD2002...,https://m.media-amazon.com/images/I/51JMJC17q-...,https://www.amazon.in//Red-Internal-Hard-Drive...


In [50]:
product_WD = pd.DataFrame(reviews[reviews['product_code']=='product_WD']).reset_index(drop=True)
product_WD =  pd.merge(product_WD, 
                       df.iloc[:len(product_WD)]\
                       .join(product_WD['product_id'])\
                       .drop_duplicates(subset='product_id', keep='first')\
                       .reset_index(drop=True)
                       , on='product_id')
product_WD.to_csv('product_WD.csv',index=False)
pd.read_csv('product_WD.csv')

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A3ADOW081PINFC,B00001MXWD,4.0,940032000,product_WD,Western Digital WD 1.5TB Elements Portable Har...,https://m.media-amazon.com/images/I/81WFWi9sKl...,https://www.amazon.in//Elements-Portable-Exter...
1,A20TBOG8IBHU44,B00001MXWD,2.0,958521600,product_WD,Western Digital WD 1.5TB Elements Portable Har...,https://m.media-amazon.com/images/I/81WFWi9sKl...,https://www.amazon.in//Elements-Portable-Exter...
2,ACCUPJZUVRNT1,B00001MXWD,1.0,946771200,product_WD,Western Digital WD 1.5TB Elements Portable Har...,https://m.media-amazon.com/images/I/81WFWi9sKl...,https://www.amazon.in//Elements-Portable-Exter...
3,A31OY8RCMP5IUR,B00001MXWD,1.0,944265600,product_WD,Western Digital WD 1.5TB Elements Portable Har...,https://m.media-amazon.com/images/I/81WFWi9sKl...,https://www.amazon.in//Elements-Portable-Exter...
4,A269OEVY5AJ95G,B00001MXWD,5.0,1021593600,product_WD,Western Digital WD 1.5TB Elements Portable Har...,https://m.media-amazon.com/images/I/81WFWi9sKl...,https://www.amazon.in//Elements-Portable-Exter...
...,...,...,...,...,...,...,...,...
80,A2V2IEC344N8ED,B000093LWD,5.0,1195084800,product_WD,Western Digital Black 4TB P10 Game Drive Porta...,https://m.media-amazon.com/images/I/91nz8ViqVL...,https://www.amazon.in//Black-Drive-Portable-Ex...
81,A1DDIPXDZTD1E1,B000093LWD,1.0,1386892800,product_WD,Western Digital Black 4TB P10 Game Drive Porta...,https://m.media-amazon.com/images/I/91nz8ViqVL...,https://www.amazon.in//Black-Drive-Portable-Ex...
82,A1NA7W62VGQ5JZ,B00009QOWD,5.0,1166054400,product_WD,"WD_BLACK™ SN850X NVMe™ SSD Gaming Storage, 2TB",https://m.media-amazon.com/images/I/61u-w0nMDT...,https://www.amazon.in//WD_BLACKTM-SN850X-NVMeT...
83,APFCBRGQMIZGG,B00009R7WD,5.0,1346630400,product_WD,Western Digital 6TB WD Red Plus NAS Internal H...,https://m.media-amazon.com/images/I/41aOVlvKlZ...,https://www.amazon.in//Western-Digital-Plus-In...


In [17]:
for product_code in reviews.product_code.unique():
    print(product_code)

product_40
product_44
product_41
product_77
product_69
product_55
product_2X
product_15
product_43
product_80
product_96
product_26
product_34
product_42
product_95
product_20
product_09
product_05
product_47
product_70
product_62
product_13
product_02
product_27
product_88
product_81
product_89
product_07
product_58
product_31
product_18
product_3X
product_6X
product_98
product_75
product_37
product_82
product_91
product_00
product_17
product_86
product_52
product_78
product_0X
product_90
product_19
product_66
product_74
product_76
product_84
product_1X
product_36
product_97
product_87
product_94
product_24
product_99
product_06
product_23
product_60
product_53
product_45
product_04
product_48
product_56
product_72
product_01
product_22
product_73
product_38
product_11
product_59
product_30
product_57
product_65
product_92
product_12
product_03
product_08
product_68
product_83
product_51
product_39
product_85
product_79
product_61
product_16
product_29
product_64
product_46
product_54

product_MX
product_N3
product_6H
product_M2
product_U7
product_3U
product_4P
product_4U
product_A8
product_Q5
product_7Q
product_7S
product_7U
product_JE
product_VD
product_6U
product_P9
product_PA
product_PH
product_W7
product_0I
product_1B
product_ZC
product_ZE
product_ZK
product_1D
product_3D
product_4O
product_7Y
product_8T
product_SO
product_SP
product_UM
product_LY
product_ME
product_P8
product_6C
product_7H
product_7I
product_B6
product_7K
product_BO
product_EG
product_FZ
product_GL
product_KB
product_KM
product_KP
product_R1
product_TP
product_TR
product_U1
product_OJ
product_0H
product_AW
product_7L
product_SM
product_4R
product_VR
product_NO
product_8S
product_8Y
product_ON
product_QG
product_MC
product_TL
product_YC
product_1H
product_1T
product_TA
product_TB
product_TE
product_NY
product_NZ
product_YK
product_YY
product_F0
product_IC
product_XF
product_Y2
product_Y7
product_YF
product_ZU
product_IL
product_J9
product_PW
product_QO
product_Z2
product_Z4
product_ZA
product_ZI

# Final Dataframe

In [2]:
product_A0 = pd.read_csv('product_A0.csv')
product_A1 = pd.read_csv('product_A1.csv')
product_A2 = pd.read_csv('product_A2.csv')
product_A3 = pd.read_csv('product_A3.csv')
product_A4 = pd.read_csv('product_A4.csv')
product_A5 = pd.read_csv('product_A5.csv')
product_A6 = pd.read_csv('product_A6.csv')
product_A7 = pd.read_csv('product_A7.csv')
product_A8 = pd.read_csv('product_A8.csv')
product_A9 = pd.read_csv('product_A9.csv')
product_AA = pd.read_csv('product_AA.csv')
product_AB = pd.read_csv('product_AB.csv')
product_AC = pd.read_csv('product_AC.csv')
product_AD = pd.read_csv('product_AD.csv')
product_AE = pd.read_csv('product_AE.csv')
product_AF = pd.read_csv('product_AF.csv')
product_AG = pd.read_csv('product_AG.csv')
product_AH = pd.read_csv('product_AH.csv')
product_AI = pd.read_csv('product_AI.csv')
product_AJ = pd.read_csv('product_AJ.csv')
product_AK = pd.read_csv('product_AK.csv')
product_AL = pd.read_csv('product_AL.csv')
product_AM = pd.read_csv('product_AM.csv')
product_AN = pd.read_csv('product_AN.csv')
product_AO = pd.read_csv('product_AO.csv')
product_AP = pd.read_csv('product_AP.csv')
product_AQ = pd.read_csv('product_AQ.csv')
product_AR = pd.read_csv('product_AR.csv')
product_AS = pd.read_csv('product_AS.csv')
product_AT = pd.read_csv('product_AT.csv')
product_AU = pd.read_csv('product_AU.csv')
product_AV = pd.read_csv('product_AV.csv')
product_AW = pd.read_csv('product_AW.csv')
product_AX = pd.read_csv('product_AX.csv')
product_AY = pd.read_csv('product_AY.csv')
product_AZ = pd.read_csv('product_AZ.csv')
product_B0 = pd.read_csv('product_B0.csv')
product_B1 = pd.read_csv('product_B1.csv')
product_B2 = pd.read_csv('product_B2.csv')
product_B3 = pd.read_csv('product_B3.csv')
product_B4 = pd.read_csv('product_B4.csv')
product_B5 = pd.read_csv('product_B5.csv')
product_B6 = pd.read_csv('product_B6.csv')
product_B7 = pd.read_csv('product_B7.csv')
product_B8 = pd.read_csv('product_B8.csv')
product_B9 = pd.read_csv('product_B9.csv')
product_BA = pd.read_csv('product_BA.csv')
product_BB = pd.read_csv('product_BB.csv')
product_BC = pd.read_csv('product_BC.csv')
product_BD = pd.read_csv('product_BD.csv')
product_BE = pd.read_csv('product_BE.csv')
product_BF = pd.read_csv('product_BF.csv')
product_BG = pd.read_csv('product_BG.csv')
product_BH = pd.read_csv('product_BH.csv')
product_BI = pd.read_csv('product_BI.csv')
product_BJ = pd.read_csv('product_BJ.csv')
product_BK = pd.read_csv('product_BK.csv')
product_BL = pd.read_csv('product_BL.csv')
product_BM = pd.read_csv('product_BM.csv')
product_BN = pd.read_csv('product_BN.csv')
product_BO = pd.read_csv('product_BO.csv')
product_BP = pd.read_csv('product_BP.csv')
product_BQ = pd.read_csv('product_BQ.csv')
product_BR = pd.read_csv('product_BR.csv')
product_BS = pd.read_csv('product_BS.csv')
product_BT = pd.read_csv('product_BT.csv')
product_BU = pd.read_csv('product_BU.csv')
product_BV = pd.read_csv('product_BV.csv')
product_BW = pd.read_csv('product_BW.csv')
product_BX = pd.read_csv('product_BX.csv')
product_BY = pd.read_csv('product_BY.csv')
product_BZ = pd.read_csv('product_BZ.csv')
product_C0 = pd.read_csv('product_C0.csv')
product_C1 = pd.read_csv('product_C1.csv')
product_C2 = pd.read_csv('product_C2.csv')
product_C3 = pd.read_csv('product_C3.csv')
product_C4 = pd.read_csv('product_C4.csv')
product_C5 = pd.read_csv('product_C5.csv')
product_C6 = pd.read_csv('product_C6.csv')
product_C7 = pd.read_csv('product_C7.csv')

product_IB = pd.read_csv('product_IB.csv')

product_MI = pd.read_csv('product_MI.csv')

product_SI = pd.read_csv('product_SI.csv')
product_SO = pd.read_csv('product_SO.csv')
product_SN = pd.read_csv('product_SN.csv')

product_WD = pd.read_csv('product_WD.csv')


In [3]:
test = product_AA.append([product_A0, product_A1, product_A2, product_A3, product_A4, product_A5, product_A6, product_A7, product_A8, product_A9,
                          product_AB, product_AC, product_AD, product_AE, product_AF, product_AG, product_AH, product_AI, product_AJ, product_AK, product_AL, product_AM, product_AN, product_AO, product_AP, product_AQ, product_AR, product_AS, product_AT, product_AU, product_AV, product_AW, product_AX, product_AY, product_AZ,
                          product_B0, product_B1, product_B2, product_B4, product_B5, product_B6, product_B7, product_B8, product_B9,
                          product_BA, product_BB, product_BC, product_BD, product_BE, product_BF, product_BG, product_BH, product_BI, product_BJ, product_BK, product_BL, product_BM, product_BN, product_BO, product_BP, product_BQ, product_BR, product_BS, product_BT, product_BU, product_BV, product_BW, product_BX, product_BY, product_BZ,
                          product_C0, product_C1, product_C2, product_C3, product_C4, product_C5, product_C6, product_C7,
                          product_IB,product_MI,
                          product_SI,product_SO,product_SN,
                          product_WD]).reset_index(drop=True)
test

,user_id,product_id,Ratings,Time_stamp,product_code,product_name,product_img,product_link
0,A1NA7W62VGQ5JZ,B00000JLAA,5.0,1173139200,product_AA,"Duracell Ultra Alkaline AA Battery, 8 Pcs",https://m.media-amazon.com/images/I/71G4VJZ+9W...,https://www.amazon.in//Duracell-Alkaline-Batte...
1,A14ME4FQBNFYWH,B00001P4AA,4.0,960595200,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
2,AMGJMFJ63DWWH,B00001P4AA,4.0,959299200,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
3,A11LT1IGS1Q5CZ,B00001P4AA,4.0,968976000,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
4,A27JOWVO9PQAI0,B00001P4AA,4.0,1008201600,product_AA,The Orignal BN46 4000 mAh Battery Compatible f...,https://m.media-amazon.com/images/I/41fPFiiTfG...,https://www.amazon.in//Orignal-Battery-Compati...
...,...,...,...,...,...,...,...,...
521586,A2V2IEC344N8ED,B000093LWD,5.0,1195084800,product_WD,Western Digital Black 4TB P10 Game Drive Porta...,https://m.media-amazon.com/images/I/91nz8ViqVL...,https://www.amazon.in//Black-Drive-Portable-Ex...
521587,A1DDIPXDZTD1E1,B000093LWD,1.0,1386892800,product_WD,Western Digital Black 4TB P10 Game Drive Porta...,https://m.media-amazon.com/images/I/91nz8ViqVL...,https://www.amazon.in//Black-Drive-Portable-Ex...
521588,A1NA7W62VGQ5JZ,B00009QOWD,5.0,1166054400,product_WD,"WD_BLACK™ SN850X NVMe™ SSD Gaming Storage, 2TB",https://m.media-amazon.com/images/I/61u-w0nMDT...,https://www.amazon.in//WD_BLACKTM-SN850X-NVMeT...
521589,APFCBRGQMIZGG,B00009R7WD,5.0,1346630400,product_WD,Western Digital 6TB WD Red Plus NAS Internal H...,https://m.media-amazon.com/images/I/41aOVlvKlZ...,https://www.amazon.in//Western-Digital-Plus-In...


In [ ]:
test.to_csv('Final_ratings.csv',index=False)
pd.read_csv('Final_ratings.csv')